This is a data preprocessing script for the Wordle game. Data is manually collected from various sources related to Wordle in Russian. 

The script generates the followind word lists:
* allowed words (wordle/resources/allowed_words.txt): this word lists contains all valid guesses for the game.
* target words (wordle/resources/target_words.txt): this word lists contains all target words from the intersection of three datasets: 1) manually collected target words 2) word frequency dictionary 3) word-clue dictionary created by the wordle/utils/clue_scraper.py.
* target word lists by difficulty (wordle/resources/*difficulty*_words.txt): the target words list divided in 3 equal parts by word frequency (*easy* - most frequent, *medium* - with medium frequency, *hard* - least frequent)

For more details, refer to the **wordle_language_porting_guide.md**

In [71]:
import json

import pandas as pd

## Data

In [23]:
# source 1: https://github.com/Darxor/wordle-ru-algorithm/blob/main/data/ru_wordle_possible.txt
allowed_words1 = ["\u0430\u0430\u043d\u0442\u0430","\u0430\u0431\u0430\u0437\u0430","\u0430\u0431\u0430\u0437\u0438","\u0430\u0431\u0430\u0439\u044f","\u0430\u0431\u0430\u043a\u0430","\u0430\u0431\u0432\u0435\u0440","\u0430\u0431\u0433\u0430\u0437","\u0430\u0431\u0437\u0430\u0446","\u0430\u0431\u0437\u0438\u043c","\u0430\u0431\u0438\u043e\u0437","\u0430\u0431\u043e\u0440\u0442","\u0430\u0431\u0440\u0438\u0441","\u0430\u0431\u0446\u0443\u0433","\u0430\u0431\u0448\u0438\u0434","\u0430\u0431\u0448\u0438\u0442","\u0430\u0431\u044c\u044e\u0437","\u0430\u0432\u0430\u043b\u044c","\u0430\u0432\u0430\u043d\u0441","\u0430\u0432\u0433\u0438\u0442","\u0430\u0432\u0435\u043d\u044e","\u0430\u0432\u0435\u0440\u0441","\u0430\u0432\u0438\u0437\u043e","\u0430\u0432\u043b\u043e\u0441","\u0430\u0432\u043e\u0441\u0442","\u0430\u0432\u043e\u0441\u044c","\u0430\u0432\u0440\u0430\u043b","\u0430\u0432\u0440\u0430\u043d","\u0430\u0432\u0440\u0430\u0442","\u0430\u0432\u0441\u0442\u0440","\u0430\u0432\u0442\u0438\u043a","\u0430\u0432\u0442\u043e\u043b","\u0430\u0433\u0430\u0432\u0430","\u0430\u0433\u0430\u043f\u044d","\u0430\u0433\u0435\u043d\u0442","\u0430\u0433\u043e\u0433\u044d","\u0430\u0433\u043e\u0440\u0430","\u0430\u0433\u0440\u0430\u0444","\u0430\u0433\u0443\u0448\u0430","\u0430\u0434\u0430\u0442\u0442","\u0430\u0434\u0432\u0438\u043b","\u0430\u0434\u0434\u043e\u043d","\u0430\u0434\u0438\u0449\u0435","\u0430\u0434\u043e\u0431\u0430","\u0430\u0434\u0440\u0435\u0441","\u0430\u0434\u0440\u043e\u043d","\u0430\u0434\u0440\u044f\u0441","\u0430\u0435\u0432\u0438\u0442","\u0430\u0436\u0433\u043e\u043d","\u0430\u0437\u0430\u0440\u0442","\u0430\u0437\u0435\u0440\u0438","\u0430\u0439\u0432\u0430\u043d","\u0430\u0439\u0432\u0430\u0440","\u0430\u0439\u0434\u043e\u043b","\u0430\u0439\u043c\u0430\u043a","\u0430\u0439\u043e\u043b\u0438","\u0430\u0439\u043f\u0430\u0434","\u0430\u0439\u043f\u043e\u0434","\u0430\u0439\u043f\u044d\u0434","\u0430\u0439\u0440\u0430\u043a","\u0430\u0439\u0440\u0430\u043d","\u0430\u0439\u0440\u043e\u043b","\u0430\u0439\u0440\u043e\u043d","\u0430\u0439\u0441\u0438\u043a","\u0430\u0439\u0442\u044b\u0441","\u0430\u0439\u0442\u044b\u0448","\u0430\u0439\u0444\u043e\u043d","\u0430\u043a\u0430\u0436\u0443","\u0430\u043a\u0430\u043d\u0442","\u0430\u043a\u0430\u043d\u0444","\u0430\u043a\u0430\u0448\u0430","\u0430\u043a\u0430\u044d\u043c","\u0430\u043a\u0438\u0434\u0430","\u0430\u043a\u043a\u0443\u043c","\u0430\u043a\u043c\u0438\u0442","\u0430\u043a\u043e\u043c\u0430","\u0430\u043a\u0440\u0438\u043b","\u0430\u043a\u0440\u043e\u043d","\u0430\u043a\u0441\u0438\u0441","\u0430\u043a\u0441\u043e\u043d","\u0430\u043a\u0442\u0435\u044f","\u0430\u043a\u0442\u0438\u0432","\u0430\u043a\u0442\u0438\u043d","\u0430\u043a\u0446\u0438\u0437","\u0430\u043a\u0446\u0438\u044f","\u0430\u043b\u0430\u0438\u0442","\u0430\u043b\u0430\u043d\u044c","\u0430\u043b\u0430\u0440\u043c","\u0430\u043b\u0433\u0438\u0434","\u0430\u043b\u0433\u0438\u044f","\u0430\u043b\u0433\u043e\u043b","\u0430\u043b\u0435\u0440\u0442","\u0430\u043b\u0438\u0430\u0441","\u0430\u043b\u0438\u0431\u0438","\u0430\u043b\u043a\u0430\u043d","\u0430\u043b\u043a\u0435\u043d","\u0430\u043b\u043a\u0438\u0434","\u0430\u043b\u043a\u0438\u043b","\u0430\u043b\u043a\u0438\u043d","\u0430\u043b\u043b\u0435\u043d","\u0430\u043b\u043b\u0435\u044f","\u0430\u043b\u043b\u0438\u043b","\u0430\u043b\u043b\u043e\u0434","\u0430\u043b\u043b\u044e\u0440","\u0430\u043b\u043c\u0430\u0437","\u0430\u043b\u0442\u0430\u043d","\u0430\u043b\u0442\u0435\u0439","\u0430\u043b\u0442\u0435\u044f","\u0430\u043b\u0442\u044b\u043d","\u0430\u043b\u0443\u043d\u0434","\u0430\u043b\u0443\u043d\u0435","\u0430\u043b\u0444\u0435\u0440","\u0430\u043b\u0447\u0431\u0430","\u0430\u043b\u044b\u0447\u0430","\u0430\u043b\u044c\u0431\u0430","\u0430\u043b\u044c\u043d\u0438","\u0430\u043b\u044c\u0444\u0430","\u0430\u043c\u0431\u0430\u0440","\u0430\u043c\u0431\u0440\u0430","\u0430\u043c\u0431\u0440\u0435","\u0430\u043c\u0432\u043e\u043d","\u0430\u043c\u0438\u0434\u0430","\u0430\u043c\u0438\u043d\u044c","\u0430\u043c\u043c\u0438\u043d","\u0430\u043c\u043f\u0435\u0440","\u0430\u043c\u043f\u0438\u0440","\u0430\u043c\u0443\u043b\u0430","\u0430\u043d\u0430\u0448\u0430","\u0430\u043d\u0431\u0430\u043d","\u0430\u043d\u0431\u0430\u0440","\u0430\u043d\u0431\u0443\u0440","\u0430\u043d\u0433\u0430\u0440","\u0430\u043d\u0433\u0430\u0441","\u0430\u043d\u0433\u043e\u0431","\u0430\u043d\u0433\u0443\u0441","\u0430\u043d\u0437\u0430\u0446","\u0430\u043d\u0437\u0443\u0440","\u0430\u043d\u0438\u043c\u0435","\u0430\u043d\u0438\u043e\u043d","\u0430\u043d\u043a\u0430\u043f","\u0430\u043d\u043a\u0435\u0440","\u0430\u043d\u043a\u043e\u0440","\u0430\u043d\u043b\u0438\u043c","\u0430\u043d\u043d\u0438\u0442","\u0430\u043d\u043e\u043d\u0430","\u0430\u043d\u043e\u043d\u0441","\u0430\u043d\u0440\u0435\u0433","\u0430\u043d\u0441\u0430\u0431","\u0430\u043d\u0442\u0430\u043b","\u0430\u043d\u0442\u0435\u0439","\u0430\u043d\u0442\u0435\u043c","\u0430\u043d\u0442\u0438\u043a","\u0430\u043d\u0442\u0440\u0435","\u0430\u043d\u0447\u0430\u0440","\u0430\u043d\u044c\u044e\u043c","\u0430\u043e\u0437\u0430\u0439","\u0430\u043e\u0440\u0442\u0430","\u0430\u043f\u0430\u0440\u0442","\u0430\u043f\u0435\u043a\u0441","\u0430\u043f\u043b\u0438\u0442","\u0430\u043f\u043d\u043e\u044d","\u0430\u043f\u043e\u0440\u0442","\u0430\u043f\u0440\u043e\u0448","\u0430\u043f\u0440\u0443\u0432","\u0430\u043f\u0441\u0438\u044f","\u0430\u0440\u0430\u043a\u0430","\u0430\u0440\u0431\u043e\u0440","\u0430\u0440\u0431\u0443\u0437","\u0430\u0440\u0433\u0430\u043b","\u0430\u0440\u0433\u0438\u0448","\u0430\u0440\u0433\u043e\u043d","\u0430\u0440\u0435\u0430\u043b","\u0430\u0440\u0435\u043a\u0430","\u0430\u0440\u0435\u043d\u0430","\u0430\u0440\u0435\u0441\u0442","\u0430\u0440\u0437\u043d\u0438","\u0430\u0440\u043a\u0430\u0434","\u0430\u0440\u043a\u0430\u043d","\u0430\u0440\u043a\u043e\u0437","\u0430\u0440\u043c\u0438\u044f","\u0430\u0440\u043c\u044e\u0440","\u0430\u0440\u043c\u044f\u043a","\u0430\u0440\u043d\u0438\u0441","\u0430\u0440\u0441\u0438\u043d","\u0430\u0440\u0441\u0438\u0441","\u0430\u0440\u0442\u043e\u0441","\u0430\u0440\u0442\u0443\u0441","\u0430\u0440\u0445\u0430\u0442","\u0430\u0440\u0445\u0435\u0439","\u0430\u0440\u0445\u0438\u0432","\u0430\u0440\u0447\u0430\u043a","\u0430\u0440\u0448\u0438\u043d","\u0430\u0441\u0430\u043d\u0430","\u0430\u0441\u043a\u043e\u0441","\u0430\u0441\u043f\u0438\u0434","\u0430\u0441\u0441\u0430\u043c","\u0430\u0441\u0442\u0430\u0442","\u0430\u0441\u0442\u043c\u0430","\u0430\u0441\u0442\u0440\u0430","\u0430\u0441\u0446\u0438\u0442","\u0430\u0441\u044c\u043a\u0430","\u0430\u0442\u0430\u043a\u0430","\u0430\u0442\u0438\u043b\u043b","\u0430\u0442\u043b\u0430\u0441","\u0430\u0442\u043c\u0430\u043d","\u0430\u0442\u043e\u043b\u043b","\u0430\u0442\u0440\u0438\u0439","\u0430\u0442\u0442\u0430\u0447","\u0430\u0442\u0442\u0438\u043a","\u0430\u0442\u0443\u043e\u0442","\u0430\u0443\u0434\u0438\u043e","\u0430\u0443\u0434\u0438\u0442","\u0430\u0443\u043b\u043e\u0441","\u0430\u0443\u0440\u0430\u0442","\u0430\u0443\u0440\u0435\u0439","\u0430\u0443\u0440\u0438\u0434","\u0430\u0443\u0442\u0440\u043e","\u0430\u0444\u0433\u0430\u043d","\u0430\u0444\u0435\u0440\u0430","\u0430\u0444\u0438\u0448\u0430","\u0430\u0445\u0438\u043b\u043b","\u0430\u0445\u043e\u0438\u0442","\u0430\u0448\u0435\u043b\u044c","\u0430\u0448\u0440\u0430\u043c","\u0430\u044d\u0440\u043e\u0431","\u0430\u044d\u0440\u043e\u043d","\u0430\u044e\u043c\u0430\u0430","\u0431\u0430\u0431\u0430\u0445","\u0431\u0430\u0431\u0431\u043b","\u0431\u0430\u0431\u043a\u0430","\u0431\u0430\u0431\u043a\u0438","\u0431\u0430\u0431\u043b\u043e","\u0431\u0430\u0431\u044c\u0451","\u0431\u0430\u0433\u0430\u0436","\u0431\u0430\u0433\u0433\u0438","\u0431\u0430\u0433\u0435\u0440","\u0431\u0430\u0433\u0435\u0442","\u0431\u0430\u0433\u043d\u043e","\u0431\u0430\u0433\u043d\u044f","\u0431\u0430\u0433\u043e\u0440","\u0431\u0430\u0434\u0430\u043d","\u0431\u0430\u0434\u0438\u043a","\u0431\u0430\u0434\u0443\u0439","\u0431\u0430\u0434\u044c\u044f","\u0431\u0430\u0437\u0430\u0440","\u0431\u0430\u0437\u0438\u043d","\u0431\u0430\u0437\u0438\u0441","\u0431\u0430\u0437\u0438\u0442","\u0431\u0430\u0438\u043d\u0433","\u0431\u0430\u0439\u0433\u0430","\u0431\u0430\u0439\u0433\u0435","\u0431\u0430\u0439\u0434\u0430","\u0431\u0430\u0439\u0437\u0430","\u0431\u0430\u0439\u043a\u0430","\u0431\u0430\u043a\u0430\u0439","\u0431\u0430\u043a\u0430\u043d","\u0431\u0430\u043a\u0435\u043d","\u0431\u0430\u043a\u0443\u043d","\u0431\u0430\u043a\u0447\u0430","\u0431\u0430\u043a\u0448\u0430","\u0431\u0430\u043b\u0430\u043d","\u0431\u0430\u043b\u0434\u0430","\u0431\u0430\u043b\u0434\u0436","\u0431\u0430\u043b\u0435\u0442","\u0431\u0430\u043b\u0438\u0437","\u0431\u0430\u043b\u043a\u0430","\u0431\u0430\u043b\u043e\u043a","\u0431\u0430\u043b\u044b\u043a","\u0431\u0430\u043c\u0438\u044f","\u0431\u0430\u043d\u0430\u043d","\u0431\u0430\u043d\u0434\u0430","\u0431\u0430\u043d\u0434\u043b","\u0431\u0430\u043d\u0434\u043e","\u0431\u0430\u043d\u0434\u0445","\u0431\u0430\u043d\u043a\u0430","\u0431\u0430\u0440\u0430\u043a","\u0431\u0430\u0440\u0430\u043d","\u0431\u0430\u0440\u0434\u0430","\u0431\u0430\u0440\u0434\u043e","\u0431\u0430\u0440\u0435\u0436","\u0431\u0430\u0440\u0436\u0430","\u0431\u0430\u0440\u0438\u0439","\u0431\u0430\u0440\u0438\u0442","\u0431\u0430\u0440\u043a\u0430","\u0431\u0430\u0440\u043c\u044b","\u0431\u0430\u0440\u043e\u043a","\u0431\u0430\u0440\u0440\u0435","\u0431\u0430\u0440\u044b\u0448","\u0431\u0430\u0441\u0438\u043a","\u0431\u0430\u0441\u043a\u0430","\u0431\u0430\u0441\u043c\u0430","\u0431\u0430\u0441\u043d\u044c","\u0431\u0430\u0441\u043d\u044f","\u0431\u0430\u0441\u043e\u043a","\u0431\u0430\u0441\u043e\u043d","\u0431\u0430\u0441\u0442\u0440","\u0431\u0430\u0442\u0430\u043d","\u0431\u0430\u0442\u0430\u0442","\u0431\u0430\u0442\u0438\u043a","\u0431\u0430\u0442\u043e\u0433","\u0431\u0430\u0442\u043e\u043d","\u0431\u0430\u0442\u0442\u043b","\u0431\u0430\u0442\u0443\u0434","\u0431\u0430\u0442\u0443\u043d","\u0431\u0430\u0442\u0443\u0442","\u0431\u0430\u0443\u0438\u0442","\u0431\u0430\u0444\u0438\u044f","\u0431\u0430\u0445\u0442\u0430","\u0431\u0430\u0445\u0447\u0430","\u0431\u0430\u0447\u043a\u0430","\u0431\u0430\u0447\u043e\u043a","\u0431\u0430\u0448\u043a\u0430","\u0431\u0430\u0448\u043b\u0438","\u0431\u0430\u0448\u043d\u044f","\u0431\u0430\u044f\u0442\u0438","\u0431\u0435\u0431\u0440\u0430","\u0431\u0435\u0431\u0443\u0442","\u0431\u0435\u0433\u0430\u0448","\u0431\u0435\u0433\u0443\u043d","\u0431\u0435\u0434\u043a\u0430","\u0431\u0435\u0434\u0440\u043e","\u0431\u0435\u0437\u0438\u043a","\u0431\u0435\u0439\u0433\u043b","\u0431\u0435\u0439\u0434\u0436","\u0431\u0435\u0439\u043a\u0430","\u0431\u0435\u0439\u0446\u044b","\u0431\u0435\u043a\u0430\u0440","\u0431\u0435\u043a\u0435\u0442","\u0431\u0435\u043a\u043e\u043d","\u0431\u0435\u043b\u0435\u043d","\u0431\u0435\u043b\u043a\u0430","\u0431\u0435\u043b\u043a\u0438","\u0431\u0435\u043b\u043e\u043a","\u0431\u0435\u043b\u044c\u0451","\u0431\u0435\u043b\u044f\u0448","\u0431\u0435\u043c\u0431\u0430","\u0431\u0435\u043c\u0438\u0442","\u0431\u0451\u043c\u0438\u0442","\u0431\u0435\u043d\u0434\u0438","\u0431\u0435\u043d\u0442\u043e","\u0431\u0435\u0440\u0430\u0442","\u0431\u0451\u0440\u0434\u043e","\u0431\u0435\u0440\u0435\u0433","\u0431\u0451\u0440\u0435\u043a","\u0431\u0435\u0440\u0435\u0442","\u0431\u0435\u0440\u043a\u0430","\u0431\u0435\u0440\u043c\u0430","\u0431\u0435\u0440\u0441\u043e","\u0431\u0435\u0440\u0446\u043e","\u0431\u0435\u0440\u0446\u044b","\u0431\u0435\u0441\u0438\u043a","\u0431\u0435\u0442\u043e\u043d","\u0431\u0437\u0434\u0451\u0436","\u0431\u0437\u0434\u0451\u0445","\u0431\u0438\u0430\u043a\u0441","\u0431\u0438\u0431\u043e\u043f","\u0431\u0438\u0431\u0443\u0442","\u0431\u0438\u0432\u0430\u043a","\u0431\u0438\u0433\u043e\u0441","\u0431\u0438\u0434\u043e\u043d","\u0431\u0438\u043a\u0441\u0430","\u0431\u0438\u043b\u0435\u0442","\u0431\u0438\u043b\u0438\u043d","\u0431\u0438\u043b\u043b\u044c","\u0431\u0438\u043b\u043e\u043d","\u0431\u0438\u043d\u0433\u043e","\u0431\u0438\u043d\u0434\u0438","\u0431\u0438\u043d\u043e\u043c","\u0431\u0438\u043e\u0442\u0430","\u0431\u0438\u043f\u0430\u043a","\u0431\u0438\u043f\u0435\u0440","\u0431\u0438\u0440\u0436\u0430","\u0431\u0438\u0440\u043a\u0430","\u0431\u0438\u0441\u0430\u043a","\u0431\u0438\u0441\u0435\u0440","\u0431\u0438\u0441\u0442\u0440","\u0431\u0438\u0442\u0432\u0430","\u0431\u0438\u0442\u0435\u0440","\u0431\u0438\u0442\u043a\u0430","\u0431\u0438\u0442\u043e\u043a","\u0431\u0438\u0442\u0443\u043c","\u0431\u0438\u0442\u044c\u0451","\u0431\u0438\u0447\u0438\u043a","\u0431\u043b\u0430\u0433\u043e","\u0431\u043b\u0430\u0436\u044c","\u0431\u043b\u0430\u043d\u043a","\u0431\u043b\u0430\u043d\u0442","\u0431\u043b\u0430\u043d\u0448","\u0431\u043b\u0430\u0441\u0442","\u0431\u043b\u0430\u0442\u043e","\u0431\u043b\u0435\u043d\u0434","\u0431\u043b\u0435\u0441\u043a","\u0431\u043b\u0438\u0437\u044c","\u0431\u043b\u0438\u043d\u0434","\u0431\u043b\u0438\u043d\u0442","\u0431\u043b\u0438\u0441\u043a","\u0431\u043b\u043e\u043d\u0434","\u0431\u043b\u043e\u0442\u044c","\u0431\u043b\u0443\u0437\u0430","\u0431\u043b\u044e\u0434\u043e","\u0431\u043b\u044f\u0445\u0430","\u0431\u043e\u0431\u043e\u043a","\u0431\u043e\u0433\u0435\u043d","\u0431\u043e\u0434\u0445\u0438","\u0431\u043e\u0434\u044f\u043a","\u0431\u043e\u0436\u0431\u0430","\u0431\u043e\u0437\u0430\u0440","\u0431\u043e\u0437\u043e\u043d","\u0431\u043e\u0438\u043d\u0433","\u0431\u043e\u0439\u043a\u0430","\u0431\u043e\u0439\u043d\u044f","\u0431\u043e\u043a\u0430\u0436","\u0431\u043e\u043a\u0430\u043b","\u0431\u043e\u043a\u0438\u0442","\u0431\u043e\u043b\u0438\u0434","\u0431\u043e\u043b\u044e\u0441","\u0431\u043e\u043c\u0431\u0430","\u0431\u043e\u043d\u0433\u043e","\u0431\u043e\u043d\u043a\u0430","\u0431\u043e\u043d\u043a\u0438","\u0431\u043e\u043d\u043c\u043e","\u0431\u043e\u043d\u0443\u0441","\u0431\u043e\u0440\u0430\u043d","\u0431\u043e\u0440\u0430\u0442","\u0431\u043e\u0440\u0434\u043e","\u0431\u043e\u0440\u0435\u0439","\u0431\u043e\u0440\u0435\u0446","\u0431\u043e\u0440\u0438\u0434","\u0431\u043e\u0440\u0438\u0439","\u0431\u043e\u0440\u043e\u043a","\u0431\u043e\u0440\u0442\u044c","\u0431\u043e\u0442\u0432\u0430","\u0431\u043e\u0442\u0438\u043a","\u0431\u043e\u0444\u043e\u0440","\u0431\u043e\u0447\u0430\u0433","\u0431\u043e\u0447\u043a\u0430","\u0431\u043e\u0447\u043e\u043a","\u0431\u043e\u0447\u0447\u0435","\u0431\u043e\u0448\u043a\u0430","\u0431\u0440\u0430\u0433\u0430","\u0431\u0440\u0430\u0434\u0430","\u0431\u0440\u0430\u043c\u0430","\u0431\u0440\u0430\u043d\u0430","\u0431\u0440\u0430\u043d\u0447","\u0431\u0440\u0430\u043d\u044c","\u0431\u0440\u0430\u0441\u0430","\u0431\u0440\u0430\u0441\u0441","\u0431\u0440\u0435\u0432\u0435","\u0431\u0440\u0435\u0434\u0430","\u0431\u0440\u0435\u0434\u044c","\u0431\u0440\u0435\u0437\u0433","\u0431\u0440\u0435\u0439\u043a","\u0431\u0440\u0435\u043c\u044f","\u0431\u0440\u0435\u043d\u0434","\u0431\u0440\u0435\u0448\u044c","\u0431\u0440\u0438\u0434\u0430","\u0431\u0440\u0438\u0434\u0436","\u0431\u0440\u0438\u043a\u0430","\u0431\u0440\u0438\u043b\u044c","\u0431\u0440\u0438\u0444\u044b","\u0431\u0440\u043e\u0432\u044c","\u0431\u0440\u043e\u0433\u0438","\u0431\u0440\u043e\u043d\u0445","\u0431\u0440\u043e\u043d\u044c","\u0431\u0440\u043e\u043d\u044f","\u0431\u0440\u043e\u0448\u044c","\u0431\u0440\u0442\u0443\u0447","\u0431\u0440\u044b\u0436\u0438","\u0431\u0440\u044b\u043b\u0430","\u0431\u0440\u044b\u043b\u044c","\u0431\u0440\u044d\u043d\u0434","\u0431\u0440\u044e\u043a\u0438","\u0431\u0440\u044e\u0445\u043e","\u0431\u0443\u0431\u0435\u043d","\u0431\u0443\u0431\u043a\u0430","\u0431\u0443\u0431\u043d\u0430","\u0431\u0443\u0431\u043d\u044b","\u0431\u0443\u0431\u043e\u043d","\u0431\u0443\u0433\u0430\u0439","\u0431\u0443\u0433\u043e\u0440","\u0431\u0443\u0434\u043a\u0430","\u0431\u0443\u0434\u0440\u0430","\u0431\u0443\u0434\u044f\u043a","\u0431\u0443\u043a\u0432\u0430","\u0431\u0443\u043a\u0435\u0442","\u0431\u0443\u043a\u043b\u0435","\u0431\u0443\u043a\u043b\u044f","\u0431\u0443\u043a\u043e\u043b","\u0431\u0443\u043a\u0441\u0430","\u0431\u0443\u043b\u0430\u0442","\u0431\u0443\u043b\u0433\u0430","\u0431\u0443\u043b\u0438\u043a","\u0431\u0443\u043b\u043a\u0430","\u0431\u0443\u043b\u043b\u0430","\u0431\u0443\u043b\u044c\u0431","\u0431\u0443\u043b\u044c\u043a","\u0431\u0443\u043c\u0435\u0440","\u0431\u0443\u0440\u0430\u0432","\u0431\u0443\u0440\u0430\u043a","\u0431\u0443\u0440\u0430\u043d","\u0431\u0443\u0440\u0430\u0442","\u0431\u0443\u0440\u0434\u0430","\u0431\u0443\u0440\u043a\u0430","\u0431\u0443\u0440\u0440\u0435","\u0431\u0443\u0440\u0441\u0430","\u0431\u0443\u0440\u0443\u043d","\u0431\u0443\u0442\u0430\u043d","\u0431\u0443\u0442\u0435\u043d","\u0431\u0443\u0442\u0435\u0440","\u0431\u0443\u0442\u0438\u043a","\u0431\u0443\u0442\u0438\u043b","\u0431\u0443\u0442\u0438\u0442","\u0431\u0443\u0442\u043d\u044f","\u0431\u0443\u0442\u043e\u043d","\u0431\u0443\u0442\u0441\u0430","\u0431\u0443\u0444\u0435\u0440","\u0431\u0443\u0444\u0435\u0442","\u0431\u0443\u0445\u0430\u0447","\u0431\u0443\u0445\u0438\u0434","\u0431\u0443\u0445\u043b\u043e","\u0431\u0443\u0445\u0442\u0430","\u0431\u0443\u0448\u043c\u0430","\u0431\u0443\u0448\u043e\u043d","\u0431\u0443\u044f\u0432\u0430","\u0431\u044b\u0434\u043b\u043e","\u0431\u044b\u043b\u0438\u0435","\u0431\u044b\u043b\u043a\u0430","\u0431\u044b\u043b\u043e\u0435","\u0431\u044b\u043b\u044c\u0451","\u0431\u044b\u0442\u0438\u0435","\u0431\u044b\u0442\u044c\u0451","\u0431\u044b\u0447\u043e\u043a","\u0431\u044c\u044e\u0438\u043a","\u0431\u044d\u043a\u0430\u043f","\u0431\u044d\u043a\u043e\u043d","\u0431\u044d\u0448\u043a\u0430","\u0431\u044e\u0432\u0430\u0440","\u0431\u044e\u0432\u0435\u0442","\u0431\u044e\u043a\u0441\u0430","\u0432\u0430\u0431\u0438\u043a","\u0432\u0430\u0432\u043a\u0430","\u0432\u0430\u0433\u043e\u043d","\u0432\u0430\u0434\u044c\u044f","\u0432\u0430\u0437\u0438\u043a","\u0432\u0430\u0437\u043e\u043d","\u0432\u0430\u043a\u0430\u0442","\u0432\u0430\u043a\u0441\u0430","\u0432\u0430\u043a\u0443\u0444","\u0432\u0430\u043b\u0435\u0436","\u0432\u0430\u043b\u0451\u043a","\u0432\u0430\u043b\u0451\u0440","\u0432\u0430\u043b\u0435\u0446","\u0432\u0430\u043b\u0438\u043a","\u0432\u0430\u043b\u043a\u0430","\u0432\u0430\u043b\u043a\u0438","\u0432\u0430\u043b\u043e\u043a","\u0432\u0430\u043b\u0443\u0439","\u0432\u0430\u043b\u0443\u043d","\u0432\u0430\u043b\u044c\u0441","\u0432\u0430\u043c\u0431\u0438","\u0432\u0430\u043d\u043d\u0430","\u0432\u0430\u043d\u0442\u0430","\u0432\u0430\u043f\u043e\u0440","\u0432\u0430\u0440\u0433\u0430","\u0432\u0430\u0440\u0435\u0437","\u0432\u0430\u0440\u0438\u043a","\u0432\u0430\u0440\u0438\u044f","\u0432\u0430\u0440\u043a\u0430","\u0432\u0430\u0440\u043d\u0430","\u0432\u0430\u0440\u043e\u043a","\u0432\u0430\u0442\u0435\u0440","\u0432\u0430\u0442\u0438\u043d","\u0432\u0430\u0442\u043a\u0430","\u0432\u0430\u0442\u0442\u044b","\u0432\u0430\u0444\u043b\u044f","\u0432\u0430\u0445\u0442\u0430","\u0432\u0431\u0440\u043e\u0441","\u0432\u0435\u0431\u0435\u0440","\u0432\u0435\u0431\u043a\u0430","\u0432\u0435\u0434\u0435\u0442","\u0432\u0435\u0434\u043e\u043c","\u0432\u0435\u0434\u0440\u043e","\u0432\u0451\u0434\u0440\u043e","\u0432\u0435\u0436\u0434\u0430","\u0432\u0435\u043a\u0448\u0430","\u0432\u0435\u043b\u0438\u043a","\u0432\u0435\u043b\u043c\u0438","\u0432\u0435\u043b\u0443\u043c","\u0432\u0435\u043b\u044c\u0434","\u0432\u0435\u043b\u044e\u0440","\u0432\u0435\u043d\u0435\u0446","\u0432\u0435\u043d\u0438\u043a","\u0432\u0435\u043d\u043a\u0430","\u0432\u0435\u043d\u043e\u043a","\u0432\u0435\u043d\u0442\u0430","\u0432\u0435\u043d\u044f\u043a","\u0432\u0435\u0440\u0431\u0430","\u0432\u0435\u0440\u0432\u044c","\u0432\u0435\u0440\u0435\u0434","\u0432\u0435\u0440\u0435\u0441","\u0432\u0435\u0440\u0435\u044f","\u0432\u0435\u0440\u0436\u0435","\u0432\u0435\u0440\u043a\u0438","\u0432\u0435\u0440\u0441\u043e","\u0432\u0435\u0440\u0444\u044c","\u0432\u0435\u0440\u0448\u0430","\u0432\u0435\u0441\u043a\u0438","\u0432\u0435\u0441\u043b\u043e","\u0432\u0435\u0441\u043d\u0430","\u0432\u0435\u0441\u043e\u043a","\u0432\u0435\u0441\u0442\u044c","\u0432\u0435\u0442\u0432\u044c","\u0432\u0435\u0442\u0435\u0440","\u0432\u0435\u0442\u043a\u0430","\u0432\u0435\u0442\u043b\u0430","\u0432\u0435\u0447\u0435\u0440","\u0432\u0435\u0448\u043a\u0430","\u0432\u0437\u0431\u0435\u0433","\u0432\u0437\u0432\u0430\u0440","\u0432\u0437\u0432\u043e\u0434","\u0432\u0437\u0432\u043e\u0437","\u0432\u0437\u0434\u043e\u0440","\u0432\u0437\u0434\u043e\u0445","\u0432\u0437\u043b\u0451\u0442","\u0432\u0437\u043b\u043e\u043c","\u0432\u0437\u043c\u0430\u0445","\u0432\u0437\u043c\u0451\u0442","\u0432\u0437\u043c\u044b\u0432","\u0432\u0437\u043d\u043e\u0441","\u0432\u0437\u0440\u0435\u0437","\u0432\u0437\u0440\u044b\u0432","\u0432\u0437\u0440\u044b\u0434","\u0432\u0437\u044a\u0451\u0431","\u0432\u0437\u044a\u0451\u043c","\u0432\u0437\u044b\u0441\u043a","\u0432\u0438\u0434\u0430\u043a","\u0432\u0438\u0434\u0435\u043e","\u0432\u0438\u0434\u0438\u043a","\u0432\u0438\u0434\u043c\u0430","\u0432\u0438\u0434\u043e\u043a","\u0432\u0438\u0434\u043e\u0441","\u0432\u0438\u0435\u043b\u0430","\u0432\u0438\u0437\u0430\u0436","\u0432\u0438\u0437\u0438\u0440","\u0432\u0438\u0437\u0438\u0442","\u0432\u0438\u0437\u043e\u0440","\u0432\u0438\u043b\u043a\u0430","\u0432\u0438\u043b\u043b\u0430","\u0432\u0438\u043b\u043e\u043a","\u0432\u0438\u043d\u0434\u0430","\u0432\u0438\u043d\u0438\u043b","\u0432\u0438\u043d\u0438\u0440","\u0432\u0438\u043d\u0446\u043e","\u0432\u0438\u043d\u0447\u0430","\u0432\u0438\u043e\u043b\u0430","\u0432\u0438\u043f\u043a\u0430","\u0432\u0438\u0440\u0430\u0436","\u0432\u0438\u0440\u0433\u0430","\u0432\u0438\u0440\u0443\u0441","\u0432\u0438\u0440\u0448\u0430","\u0432\u0438\u0441\u043a\u0430","\u0432\u0438\u0441\u043a\u0438","\u0432\u0438\u0441\u043e\u043a","\u0432\u0438\u0441\u0442\u043b","\u0432\u0438\u0441\u044f\u043a","\u0432\u0438\u0442\u043e\u043a","\u0432\u0438\u0442\u044c\u0451","\u0432\u0438\u0445\u043e\u0440","\u0432\u0438\u0445\u0440\u044c","\u0432\u0438\u0448\u043d\u044f","\u0432\u043a\u043b\u0430\u0434","\u0432\u043b\u0430\u0433\u0430","\u0432\u043e\u0431\u0431\u043b","\u0432\u043e\u0434\u043a\u0430","\u0432\u043e\u0436\u0436\u0430","\u0432\u043e\u0437\u0436\u0430","\u0432\u043e\u0437\u0438\u043a","\u0432\u043e\u0437\u043a\u0430","\u0432\u043e\u0437\u043d\u044f","\u0432\u043e\u0437\u043e\u043a","\u0432\u043e\u0439\u0434\u0430","\u0432\u043e\u0439\u043d\u0430","\u0432\u043e\u043a\u0430\u043b","\u0432\u043e\u043b\u0430\u043d","\u0432\u043e\u043b\u0433\u0430","\u0432\u043e\u043b\u043d\u0430","\u0432\u043e\u043b\u043e\u043a","\u0432\u043e\u043b\u043e\u0441","\u0432\u043e\u043b\u043e\u0444","\u0432\u043e\u043b\u044c\u0442","\u0432\u043e\u043b\u044e\u043c","\u0432\u043e\u043f\u043b\u044c","\u0432\u043e\u0440\u0433\u0430","\u0432\u043e\u0440\u043e\u0442","\u0432\u043e\u0440\u043e\u0445","\u0432\u043e\u0440\u044c\u0451","\u0432\u043e\u0442\u0438\u0432","\u0432\u043e\u0442\u0443\u043c","\u0432\u043e\u0445\u0440\u0430","\u0432\u043e\u0448\u0438\u043a","\u0432\u043f\u0443\u0441\u043a","\u0432\u0440\u0430\u043a\u0438","\u0432\u0440\u0430\u0442\u0430","\u0432\u0440\u0435\u043c\u044f","\u0432\u0441\u043f\u044b\u0445","\u0432\u0441\u0442\u043e\u043a","\u0432\u0441\u0445\u043e\u0434","\u0432\u0442\u043e\u0440\u0430","\u0432\u0443\u0430\u043b\u044c","\u0432\u0443\u0444\u0435\u0440","\u0432\u0447\u0435\u0440\u0430","\u0432\u044a\u0435\u0437\u0434","\u0432\u044b\u0431\u0435\u0433","\u0432\u044b\u0431\u043e\u0439","\u0432\u044b\u0431\u043e\u0440","\u0432\u044b\u0432\u0430\u043b","\u0432\u044b\u0432\u0438\u0445","\u0432\u044b\u0432\u043e\u0434","\u0432\u044b\u0432\u043e\u0437","\u0432\u044b\u0433\u0430\u0440","\u0432\u044b\u0433\u0438\u0431","\u0432\u044b\u0433\u043e\u043d","\u0432\u044b\u0433\u0443\u043b","\u0432\u044b\u0434\u0435\u043b","\u0432\u044b\u0434\u043e\u0445","\u0432\u044b\u0434\u0443\u0432","\u0432\u044b\u0434\u0443\u0439","\u0432\u044b\u0435\u0437\u0434","\u0432\u044b\u0436\u0438\u0433","\u0432\u044b\u0436\u0438\u043c","\u0432\u044b\u0437\u043e\u0432","\u0432\u044b\u043a\u0430\u0442","\u0432\u044b\u043a\u043e\u043b","\u0432\u044b\u043a\u043e\u0441","\u0432\u044b\u043a\u0443\u043f","\u0432\u044b\u043a\u0443\u0441","\u0432\u044b\u043b\u0430\u0437","\u0432\u044b\u043b\u0435\u0442","\u0432\u044b\u043b\u0438\u0432","\u0432\u044b\u043b\u043e\u0432","\u0432\u044b\u043b\u043e\u043c","\u0432\u044b\u043c\u043e\u043b","\u0432\u044b\u043c\u043e\u0440","\u0432\u044b\u043d\u043e\u0441","\u0432\u044b\u043f\u0430\u0434","\u0432\u044b\u043f\u0430\u043b","\u0432\u044b\u043f\u0430\u0440","\u0432\u044b\u043f\u0430\u0441","\u0432\u044b\u043f\u0435\u043a","\u0432\u044b\u043f\u043e\u0440","\u0432\u044b\u043f\u043e\u0442","\u0432\u044b\u043f\u0443\u043a","\u0432\u044b\u0440\u0435\u0437","\u0432\u044b\u0440\u0438\u0439","\u0432\u044b\u0440\u0443\u0431","\u0432\u044b\u0441\u0435\u0432","\u0432\u044b\u0441\u0435\u0434","\u0432\u044b\u0441\u0435\u0440","\u0432\u044b\u0442\u044c\u0451","\u0432\u044b\u0445\u043e\u0434","\u0432\u044b\u0448\u0430\u043a","\u0432\u044b\u0448\u0438\u0431","\u0432\u044b\u0448\u043a\u0430","\u0432\u044b\u0449\u0438\u043f","\u0432\u044c\u044e\u0433\u0430","\u0432\u044d\u0439\u0446\u0438","\u0432\u044f\u0437\u043a\u0430","\u0432\u044f\u0437\u043e\u043a","\u0432\u044f\u0437\u044c\u0435","\u0432\u044f\u043b\u044b\u0439","\u0433\u0430\u0431\u0431\u0430","\u0433\u0430\u0432\u0430\u0440","\u0433\u0430\u0432\u043e\u0442","\u0433\u0430\u0433\u0430\u0442","\u0433\u0430\u0434\u0435\u0439","\u0433\u0430\u0434\u0438\u043a","\u0433\u0430\u0437\u0435\u0440","\u0433\u0430\u0437\u0438\u043a","\u0433\u0430\u0437\u043e\u043a","\u0433\u0430\u0437\u043e\u043d","\u0433\u0430\u0439\u043a\u0430","\u0433\u0430\u0439\u043d\u043e","\u0433\u0430\u043b\u0434\u0430","\u0433\u0430\u043b\u0438\u0442","\u0433\u0430\u043b\u043a\u0430","\u0433\u0430\u043b\u043b\u0430","\u0433\u0430\u043b\u043e\u043f","\u0433\u0430\u043b\u043e\u0441","\u0433\u0430\u043b\u0443\u043d","\u0433\u0430\u043b\u044c\u044f","\u0433\u0430\u043b\u044e\u043d","\u0433\u0430\u043c\u0430\u0437","\u0433\u0430\u043c\u0430\u043a","\u0433\u0430\u043c\u0430\u043d","\u0433\u0430\u043c\u0431\u0430","\u0433\u0430\u043c\u043c\u0430","\u0433\u0430\u043c\u043e\u043d","\u0433\u0430\u043c\u0443\u0442","\u0433\u0430\u043d\u0430\u0448","\u0433\u0430\u043d\u0438\u0442","\u0433\u0430\u043e\u0447\u044d","\u0433\u0430\u0440\u0430\u0436","\u0433\u0430\u0440\u0434\u0430","\u0433\u0430\u0440\u0434\u0435","\u0433\u0430\u0440\u0435\u043c","\u0433\u0430\u0440\u043d\u0446","\u0433\u0430\u0440\u0443\u043c","\u0433\u0430\u0440\u0443\u0441","\u0433\u0430\u0441\u043f\u0441","\u0433\u0430\u0443\u0434\u0430","\u0433\u0430\u0443\u0441\u0441","\u0433\u0430\u0447\u0435\u043a","\u0433\u0430\u0448\u0438\u0448","\u0433\u0432\u0430\u043b\u0442","\u0433\u0432\u0430\u044f\u043a","\u0433\u0435\u0432\u0435\u044f","\u0433\u0451\u0434\u0437\u0430","\u0433\u0435\u0439\u043c\u0430","\u0433\u0435\u0439\u0448\u0430","\u0433\u0435\u043a\u0442\u0430","\u0433\u0435\u043b\u0438\u0439","\u0433\u0435\u043b\u0438\u043a","\u0433\u0435\u043b\u0438\u0442","\u0433\u0451\u043b\u0438\u0442","\u0433\u0435\u043c\u0438\u043d","\u0433\u0435\u043c\u043c\u0430","\u0433\u0435\u043c\u043e\u0440","\u0433\u0435\u043d\u0435\u0437","\u0433\u0435\u043d\u0438\u0439","\u0433\u0435\u043d\u0438\u043d","\u0433\u0435\u043d\u043e\u043c","\u0433\u0435\u043d\u0440\u0438","\u0433\u0435\u043e\u0438\u0434","\u0433\u0435\u0440\u0430\u0442","\u0433\u0435\u0440\u043c\u0430","\u0433\u0435\u0440\u044b\u0447","\u0433\u0451\u0442\u0438\u0442","\u0433\u0435\u0442\u0440\u0430","\u0433\u0435\u0442\u0442\u043e","\u0433\u0436\u0435\u043b\u044c","\u0433\u0438\u0431\u043a\u0430","\u0433\u0438\u0437\u043c\u043e","\u0433\u0438\u043b\u0430\u0441","\u0433\u0438\u043b\u0435\u044f","\u0433\u0438\u043c\u0435\u043d","\u0433\u0438\u043c\u043e\u0440","\u0433\u0438\u043d\u0435\u044f","\u0433\u0438\u043e\u0438\u0434","\u0433\u0438\u043f\u0430\u0440","\u0433\u0438\u043f\u044e\u0440","\u0433\u0438\u0440\u043a\u0430","\u0433\u0438\u0440\u043b\u043e","\u0433\u0438\u0442\u043e\u0432","\u0433\u0438\u0444\u043a\u0430","\u0433\u0438\u0447\u043a\u0430","\u0433\u043b\u0430\u0432\u0430","\u0433\u043b\u0430\u0432\u043a","\u0433\u043b\u0430\u0434\u044c","\u0433\u043b\u0430\u0437\u0430","\u0433\u043b\u0430\u0439\u0434","\u0433\u043b\u0435\u0431\u0430","\u0433\u043b\u0435\u0444\u0430","\u0433\u043b\u0438\u043d\u0430","\u0433\u043b\u0438\u043d\u0442","\u0433\u043b\u0438\u043e\u0437","\u0433\u043b\u0438\u0442\u0447","\u0433\u043b\u043e\u0433\u0433","\u0433\u043b\u0443\u0430\u0440","\u0433\u043b\u0443\u0431\u044c","\u0433\u043b\u0443\u0437\u0433","\u0433\u043b\u0443\u0437\u0434","\u0433\u043b\u0443\u0448\u044c","\u0433\u043b\u044b\u0431\u0430","\u0433\u043b\u044b\u0431\u044c","\u0433\u043b\u044e\u043c\u0430","\u0433\u043b\u044e\u043e\u043d","\u0433\u043b\u044f\u0441\u0435","\u0433\u043c\u0438\u043d\u0430","\u0433\u043d\u0435\u0439\u0441","\u0433\u043d\u0438\u0434\u0430","\u0433\u043d\u0438\u043b\u044c","\u0433\u043d\u043e\u0437\u0430","\u0433\u043d\u043e\u043c\u0430","\u0433\u043d\u0443\u0441\u044c","\u0433\u043e\u0431\u0430\u043d","\u0433\u043e\u0431\u043e\u0439","\u0433\u043e\u0432\u043d\u043e","\u0433\u043e\u0432\u043e\u0440","\u0433\u043e\u0433\u043b\u044b","\u0433\u043e\u0433\u043e\u0442","\u0433\u043e\u0434\u0438\u043a","\u0433\u043e\u0434\u043e\u043a","\u0433\u043e\u043b\u0435\u0446","\u0433\u043e\u043b\u0438\u043a","\u0433\u043e\u043b\u043e\u0434","\u0433\u043e\u043b\u043e\u0441","\u0433\u043e\u043b\u044b\u0448","\u0433\u043e\u043b\u044c\u0451","\u0433\u043e\u043b\u044c\u0444","\u0433\u043e\u043c\u0430\u0440","\u0433\u043e\u043c\u043d\u043e","\u0433\u043e\u043c\u043e\u043d","\u0433\u043e\u043d\u0437\u043e","\u0433\u043e\u043d\u0438\u0442","\u0433\u043e\u043d\u043a\u0430","\u0433\u043e\u043d\u043e\u043a","\u0433\u043e\u043d\u043e\u0440","\u0433\u043e\u043f\u0430\u043a","\u0433\u043e\u0440\u0435\u0446","\u0433\u043e\u0440\u0436\u0430","\u0433\u043e\u0440\u0438\u0442","\u0433\u043e\u0440\u043a\u0430","\u0433\u043e\u0440\u043b\u043e","\u0433\u043e\u0440\u043d\u043e","\u0433\u043e\u0440\u043e\u0434","\u0433\u043e\u0440\u043e\u0445","\u0433\u043e\u0440\u043f\u043e","\u0433\u043e\u0440\u0441\u0442","\u0433\u043e\u0444\u0440\u0430","\u0433\u043e\u0444\u0440\u0435","\u0433\u043e\u0445\u0443\u0430","\u0433\u0440\u0430\u0432\u0435","\u0433\u0440\u0430\u0439\u043c","\u0433\u0440\u0430\u043c\u043c","\u0433\u0440\u0430\u043d\u0436","\u0433\u0440\u0430\u043d\u0442","\u0433\u0440\u0430\u043d\u044c","\u0433\u0440\u0430\u0443\u0441","\u0433\u0440\u0430\u0444\u0430","\u0433\u0440\u0435\u0431\u044c","\u0433\u0440\u0435\u0436\u0430","\u0433\u0440\u0451\u0437\u0430","\u0433\u0440\u0435\u0439\u0434","\u0433\u0440\u0435\u043d\u0430","\u0433\u0440\u0435\u0447\u0430","\u0433\u0440\u0438\u0431\u0430","\u0433\u0440\u0438\u0431\u044b","\u0433\u0440\u0438\u0432\u0430","\u0433\u0440\u0438\u0434\u044c","\u0433\u0440\u0438\u043b\u044c","\u0433\u0440\u0438\u043d\u0434","\u0433\u0440\u0438\u043e\u0442","\u0433\u0440\u0438\u043f\u043f","\u0433\u0440\u0438\u043f\u0441","\u0433\u0440\u043e\u0437\u0430","\u0433\u0440\u043e\u0441\u0441","\u0433\u0440\u043e\u0443\u043b","\u0433\u0440\u0443\u0434\u0430","\u0433\u0440\u0443\u0434\u044c","\u0433\u0440\u0443\u043d\u0442","\u0433\u0440\u0443\u0448\u0430","\u0433\u0440\u044b\u0436\u0430","\u0433\u0440\u044e\u0439\u0442","\u0433\u0440\u044f\u0434\u0430","\u0433\u0440\u044f\u0437\u044c","\u0433\u0443\u0430\u0432\u0430","\u0433\u0443\u0430\u043d\u043e","\u0433\u0443\u0430\u0448\u044c","\u0433\u0443\u0431\u043a\u0430","\u0433\u0443\u0431\u044c\u0451","\u0433\u0443\u0433\u043d\u044f","\u0433\u0443\u0433\u043e\u043b","\u0433\u0443\u0434\u0451\u0436","\u0433\u0443\u0434\u043e\u043a","\u0433\u0443\u0436\u0438\u043a","\u0433\u0443\u0436\u0438\u0440","\u0433\u0443\u0437\u043a\u0430","\u0433\u0443\u0437\u043d\u043e","\u0433\u0443\u0438\u0440\u043e","\u0433\u0443\u043b\u044f\u0448","\u0433\u0443\u043c\u0431\u043e","\u0433\u0443\u043c\u0438\u0442","\u0433\u0443\u043c\u043c\u0430","\u0433\u0443\u043c\u043c\u0438","\u0433\u0443\u043c\u043d\u043e","\u0433\u0443\u043c\u043e\u0437","\u0433\u0443\u043c\u043e\u0440","\u0433\u0443\u043c\u0443\u0441","\u0433\u0443\u043d\u0444\u0443","\u0433\u0443\u0440\u0434\u0430","\u0433\u0443\u0440\u0438\u0439","\u0433\u0443\u0440\u044c\u044f","\u0433\u0443\u0441\u0430\u043a","\u0433\u0443\u0441\u0451\u043a","\u0433\u0443\u0441\u043b\u0435","\u0433\u0443\u0441\u043b\u0438","\u0433\u044d\u0431\u043d\u044f","\u0433\u044e\u0432\u0435\u0447","\u0434\u0430\u0431\u043a\u0435","\u0434\u0430\u0432\u0451\u0436","\u0434\u0430\u0432\u043a\u0430","\u0434\u0430\u0434\u0430\u043d","\u0434\u0430\u0439\u043a\u0430","\u0434\u0430\u043b\u0438\u044f","\u0434\u0430\u043c\u0431\u0430","\u0434\u0430\u043c\u043a\u0430","\u0434\u0430\u043c\u043d\u043e","\u0434\u0430\u043d\u0433\u043e","\u0434\u0430\u043d\u0438\u0439","\u0434\u0430\u043d\u0441\u0430","\u0434\u0430\u0440\u0442\u0441","\u0434\u0430\u0441\u0438\u044f","\u0434\u0430\u0442\u0435\u0440","\u0434\u0430\u0442\u0438\u0432","\u0434\u0430\u0444\u043d\u0430","\u0434\u0430\u0446\u0430\u043d","\u0434\u0430\u0446\u0438\u0442","\u0434\u0430\u0447\u043a\u0430","\u0434\u0432\u0435\u0440\u044c","\u0434\u0432\u0438\u0433\u0430","\u0434\u0432\u043e\u044f\u043a","\u0434\u0435\u0431\u0430\u0433","\u0434\u0435\u0431\u0430\u0439","\u0434\u0435\u0431\u0435\u0442","\u0434\u0435\u0431\u0438\u0442","\u0434\u0435\u0431\u043e\u0448","\u0434\u0435\u0431\u0440\u0438","\u0434\u0435\u0431\u044e\u0442","\u0434\u0435\u0432\u0438\u0437","\u0434\u0435\u0432\u043e\u043d","\u0434\u0451\u0436\u043a\u0430","\u0434\u0435\u0437\u0438\u043a","\u0434\u0435\u0438\u0437\u043c","\u0434\u0435\u043a\u0430\u043d","\u0434\u0435\u043a\u043e\u0440","\u0434\u0435\u043b\u0451\u0436","\u0434\u0435\u043b\u0438\u0441","\u0434\u0435\u043c\u043a\u0430","\u0434\u0435\u043c\u043e\u043d","\u0434\u0435\u043c\u043e\u0441","\u0434\u0435\u043d\u0433\u0435","\u0434\u0435\u043d\u0451\u043a","\u0434\u0435\u043d\u0438\u043c","\u0434\u0435\u043d\u044c\u0435","\u0434\u0435\u043f\u0440\u0430","\u0434\u0435\u0440\u0431\u0430","\u0434\u0435\u0440\u0431\u0438","\u0434\u0451\u0440\u0435\u043d","\u0434\u0451\u0440\u043a\u0430","\u0434\u0435\u0440\u043c\u0430","\u0434\u0435\u0440\u0442\u044c","\u0434\u0451\u0440\u0442\u044c","\u0434\u0435\u0441\u043d\u0430","\u0434\u0435\u0441\u0442\u044c","\u0434\u0435\u0442\u0432\u0430","\u0434\u0435\u0444\u0438\u0441","\u0434\u0436\u0430\u0439\u0432","\u0434\u0436\u0430\u043d\u043a","\u0434\u0436\u0435\u0440\u043a","\u0434\u0436\u0438\u0433\u0430","\u0434\u0436\u0438\u043b\u0430","\u0434\u0436\u0438\u043d\u044b","\u0434\u0436\u043e\u043a\u0438","\u0434\u0437\u0435\u0442\u0430","\u0434\u0437\u043e\u0440\u0438","\u0434\u0437\u044e\u0434\u043e","\u0434\u0438\u0430\u0434\u0430","\u0434\u0438\u0431\u0438\u0442","\u0434\u0438\u0432\u0430\u043d","\u0434\u0438\u0433\u0443\u043a","\u0434\u0438\u0435\u0442\u0430","\u0434\u0438\u043b\u0434\u043e","\u0434\u0438\u043b\u0435\u0439","\u0434\u0438\u043b\u0435\u043d","\u0434\u0438\u043b\u044c\u0441","\u0434\u0438\u043c\u0435\u0440","\u0434\u0438\u043d\u0430\u0440","\u0434\u0438\u043d\u0430\u0441","\u0434\u0438\u043d\u0433\u0438","\u0434\u0438\u043d\u043e\u0434","\u0434\u0438\u043f\u0438\u043d","\u0434\u0438\u043f\u0438\u0440","\u0434\u0438\u0440\u0438\u0442","\u0434\u0438\u0441\u043a\u043e","\u0434\u0438\u0444\u043e\u043d","\u0434\u0438\u0445\u0442\u0430","\u0434\u0438\u0447\u043a\u0430","\u0434\u0438\u0447\u043e\u043a","\u0434\u0438\u044d\u0434\u0440","\u0434\u043b\u0430\u043d\u044c","\u0434\u043b\u0438\u043d\u0430","\u0434\u043d\u0438\u0449\u0435","\u0434\u043d\u044e\u0445\u0430","\u0434\u043e\u0431\u043e\u0440","\u0434\u043e\u0431\u0440\u043e","\u0434\u043e\u0432\u043e\u0434","\u0434\u043e\u0432\u043e\u0437","\u0434\u043e\u0433\u043c\u0430","\u0434\u043e\u0433\u0440\u0438","\u0434\u043e\u0434\u0437\u0451","\u0434\u043e\u0434\u0438\u043a","\u0434\u043e\u0434\u043e\u0439","\u0434\u043e\u0435\u0437\u0434","\u0434\u043e\u0436\u0434\u044c","\u0434\u043e\u0436\u0438\u0433","\u0434\u043e\u0436\u0438\u043c","\u0434\u043e\u0437\u043e\u0440","\u0434\u043e\u0439\u043a\u0430","\u0434\u043e\u0439\u043b\u043e","\u0434\u043e\u0439\u043d\u0430","\u0434\u043e\u0439\u0440\u0430","\u0434\u043e\u043a\u0430\u043b","\u0434\u043e\u043b\u0431\u0438","\u0434\u043e\u043b\u0438\u0432","\u0434\u043e\u043b\u043c\u0430","\u0434\u043e\u043c\u0451\u043a","\u0434\u043e\u043c\u0435\u043d","\u0434\u043e\u043c\u0438\u043a","\u0434\u043e\u043c\u043d\u0430","\u0434\u043e\u043c\u043e\u043a","\u0434\u043e\u043c\u0440\u0430","\u0434\u043e\u043d\u0430\u0442","\u0434\u043e\u043d\u0435\u0440","\u0434\u043e\u043d\u043a\u0430","\u0434\u043e\u043d\u043e\u0441","\u0434\u043e\u043d\u0443\u043c","\u0434\u043e\u043d\u0446\u0435","\u0434\u043e\u043f\u0430\u043d","\u0434\u043e\u0440\u0433\u0438","\u0434\u043e\u0440\u043a\u0430","\u0434\u043e\u0441\u0435\u0432","\u0434\u043e\u0441\u043a\u0430","\u0434\u043e\u0441\u043e\u043b","\u0434\u043e\u0441\u044b\u043b","\u0434\u043e\u0441\u044c\u0435","\u0434\u043e\u0445\u043e\u0434","\u0434\u043e\u0447\u043a\u0430","\u0434\u043e\u0448\u0438\u043a","\u0434\u043e\u0448\u043a\u0430","\u0434\u0440\u0430\u0433\u0430","\u0434\u0440\u0430\u0436\u0435","\u0434\u0440\u0430\u0439\u0432","\u0434\u0440\u0430\u043a\u0430","\u0434\u0440\u0430\u043c\u0430","\u0434\u0440\u0430\u043d\u044c","\u0434\u0440\u0430\u0444\u0442","\u0434\u0440\u0435\u0432\u043e","\u0434\u0440\u0435\u0434\u044b","\u0434\u0440\u0435\u0439\u0444","\u0434\u0440\u0435\u043b\u044c","\u0434\u0440\u0451\u043c\u0430","\u0434\u0440\u0435\u043d\u0430","\u0434\u0440\u0438\u043d\u043a","\u0434\u0440\u0438\u0441\u044c","\u0434\u0440\u0438\u0444\u0442","\u0434\u0440\u043e\u0431\u044c","\u0434\u0440\u043e\u0432\u0430","\u0434\u0440\u043e\u0433\u0430","\u0434\u0440\u043e\u0433\u0438","\u0434\u0440\u043e\u0436\u044c","\u0434\u0440\u043e\u0443\u043d","\u0434\u0440\u0443\u0437\u0430","\u0434\u0440\u044b\u0437\u0433","\u0434\u0440\u044b\u043d\u0430","\u0434\u0440\u044f\u0437\u0433","\u0434\u0440\u044f\u043d\u044c","\u0434\u0443\u0430\u0442\u0435","\u0434\u0443\u0431\u0430\u043a","\u0434\u0443\u0431\u0435\u0446","\u0434\u0443\u0431\u043a\u0430","\u0434\u0443\u0431\u043b\u044c","\u0434\u0443\u0431\u043e\u043a","\u0434\u0443\u0431\u044c\u0451","\u0434\u0443\u0432\u0430\u043b","\u0434\u0443\u0432\u0430\u043d","\u0434\u0443\u0434\u0451\u0436","\u0434\u0443\u0434\u043a\u0430","\u0434\u0443\u0434\u0443\u043a","\u0434\u0443\u0436\u043a\u0430","\u0434\u0443\u043a\u0430\u0442","\u0434\u0443\u043c\u043a\u0430","\u0434\u0443\u043d\u0430\u043c","\u0434\u0443\u043d\u0438\u0442","\u0434\u0443\u043d\u0441\u0442","\u0434\u0443\u043e\u043b\u044c","\u0434\u0443\u043e\u043c\u043e","\u0434\u0443\u043f\u043b\u043e","\u0434\u0443\u043f\u043b\u044c","\u0434\u0443\u0440\u0430\u043a","\u0434\u0443\u0440\u0438\u0442","\u0434\u0443\u0440\u043a\u0430","\u0434\u0443\u0440\u0440\u0430","\u0434\u0443\u0440\u0440\u043e","\u0434\u0443\u0442\u0430\u0440","\u0434\u0443\u0442\u0438\u043a","\u0434\u0443\u0442\u044b\u0439","\u0434\u0443\u0442\u044c\u0451","\u0434\u0443\u0445\u0430\u043d","\u0434\u0443\u0448\u043a\u0430","\u0434\u0443\u0448\u043e\u043a","\u0434\u0443\u044d\u043b\u044c","\u0434\u044b\u043c\u043a\u0430","\u0434\u044b\u043c\u043e\u043a","\u0434\u044b\u0440\u043a\u0430","\u0434\u044b\u0445\u043b\u043e","\u0434\u044b\u0448\u043b\u043e","\u0434\u044c\u044e\u0430\u0440","\u0434\u044e\u0431\u0435\u043a","\u0434\u044e\u043a\u0435\u0440","\u0434\u044e\u0440\u0438\u0442","\u0434\u044e\u0448\u0435\u0441","\u0435\u0431\u0430\u043b\u043e","\u0451\u0431\u0430\u043d\u044c","\u0435\u0431\u0435\u043b\u044c","\u0435\u0431\u0435\u043d\u044f","\u0451\u0431\u0438\u043a\u043e","\u0435\u0431\u043b\u0435\u0442","\u0435\u0431\u043b\u0435\u0446","\u0435\u0431\u043e\u0442\u0430","\u0435\u0431\u0443\u043a\u0438","\u0435\u0432\u0440\u0438\u043a","\u0435\u0434\u043e\u043c\u0430","\u0435\u0434\u044c\u0431\u0430","\u0435\u0437\u0434\u043a\u0430","\u0435\u043b\u0430\u043d\u044c","\u0435\u043b\u0431\u0430\u043d","\u0435\u043b\u0434\u0430\u043a","\u0435\u043b\u0434\u0430\u0447","\u0435\u043b\u0438\u043d\u0430","\u0435\u043c\u0435\u0439\u043b","\u0435\u043c\u0448\u0430\u043d","\u0435\u0440\u0430\u043d\u044c","\u0435\u0440\u0433\u0430\u043a","\u0435\u0440\u0435\u0441\u044c","\u0435\u0440\u043c\u043e\u0441","\u0435\u0440\u043d\u0438\u043a","\u0451\u0440\u043d\u0438\u043a","\u0435\u0441\u0435\u043d\u044c","\u0435\u0441\u0435\u043d\u044f","\u0435\u044e\u043d\u0438\u0442","\u0436\u0430\u0431\u043a\u0430","\u0436\u0430\u0431\u0440\u0430","\u0436\u0430\u0436\u0434\u0430","\u0436\u0430\u043a\u0430\u043d","\u0436\u0430\u043a\u0435\u0442","\u0436\u0430\u043a\u043e\u0431","\u0436\u0430\u043b\u043a\u043e","\u0436\u0430\u043c\u043a\u0430","\u0436\u0430\u0440\u043a\u0430","\u0436\u0430\u0440\u043e\u043a","\u0436\u0430\u0442\u0432\u0430","\u0436\u0430\u0442\u043a\u0430","\u0436\u0432\u0430\u043b\u043e","\u0436\u0451\u0432\u043a\u0430","\u0436\u0435\u0432\u043e\u043a","\u0436\u0435\u043b\u0432\u044c","\u0436\u0451\u043b\u043e\u0431","\u0436\u0435\u043b\u0442\u044c","\u0436\u0435\u043b\u0447\u044c","\u0436\u0451\u043b\u0447\u044c","\u0436\u0435\u043e\u0434\u0430","\u0436\u0435\u0440\u0434\u044c","\u0436\u0435\u0440\u043b\u043e","\u0436\u0435\u0441\u0442\u044c","\u0436\u0435\u0442\u043e\u043d","\u0436\u0438\u0432\u043e\u0442","\u0436\u0438\u0432\u044c\u0451","\u0436\u0438\u0433\u0430\u043d","\u0436\u0438\u0434\u0432\u0430","\u0436\u0438\u0436\u043a\u0430","\u0436\u0438\u0437\u043d\u044c","\u0436\u0438\u043b\u0435\u0442","\u0436\u0438\u043b\u043a\u0430","\u0436\u0438\u043b\u044c\u0451","\u0436\u0438\u0440\u043e\u043a","\u0436\u0438\u0441\u0442\u044c","\u0436\u0438\u0442\u0438\u0435","\u0436\u0438\u0442\u044c\u0451","\u0436\u043c\u0435\u043d\u044f","\u0436\u043c\u0443\u0434\u044c","\u0436\u043d\u0438\u0432\u0430","\u0436\u043d\u0438\u0432\u043e","\u0436\u043e\u043f\u043a\u0430","\u0436\u043e\u043f\u0443\u0430","\u0436\u0443\u0434\u0435\u0446","\u0436\u0443\u043b\u044c\u0451","\u0436\u0443\u043c\u0430\u0440","\u0436\u0443\u043d\u0442\u0430","\u0436\u0443\u043f\u0430\u043d","\u0436\u0443\u043f\u0435\u043b","\u0436\u0443\u043f\u043a\u0430","\u0436\u0443\u0442\u0438\u043a","\u0436\u0443\u0447\u043e\u043a","\u0437\u0430\u0431\u0435\u0433","\u0437\u0430\u0431\u0438\u0432","\u0437\u0430\u0431\u043e\u0439","\u0437\u0430\u0431\u043e\u0440","\u0437\u0430\u0432\u0430\u043b","\u0437\u0430\u0432\u0435\u0441","\u0437\u0430\u0432\u0435\u0442","\u0437\u0430\u0432\u043e\u0434","\u0437\u0430\u0432\u043e\u0437","\u0437\u0430\u0433\u0430\u0434","\u0437\u0430\u0433\u0430\u0440","\u0437\u0430\u0433\u0438\u0431","\u0437\u0430\u0433\u043e\u043d","\u0437\u0430\u0433\u0443\u043b","\u0437\u0430\u0434\u0435\u043b","\u0437\u0430\u0434\u0438\u043a","\u0437\u0430\u0434\u0438\u0440","\u0437\u0430\u0434\u043e\u043a","\u0437\u0430\u0434\u043e\u0440","\u0437\u0430\u0451\u0431\u0430","\u0437\u0430\u0435\u0434\u0430","\u0437\u0430\u0435\u0437\u0434","\u0437\u0430\u0436\u0438\u043c","\u0437\u0430\u0436\u0438\u043d","\u0437\u0430\u0436\u043e\u0440","\u0437\u0430\u0437\u043e\u0440","\u0437\u0430\u0437\u044b\u0432","\u0437\u0430\u043a\u0430\u0437","\u0437\u0430\u043a\u0430\u043b","\u0437\u0430\u043a\u0430\u0442","\u0437\u0430\u043a\u043e\u043b","\u0437\u0430\u043a\u043e\u043d","\u0437\u0430\u043a\u043e\u043f","\u0437\u0430\u043a\u043e\u0441","\u0437\u0430\u043a\u0443\u0441","\u0437\u0430\u043a\u0443\u0442","\u0437\u0430\u043a\u044f\u0442","\u0437\u0430\u043b\u0430\u0437","\u0437\u0430\u043b\u0451\u0442","\u0437\u0430\u043b\u0435\u0446","\u0437\u0430\u043b\u0438\u0432","\u0437\u0430\u043b\u0438\u0437","\u0437\u0430\u043b\u043e\u0433","\u0437\u0430\u043b\u043e\u043c","\u0437\u0430\u043c\u0430\u043d","\u0437\u0430\u043c\u0430\u0445","\u0437\u0430\u043c\u0435\u0440","\u0437\u0430\u043c\u0435\u0441","\u0437\u0430\u043c\u0451\u0442","\u0437\u0430\u043c\u0438\u043d","\u0437\u0430\u043c\u043e\u043a","\u0437\u0430\u043c\u043e\u0440","\u0437\u0430\u043c\u043e\u0442","\u0437\u0430\u043c\u0448\u0430","\u0437\u0430\u043d\u0434\u0435","\u0437\u0430\u043d\u0434\u0440","\u0437\u0430\u043d\u043e\u0441","\u0437\u0430\u043f\u0430\u0434","\u0437\u0430\u043f\u0430\u043b","\u0437\u0430\u043f\u0430\u0441","\u0437\u0430\u043f\u0430\u0445","\u0437\u0430\u043f\u0435\u0432","\u0437\u0430\u043f\u0438\u043b","\u0437\u0430\u043f\u043e\u0439","\u0437\u0430\u043f\u043e\u043d","\u0437\u0430\u043f\u043e\u0440","\u0437\u0430\u0440\u0435\u0432","\u0437\u0430\u0440\u0435\u0437","\u0437\u0430\u0440\u0438\u043a","\u0437\u0430\u0440\u0438\u043d","\u0437\u0430\u0440\u043e\u0434","\u0437\u0430\u0440\u043e\u043a","\u0437\u0430\u0440\u0443\u0431","\u0437\u0430\u0440\u044f\u0434","\u0437\u0430\u0441\u0435\u0432","\u0437\u0430\u0441\u043e\u0432","\u0437\u0430\u0441\u043e\u043b","\u0437\u0430\u0441\u043e\u0440","\u0437\u0430\u0441\u043e\u0441","\u0437\u0430\u0441\u044b\u043b","\u0437\u0430\u0442\u0430\u0440","\u0437\u0430\u0442\u0451\u043a","\u0437\u0430\u0442\u0451\u0441","\u0437\u0430\u0442\u0435\u044f","\u0437\u0430\u0442\u043e\u043a","\u0437\u0430\u0442\u043e\u043d","\u0437\u0430\u0442\u043e\u0440","\u0437\u0430\u0442\u044b\u043a","\u0437\u0430\u0443\u043c\u044c","\u0437\u0430\u0445\u043e\u0434","\u0437\u0430\u0446\u0435\u043f","\u0437\u0430\u0447\u0451\u0441","\u0437\u0430\u0447\u0451\u0442","\u0437\u0430\u0447\u0438\u043d","\u0437\u0430\u0448\u0430\u0433","\u0437\u0430\u0449\u0435\u043f","\u0437\u0430\u0449\u0438\u043f","\u0437\u0430\u044f\u0432\u0430","\u0437\u0432\u0435\u043d\u043e","\u0437\u0432\u0435\u043d\u044c","\u0437\u0435\u0431\u0440\u0430","\u0437\u0435\u0432\u043e\u043a","\u0437\u0435\u043b\u044c\u0435","\u0437\u0435\u043b\u044c\u0446","\u0437\u0435\u043c\u043b\u044f","\u0437\u0435\u043d\u0438\u0442","\u0437\u0435\u043d\u043a\u0438","\u0437\u0435\u0440\u043d\u043e","\u0437\u0435\u0440\u043d\u044c","\u0437\u0435\u0444\u0438\u0440","\u0437\u0438\u043b\u043e\u043a","\u0437\u0438\u043c\u043a\u0430","\u0437\u0438\u043f\u0443\u043d","\u0437\u0438\u0445\u0435\u0440","\u0437\u043b\u0430\u0442\u043e","\u0437\u043b\u043e\u0431\u0430","\u0437\u043d\u0430\u043c\u044f","\u0437\u043d\u0430\u0442\u044c","\u0437\u043e\u0431\u0438\u043a","\u0437\u043e\u043b\u043a\u0430","\u0437\u043e\u043c\u0430\u043d","\u0437\u043e\u043e\u0438\u0434","\u0437\u043e\u0440\u0438\u0442","\u0437\u0440\u0430\u0437\u0430","\u0437\u0443\u0431\u0435\u0446","\u0437\u0443\u0431\u0438\u043a","\u0437\u0443\u0431\u043e\u043a","\u0437\u0443\u0434\u0451\u0436","\u0437\u0443\u043b\u044e\u043c","\u0437\u0443\u043c\u0431\u0430","\u0437\u0443\u043c\u043f\u0444","\u0437\u0443\u0440\u043d\u0430","\u0437\u0443\u0448\u043a\u0430","\u0437\u044b\u0431\u043a\u0430","\u0437\u044b\u0431\u0443\u043d","\u0437\u044f\u0431\u0440\u0430","\u0438\u0431\u0431\u0443\u0440","\u0438\u0432\u0435\u043d\u0442","\u0438\u0432\u0438\u043d\u0430","\u0438\u0432\u043d\u044f\u043a","\u0438\u0432\u0440\u0438\u0442","\u0438\u0433\u0438\u043b\u044c","\u0438\u0433\u043d\u043e\u0440","\u0438\u0433\u0440\u0435\u043a","\u0438\u0434\u0430\u0438\u0442","\u0438\u0434\u0435\u0430\u043b","\u0438\u0434\u0436\u043c\u0430","\u0438\u0434\u0438\u043e\u043c","\u0438\u0436\u0438\u0446\u0430","\u0438\u0436\u043e\u0440\u0430","\u0438\u0437\u0432\u0435\u0442","\u0438\u0437\u0432\u0438\u0432","\u0438\u0437\u0432\u0438\u043d","\u0438\u0437\u0432\u043e\u0434","\u0438\u0437\u0432\u043e\u0437","\u0438\u0437\u0433\u0438\u0431","\u0438\u0437\u043b\u0430\u0434","\u0438\u0437\u043b\u0451\u0442","\u0438\u0437\u043b\u043e\u0433","\u0438\u0437\u043b\u043e\u043c","\u0438\u0437\u043c\u043e\u043b","\u0438\u0437\u043c\u043e\u0440","\u0438\u0437\u043d\u043e\u0441","\u0438\u0437\u044a\u0451\u0431","\u0438\u0437\u044a\u044f\u043d","\u0438\u0437\u044b\u0441\u043a","\u0438\u043a\u0430\u0438\u0442","\u0438\u043a\u0430\u043b\u044c","\u0438\u043a\u043e\u043d\u0430","\u0438\u043a\u043e\u0442\u0430","\u0438\u043a\u0440\u044f\u043a","\u0438\u043a\u0441\u0438\u044f","\u0438\u043b\u0435\u0438\u0442","\u0438\u043b\u0435\u0443\u0441","\u0438\u043b\u043b\u0438\u0442","\u0438\u043b\u044c\u043c\u0430","\u0438\u043c\u0430\u0433\u043e","\u0438\u043c\u0435\u0439\u043b","\u0438\u043c\u0438\u0434\u0436","\u0438\u043d\u0432\u0430\u0440","\u0438\u043d\u0434\u0435\u043b","\u0438\u043d\u0434\u0438\u0439","\u0438\u043d\u0434\u0438\u0442","\u0438\u043d\u0434\u043e\u043b","\u0438\u043d\u0436\u0438\u0440","\u0438\u043d\u0438\u043e\u043d","\u0438\u043d\u043f\u0443\u0442","\u0438\u043d\u0441\u0442\u0430","\u0438\u043d\u0442\u0438\u043c","\u0438\u043d\u0442\u0440\u043e","\u0438\u043d\u0443\u043b\u0430","\u0438\u043d\u0444\u0438\u0437","\u0438\u043e\u0434\u0430\u0442","\u0438\u043e\u0434\u0438\u0434","\u0438\u043e\u0434\u044e\u0440","\u0438\u043e\u043b\u0438\u0442","\u0438\u043e\u043d\u0438\u0439","\u0438\u043e\u043d\u0438\u043a","\u0438\u043e\u043d\u0438\u0442","\u0438\u043f\u0440\u0438\u0442","\u0438\u0440\u043c\u043e\u0441","\u0438\u0441\u043a\u0438\u043d","\u0438\u0441\u043a\u043e\u043f","\u0438\u0441\u043a\u0440\u0430","\u0438\u0441\u043a\u0443\u0441","\u0438\u0441\u043b\u0430\u043c","\u0438\u0441\u043f\u043e\u0434","\u0438\u0441\u043f\u0443\u0433","\u0438\u0441\u0441\u043e\u043f","\u0438\u0441\u0442\u043e\u0434","\u0438\u0441\u0442\u043e\u043a","\u0438\u0441\u0445\u043e\u0434","\u0438\u0442\u043e\u0438\u0442","\u0438\u0444\u0438\u043a\u0430","\u0438\u0445\u0440\u0430\u043c","\u0438\u0445\u0442\u0438\u0441","\u0438\u0448\u0438\u0430\u0441","\u0439\u0438\u0434\u0438\u0448","\u0439\u043e\u0434\u0430\u0442","\u0439\u043e\u0434\u0438\u0434","\u0439\u043e\u0434\u043b\u044c","\u0439\u0443\u0445\u043d\u044f","\u043a\u0430\u0431\u0430\u043a","\u043a\u0430\u0431\u0443\u043b","\u043a\u0430\u0432\u0430\u0439","\u043a\u0430\u0432\u0430\u043b","\u043a\u0430\u0432\u0435\u0440","\u043a\u0430\u0432\u0443\u043d","\u043a\u0430\u0433\u0430\u043b","\u043a\u0430\u0433\u0430\u0442","\u043a\u0430\u0433\u043e\u0440","\u043a\u0430\u0434\u0430\u0433","\u043a\u0430\u0434\u0430\u0440","\u043a\u0430\u0434\u0434\u0438","\u043a\u0430\u0434\u043a\u0430","\u043a\u0430\u0434\u0440\u044b","\u043a\u0430\u0434\u044b\u043a","\u043a\u0430\u0437\u0430\u043d","\u043a\u0430\u0437\u043d\u0430","\u043a\u0430\u0437\u043d\u044c","\u043a\u0430\u0437\u0443\u0441","\u043a\u0430\u0439\u0435\u043d","\u043a\u0430\u0439\u043a\u0430","\u043a\u0430\u0439\u043b\u0430","\u043a\u0430\u0439\u043b\u043e","\u043a\u0430\u0439\u043c\u0430","\u043a\u0430\u0439\u043d\u0430","\u043a\u0430\u043a\u0430\u043e","\u043a\u0430\u043b\u0430\u043c","\u043a\u0430\u043b\u0430\u0447","\u043a\u0430\u043b\u0430\u0448","\u043a\u0430\u043b\u0438\u0439","\u043a\u0430\u043b\u043b\u0430","\u043a\u0430\u043b\u043f\u0430","\u043a\u0430\u043b\u044b\u043c","\u043a\u0430\u043b\u044c\u044f","\u043a\u0430\u043b\u044f\u043c","\u043a\u0430\u043c\u0430\u0441","\u043a\u0430\u043c\u0435\u043e","\u043a\u0430\u043c\u0435\u044f","\u043a\u0430\u043c\u0438\u043d","\u043a\u0430\u043c\u043a\u0430","\u043a\u0430\u043c\u0443\u0441","\u043a\u0430\u043c\u0443\u0442","\u043a\u0430\u043c\u0447\u0430","\u043a\u0430\u043c\u044b\u0448","\u043a\u0430\u043d\u0430\u043b","\u043a\u0430\u043d\u0430\u0442","\u043a\u0430\u043d\u0432\u0430","\u043a\u0430\u043d\u0437\u0443","\u043a\u0430\u043d\u0438\u0442","\u043a\u0430\u043d\u043d\u0430","\u043a\u0430\u043d\u043e\u043d","\u043a\u0430\u043d\u043e\u044d","\u043a\u0430\u043d\u0443\u043d","\u043a\u0430\u043f\u0435\u0440","\u043a\u0430\u043f\u043b\u0438","\u043a\u0430\u043f\u043b\u044f","\u043a\u0430\u043f\u043e\u043a","\u043a\u0430\u043f\u043e\u0440","\u043a\u0430\u043f\u043e\u0442","\u043a\u0430\u043f\u043f\u0430","\u043a\u0430\u043f\u0440\u0438","\u043a\u0430\u043f\u0443\u0442","\u043a\u0430\u043f\u0447\u0430","\u043a\u0430\u0440\u0430\u0442","\u043a\u0430\u0440\u0432\u044b","\u043a\u0430\u0440\u0433\u0430","\u043a\u0430\u0440\u0433\u043e","\u043a\u0430\u0440\u0434\u0430","\u043a\u0430\u0440\u0435\u0442","\u043a\u0430\u0440\u0438\u044f","\u043a\u0430\u0440\u043c\u0430","\u043a\u0430\u0440\u0440\u0430","\u043a\u0430\u0440\u0440\u0438","\u043a\u0430\u0440\u0441\u0442","\u043a\u0430\u0440\u0442\u0430","\u043a\u0430\u0440\u0442\u044b","\u043a\u0430\u0440\u0447\u0430","\u043a\u0430\u0440\u0448\u0430","\u043a\u0430\u0441\u0431\u0430","\u043a\u0430\u0441\u043a\u0430","\u043a\u0430\u0441\u043a\u043e","\u043a\u0430\u0441\u0441\u0430","\u043a\u0430\u0441\u0442\u0430","\u043a\u0430\u0442\u0430\u0440","\u043a\u0430\u0442\u0435\u0440","\u043a\u0430\u0442\u0435\u0442","\u043a\u0430\u0442\u0438\u043d","\u043a\u0430\u0442\u043e\u0434","\u043a\u0430\u0442\u043e\u043a","\u043a\u0430\u0442\u0443\u0445","\u043a\u0430\u0442\u044b\u043a","\u043a\u0430\u0442\u044b\u0448","\u043a\u0430\u0442\u044f\u0445","\u043a\u0430\u0443\u0437\u0430","\u043a\u0430\u0443\u0440\u0438","\u043a\u0430\u0446\u0435\u044f","\u043a\u0430\u0447\u0438\u043c","\u043a\u0430\u0447\u043a\u0430","\u043a\u0430\u0447\u043e\u043a","\u043a\u0430\u0448\u043a\u0430","\u043a\u0430\u0448\u043d\u0435","\u043a\u0430\u0448\u043f\u043e","\u043a\u0430\u044e\u0442\u0430","\u043a\u0432\u0430\u0434\u0440","\u043a\u0432\u0430\u043b\u0430","\u043a\u0432\u0430\u043d\u0442","\u043a\u0432\u0430\u0440\u043a","\u043a\u0432\u0430\u0440\u0446","\u043a\u0432\u0430\u0447\u0430","\u043a\u0432\u0435\u0441\u0442","\u043a\u0432\u0438\u043b\u0442","\u043a\u0432\u0438\u044e\u0442","\u043a\u0432\u043e\u0442\u0430","\u043a\u0435\u0431\u0430\u0431","\u043a\u0435\u0433\u043b\u0438","\u043a\u0435\u0433\u043b\u044c","\u043a\u0435\u0433\u043b\u044f","\u043a\u0435\u0434\u0438\u043a","\u043a\u0435\u0438\u043d\u0433","\u043a\u0435\u043a\u0438\u0442","\u043a\u0435\u043a\u0443\u0440","\u043a\u0435\u043a\u0447\u0438","\u043a\u0435\u043b\u0438\u044f","\u043a\u0435\u043b\u044c\u0442","\u043a\u0435\u043b\u044c\u044f","\u043a\u0435\u043d\u0430\u0444","\u043a\u0435\u043d\u0435\u0448","\u043a\u0435\u043f\u043a\u0430","\u043a\u0435\u0440\u0431\u044c","\u043a\u0435\u0440\u0438\u0442","\u043a\u0435\u0440\u043c\u0430","\u043a\u0435\u0442\u0435\u043d","\u043a\u0435\u0442\u043e\u0437","\u043a\u0435\u0442\u043e\u043d","\u043a\u0435\u0444\u0438\u0440","\u043a\u0435\u0447\u0443\u0430","\u043a\u0435\u0448\u044c\u044e","\u043a\u0438\u0431\u043b\u0430","\u043a\u0438\u0431\u0443\u0446","\u043a\u0438\u0432\u0435\u0440","\u043a\u0438\u0432\u043e\u043a","\u043a\u0438\u0432\u043e\u0442","\u043a\u0438\u0434\u043e\u043a","\u043a\u0438\u0437\u0438\u043b","\u043a\u0438\u0437\u044f\u043a","\u043a\u0438\u043a\u0435\u0440","\u043a\u0438\u043b\u0438\u043a","\u043a\u0438\u043b\u0438\u043c","\u043a\u0438\u043b\u044c\u0442","\u043a\u0438\u043c\u0447\u0438","\u043a\u0438\u043d\u0435\u0437","\u043a\u0438\u043d\u0437\u0430","\u043a\u0438\u043d\u043e\u0430","\u043a\u0438\u043d\u0446\u043e","\u043a\u0438\u043e\u0441\u043a","\u043a\u0438\u043f\u0435\u0436","\u043a\u0438\u043f\u0435\u0440","\u043a\u0438\u043f\u0435\u0448","\u043a\u0438\u043f\u0438\u0448","\u043a\u0438\u0440\u0437\u0430","\u043a\u0438\u0440\u043a\u0430","\u043a\u0438\u0440\u0445\u0430","\u043a\u0438\u0441\u0432\u0430","\u043a\u0438\u0441\u0435\u0442","\u043a\u0438\u0441\u0435\u044f","\u043a\u0438\u0441\u043a\u0430","\u043a\u0438\u0441\u0442\u0430","\u043a\u0438\u0441\u0442\u044c","\u043a\u0438\u0441\u044d\u043d","\u043a\u0438\u0442\u0430\u0431","\u043a\u0438\u0442\u0438\u0442","\u043a\u0438\u0444\u043e\u0437","\u043a\u0438\u0447\u043a\u0430","\u043a\u0438\u0448\u043a\u0430","\u043a\u0438\u0448\u043a\u0438","\u043a\u043b\u0430\u0432\u0430","\u043a\u043b\u0430\u0434\u0430","\u043a\u043b\u0430\u0434\u044c","\u043a\u043b\u0430\u043a\u0430","\u043a\u043b\u0430\u043f\u044b","\u043a\u043b\u0430\u0440\u043a","\u043a\u043b\u0430\u0441\u0441","\u043a\u043b\u0430\u0442\u0447","\u043a\u043b\u0435\u0440\u0441","\u043a\u043b\u0435\u0442\u044c","\u043a\u043b\u0435\u0449\u0438","\u043a\u043b\u0438\u043a\u0430","\u043a\u043b\u0438\u043a\u043e","\u043a\u043b\u0438\u043d\u0447","\u043a\u043b\u0438\u043f\u0441","\u043a\u043b\u0438\u0446\u0430","\u043a\u043b\u0438\u0448\u0435","\u043a\u043b\u043e\u0430\u043a","\u043a\u043b\u043e\u043a\u0435","\u043a\u043b\u0443\u043d\u044f","\u043a\u043b\u0443\u043f\u043f","\u043a\u043b\u044e\u043a\u0430","\u043a\u043d\u0435\u043b\u044c","\u043a\u043d\u0435\u0445\u0442","\u043a\u043d\u0438\u0433\u0430","\u043a\u043d\u0438\u0446\u0430","\u043a\u043d\u043e\u0440\u0440","\u043a\u043e\u0431\u0435\u043b","\u043a\u043e\u0431\u0435\u044f","\u043a\u043e\u0431\u0437\u0430","\u043a\u043e\u0431\u043e\u043b","\u043a\u043e\u0431\u0443\u0437","\u043a\u043e\u0431\u044b\u0437","\u043a\u043e\u0432\u0430\u0440","\u043a\u043e\u0432\u0451\u0440","\u043a\u043e\u0432\u0438\u0434","\u043a\u043e\u0432\u0438\u0442","\u043a\u043e\u0432\u043a\u0430","\u043a\u043e\u0434\u0430\u043a","\u043a\u043e\u0434\u0435\u043a","\u043a\u043e\u0434\u0435\u0440","\u043a\u043e\u0434\u0438\u044f","\u043a\u043e\u0434\u043b\u0430","\u043a\u043e\u0434\u043e\u043d","\u043a\u043e\u0436\u0430\u043d","\u043a\u043e\u0436\u0443\u0445","\u043a\u043e\u0436\u044c\u0451","\u043a\u043e\u0437\u0430\u043d","\u043a\u043e\u0437\u043b\u044b","\u043a\u043e\u0437\u043d\u044b","\u043a\u043e\u0437\u043d\u044c","\u043a\u043e\u0437\u043e\u043d","\u043a\u043e\u0438\u043a\u0441","\u043a\u043e\u0439\u043a\u0430","\u043a\u043e\u0439\u043c\u0430","\u043a\u043e\u0439\u043d\u0435","\u043a\u043e\u0439\u043d\u044d","\u043a\u043e\u0439\u043e\u043b","\u043a\u043e\u043a\u043b\u0435","\u043a\u043e\u043a\u043d\u0438","\u043a\u043e\u043a\u043e\u043d","\u043a\u043e\u043a\u043e\u0440","\u043a\u043e\u043a\u043e\u0441","\u043a\u043e\u043a\u0443\u0441","\u043a\u043e\u043b\u0431\u0430","\u043a\u043e\u043b\u0435\u0440","\u043a\u043e\u043b\u0435\u0442","\u043a\u043e\u043b\u0435\u044f","\u043a\u043e\u043b\u0438\u043a","\u043a\u043e\u043b\u0438\u0442","\u043a\u043e\u043b\u043a\u0430","\u043a\u043e\u043b\u043e\u0431","\u043a\u043e\u043b\u043e\u043a","\u043a\u043e\u043b\u043e\u043d","\u043a\u043e\u043b\u043e\u0441","\u043a\u043e\u043b\u043e\u0442","\u043a\u043e\u043b\u0443\u043d","\u043a\u043e\u043b\u0447\u0430","\u043a\u043e\u043b\u044c\u0435","\u043a\u043e\u043b\u044c\u0451","\u043a\u043e\u043b\u044c\u0442","\u043a\u043e\u043b\u044e\u0440","\u043a\u043e\u043c\u0431\u0438","\u043a\u043e\u043c\u0431\u043e","\u043a\u043e\u043c\u043c\u0430","\u043a\u043e\u043c\u043c\u0438","\u043a\u043e\u043c\u043e\u0434","\u043a\u043e\u043c\u043e\u043a","\u043a\u043e\u043d\u0433\u0430","\u043a\u043e\u043d\u0433\u043e","\u043a\u043e\u043d\u0451\u043a","\u043a\u043e\u043d\u0435\u0446","\u043a\u043e\u043d\u0438\u0434","\u043a\u043e\u043d\u0438\u043a","\u043a\u043e\u043d\u043a\u0430","\u043a\u043e\u043d\u0442\u0435","\u043a\u043e\u043d\u0442\u043e","\u043a\u043e\u043d\u0443\u0441","\u043a\u043e\u043d\u0444\u0430","\u043a\u043e\u043d\u0445\u0430","\u043a\u043e\u043f\u0430\u043b","\u043a\u043e\u043f\u0451\u0440","\u043a\u043e\u043f\u0435\u0446","\u043a\u043e\u043f\u0438\u0435","\u043a\u043e\u043f\u0438\u0440","\u043a\u043e\u043f\u0438\u044f","\u043a\u043e\u043f\u043a\u0430","\u043a\u043e\u043f\u043d\u0430","\u043a\u043e\u043f\u043f\u0430","\u043a\u043e\u043f\u0440\u0430","\u043a\u043e\u043f\u044b\u043b","\u043a\u043e\u043f\u044c\u0451","\u043a\u043e\u0440\u0434\u0430","\u043a\u043e\u0440\u0435\u0446","\u043a\u043e\u0440\u043a\u0430","\u043a\u043e\u0440\u043c\u0430","\u043a\u043e\u0440\u043e\u0431","\u043a\u043e\u0440\u0447\u0430","\u043a\u043e\u0440\u044c\u0451","\u043a\u043e\u0441\u043a\u0430","\u043a\u043e\u0441\u043c\u044b","\u043a\u043e\u0441\u043e\u043a","\u043a\u043e\u0441\u0442\u0438","\u043a\u043e\u0441\u0442\u044c","\u043a\u043e\u0441\u044c\u0451","\u043a\u043e\u0441\u044f\u043a","\u043a\u043e\u0442\u0451\u043b","\u043a\u043e\u0442\u0438\u043f","\u043a\u043e\u0442\u043b\u044b","\u043a\u043e\u0442\u0443\u0445","\u043a\u043e\u0444\u0435\u0439","\u043a\u043e\u0444\u0438\u0439","\u043a\u043e\u0444\u0442\u0430","\u043a\u043e\u0445\u0438\u0442","\u043a\u043e\u0445\u0438\u044f","\u043a\u043e\u0447\u0430\u043d","\u043a\u043e\u0447\u043a\u0430","\u043a\u043e\u0448\u0435\u0440","\u043a\u043e\u0448\u043a\u0430","\u043a\u043e\u0448\u043c\u0430","\u043a\u0440\u0430\u0431\u0431","\u043a\u0440\u0430\u0433\u0430","\u043a\u0440\u0430\u0436\u0430","\u043a\u0440\u0430\u0437\u0430","\u043a\u0440\u0430\u043c\u043f","\u043a\u0440\u0430\u043d\u043a","\u043a\u0440\u0430\u043d\u0442","\u043a\u0440\u0430\u043d\u0446","\u043a\u0440\u0430\u043d\u0447","\u043a\u0440\u0430\u043f\u043f","\u043a\u0440\u0430\u0441\u0430","\u043a\u0440\u0430\u0441\u0442","\u043a\u0440\u0430\u0444\u0442","\u043a\u0440\u0435\u0434\u043e","\u043a\u0440\u0435\u043c\u0430","\u043a\u0440\u0435\u043d\u044c","\u043a\u0440\u0435\u043f\u044c","\u043a\u0440\u0435\u0441\u0441","\u043a\u0440\u0435\u0441\u0442","\u043a\u0440\u0438\u0433\u0430","\u043a\u0440\u0438\u043b\u044c","\u043a\u0440\u0438\u043c\u0438","\u043a\u0440\u0438\u043d\u0436","\u043a\u0440\u0438\u0446\u0430","\u043a\u0440\u043e\u0432\u044c","\u043a\u0440\u043e\u043a\u0438","\u043a\u0440\u043e\u043b\u044c","\u043a\u0440\u043e\u043d\u0430","\u043a\u0440\u043e\u0441\u0441","\u043a\u0440\u043e\u0442\u0430","\u043a\u0440\u043e\u0445\u0430","\u043a\u0440\u043e\u0448\u0435","\u043a\u0440\u0443\u0438\u0437","\u043a\u0440\u0443\u043f\u0430","\u043a\u0440\u0443\u0442\u044c","\u043a\u0440\u0443\u0447\u0430","\u043a\u0440\u0443\u0447\u044c","\u043a\u0440\u044b\u0433\u0430","\u043a\u0440\u044b\u043b\u043e","\u043a\u0440\u044b\u0441\u0430","\u043a\u0440\u044b\u0448\u0430","\u043a\u0440\u044d\u043f\u0441","\u043a\u0441\u0435\u043d\u044c","\u043a\u0441\u0438\u0432\u0430","\u043a\u0441\u0438\u0441\u0442","\u043a\u0443\u0431\u0438\u043a","\u043a\u0443\u0431\u0438\u0442","\u043a\u0443\u0431\u043b\u043e","\u043a\u0443\u0431\u043e\u043a","\u043a\u0443\u0432\u0435\u0437","\u043a\u0443\u0434\u043b\u0430","\u043a\u0443\u0434\u0440\u0438","\u043a\u0443\u0437\u043d\u044f","\u043a\u0443\u0437\u043e\u0432","\u043a\u0443\u043a\u0430\u043d","\u043a\u0443\u043a\u0438\u0448","\u043a\u0443\u043a\u043b\u0430","\u043a\u0443\u043a\u043e\u0436","\u043a\u0443\u043a\u0440\u0438","\u043a\u0443\u043a\u0441\u0443","\u043a\u0443\u043a\u0443\u0438","\u043a\u0443\u043b\u0430\u043a","\u043a\u0443\u043b\u0451\u043a","\u043a\u0443\u043b\u0435\u0440","\u043a\u0443\u043b\u0435\u0448","\u043a\u0443\u043b\u0438\u0447","\u043a\u0443\u043b\u043e\u043d","\u043a\u0443\u043b\u044c\u0451","\u043a\u0443\u043b\u044c\u0442","\u043a\u0443\u043c\u0430\u043d","\u043a\u0443\u043c\u0430\u0440","\u043a\u0443\u043c\u0430\u0447","\u043a\u0443\u043c\u0438\u043d","\u043a\u0443\u043c\u0438\u0440","\u043a\u0443\u043c\u043e\u043b","\u043a\u0443\u043c\u0443\u0437","\u043a\u0443\u043c\u044b\u0441","\u043a\u0443\u043d\u043a\u0430","\u043a\u0443\u043d\u0444\u0443","\u043a\u0443\u043f\u0430\u0436","\u043a\u0443\u043f\u043b\u044f","\u043a\u0443\u043f\u043e\u043b","\u043a\u0443\u043f\u043e\u043d","\u043a\u0443\u0440\u0430\u0436","\u043a\u0443\u0440\u0430\u0439","\u043a\u0443\u0440\u0430\u043a","\u043a\u0443\u0440\u0430\u0448","\u043a\u0443\u0440\u0435\u0441","\u043a\u0443\u0440\u0435\u0448","\u043a\u0443\u0440\u0437\u0435","\u043a\u0443\u0440\u0438\u044f","\u043a\u0443\u0440\u043a\u0430","\u043a\u0443\u0440\u043e\u043a","\u043a\u0443\u0440\u043e\u0441","\u043a\u0443\u0440\u0442\u0430","\u043a\u0443\u0440\u0443\u043c","\u043a\u0443\u0440\u0443\u0440","\u043a\u0443\u0440\u0443\u0448","\u043a\u0443\u0440\u044c\u044f","\u043a\u0443\u0441\u043e\u043a","\u043a\u0443\u0442\u0430\u0431","\u043a\u0443\u0442\u0430\u043d","\u043a\u0443\u0442\u0430\u0441","\u043a\u0443\u0442\u0451\u0436","\u043a\u0443\u0442\u0438\u043d","\u043a\u0443\u0442\u0438\u0441","\u043a\u0443\u0442\u043e\u043a","\u043a\u0443\u0442\u0440\u0430","\u043a\u0443\u0442\u044c\u044f","\u043a\u0443\u0442\u044e\u043c","\u043a\u0443\u0444\u0438\u044f","\u043a\u0443\u0445\u0432\u0430","\u043a\u0443\u0445\u0435\u043d","\u043a\u0443\u0445\u043d\u044f","\u043a\u0443\u0445\u0442\u0430","\u043a\u0443\u0447\u043a\u0430","\u043a\u0443\u0448\u0430\u043a","\u043a\u044d\u043c\u043f\u043e","\u043a\u044d\u043d\u0434\u043e","\u043a\u044d\u0440\u043e\u0431","\u043a\u044d\u0442\u0442\u0438","\u043a\u044e\u0432\u0435\u0442","\u043a\u044e\u043b\u043e\u0442","\u043a\u044e\u0440\u0438\u0439","\u043a\u044e\u0440\u0438\u0442","\u043a\u044e\u0444\u0442\u0430","\u043a\u044f\u0440\u0438\u0437","\u043a\u044f\u0441\u0440\u0430","\u043b\u0430\u0431\u0430\u0437","\u043b\u0430\u0432\u0430\u0436","\u043b\u0430\u0432\u0430\u0448","\u043b\u0430\u0432\u0434\u0430","\u043b\u0430\u0432\u0438\u0441","\u043b\u0430\u0432\u043a\u0430","\u043b\u0430\u0432\u0440\u0430","\u043b\u0430\u0433\u0435\u0440","\u043b\u0430\u0433\u0443\u043d","\u043b\u0430\u0434\u0430\u043d","\u043b\u0430\u0434\u043a\u0430","\u043b\u0430\u0434\u043e\u043a","\u043b\u0430\u0434\u044c\u044f","\u043b\u0430\u0437\u0435\u0440","\u043b\u0430\u0437\u0435\u044f","\u043b\u0430\u0437\u043e\u043a","\u043b\u0430\u0439\u0431\u0430","\u043b\u0430\u0439\u0434\u0430","\u043b\u0430\u0439\u043a\u0430","\u043b\u0430\u043a\u0430\u0431","\u043b\u0430\u043a\u0441\u0430","\u043b\u0430\u043b\u043a\u0430","\u043b\u0430\u043c\u0431\u0430","\u043b\u0430\u043c\u0438\u043d","\u043b\u0430\u043c\u043f\u0430","\u043b\u0430\u043d\u0434\u0430","\u043b\u0430\u043d\u0434\u043e","\u043b\u0430\u043f\u043a\u0430","\u043b\u0430\u043f\u0442\u0430","\u043b\u0430\u043f\u0448\u0430","\u043b\u0430\u0440\u0432\u0430","\u043b\u0430\u0440\u0434\u043e","\u043b\u0430\u0440\u0451\u043a","\u043b\u0430\u0440\u0435\u0446","\u043b\u0430\u0441\u043a\u0430","\u043b\u0430\u0441\u0441\u0438","\u043b\u0430\u0441\u0441\u043e","\u043b\u0430\u0441\u0442\u0430","\u043b\u0430\u0442\u043a\u0430","\u043b\u0430\u0442\u0442\u0435","\u043b\u0430\u0442\u0443\u043a","\u043b\u0430\u0443\u043d\u0436","\u043b\u0430\u0444\u0435\u0442","\u043b\u0430\u0444\u0438\u0442","\u043b\u0430\u0446\u0446\u0438","\u043b\u0430\u0446\u0446\u043e","\u043b\u0430\u0447\u043e\u043a","\u043b\u0431\u0438\u043d\u0430","\u043b\u0431\u0438\u0449\u0435","\u043b\u0435\u0432\u0430\u043a","\u043b\u0435\u0432\u0435\u043b","\u043b\u0435\u0433\u0443\u0430","\u043b\u0435\u0434\u043e\u043a","\u043b\u0435\u0436\u0430\u043a","\u043b\u0451\u0436\u043a\u0430","\u043b\u0435\u0439\u0430\u0441","\u043b\u0435\u0439\u0431\u043b","\u043b\u0435\u0439\u043a\u0430","\u043b\u0435\u0439\u043c\u0430","\u043b\u0435\u043a\u0438\u0444","\u043b\u0435\u043a\u0441\u0430","\u043b\u0435\u043c\u0435\u0445","\u043b\u0435\u043c\u043c\u0430","\u043b\u0435\u043d\u043e\u043a","\u043b\u0435\u043d\u0442\u0430","\u043b\u0435\u043d\u0446\u0430","\u043b\u0435\u043e\u043d\u0435","\u043b\u0435\u043f\u0435\u0442","\u043b\u0435\u043f\u043a\u0430","\u043b\u0435\u043f\u0440\u0430","\u043b\u0435\u043f\u0442\u0430","\u043b\u0435\u0440\u043a\u0430","\u043b\u0435\u0441\u043a\u0430","\u043b\u0435\u0441\u043e\u043a","\u043b\u0435\u0441\u0442\u044c","\u043b\u0435\u0441\u0445\u0430","\u043b\u0435\u0442\u043a\u0430","\u043b\u0451\u0442\u043a\u0430","\u043b\u0435\u0442\u043e\u043a","\u043b\u0436\u0435\u0446\u0430","\u043b\u0436\u0438\u0446\u0430","\u043b\u0438\u0430\u0437\u0430","\u043b\u0438\u0430\u043d\u0430","\u043b\u0438\u0431\u0440\u0430","\u043b\u0438\u0432\u0430\u043d","\u043b\u0438\u0432\u0435\u0440","\u043b\u0438\u0434\u0430\u0440","\u043b\u0438\u0437\u0430\u0442","\u043b\u0438\u0437\u0438\u043d","\u043b\u0438\u0437\u0438\u0441","\u043b\u0438\u0437\u043e\u043b","\u043b\u0438\u0437\u0443\u043d","\u043b\u0438\u043a\u0451\u0440","\u043b\u0438\u043a\u0438\u0439","\u043b\u0438\u043b\u0435\u044f","\u043b\u0438\u043b\u0438\u044f","\u043b\u0438\u043c\u0430\u043d","\u043b\u0438\u043c\u0431\u043e","\u043b\u0438\u043c\u0438\u0442","\u043b\u0438\u043c\u043e\u043d","\u043b\u0438\u043c\u0444\u0430","\u043b\u0438\u043d\u0433\u0430","\u043b\u0438\u043d\u0451\u043a","\u043b\u0438\u043d\u0435\u0440","\u043b\u0438\u043d\u0437\u0430","\u043b\u0438\u043d\u0438\u044f","\u043b\u0438\u043d\u043e\u043b","\u043b\u0438\u043f\u0435\u0446","\u043b\u0438\u043f\u0438\u0434","\u043b\u0438\u043f\u043a\u0430","\u043b\u0438\u043f\u0441\u0438","\u043b\u0438\u0442\u0432\u0430","\u043b\u0438\u0442\u0435\u0440","\u043b\u0438\u0442\u0438\u0439","\u043b\u0438\u0442\u0438\u044f","\u043b\u0438\u0442\u043a\u0430","\u043b\u0438\u0442\u043e\u043d","\u043b\u0438\u0442\u0440\u0430","\u043b\u0438\u0442\u044c\u0451","\u043b\u0438\u0444\u0430\u043a","\u043b\u0438\u0444\u043e\u043d","\u043b\u0438\u0445\u0432\u0430","\u043b\u0438\u0445\u0435\u043d","\u043b\u0438\u0446\u0435\u0439","\u043b\u0438\u0447\u043a\u0430","\u043b\u0438\u0448\u0430\u0439","\u043b\u0438\u0448\u0435\u043a","\u043b\u0438\u0448\u043a\u043e","\u043b\u043e\u0431\u0431\u0438","\u043b\u043e\u0431\u0438\u043a","\u043b\u043e\u0431\u0438\u043e","\u043b\u043e\u0431\u043e\u043a","\u043b\u043e\u0432\u043b\u044f","\u043b\u043e\u0433\u0438\u043d","\u043b\u043e\u0433\u043e\u0441","\u043b\u043e\u0434\u0435\u043d","\u043b\u043e\u0434\u043a\u0430","\u043b\u043e\u0436\u043a\u0430","\u043b\u043e\u0436\u043e\u043a","\u043b\u043e\u043a\u0432\u0430","\u043b\u043e\u043a\u0435\u0440","\u043b\u043e\u043a\u043e\u043d","\u043b\u043e\u043a\u0443\u0441","\u043b\u043e\u043c\u0438\u043a","\u043b\u043e\u043c\u043a\u0430","\u043b\u043e\u043c\u043e\u043a","\u043b\u043e\u043d\u0436\u0430","\u043b\u043e\u043f\u0443\u0445","\u043b\u043e\u0440\u0443\u043c","\u043b\u043e\u0442\u043e\u043a","\u043b\u043e\u0442\u043e\u0441","\u043b\u043e\u0444\u0435\u0440","\u043b\u043e\u0445\u043c\u0430","\u043b\u043e\u0445\u043c\u044b","\u043b\u043e\u0445\u043e\u0441","\u043b\u043e\u0446\u0438\u044f","\u043b\u0443\u0431\u043e\u043a","\u043b\u0443\u0436\u043e\u043a","\u043b\u0443\u0437\u0433\u0430","\u043b\u0443\u043a\u043d\u043e","\u043b\u0443\u043a\u0443\u043c","\u043b\u0443\u043d\u0434\u0430","\u043b\u0443\u043d\u0438\u0442","\u043b\u0443\u043d\u043a\u0430","\u043b\u0443\u043f\u0438\u043d","\u043b\u0443\u043f\u043a\u0430","\u043b\u0443\u0444\u0444\u0430","\u043b\u0443\u0447\u0438\u043a","\u043b\u0443\u0447\u043e\u043a","\u043b\u044b\u0434\u043a\u0430","\u043b\u044b\u0436\u043d\u044f","\u043b\u044b\u0441\u043a\u0430","\u043b\u044b\u0441\u0442\u043e","\u043b\u044b\u0442\u043a\u0430","\u043b\u044b\u0442\u043a\u0438","\u043b\u044b\u0447\u043a\u0430","\u043b\u044b\u0447\u043a\u043e","\u043b\u044c\u044f\u043b\u043e","\u043b\u044e\u0431\u043a\u0430","\u043b\u044e\u0433\u0435\u0440","\u043b\u044e\u0434\u0443\u0441","\u043b\u044e\u043c\u0435\u043d","\u043b\u044e\u043d\u0435\u0442","\u043b\u044e\u043f\u0438\u043d","\u043b\u044e\u043f\u0443\u0441","\u043b\u044e\u0441\u0442\u0440","\u043b\u044e\u0442\u0438\u043a","\u043b\u044e\u0442\u043d\u044f","\u043b\u044e\u0444\u0444\u0430","\u043b\u044e\u0447\u043e\u043a","\u043b\u044f\u0433\u0430\u043d","\u043b\u044f\u0436\u043a\u0430","\u043b\u044f\u043c\u043a\u0430","\u043b\u044f\u043f\u0438\u0441","\u043b\u044f\u0441\u0441\u0435","\u043c\u0430\u0433\u0430\u0437","\u043c\u0430\u0433\u0435\u0439","\u043c\u0430\u0433\u0438\u044f","\u043c\u0430\u0433\u043c\u0430","\u043c\u0430\u0434\u0438\u044f","\u043c\u0430\u0435\u0442\u0430","\u043c\u0430\u0436\u043e\u0440","\u043c\u0430\u0437\u0430\u0440","\u043c\u0430\u0437\u0434\u0430","\u043c\u0430\u0437\u0435\u0440","\u043c\u0430\u0437\u0438\u043a","\u043c\u0430\u0437\u043a\u0430","\u043c\u0430\u0437\u043d\u044f","\u043c\u0430\u0437\u043e\u043a","\u043c\u0430\u0437\u0443\u0442","\u043c\u0430\u0439\u043a\u0430","\u043c\u0430\u0439\u043d\u0430","\u043c\u0430\u043a\u0430\u043e","\u043c\u0430\u043a\u0430\u0440","\u043c\u0430\u043a\u0435\u0442","\u043c\u0430\u043a\u0441\u0430","\u043c\u0430\u043a\u0441\u0438","\u043c\u0430\u043b\u0438\u043a","\u043c\u0430\u043b\u043a\u0430","\u043c\u0430\u043b\u044c\u043c","\u043c\u0430\u043c\u0431\u0430","\u043c\u0430\u043c\u0431\u043e","\u043c\u0430\u043c\u043e\u043d","\u043c\u0430\u043c\u0441\u044b","\u043c\u0430\u043d\u0430\u0441","\u043c\u0430\u043d\u0430\u0442","\u043c\u0430\u043d\u0433\u0430","\u043c\u0430\u043d\u0433\u043e","\u043c\u0430\u043d\u0433\u0440","\u043c\u0430\u043d\u0434\u0430","\u043c\u0430\u043d\u0434\u0443","\u043c\u0430\u043d\u0435\u0436","\u043c\u0430\u043d\u0435\u0440","\u043c\u0430\u043d\u0438\u0435","\u043c\u0430\u043d\u0438\u044f","\u043c\u0430\u043d\u043a\u0430","\u043c\u0430\u043d\u043a\u0438","\u043c\u0430\u043d\u043a\u043e","\u043c\u0430\u043d\u043d\u0430","\u043c\u0430\u043d\u043e\u043a","\u043c\u0430\u043d\u043e\u0440","\u043c\u0430\u043d\u0442\u0430","\u043c\u0430\u043d\u0442\u043e","\u043c\u0430\u043d\u0442\u0443","\u043c\u0430\u043d\u0442\u044b","\u043c\u0430\u043d\u044c\u0435","\u043c\u0430\u0440\u0433\u043e","\u043c\u0430\u0440\u0436\u0430","\u043c\u0430\u0440\u043a\u0430","\u043c\u0430\u0440\u043b\u044f","\u043c\u0430\u0440\u043c\u0430","\u043c\u0430\u0440\u0448\u0438","\u043c\u0430\u0441\u043a\u0430","\u043c\u0430\u0441\u043b\u043e","\u043c\u0430\u0441\u0441\u0430","\u043c\u0430\u0441\u0442\u044c","\u043c\u0430\u0442\u0430\u043d","\u043c\u0430\u0442\u043a\u0430","\u043c\u0430\u0442\u044e\u0433","\u043c\u0430\u0442\u044e\u043a","\u043c\u0430\u0444\u0438\u0442","\u043c\u0430\u0444\u0438\u044f","\u043c\u0430\u0444\u043e\u043d","\u043c\u0430\u0445\u0430\u0447","\u043c\u0430\u0445\u0440\u0430","\u043c\u0430\u0445\u0440\u044b","\u043c\u0430\u0446\u0438\u0441","\u043c\u0430\u0446\u0443\u043d","\u043c\u0430\u0447\u0442\u0430","\u043c\u0430\u0448\u0442\u0430","\u043c\u0435\u0433\u043e\u043c","\u043c\u0435\u0434\u0438\u0430","\u043c\u0435\u0434\u0438\u0439","\u043c\u0435\u0434\u0438\u043d","\u043c\u0435\u0434\u043e\u043a","\u043c\u0435\u0434\u044c\u0435","\u043c\u0435\u0434\u044f\u043a","\u043c\u0435\u0437\u0433\u0430","\u043c\u0435\u0437\u0438\u043b","\u043c\u0435\u0437\u043e\u043d","\u043c\u0435\u0439\u043e\u0437","\u043c\u0435\u043b\u0438\u044f","\u043c\u0435\u043b\u043e\u043a","\u043c\u0435\u043b\u043e\u0441","\u043c\u0435\u043c\u0430\u0441","\u043c\u0435\u043c\u0435\u0441","\u043c\u0435\u043c\u043e\u0441","\u043c\u0435\u043d\u0434\u0438","\u043c\u0435\u043d\u0436\u0430","\u043c\u0435\u043d\u043a\u0430","\u043c\u0435\u0440\u0432\u0430","\u043c\u0435\u0440\u0438\u043d","\u043c\u0435\u0440\u043a\u0430","\u043c\u0435\u0440\u043e\u043c","\u043c\u0435\u0441\u0441\u0430","\u043c\u0435\u0441\u0442\u043e","\u043c\u0435\u0441\u0442\u044c","\u043c\u0435\u0441\u044f\u0446","\u043c\u0435\u0442\u0430\u043b","\u043c\u0435\u0442\u0430\u043d","\u043c\u0435\u0442\u0438\u0437","\u043c\u0435\u0442\u0438\u043b","\u043c\u0435\u0442\u0438\u043d","\u043c\u0435\u0442\u043a\u0430","\u043c\u0435\u0442\u043b\u0430","\u043c\u0435\u0442\u043e\u0434","\u043c\u0435\u0442\u043e\u043b","\u043c\u0435\u0442\u043e\u043f","\u043c\u0435\u0442\u0440\u043e","\u043c\u0435\u0446\u0446\u043e","\u043c\u0435\u0447\u0442\u0430","\u043c\u0435\u0448\u043a\u0430","\u043c\u0435\u0448\u043e\u043a","\u043c\u0438\u0430\u0437\u043c","\u043c\u0438\u0432\u0430\u043b","\u043c\u0438\u0433\u043c\u0430","\u043c\u0438\u0437\u0435\u0440","\u043c\u0438\u043a\u043e\u0437","\u043c\u0438\u043a\u0441\u0442","\u043c\u0438\u043b\u0438\u0430","\u043c\u0438\u043b\u043a\u0430","\u043c\u0438\u043b\u043e\u043d","\u043c\u0438\u043d\u0435\u0442","\u043c\u0438\u043d\u0435\u044f","\u043c\u0438\u043d\u0438\u043a","\u043c\u0438\u043d\u043e\u0440","\u043c\u0438\u043d\u0443\u0441","\u043c\u0438\u043e\u043c\u0430","\u043c\u0438\u0440\u0430\u0436","\u043c\u0438\u0440\u043a\u0430","\u043c\u0438\u0440\u043e\u043a","\u043c\u0438\u0440\u0440\u0430","\u043c\u0438\u0441\u043a\u0430","\u043c\u0438\u0442\u0430\u043f","\u043c\u0438\u0442\u043e\u0437","\u043c\u0438\u0442\u0440\u0430","\u043c\u043b\u0435\u043a\u043e","\u043c\u043d\u0435\u043c\u0430","\u043c\u043e\u0430\u0440\u0435","\u043c\u043e\u0433\u0430\u0440","\u043c\u043e\u0433\u0443\u043b","\u043c\u043e\u0434\u0430\u043b","\u043c\u043e\u0434\u0435\u043c","\u043c\u043e\u0434\u0438\u0439","\u043c\u043e\u0434\u0443\u0441","\u043c\u043e\u0437\u0433\u0438","\u043c\u043e\u0438\u0437\u043c","\u043c\u043e\u0439\u043a\u0430","\u043c\u043e\u043a\u0430\u043f","\u043c\u043e\u043a\u0438\u043a","\u043c\u043e\u043a\u043a\u043e","\u043c\u043e\u043a\u0448\u0430","\u043c\u043e\u043b\u0432\u0430","\u043c\u043e\u043b\u0432\u044c","\u043c\u043e\u043b\u043a\u0430","\u043c\u043e\u043b\u043e\u0442","\u043c\u043e\u043b\u044f\u043d","\u043c\u043e\u043b\u044f\u0440","\u043c\u043e\u043c\u043c\u0435","\u043c\u043e\u043c\u043c\u044d","\u043c\u043e\u043d\u0434\u043e","\u043c\u043e\u043d\u0438\u043a","\u043c\u043e\u043d\u043e\u043c","\u043c\u043e\u043f\u0435\u0434","\u043c\u043e\u0440\u0434\u0430","\u043c\u043e\u0440\u0438\u043d","\u043c\u043e\u0440\u0438\u0442","\u043c\u043e\u0440\u0438\u044f","\u043c\u043e\u0440\u043e\u0437","\u043c\u043e\u0440\u043e\u043a","\u043c\u043e\u0440\u0444\u0430","\u043c\u043e\u0440\u0446\u043e","\u043c\u043e\u0440\u044f\u043a","\u043c\u043e\u0441\u043e\u043b","\u043c\u043e\u0442\u0435\u0442","\u043c\u043e\u0442\u0438\u0432","\u043c\u043e\u0442\u0438\u043a","\u043c\u043e\u0442\u043a\u0430","\u043c\u043e\u0442\u043d\u044f","\u043c\u043e\u0442\u043e\u043a","\u043c\u043e\u0442\u043e\u0440","\u043c\u043e\u0442\u0442\u043e","\u043c\u043e\u0444\u0435\u0442","\u043c\u043e\u0445\u0435\u0440","\u043c\u043e\u0445\u0438\u0442","\u043c\u043e\u0445\u043d\u044b","\u043c\u043e\u0447\u043a\u0430","\u043c\u043e\u0448\u043a\u0430","\u043c\u043e\u0448\u043d\u0430","\u043c\u0440\u0430\u0437\u044c","\u043c\u0440\u0435\u0436\u0430","\u043c\u0440\u0435\u0442\u044c","\u043c\u0443\u0430\u0440\u0435","\u043c\u0443\u0433\u0430\u043c","\u043c\u0443\u0434\u043d\u044f","\u043c\u0443\u0434\u0440\u0430","\u043c\u0443\u0437\u0433\u0430","\u043c\u0443\u0437\u0435\u0439","\u043c\u0443\u0437\u043b\u043e","\u043c\u0443\u0437\u043e\u043d","\u043c\u0443\u043b\u044c\u0442","\u043c\u0443\u043b\u044f\u0436","\u043c\u0443\u043c\u0438\u0451","\u043c\u0443\u043c\u0438\u044f","\u043c\u0443\u043d\u0433\u0443","\u043c\u0443\u0440\u0430\u043b","\u043c\u0443\u0440\u0438\u0442","\u043c\u0443\u0440\u043b\u043e","\u043c\u0443\u0440\u0442\u0438","\u043c\u0443\u0440\u044c\u044f","\u043c\u0443\u0441\u0430\u0442","\u043c\u0443\u0441\u043e\u0440","\u043c\u0443\u0442\u043e\u043d","\u043c\u0443\u0442\u0443\u043b","\u043c\u0443\u0444\u0442\u0430","\u043c\u0443\u0446\u0438\u043d","\u043c\u0443\u0447\u043a\u0430","\u043c\u0443\u0448\u043a\u0430","\u043c\u0448\u0430\u0440\u0430","\u043c\u0448\u0435\u043b\u044c","\u043c\u044b\u0441\u0438\u043a","\u043c\u044b\u0441\u043b\u044c","\u043c\u044b\u0441\u043e\u043a","\u043c\u044b\u0442\u043d\u044f","\u043c\u044b\u0442\u044c\u0451","\u043c\u044b\u0447\u043a\u0430","\u043c\u044b\u0448\u0435\u0439","\u043c\u044b\u0448\u043a\u0430","\u043c\u044b\u0448\u0446\u0430","\u043c\u044d\u043b\u0430\u043d","\u043c\u044d\u0440\u0438\u044f","\u043c\u044d\u0440\u043e\u043d","\u043c\u044d\u0448\u0430\u043f","\u043c\u044e\u0437\u0435\u0442","\u043c\u044e\u0437\u043b\u0435","\u043c\u044f\u043a\u0438\u0448","\u043c\u044f\u043b\u043a\u0430","\u043c\u044f\u043d\u0434\u0430","\u043c\u044f\u0441\u043a\u043e","\u043c\u044f\u0441\u0446\u043e","\u043c\u044f\u0442\u0435\u0436","\u043c\u044f\u0442\u043a\u0430","\u043c\u044f\u0442\u044c\u0451","\u043c\u044f\u0447\u0438\u043a","\u043d\u0430\u0431\u0430\u0442","\u043d\u0430\u0431\u0435\u0433","\u043d\u0430\u0431\u043a\u0430","\u043d\u0430\u0431\u043b\u0430","\u043d\u0430\u0431\u043e\u0439","\u043d\u0430\u0431\u043e\u0440","\u043d\u0430\u0432\u0430\u043b","\u043d\u0430\u0432\u0430\u0440","\u043d\u0430\u0432\u0435\u0441","\u043d\u0430\u0432\u0435\u0442","\u043d\u0430\u0432\u0438\u0439","\u043d\u0430\u0432\u0438\u0441","\u043d\u0430\u0432\u043e\u0437","\u043d\u0430\u0432\u043e\u0439","\u043d\u0430\u0432\u044b\u043a","\u043d\u0430\u0433\u0430\u043d","\u043d\u0430\u0433\u0430\u0440","\u043d\u0430\u0433\u0438\u0431","\u043d\u0430\u0433\u043e\u043d","\u043d\u0430\u0433\u0443\u043b","\u043d\u0430\u0434\u0435\u043b","\u043d\u0430\u0434\u0438\u0440","\u043d\u0430\u0434\u043e\u0439","\u043d\u0430\u0434\u043e\u043c","\u043d\u0430\u0434\u0443\u0432","\u043d\u0430\u0435\u0437\u0434","\u043d\u0430\u0436\u0438\u043c","\u043d\u0430\u0436\u0438\u043d","\u043d\u0430\u0437\u0451\u043c","\u043d\u0430\u0439\u043a\u0438","\u043d\u0430\u0439\u0440\u0430","\u043d\u0430\u043a\u0430\u0437","\u043d\u0430\u043a\u0430\u043b","\u043d\u0430\u043a\u0430\u0442","\u043d\u0430\u043a\u0438\u0434","\u043d\u0430\u043a\u043e\u043f","\u043d\u0430\u043a\u043e\u0441","\u043d\u0430\u043a\u0440\u044b","\u043d\u0430\u043b\u0435\u043f","\u043d\u0430\u043b\u0451\u0442","\u043d\u0430\u043b\u0438\u0432","\u043d\u0430\u043b\u0438\u043a","\u043d\u0430\u043b\u043e\u0433","\u043d\u0430\u043b\u043e\u0439","\u043d\u0430\u043c\u0430\u0437","\u043d\u0430\u043c\u0451\u043a","\u043d\u0430\u043c\u0451\u0442","\u043d\u0430\u043c\u0438\u043d","\u043d\u0430\u043c\u043e\u043b","\u043d\u0430\u043c\u044b\u0432","\u043d\u0430\u043d\u0433\u0430","\u043d\u0430\u043d\u043a\u0430","\u043d\u0430\u043d\u043e\u0431","\u043d\u0430\u043d\u043e\u0441","\u043d\u0430\u043f\u0430\u0441","\u043d\u0430\u043f\u0435\u0432","\u043d\u0430\u043f\u043e\u0439","\u043d\u0430\u043f\u043e\u0440","\u043d\u0430\u043f\u043f\u0430","\u043d\u0430\u0440\u0434\u044b","\u043d\u0430\u0440\u0435\u0437","\u043d\u0430\u0440\u043e\u0434","\u043d\u0430\u0440\u0442\u0430","\u043d\u0430\u0440\u0442\u044b","\u043d\u0430\u0440\u044b\u0432","\u043d\u0430\u0440\u044f\u0434","\u043d\u0430\u0441\u0430\u0434","\u043d\u0430\u0441\u043e\u0441","\u043d\u0430\u0441\u044b\u043f","\u043d\u0430\u0442\u0451\u043a","\u043d\u0430\u0442\u0451\u0441","\u043d\u0430\u0442\u044f\u0433","\u043d\u0430\u0443\u043a\u0430","\u043d\u0430\u0443\u0440\u0443","\u043d\u0430\u0444\u0442\u0430","\u043d\u0430\u0445\u0438\u044f","\u043d\u0430\u0445\u043e\u0434","\u043d\u0430\u0446\u0438\u044f","\u043d\u0430\u0447\u0430\u043b","\u043d\u0430\u0447\u0451\u0441","\u043d\u0430\u0447\u0451\u0442","\u043d\u0430\u0447\u0438\u043d","\u043d\u0430\u0447\u043e\u0441","\u043d\u0433\u0430\u043b\u0430","\u043d\u0435\u0431\u0435\u043b","\u043d\u0435\u0432\u043c\u0430","\u043d\u0435\u0432\u043e\u0434","\u043d\u0435\u0432\u0443\u0441","\u043d\u0435\u0434\u0440\u0430","\u043d\u0435\u0434\u0443\u0433","\u043d\u0435\u0439\u0438\u0442","\u043d\u0435\u043d\u0438\u044f","\u043d\u0435\u043f\u0435\u0440","\u043d\u0435\u0440\u043e\u043b","\u043d\u0435\u0443\u044e\u0442","\u043d\u0435\u0444\u0435\u0448","\u043d\u0435\u0444\u0442\u044c","\u043d\u0435\u0447\u0435\u0442","\u043d\u0438\u0432\u043a\u0430","\u043d\u0438\u0432\u043e\u0437","\u043d\u0438\u0437\u043a\u0430","\u043d\u0438\u0437\u043e\u043a","\u043d\u0438\u043a\u0430\u0431","\u043d\u0438\u043b\u0438\u0442","\u043d\u0438\u043c\u0438\u0442","\u043d\u0438\u0441\u0430\u043d","\u043d\u0438\u0442\u043a\u0430","\u043d\u0438\u0442\u043e\u043d","\u043d\u0438\u0447\u044c\u044f","\u043d\u0438\u0448\u043a\u0430","\u043d\u0438\u044f\u043c\u0430","\u043d\u043e\u0431\u043b\u044c","\u043d\u043e\u0432\u043e\u0435","\u043d\u043e\u0432\u044c\u0451","\u043d\u043e\u0434\u044c\u044f","\u043d\u043e\u0436\u0438\u043a","\u043d\u043e\u0436\u043a\u0430","\u043d\u043e\u0436\u043d\u044b","\u043d\u043e\u043b\u0438\u043a","\u043d\u043e\u043c\u0435\u043d","\u043d\u043e\u043c\u0435\u0440","\u043d\u043e\u043c\u043e\u0441","\u043d\u043e\u043d\u0435\u0442","\u043d\u043e\u0440\u0438\u0442","\u043d\u043e\u0440\u0438\u044f","\u043d\u043e\u0440\u043a\u0430","\u043d\u043e\u0440\u043c\u0430","\u043d\u043e\u0440\u043e\u0432","\u043d\u043e\u0440\u043e\u0441","\u043d\u043e\u0441\u0438\u043a","\u043d\u043e\u0441\u043a\u0430","\u043d\u043e\u0441\u043e\u043a","\u043d\u043e\u0442\u0438\u0441","\u043d\u043e\u0442\u043a\u0430","\u043d\u043e\u0447\u0432\u044b","\u043d\u043e\u0447\u043a\u0430","\u043d\u043e\u0448\u043f\u0430","\u043d\u043e\u044d\u043b\u044c","\u043d\u0440\u0430\u0432\u044b","\u043d\u0443\u0431\u0443\u043a","\u043d\u0443\u0434\u0438\u043a","\u043d\u0443\u0436\u0434\u0430","\u043d\u0443\u043b\u0438\u043a","\u043d\u0443\u043c\u0435\u043d","\u043d\u0443\u043c\u0435\u0440","\u043d\u0443\u0440\u0430\u0433","\u043d\u0443\u0442\u0440\u043e","\u043d\u044b\u043d\u043b\u0443","\u043d\u044b\u0440\u043e\u043a","\u043d\u044b\u0442\u044c\u0451","\u043d\u044b\u0447\u043a\u0430","\u043d\u044d\u043d\u0433\u043e","\u043d\u044d\u0446\u043a\u0435","\u043d\u044d\u0446\u043a\u044d","\u043d\u044e\u0430\u043d\u0441","\u043e\u0430\u0437\u0438\u0441","\u043e\u0431\u0432\u0430\u043b","\u043e\u0431\u0432\u0435\u0441","\u043e\u0431\u0432\u043e\u0434","\u043e\u0431\u0432\u043e\u0437","\u043e\u0431\u0433\u043e\u043d","\u043e\u0431\u0434\u0438\u0440","\u043e\u0431\u0434\u0443\u0432","\u043e\u0431\u0435\u0434\u044c","\u043e\u0431\u0436\u0438\u0433","\u043e\u0431\u0436\u0438\u043c","\u043e\u0431\u0436\u0438\u043d","\u043e\u0431\u0436\u043e\u0433","\u043e\u0431\u0437\u043e\u043b","\u043e\u0431\u0437\u043e\u0440","\u043e\u0431\u0438\u0434\u0430","\u043e\u0431\u043a\u0430\u0442","\u043e\u0431\u043a\u043e\u043c","\u043e\u0431\u043a\u043e\u043f","\u043e\u0431\u043a\u043e\u0441","\u043e\u0431\u043b\u0430\u043a","\u043e\u0431\u043b\u0430\u0441","\u043e\u0431\u043b\u0451\u0442","\u043e\u0431\u043b\u0438\u0432","\u043e\u0431\u043b\u0438\u043a","\u043e\u0431\u043b\u043e\u0432","\u043e\u0431\u043b\u043e\u0433","\u043e\u0431\u043b\u043e\u0439","\u043e\u0431\u043b\u043e\u043c","\u043e\u0431\u043c\u0430\u043d","\u043e\u0431\u043c\u0435\u043d","\u043e\u0431\u043c\u0435\u0440","\u043e\u0431\u043c\u0451\u0442","\u043e\u0431\u043c\u0438\u043d","\u043e\u0431\u043c\u043e\u043b","\u043e\u0431\u043c\u044b\u0432","\u043e\u0431\u043d\u0430\u043b","\u043e\u0431\u043d\u043e\u0441","\u043e\u0431\u043e\u0440\u0430","\u043e\u0431\u0440\u0430\u0437","\u043e\u0431\u0440\u0430\u0442","\u043e\u0431\u0440\u0435\u0437","\u043e\u0431\u0440\u0438\u0441","\u043e\u0431\u0440\u043e\u043a","\u043e\u0431\u0440\u043e\u043d","\u043e\u0431\u0440\u0443\u0431","\u043e\u0431\u0440\u0443\u0447","\u043e\u0431\u0440\u044b\u0432","\u043e\u0431\u0440\u044f\u0434","\u043e\u0431\u0441\u0435\u0432","\u043e\u0431\u0441\u0451\u0440","\u043e\u0431\u0443\u0432\u044c","\u043e\u0431\u0443\u0436\u0430","\u043e\u0431\u0443\u0437\u0430","\u043e\u0431\u0445\u043e\u0434","\u043e\u0431\u0449\u0430\u043a","\u043e\u0431\u0449\u0435\u0435","\u043e\u0431\u044a\u0451\u0431","\u043e\u0431\u044a\u0451\u043c","\u043e\u0431\u044b\u0441\u043a","\u043e\u0432\u043e\u0438\u0434","\u043e\u0432\u0440\u0430\u0433","\u043e\u0432\u0441\u0435\u0446","\u043e\u0432\u0441\u044e\u0433","\u043e\u0432\u0443\u043b\u044c","\u043e\u0433\u0438\u0432\u0430","\u043e\u0433\u043b\u044f\u0434","\u043e\u0433\u043e\u043d\u044c","\u043e\u0433\u0440\u0435\u0445","\u043e\u0434\u0430\u043b\u044c","\u043e\u0434\u0434\u0438\u0442","\u043e\u0434\u0451\u0436\u0430","\u043e\u0434\u0435\u043e\u043d","\u043e\u0434\u0443\u0440\u044c","\u043e\u0435\u043b\u0438\u0442","\u043e\u0436\u0438\u043d\u0430","\u043e\u0437\u0435\u0440\u043e","\u043e\u0437\u0438\u043c\u044c","\u043e\u0437\u043d\u043e\u0431","\u043e\u0438\u0434\u0438\u0439","\u043e\u0438\u0434\u0438\u044f","\u043e\u0439\u043b\u0435\u0440","\u043e\u043a\u0430\u0451\u043c","\u043e\u043a\u0435\u0430\u043d","\u043e\u043a\u0438\u0441\u044c","\u043e\u043a\u0438\u044f\u043d","\u043e\u043a\u043b\u0430\u0434","\u043e\u043a\u043b\u0438\u043a","\u043e\u043a\u043e\u0432\u044b","\u043e\u043a\u043e\u0451\u043c","\u043e\u043a\u043e\u0440\u043c","\u043e\u043a\u0440\u0430\u0441","\u043e\u043a\u0440\u0438\u043a","\u043e\u043a\u0440\u043e\u043b","\u043e\u043a\u0440\u0443\u0433","\u043e\u043a\u0441\u0438\u0434","\u043e\u043a\u0441\u0438\u043c","\u043e\u043a\u0441\u0438\u044f","\u043e\u043a\u0442\u0430\u043b","\u043e\u043a\u0442\u0430\u043d","\u043e\u043a\u0442\u0435\u043d","\u043e\u043a\u0442\u0435\u0442","\u043e\u043a\u0442\u0438\u043b","\u043e\u043b\u0435\u0430\u0442","\u043e\u043b\u0435\u0438\u043d","\u043e\u043b\u0435\u0443\u043c","\u043e\u043b\u0438\u0432\u0430","\u043e\u043b\u0438\u043c\u043f","\u043e\u043b\u0438\u0444\u0430","\u043e\u043b\u043e\u0432\u043e","\u043e\u043b\u0442\u0430\u0440","\u043e\u043b\u044c\u0445\u0430","\u043e\u043c\u0430\u0445\u0430","\u043e\u043c\u0435\u0433\u0430","\u043e\u043c\u0435\u0436\u044c","\u043e\u043c\u0435\u043b\u0430","\u043e\u043c\u043b\u0435\u0442","\u043e\u043c\u043c\u0430\u0436","\u043e\u043c\u0440\u0430\u043a","\u043e\u043c\u0444\u0430\u043b","\u043e\u043d\u0430\u0433\u0440","\u043e\u043d\u0438\u043a\u0441","\u043e\u043d\u0441\u044d\u043d","\u043e\u043d\u0443\u0447\u0430","\u043e\u043d\u0448\u043e\u0440","\u043e\u043e\u043b\u0438\u0442","\u043e\u043e\u0446\u0438\u0442","\u043e\u043f\u0430\u043b\u0430","\u043e\u043f\u0430\u043b\u044c","\u043e\u043f\u0430\u0440\u0430","\u043e\u043f\u0435\u043a\u0430","\u043e\u043f\u0435\u043b\u044c","\u043e\u043f\u0435\u0440\u0430","\u043e\u043f\u0438\u0430\u043d","\u043e\u043f\u0438\u0430\u0442","\u043e\u043f\u0438\u0441\u044c","\u043e\u043f\u0438\u0443\u043c","\u043e\u043f\u043b\u043e\u0442","\u043e\u043f\u043b\u044b\u0432","\u043e\u043f\u043e\u0435\u043a","\u043e\u043f\u043e\u043a\u0430","\u043e\u043f\u043e\u043d\u0430","\u043e\u043f\u043e\u0440\u0430","\u043e\u043f\u0440\u043e\u0441","\u043e\u043f\u0445\u043e\u0437","\u043e\u043f\u0446\u0438\u044f","\u043e\u0440\u0430\u0432\u0430","\u043e\u0440\u0430\u043b\u0430","\u043e\u0440\u0430\u043b\u0435","\u043e\u0440\u0430\u043b\u043e","\u043e\u0440\u0430\u0440\u044c","\u043e\u0440\u0433\u0430\u043d","\u043e\u0440\u0433\u0438\u044f","\u043e\u0440\u0434\u0435\u043d","\u043e\u0440\u0434\u0435\u0440","\u043e\u0440\u0435\u0432\u043e","\u043e\u0440\u0435\u043e\u043b","\u043e\u0440\u0436\u0430\u0434","\u043e\u0440\u0438\u0441\u0441","\u043e\u0440\u043a\u0430\u043d","\u043e\u0440\u043b\u0435\u0446","\u043e\u0440\u043b\u043e\u043d","\u043e\u0440\u043b\u044f\u043a","\u043e\u0440\u0441\u0438\u043d","\u043e\u0440\u0442\u0435\u0437","\u043e\u0440\u0442\u0435\u0442","\u043e\u0440\u0442\u0438\u0442","\u043e\u0440\u0445\u0438\u0442","\u043e\u0440\u0448\u0430\u0434","\u043e\u0441\u0430\u0434\u0430","\u043e\u0441\u0435\u043d\u044c","\u043e\u0441\u0438\u043d\u0430","\u043e\u0441\u043a\u0430\u043b","\u043e\u0441\u043b\u043e\u043f","\u043e\u0441\u043c\u0438\u0439","\u043e\u0441\u043c\u043e\u043b","\u043e\u0441\u043c\u043e\u0441","\u043e\u0441\u043e\u043a\u0430","\u043e\u0441\u0442\u0430\u0432","\u043e\u0441\u0442\u0430\u043d","\u043e\u0441\u0442\u0438\u043d","\u043e\u0441\u0442\u0438\u0442","\u043e\u0441\u0442\u043e\u0432","\u043e\u0441\u0442\u043e\u043b","\u043e\u0441\u0442\u044e\u043a","\u043e\u0441\u044b\u043f\u044c","\u043e\u0442\u0430\u0432\u0430","\u043e\u0442\u0430\u0440\u0430","\u043e\u0442\u0431\u0435\u0433","\u043e\u0442\u0431\u0438\u0432","\u043e\u0442\u0431\u043e\u0439","\u043e\u0442\u0431\u043e\u0440","\u043e\u0442\u0432\u0430\u043b","\u043e\u0442\u0432\u0430\u0440","\u043e\u0442\u0432\u0435\u0441","\u043e\u0442\u0432\u0435\u0442","\u043e\u0442\u0432\u043e\u0434","\u043e\u0442\u0432\u043e\u0437","\u043e\u0442\u0433\u0438\u0431","\u043e\u0442\u0433\u043e\u043d","\u043e\u0442\u0433\u0443\u043b","\u043e\u0442\u0434\u0435\u043b","\u043e\u0442\u0434\u0443\u0445","\u043e\u0442\u0434\u044b\u0445","\u043e\u0442\u0435\u043b\u044c","\u043e\u0442\u0436\u0438\u0433","\u043e\u0442\u0436\u0438\u043c","\u043e\u0442\u0437\u043e\u043b","\u043e\u0442\u0437\u044b\u0432","\u043e\u0442\u043a\u0430\u0437","\u043e\u0442\u043a\u0430\u0442","\u043e\u0442\u043a\u043e\u043b","\u043e\u0442\u043a\u043e\u0441","\u043e\u0442\u043a\u0443\u043f","\u043e\u0442\u043a\u0443\u0441","\u043e\u0442\u043b\u0430\u0441","\u043e\u0442\u043b\u0451\u0442","\u043e\u0442\u043b\u0438\u0432","\u043e\u0442\u043b\u0438\u043f","\u043e\u0442\u043b\u043e\u0432","\u043e\u0442\u043b\u043e\u0433","\u043e\u0442\u043b\u043e\u043c","\u043e\u0442\u043b\u0443\u043f","\u043e\u0442\u043c\u0430\u0437","\u043e\u0442\u043c\u0430\u0445","\u043e\u0442\u043c\u0435\u0440","\u043e\u0442\u043c\u044b\u0432","\u043e\u0442\u043d\u043e\u0441","\u043e\u0442\u043f\u0430\u0434","\u043e\u0442\u043f\u0430\u043b","\u043e\u0442\u043f\u043e\u0440","\u043e\u0442\u0440\u0435\u0437","\u043e\u0442\u0440\u043e\u0433","\u043e\u0442\u0440\u0443\u0431","\u043e\u0442\u0440\u044b\u0432","\u043e\u0442\u0440\u044f\u0434","\u043e\u0442\u0441\u0435\u0432","\u043e\u0442\u0441\u0435\u043a","\u043e\u0442\u0441\u043e\u0441","\u043e\u0442\u0441\u044b\u043b","\u043e\u0442\u0441\u044b\u043f","\u043e\u0442\u0442\u0451\u043a","\u043e\u0442\u0442\u043e\u043a","\u043e\u0442\u0442\u044f\u0433","\u043e\u0442\u0445\u043e\u0434","\u043e\u0442\u0446\u0435\u043f","\u043e\u0442\u0447\u0430\u043b","\u043e\u0442\u0447\u0451\u0442","\u043e\u0442\u0448\u0438\u0431","\u043e\u0442\u0449\u0435\u043f","\u043e\u0442\u044a\u0451\u043c","\u043e\u0444\u043e\u0440\u0442","\u043e\u0444\u0441\u0435\u0442","\u043e\u0444\u0443\u0440\u043e","\u043e\u0444\u0444\u0435\u0440","\u043e\u0444\u0448\u043e\u0440","\u043e\u0445\u0432\u0430\u0442","\u043e\u0445\u043b\u043e\u0441","\u043e\u0445\u043b\u0443\u043f","\u043e\u0445\u043e\u0442\u0430","\u043e\u0447\u0435\u0440\u043a","\u043e\u0447\u043a\u0443\u0440","\u043e\u0448\u0435\u0435\u043a","\u043e\u0449\u0443\u043f\u044c","\u043f\u0430\u0433\u043e\u043d","\u043f\u0430\u0434\u0434\u043b","\u043f\u0430\u0434\u0435\u0436","\u043f\u0430\u0434\u0451\u0436","\u043f\u0430\u0434\u0438\u043a","\u043f\u0430\u0434\u043b\u043e","\u043f\u0430\u0434\u043c\u0430","\u043f\u0430\u0434\u043e\u0433","\u043f\u0430\u0434\u0440\u0430","\u043f\u0430\u0434\u0443\u0431","\u043f\u0430\u0434\u0443\u043d","\u043f\u0430\u0437\u0438\u043a","\u043f\u0430\u0437\u043e\u043a","\u043f\u0430\u0439\u0432\u0430","\u043f\u0430\u0439\u0437\u0430","\u043f\u0430\u0439\u043a\u0430","\u043f\u0430\u0439\u043e\u043b","\u043f\u0430\u0439\u0441\u0430","\u043f\u0430\u0439\u0442\u0430","\u043f\u0430\u043a\u0435\u0440","\u043f\u0430\u043a\u0435\u0442","\u043f\u0430\u043a\u043b\u044f","\u043f\u0430\u043a\u0448\u0430","\u043f\u0430\u043b\u0430\u0441","\u043f\u0430\u043b\u0430\u0443","\u043f\u0430\u043b\u0430\u0446","\u043f\u0430\u043b\u0430\u0448","\u043f\u0430\u043b\u0435\u0445","\u043f\u0430\u043b\u0435\u0446","\u043f\u0430\u043b\u0435\u044f","\u043f\u0430\u043b\u0438\u0439","\u043f\u0430\u043b\u043a\u0430","\u043f\u0430\u043b\u0443\u0431","\u043f\u0430\u043c\u043f\u0430","\u043f\u0430\u043d\u0430\u0448","\u043f\u0430\u043d\u0434\u0430","\u043f\u0430\u043d\u0451\u0440","\u043f\u0430\u043d\u043d\u0430","\u043f\u0430\u043d\u043d\u043e","\u043f\u0430\u043d\u0442\u044b","\u043f\u0430\u043d\u044c\u0435","\u043f\u0430\u043f\u043a\u0430","\u043f\u0430\u0440\u0430\u0434","\u043f\u0430\u0440\u0430\u0444","\u043f\u0430\u0440\u0435\u0437","\u043f\u0430\u0440\u0435\u043e","\u043f\u0430\u0440\u0438\u043a","\u043f\u0430\u0440\u043a\u0430","\u043f\u0430\u0440\u043c\u0430","\u043f\u0430\u0440\u043e\u0434","\u043f\u0430\u0440\u043e\u043a","\u043f\u0430\u0440\u043e\u043c","\u043f\u0430\u0440\u0442\u0430","\u043f\u0430\u0440\u0442\u0438","\u043f\u0430\u0440\u0443\u043d","\u043f\u0430\u0440\u0443\u0441","\u043f\u0430\u0440\u0443\u0448","\u043f\u0430\u0440\u0444\u0435","\u043f\u0430\u0440\u0447\u0430","\u043f\u0430\u0440\u0448\u0430","\u043f\u0430\u0440\u0448\u044c","\u043f\u0430\u0440\u044c\u044f","\u043f\u0430\u0441\u0438\u043a","\u043f\u0430\u0441\u043a\u0430","\u043f\u0430\u0441\u043c\u0430","\u043f\u0430\u0441\u043c\u043e","\u043f\u0430\u0441\u043e\u043a","\u043f\u0430\u0441\u0442\u0430","\u043f\u0430\u0441\u0442\u044c","\u043f\u0430\u0441\u0445\u0430","\u043f\u0430\u0442\u0430\u0448","\u043f\u0430\u0442\u0438\u043e","\u043f\u0430\u0442\u043a\u0430","\u043f\u0430\u0442\u043b\u044b","\u043f\u0430\u0443\u0437\u0430","\u043f\u0430\u0443\u043d\u0434","\u043f\u0430\u0444\u043e\u0441","\u043f\u0430\u0445\u0432\u0438","\u043f\u0430\u0445\u0432\u044b","\u043f\u0430\u0445\u0438\u0442","\u043f\u0430\u0445\u0442\u0430","\u043f\u0430\u0447\u043a\u0430","\u043f\u0430\u0448\u043d\u044f","\u043f\u0430\u0448\u043e\u0442","\u043f\u0435\u0434\u0438\u043a","\u043f\u0435\u0439\u043e\u0442","\u043f\u0435\u043a\u0430\u043d","\u043f\u0435\u043a\u043b\u043e","\u043f\u0451\u043a\u043b\u043e","\u043f\u0435\u043b\u0438\u0442","\u043f\u0435\u043c\u0437\u0430","\u043f\u0435\u043d\u0430\u043b","\u043f\u0435\u043d\u0433\u0451","\u043f\u0435\u043d\u0433\u043e","\u043f\u0435\u043d\u0451\u043a","\u043f\u0435\u043d\u0438\u0435","\u043f\u0435\u043d\u0438\u0441","\u043f\u0435\u043d\u043a\u0430","\u043f\u0435\u043d\u043d\u0435","\u043f\u0435\u043d\u043d\u0438","\u043f\u0435\u043d\u044c\u0435","\u043f\u0435\u043f\u0435\u043b","\u043f\u0435\u043f\u0441\u0438","\u043f\u0435\u0440\u0433\u0430","\u043f\u0435\u0440\u0434\u044c","\u043f\u0435\u0440\u0435\u0434","\u043f\u0435\u0440\u0451\u0434","\u043f\u0435\u0440\u0435\u0446","\u043f\u0435\u0440\u0438\u0442","\u043f\u0451\u0440\u043a\u0430","\u043f\u0435\u0440\u043c\u044c","\u043f\u0435\u0440\u0440\u0438","\u043f\u0435\u0440\u0441\u0438","\u043f\u0435\u0440\u0441\u0442","\u043f\u0435\u0440\u0443\u043d","\u043f\u0435\u0440\u044c\u0435","\u043f\u0435\u0441\u0435\u0446","\u043f\u0435\u0441\u043d\u044c","\u043f\u0435\u0441\u043d\u044f","\u043f\u0435\u0441\u043e\u043a","\u043f\u0435\u0441\u0442\u043e","\u043f\u0435\u0442\u0438\u0435","\u043f\u0435\u0442\u0438\u0442","\u043f\u0435\u0442\u043b\u044f","\u043f\u0435\u0442\u044c\u0435","\u043f\u0435\u0447\u0438\u0442","\u043f\u0435\u0447\u043a\u0430","\u043f\u0435\u0448\u043a\u0430","\u043f\u0435\u0448\u043d\u044f","\u043f\u0435\u0449\u0435\u0440","\u043f\u0435\u0449\u0443\u0440","\u043f\u0438\u0430\u043b\u0430","\u043f\u0438\u0430\u043d\u043e","\u043f\u0438\u0432\u0430\u0441","\u043f\u0438\u0432\u043a\u043e","\u043f\u0438\u0432\u0446\u043e","\u043f\u0438\u0433\u0443\u0441","\u043f\u0438\u0436\u043c\u0430","\u043f\u0438\u0436\u043d\u044f","\u043f\u0438\u0437\u0434\u0430","\u043f\u0438\u043a\u0430\u043d","\u043f\u0438\u043a\u0430\u043f","\u043f\u0438\u043a\u0435\u0442","\u043f\u0438\u043a\u043e\u043d","\u043f\u0438\u043a\u0441\u0438","\u043f\u0438\u043a\u0447\u0430","\u043f\u0438\u043b\u0430\u0432","\u043f\u0438\u043b\u043a\u0430","\u043f\u0438\u043b\u043e\u043d","\u043f\u0438\u043b\u043e\u0442","\u043f\u0438\u043b\u0443\u043c","\u043f\u0438\u043b\u0443\u0441","\u043f\u0438\u043c\u043f\u0430","\u043f\u0438\u043d\u0435\u043d","\u043f\u0438\u043d\u0438\u044f","\u043f\u0438\u043d\u043a\u0430","\u043f\u0438\u043d\u043d\u0430","\u043f\u0438\u043d\u043e\u043a","\u043f\u0438\u043d\u0442\u0430","\u043f\u0438\u043f\u0435\u0446","\u043f\u0438\u043f\u0438\u0441","\u043f\u0438\u043f\u043a\u0430","\u043f\u0438\u0440\u0430\u043d","\u043f\u0438\u0440\u0438\u0442","\u043f\u0438\u0440\u043a\u0435","\u043f\u0438\u0440\u043e\u0433","\u043f\u0438\u0440\u043e\u043a","\u043f\u0438\u0440\u043e\u043d","\u043f\u0438\u0440\u043e\u043f","\u043f\u0438\u0441\u043a\u043e","\u043f\u0438\u0441\u044e\u043a","\u043f\u0438\u0441\u044e\u043d","\u043f\u0438\u0442\u0438\u0435","\u043f\u0438\u0442\u043e\u043d","\u043f\u0438\u0442\u044c\u0451","\u043f\u0438\u0444\u043e\u0441","\u043f\u0438\u0445\u0442\u0430","\u043f\u0438\u0446\u0446\u0430","\u043f\u0438\u0447\u043e\u043a","\u043f\u0438\u0449\u0438\u043a","\u043f\u043b\u0430\u0432\u044c","\u043f\u043b\u0430\u0437\u0430","\u043f\u043b\u0430\u043c\u044f","\u043f\u043b\u0430\u0441\u0442","\u043f\u043b\u0430\u0442\u0430","\u043f\u043b\u0430\u0442\u043e","\u043f\u043b\u0430\u0443\u043d","\u043f\u043b\u0430\u0445\u0430","\u043f\u043b\u0435\u0431\u0441","\u043f\u043b\u0435\u0432\u0430","\u043f\u043b\u0435\u0435\u0440","\u043f\u043b\u0435\u0439\u0442","\u043f\u043b\u0435\u043c\u044f","\u043f\u043b\u0435\u043d\u0430","\u043f\u043b\u0435\u0441\u043a","\u043f\u043b\u0451\u0441\u043e","\u043f\u043b\u0435\u0442\u044c","\u043f\u043b\u0435\u0447\u043e","\u043f\u043b\u0435\u0448\u044c","\u043f\u043b\u0438\u043d\u0442","\u043f\u043b\u0438\u0442\u0430","\u043f\u043b\u0438\u0446\u0430","\u043f\u043b\u043e\u043a\u0430","\u043f\u043b\u043e\u0442\u044c","\u043f\u043b\u044e\u0445\u0430","\u043f\u043d\u0438\u0449\u0435","\u043f\u043e\u0431\u0435\u0433","\u043f\u043e\u0431\u043e\u0438","\u043f\u043e\u0431\u043e\u0440","\u043f\u043e\u0431\u044b\u0442","\u043f\u043e\u0432\u0430\u043b","\u043f\u043e\u0432\u0435\u0440","\u043f\u043e\u0432\u0435\u0442","\u043f\u043e\u0432\u043e\u0434","\u043f\u043e\u0432\u043e\u0437","\u043f\u043e\u0432\u043e\u0439","\u043f\u043e\u0432\u044f\u0442","\u043f\u043e\u0433\u0438\u0431","\u043f\u043e\u0433\u043e\u043d","\u043f\u043e\u0434\u0435\u043b","\u043f\u043e\u0434\u0438\u0439","\u043f\u043e\u0434\u043e\u0433","\u043f\u043e\u0434\u043e\u0439","\u043f\u043e\u0434\u043e\u043a","\u043f\u043e\u0434\u043e\u043b","\u043f\u043e\u0435\u0434\u044c","\u043f\u043e\u0435\u0437\u0434","\u043f\u043e\u0436\u0430\u0440","\u043f\u043e\u0436\u043d\u044f","\u043f\u043e\u0436\u043e\u0433","\u043f\u043e\u0437\u0451\u043c","\u043f\u043e\u0437\u043e\u0440","\u043f\u043e\u0437\u044b\u0432","\u043f\u043e\u0438\u0441\u043a","\u043f\u043e\u0439\u043a\u0430","\u043f\u043e\u0439\u043b\u043e","\u043f\u043e\u0439\u043c\u0430","\u043f\u043e\u043a\u0430\u0437","\u043f\u043e\u043a\u0430\u043b","\u043f\u043e\u043a\u0430\u0442","\u043f\u043e\u043a\u0435\u0440","\u043f\u043e\u043a\u043e\u0438","\u043f\u043e\u043a\u043e\u0439","\u043f\u043e\u043a\u043e\u043b","\u043f\u043e\u043a\u043e\u043d","\u043f\u043e\u043a\u043e\u0440","\u043f\u043e\u043a\u043e\u0441","\u043f\u043e\u043a\u0443\u0441","\u043f\u043e\u043b\u0431\u0430","\u043f\u043e\u043b\u0451\u0442","\u043f\u043e\u043b\u0438\u0432","\u043f\u043e\u043b\u0438\u0441","\u043f\u043e\u043b\u043a\u0430","\u043f\u043e\u043b\u043e\u0433","\u043f\u043e\u043b\u043e\u0437","\u043f\u043e\u043b\u043e\u0439","\u043f\u043e\u043b\u043e\u043a","\u043f\u043e\u043b\u043e\u043c","\u043f\u043e\u043b\u043e\u043d","\u043f\u043e\u043b\u0442\u044c","\u043f\u043e\u043b\u044e\u0441","\u043f\u043e\u043b\u044f\u0440","\u043f\u043e\u043c\u0451\u0442","\u043f\u043e\u043c\u0438\u043d","\u043f\u043e\u043c\u043e\u0438","\u043f\u043e\u043c\u043e\u043b","\u043f\u043e\u043c\u043f\u0430","\u043f\u043e\u043d\u043e\u0440","\u043f\u043e\u043d\u043e\u0441","\u043f\u043e\u043d\u0443\u0440","\u043f\u043e\u043d\u0447\u043e","\u043f\u043e\u043d\u044e\u0445","\u043f\u043e\u043f\u0430\u0441","\u043f\u043e\u043f\u0435\u0446","\u043f\u043e\u043f\u043a\u0430","\u043f\u043e\u043f\u0441\u0430","\u043f\u043e\u043f\u044b\u0442","\u043f\u043e\u0440\u0435\u0437","\u043f\u043e\u0440\u0435\u0439","\u043f\u043e\u0440\u043a\u0430","\u043f\u043e\u0440\u043d\u043e","\u043f\u043e\u0440\u043e\u0433","\u043f\u043e\u0440\u043e\u043a","\u043f\u043e\u0440\u043e\u0445","\u043f\u043e\u0440\u0441\u043a","\u043f\u043e\u0440\u0442\u043e","\u043f\u043e\u0440\u0442\u044b","\u043f\u043e\u0440\u0443\u0431","\u043f\u043e\u0440\u0443\u0448","\u043f\u043e\u0440\u0447\u0430","\u043f\u043e\u0440\u0448\u0435","\u043f\u043e\u0440\u044b\u0432","\u043f\u043e\u0441\u0430\u0434","\u043f\u043e\u0441\u0435\u0432","\u043f\u043e\u0441\u043e\u043b","\u043f\u043e\u0441\u043e\u0445","\u043f\u043e\u0441\u0443\u043b","\u043f\u043e\u0441\u044b\u043b","\u043f\u043e\u0442\u0430\u0448","\u043f\u043e\u0442\u0451\u043a","\u043f\u043e\u0442\u0438\u043a","\u043f\u043e\u0442\u0438\u0440","\u043f\u043e\u0442\u043e\u043a","\u043f\u043e\u0442\u043e\u043f","\u043f\u043e\u0442\u044f\u0433","\u043f\u043e\u0445\u043e\u0434","\u043f\u043e\u0447\u0430\u043d","\u043f\u043e\u0447\u0432\u0430","\u043f\u043e\u0447\u0451\u0442","\u043f\u043e\u0447\u0438\u043d","\u043f\u043e\u0447\u043a\u0430","\u043f\u043e\u0447\u0442\u0430","\u043f\u043e\u0448\u0435\u0442","\u043f\u043e\u0448\u0438\u0431","\u043f\u043e\u0448\u0438\u0432","\u043f\u043e\u044d\u0437\u0430","\u043f\u043e\u044d\u043c\u0430","\u043f\u0440\u0430\u0432\u043e","\u043f\u0440\u0430\u0439\u0434","\u043f\u0440\u0430\u0439\u043c","\u043f\u0440\u0430\u0439\u0441","\u043f\u0440\u0430\u043d\u0430","\u043f\u0440\u0430\u043d\u043a","\u043f\u0440\u0430\u0449\u0430","\u043f\u0440\u0435\u0437\u0430","\u043f\u0440\u0435\u043b\u044c","\u043f\u0440\u0435\u043e\u043d","\u043f\u0440\u0435\u0441\u0441","\u043f\u0440\u0438\u0451\u043c","\u043f\u0440\u0438\u043c\u0430","\u043f\u0440\u0438\u043d\u0442","\u043f\u0440\u0438\u043e\u043d","\u043f\u0440\u0438\u0443\u0437","\u043f\u0440\u0438\u0447\u0442","\u043f\u0440\u0438\u044e\u0442","\u043f\u0440\u043e\u0431\u0430","\u043f\u0440\u043e\u0432\u043e","\u043f\u0440\u043e\u0433\u0430","\u043f\u0440\u043e\u0434\u0430","\u043f\u0440\u043e\u0451\u0431","\u043f\u0440\u043e\u0451\u043c","\u043f\u0440\u043e\u0437\u0430","\u043f\u0440\u043e\u043c\u043e","\u043f\u0440\u043e\u043f\u0441","\u043f\u0440\u043e\u0441\u043e","\u043f\u0440\u043e\u0443\u0445","\u043f\u0440\u0443\u0445\u0430","\u043f\u0440\u0448\u0443\u0442","\u043f\u0440\u044b\u0441\u043a","\u043f\u0440\u044b\u0442\u044c","\u043f\u0440\u044f\u0434\u044c","\u043f\u0440\u044f\u0436\u0430","\u043f\u0441\u0438\u043b\u0438","\u043f\u0441\u0438\u043d\u0430","\u043f\u0441\u0438\u043e\u043d","\u043f\u0441\u0438\u0448\u0435","\u043f\u0441\u043e\u0440\u0430","\u043f\u0443\u0430\u043d\u0442","\u043f\u0443\u0430\u0440\u0435","\u043f\u0443\u0433\u0430\u0447","\u043f\u0443\u0434\u0438\u043a","\u043f\u0443\u0434\u0440\u0430","\u043f\u0443\u0437\u0446\u043e","\u043f\u0443\u043a\u0430\u043d","\u043f\u0443\u043a\u043b\u044f","\u043f\u0443\u043b\u0435\u043d","\u043f\u0443\u043b\u044c\u0441","\u043f\u0443\u043b\u044c\u0442","\u043f\u0443\u043d\u0435\u0446","\u043f\u0443\u043d\u043a\u0442","\u043f\u0443\u043f\u043e\u043a","\u043f\u0443\u0440\u0433\u0430","\u043f\u0443\u0440\u0434\u0430","\u043f\u0443\u0440\u0438\u043d","\u043f\u0443\u0440\u043a\u0430","\u043f\u0443\u0441\u0435\u0442","\u043f\u0443\u0442\u0435\u0446","\u043f\u0443\u0442\u0438\u043a","\u043f\u0443\u0442\u0438\u043d","\u043f\u0443\u0442\u043b\u043e","\u043f\u0443\u0442\u0442\u0438","\u043f\u0443\u0444\u0438\u043a","\u043f\u0443\u0446\u0438\u043d","\u043f\u0443\u0447\u043e\u043a","\u043f\u0443\u0448\u043a\u0430","\u043f\u0443\u0448\u043e\u043a","\u043f\u0443\u0448\u0442\u0443","\u043f\u0448\u0435\u043d\u043e","\u043f\u044b\u0436\u0438\u043a","\u043f\u044b\u0440\u0435\u0439","\u043f\u044b\u0440\u043a\u0430","\u043f\u044b\u0442\u043a\u0430","\u043f\u044b\u0448\u043a\u0430","\u043f\u044c\u0435\u0437\u0430","\u043f\u044c\u0435\u0441\u0430","\u043f\u044c\u044f\u043d\u044c","\u043f\u044f\u043b\u043a\u0430","\u043f\u044f\u043d\u0441\u0435","\u043f\u044f\u0441\u0442\u044c","\u043f\u044f\u0442\u0430\u043a","\u043f\u044f\u0442\u043a\u0430","\u043f\u044f\u0442\u043d\u043e","\u043f\u044f\u0442\u043e\u043a","\u0440\u0430\u0431\u0430\u0442","\u0440\u0430\u0433\u0433\u0430","\u0440\u0430\u0434\u0430\u0440","\u0440\u0430\u0434\u0438\u0439","\u0440\u0430\u0434\u0438\u043e","\u0440\u0430\u0434\u043e\u043d","\u0440\u0430\u0437\u0435\u0440","\u0440\u0430\u0437\u0438\u043a","\u0440\u0430\u0437\u043e\u043a","\u0440\u0430\u0437\u043e\u0440","\u0440\u0430\u0437\u0443\u043c","\u0440\u0430\u0438\u043d\u0430","\u0440\u0430\u0439\u0434\u0430","\u0440\u0430\u0439\u0437\u043e","\u0440\u0430\u0439\u043e\u043d","\u0440\u0430\u0439\u0442\u0430","\u0440\u0430\u0439\u0444\u043e","\u0440\u0430\u043a\u0438\u044f","\u0440\u0430\u043a\u043b\u044f","\u0440\u0430\u043b\u043b\u0438","\u0440\u0430\u043c\u0435\u043d","\u0440\u0430\u043c\u0435\u0442","\u0440\u0430\u043c\u043a\u0430","\u0440\u0430\u043c\u043f\u0430","\u0440\u0430\u043c\u044d\u043d","\u0440\u0430\u043d\u0435\u0442","\u0440\u0430\u043d\u0435\u0446","\u0440\u0430\u043d\u043a\u0430","\u0440\u0430\u043d\u0447\u043e","\u0440\u0430\u043d\u044c\u0435","\u0440\u0430\u043d\u044c\u0451","\u0440\u0430\u043f\u0438\u0434","\u0440\u0430\u0441\u0442\u0440","\u0440\u0430\u0442\u0438\u043d","\u0440\u0430\u0442\u0438\u0442","\u0440\u0430\u0442\u0442\u0438","\u0440\u0430\u0443\u043d\u0434","\u0440\u0430\u0444\u0438\u0434","\u0440\u0430\u0444\u0438\u043a","\u0440\u0430\u0444\u0438\u044f","\u0440\u0430\u0445\u0438\u0441","\u0440\u0430\u0445\u0438\u0442","\u0440\u0430\u0446\u0435\u044f","\u0440\u0430\u0446\u0438\u043e","\u0440\u0430\u0446\u0438\u044f","\u0440\u0430\u0447\u043d\u044f","\u0440\u0432\u0430\u043d\u044c","\u0440\u0432\u043e\u0442\u0430","\u0440\u0434\u0435\u0441\u0442","\u0440\u0435\u0431\u0430\u0431","\u0440\u0435\u0431\u0430\u0439","\u0440\u0435\u0431\u0435\u043a","\u0440\u0435\u0431\u0440\u043e","\u0440\u0435\u0431\u0443\u0441","\u0440\u0435\u0431\u0443\u0442","\u0440\u0435\u0432\u0438\u0440","\u0440\u0451\u0432\u043a\u0430","\u0440\u0435\u0432\u0443\u043d","\u0440\u0435\u0432\u044c\u044e","\u0440\u0435\u0433\u0430\u043d","\u0440\u0435\u0433\u0430\u0442","\u0440\u0435\u0433\u0431\u0438","\u0440\u0435\u0433\u0433\u0438","\u0440\u0435\u0433\u043e\u0442","\u0440\u0435\u0434\u0430\u043d","\u0440\u0435\u0434\u0438\u0441","\u0440\u0435\u0434\u0438\u0444","\u0440\u0435\u0434\u0438\u044f","\u0440\u0435\u0434\u0443\u0442","\u0440\u0435\u0436\u0438\u043a","\u0440\u0435\u0436\u0438\u043c","\u0440\u0435\u0437\u0430\u043a","\u0440\u0435\u0437\u0435\u043d","\u0440\u0435\u0437\u0435\u0446","\u0440\u0435\u0437\u0438\u0442","\u0440\u0435\u0437\u043a\u0430","\u0440\u0435\u0437\u043d\u044f","\u0440\u0435\u0437\u043e\u043b","\u0440\u0435\u0437\u043e\u043d","\u0440\u0435\u0437\u0443\u0441","\u0440\u0435\u0439\u0434\u0430","\u0440\u0435\u0439\u043a\u0430","\u0440\u0435\u0439\u043a\u0438","\u0440\u0435\u043a\u0430\u043f","\u0440\u0435\u043a\u043e\u043d","\u0440\u0435\u043a\u0442\u043e","\u0440\u0435\u043b\u0438\u0437","\u0440\u0435\u043b\u0438\u043d","\u0440\u0451\u043b\u043a\u0430","\u0440\u0435\u043b\u043e\u043d","\u0440\u0435\u043b\u044c\u0441","\u0440\u0435\u043c\u0435\u0448","\u0440\u0435\u043c\u0438\u0437","\u0440\u0435\u043d\u0430\u0442","\u0440\u0435\u043d\u0435\u0442","\u0440\u0435\u043d\u0438\u0439","\u0440\u0435\u043d\u0438\u043d","\u0440\u0451\u043d\u0438\u0442","\u0440\u0435\u043d\u0442\u0430","\u0440\u0435\u043f\u0435\u0439","\u0440\u0435\u043f\u0435\u0440","\u0440\u0435\u043f\u0438\u0442","\u0440\u0435\u043f\u043a\u0430","\u0440\u0435\u043f\u044c\u0451","\u0440\u0435\u0442\u0440\u043e","\u0440\u0435\u0442\u044d\u0448","\u0440\u0435\u0445\u0430\u0431","\u0440\u0435\u0447\u043a\u0430","\u0440\u0435\u0448\u043a\u0430","\u0440\u0451\u0448\u0442\u0438","\u0440\u0436\u0430\u0432\u044c","\u0440\u0436\u0430\u043a\u0430","\u0440\u0436\u0438\u0449\u0435","\u0440\u0438\u0431\u0430\u0439","\u0440\u0438\u0431\u043e\u043f","\u0440\u0438\u0432\u0435\u0440","\u0440\u0438\u0433\u0438\u0434","\u0440\u0438\u0434\u0435\u0440","\u0440\u0438\u0435\u043b\u044c","\u0440\u0438\u0437\u043a\u0430","\u0440\u0438\u043b\u0438\u043d","\u0440\u0438\u043c\u0435\u0440","\u0440\u0438\u043d\u0434\u0430","\u0440\u0438\u043d\u0438\u0442","\u0440\u0438\u0441\u043a\u0430","\u0440\u0438\u0442\u043e\u043d","\u0440\u0438\u0444\u0435\u0439","\u0440\u0438\u0444\u043b\u044f","\u0440\u0438\u0444\u043c\u0430","\u0440\u0438\u0446\u0438\u043d","\u0440\u043e\u0432\u0435\u0440","\u0440\u043e\u0432\u0438\u043a","\u0440\u043e\u0433\u043e\u0437","\u0440\u043e\u0433\u043e\u0440","\u0440\u043e\u0434\u0430\u043d","\u0440\u043e\u0434\u0435\u043e","\u0440\u043e\u0434\u0435\u0440","\u0440\u043e\u0434\u0438\u0439","\u0440\u043e\u0434\u043d\u044f","\u0440\u043e\u0436\u043a\u0438","\u0440\u043e\u0436\u043d\u043e","\u0440\u043e\u0436\u043e\u043a","\u0440\u043e\u0436\u043e\u043d","\u0440\u043e\u0437\u0430\u043d","\u0440\u043e\u0437\u0433\u0430","\u0440\u043e\u0437\u043d\u044c","\u0440\u043e\u0439\u0431\u0430","\u0440\u043e\u0439\u043a\u0430","\u0440\u043e\u043a\u043e\u0442","\u0440\u043e\u043b\u0438\u043a","\u0440\u043e\u043c\u0430\u043d","\u0440\u043e\u043c\u0435\u0446","\u0440\u043e\u043c\u043e\u043a","\u0440\u043e\u043c\u0448\u0430","\u0440\u043e\u043d\u0434\u043e","\u0440\u043e\u043f\u0430\u043a","\u0440\u043e\u043f\u043e\u0442","\u0440\u043e\u0441\u0442\u0440","\u0440\u043e\u0442\u0438\u043a","\u0440\u043e\u0442\u043e\u043a","\u0440\u043e\u0442\u043e\u043d","\u0440\u043e\u0442\u043e\u0440","\u0440\u043e\u0445\u043b\u044f","\u0440\u043e\u044f\u043b\u044c","\u0440\u0442\u0438\u0449\u0435","\u0440\u0442\u0443\u0442\u044c","\u0440\u0443\u0431\u0430\u0431","\u0440\u0443\u0431\u0430\u0438","\u0440\u0443\u0431\u0430\u0441","\u0440\u0443\u0431\u0435\u0436","\u0440\u0443\u0431\u0451\u0436","\u0440\u0443\u0431\u0435\u0446","\u0440\u0443\u0431\u0438\u043d","\u0440\u0443\u0431\u043a\u0430","\u0440\u0443\u0431\u043b\u044c","\u0440\u0443\u0431\u043e\u0431","\u0440\u0443\u0433\u043d\u044f","\u0440\u0443\u0434\u0438\u0442","\u0440\u0443\u0434\u044f\u043a","\u0440\u0443\u0436\u044c\u0451","\u0440\u0443\u0438\u043d\u0430","\u0440\u0443\u043a\u0430\u0432","\u0440\u0443\u043b\u0451\u0436","\u0440\u0443\u043b\u0435\u0442","\u0440\u0443\u043b\u043e\u043d","\u0440\u0443\u043c\u0431\u0430","\u0440\u0443\u043d\u0434\u0438","\u0440\u0443\u043f\u0438\u044f","\u0440\u0443\u043f\u043e\u0440","\u0440\u0443\u0441\u0430\u0431","\u0440\u0443\u0441\u0438\u043d","\u0440\u0443\u0441\u043b\u043e","\u0440\u0443\u0441\u043d\u044f","\u0440\u0443\u0442\u0435\u0440","\u0440\u0443\u0442\u0438\u043b","\u0440\u0443\u0442\u0438\u043d","\u0440\u0443\u0447\u0435\u0439","\u0440\u0443\u0447\u043a\u0430","\u0440\u0443\u0448\u043a\u0430","\u0440\u044b\u0432\u043e\u043a","\u0440\u044b\u0436\u0438\u0439","\u0440\u044b\u0436\u0438\u043a","\u0440\u044b\u0436\u044c\u0451","\u0440\u044b\u043d\u0434\u0430","\u0440\u044b\u043d\u043e\u043a","\u0440\u044b\u0441\u0446\u0430","\u0440\u044b\u0442\u044c\u0451","\u0440\u044b\u0447\u0430\u0433","\u0440\u044d\u0439\u0438\u0442","\u0440\u044d\u0439\u043a\u0438","\u0440\u044d\u043a\u0435\u0442","\u0440\u044d\u043b\u0435\u0439","\u0440\u044d\u043c\u0431\u043e","\u0440\u044d\u043d\u0433\u0430","\u0440\u044e\u0438\u043d\u044c","\u0440\u044e\u043c\u043a\u0430","\u0440\u044e\u0448\u043a\u0430","\u0440\u044f\u0431\u0438\u043a","\u0440\u044f\u0434\u043d\u043e","\u0440\u044f\u0434\u043e\u043a","\u0440\u044f\u0441\u043a\u0430","\u0440\u044f\u0441\u043d\u0430","\u0440\u044f\u0448\u043a\u0430","\u0441\u0430\u0431\u0430\u043d","\u0441\u0430\u0431\u0436\u0435","\u0441\u0430\u0431\u0437\u0430","\u0441\u0430\u0431\u043b\u044f","\u0441\u0430\u0431\u0443\u0440","\u0441\u0430\u0432\u0430\u043d","\u0441\u0430\u0432\u0430\u0440","\u0441\u0430\u0432\u0430\u0442","\u0441\u0430\u0433\u0443\u043c","\u0441\u0430\u0434\u0438\u043a","\u0441\u0430\u0434\u043a\u0430","\u0441\u0430\u0434\u043d\u043e","\u0441\u0430\u0434\u043e\u043a","\u0441\u0430\u0439\u043a\u0430","\u0441\u0430\u0439\u043a\u043b","\u0441\u0430\u0439\u043c\u0430","\u0441\u0430\u043a\u0432\u0430","\u0441\u0430\u043a\u043b\u044f","\u0441\u0430\u043a\u043c\u0430","\u0441\u0430\u043a\u0442\u0430","\u0441\u0430\u043b\u0430\u0434","\u0441\u0430\u043b\u0430\u0442","\u0441\u0430\u043b\u0438\u0442","\u0441\u0430\u043b\u043a\u0438","\u0441\u0430\u043b\u043c\u0430","\u0441\u0430\u043b\u043e\u043b","\u0441\u0430\u043b\u043e\u043d","\u0441\u0430\u043b\u043e\u043f","\u0441\u0430\u043b\u0443\u043d","\u0441\u0430\u043b\u044e\u0442","\u0441\u0430\u043b\u044f\u0442","\u0441\u0430\u043c\u0430\u043d","\u0441\u0430\u043c\u0431\u0430","\u0441\u0430\u043c\u0431\u043e","\u0441\u0430\u043c\u043e\u0430","\u0441\u0430\u043c\u0441\u0430","\u0441\u0430\u043c\u0443\u043c","\u0441\u0430\u043d\u0431\u043e","\u0441\u0430\u043d\u0433\u043e","\u0441\u0430\u043d\u0434\u0438","\u0441\u0430\u043d\u043a\u0438","\u0441\u0430\u043f\u043a\u0430","\u0441\u0430\u043f\u043e\u0433","\u0441\u0430\u043f\u0443\u043d","\u0441\u0430\u0440\u0430\u0439","\u0441\u0430\u0440\u0430\u043d","\u0441\u0430\u0440\u0436\u0430","\u0441\u0430\u0440\u043c\u0430","\u0441\u0430\u0440\u043e\u043d","\u0441\u0430\u0440\u043e\u0441","\u0441\u0430\u0442\u0438\u043d","\u0441\u0430\u0443\u043d\u0430","\u0441\u0430\u0443\u043d\u0434","\u0441\u0430\u0444\u0440\u0430","\u0441\u0430\u0445\u0430\u0440","\u0441\u0430\u0447\u043e\u043a","\u0441\u0431\u043b\u0451\u0432","\u0441\u0431\u043e\u0440\u0430","\u0441\u0431\u0440\u043e\u0434","\u0441\u0431\u0440\u043e\u0441","\u0441\u0431\u0440\u0443\u044f","\u0441\u0432\u0430\u0434\u0430","\u0441\u0432\u0430\u0437\u0438","\u0441\u0432\u0430\u0439\u043f","\u0441\u0432\u0430\u0440\u0430","\u0441\u0432\u0430\u0442\u0438","\u0441\u0432\u0435\u0438\u0442","\u0441\u0432\u0435\u0440\u043a","\u0441\u0432\u0435\u0447\u0430","\u0441\u0432\u0435\u0449\u0430","\u0441\u0432\u0438\u043b\u044c","\u0441\u0432\u0438\u043d\u0433","\u0441\u0432\u0438\u0441\u0442","\u0441\u0432\u0438\u0442\u0430","\u0441\u0432\u0438\u0442\u0447","\u0441\u0432\u043e\u0440\u0430","\u0441\u0432\u044f\u0437\u0430","\u0441\u0432\u044f\u0437\u044c","\u0441\u0433\u043b\u0430\u0437","\u0441\u0434\u0430\u0447\u0430","\u0441\u0434\u0432\u0438\u0433","\u0441\u0434\u0435\u043b\u043e","\u0441\u0434\u043e\u0431\u0430","\u0441\u0435\u0430\u043d\u0441","\u0441\u0435\u0431\u0443\u043c","\u0441\u0435\u0432\u0431\u0430","\u0441\u0435\u0432\u0435\u0440","\u0441\u0435\u0432\u043e\u043a","\u0441\u0435\u0434\u0430\u043d","\u0441\u0451\u0434\u0437\u0451","\u0441\u0451\u0434\u0437\u0438","\u0441\u0435\u0434\u043b\u043e","\u0441\u0435\u0435\u0432\u043e","\u0441\u0435\u0437\u0430\u043c","\u0441\u0435\u0437\u043e\u043d","\u0441\u0435\u0439\u0431\u0430","\u0441\u0435\u0439\u0448\u0430","\u0441\u0435\u043a\u0430\u0441","\u0441\u0435\u043a\u0430\u0447","\u0441\u0435\u043a\u0438\u0441","\u0441\u0435\u043a\u0442\u0430","\u0441\u0435\u043b\u0435\u043d","\u0441\u0435\u043b\u0438\u043d","\u0441\u0435\u043b\u0444\u0438","\u0441\u0435\u043b\u044f\u043c","\u0441\u0435\u043c\u0438\u043a","\u0441\u0435\u043c\u0438\u0441","\u0441\u0435\u043c\u043a\u0430","\u0441\u0435\u043c\u043f\u043b","\u0441\u0435\u043c\u044c\u044f","\u0441\u0435\u043d\u0430\u0436","\u0441\u0435\u043d\u0430\u0442","\u0441\u0435\u043d\u0435\u0442","\u0441\u0435\u043d\u043a\u0438","\u0441\u0435\u043d\u043d\u0430","\u0441\u0435\u043d\u0446\u043e","\u0441\u0435\u043d\u0446\u044b","\u0441\u0451\u043d\u044d\u043d","\u0441\u0435\u043f\u0438\u044f","\u0441\u0435\u043f\u0442\u0430","\u0441\u0435\u0440\u0430\u043a","\u0441\u0451\u0440\u0434\u0436","\u0441\u0435\u0440\u0438\u043a","\u0441\u0435\u0440\u0438\u043d","\u0441\u0435\u0440\u0438\u0440","\u0441\u0435\u0440\u0438\u0444","\u0441\u0435\u0440\u0438\u044f","\u0441\u0435\u0440\u043a\u0430","\u0441\u0435\u0440\u0441\u043e","\u0441\u0435\u0440\u0443\u043c","\u0441\u0435\u0440\u044f\u043a","\u0441\u0435\u0442\u0430\u043f","\u0441\u0435\u0442\u043a\u0430","\u0441\u0435\u0447\u043a\u0430","\u0441\u0438\u0430\u043b\u044c","\u0441\u0438\u0432\u0435\u0440","\u0441\u0438\u0433\u0438\u043b","\u0441\u0438\u0433\u043c\u0430","\u0441\u0438\u0433\u043d\u0430","\u0441\u0438\u0434\u0436\u043e","\u0441\u0438\u0434\u043a\u0430","\u0441\u0438\u0434\u043e\u0440","\u0441\u0438\u0434\u044e\u043a","\u0441\u0438\u0435\u043d\u0430","\u0441\u0438\u0436\u043a\u0430","\u0441\u0438\u043a\u043b\u044c","\u0441\u0438\u043a\u043e\u0437","\u0441\u0438\u043b\u0430\u043d","\u0441\u0438\u043b\u0430\u0442","\u0441\u0438\u043b\u043a\u0430","\u0441\u0438\u043b\u043e\u043a","\u0441\u0438\u043b\u043e\u043d","\u0441\u0438\u043b\u043e\u0441","\u0441\u0438\u043b\u0443\u0440","\u0441\u0438\u043b\u044c\u0451","\u0441\u0438\u043c\u043a\u0430","\u0441\u0438\u043d\u0430\u043f","\u0441\u0438\u043d\u0433\u043b","\u0441\u0438\u043d\u0438\u0439","\u0441\u0438\u043d\u043e\u0434","\u0441\u0438\u043d\u0442\u043e","\u0441\u0438\u043d\u0443\u0441","\u0441\u0438\u043d\u044f\u043a","\u0441\u0438\u0440\u043e\u043f","\u0441\u0438\u0441\u0442\u0440","\u0441\u0438\u0442\u0430\u0440","\u0441\u0438\u0442\u0435\u0446","\u0441\u0438\u0442\u0440\u043e","\u0441\u0438\u0442\u0446\u0435","\u0441\u0438\u0444\u043e\u043d","\u0441\u043a\u0430\u0439\u043f","\u0441\u043a\u0430\u0439\u0441","\u0441\u043a\u0430\u043b\u0430","\u0441\u043a\u0430\u043b\u043e","\u0441\u043a\u0430\u043b\u044b","\u0441\u043a\u0430\u043d\u044c","\u0441\u043a\u0430\u0440\u0431","\u0441\u043a\u0430\u0440\u043d","\u0441\u043a\u0430\u0443\u0442","\u0441\u043a\u0430\u0447\u044c","\u0441\u043a\u0432\u0430\u0434","\u0441\u043a\u0432\u0435\u0440","\u0441\u043a\u0432\u0438\u0434","\u0441\u043a\u0432\u043e\u0442","\u0441\u043a\u0432\u043e\u0448","\u0441\u043a\u0435\u0439\u0442","\u0441\u043a\u0435\u043d\u0430","\u0441\u043a\u0435\u043d\u044d","\u0441\u043a\u0435\u0442\u0447","\u0441\u043a\u0438\u043b\u043b","\u0441\u043a\u0438\u0440\u0434","\u0441\u043a\u043b\u0430\u0434","\u0441\u043a\u043b\u0435\u043f","\u0441\u043a\u043b\u0451\u043f","\u0441\u043a\u043b\u0438\u0437","\u0441\u043a\u043b\u043e\u043d","\u0441\u043a\u043e\u0431\u0430","\u0441\u043a\u043e\u0440\u0430","\u0441\u043a\u043e\u0440\u043c","\u0441\u043a\u043e\u0442\u0447","\u0441\u043a\u0440\u0430\u0431","\u0441\u043a\u0440\u0430\u0434","\u0441\u043a\u0440\u0430\u043f","\u0441\u043a\u0440\u0435\u043f","\u0441\u043a\u0440\u0438\u043d","\u0441\u043a\u0440\u0438\u043f","\u0441\u043a\u0440\u044d\u0431","\u0441\u043a\u0443\u0431\u0430","\u0441\u043a\u0443\u0434\u0430","\u0441\u043a\u0443\u0434\u0438","\u0441\u043a\u0443\u0434\u043e","\u0441\u043a\u0443\u043a\u0430","\u0441\u043a\u0443\u043b\u0430","\u0441\u043b\u0430\u0432\u0430","\u0441\u043b\u0430\u0434\u0436","\u0441\u043b\u0430\u0439\u0434","\u0441\u043b\u0430\u0439\u043c","\u0441\u043b\u0430\u0439\u0441","\u0441\u043b\u0430\u043d\u044c","\u0441\u043b\u0435\u0433\u0430","\u0441\u043b\u0435\u0437\u0430","\u0441\u043b\u0435\u043d\u0433","\u0441\u043b\u0438\u0432\u0430","\u0441\u043b\u0438\u0437\u044c","\u0441\u043b\u0438\u043d\u0433","\u0441\u043b\u0438\u043f\u044b","\u0441\u043b\u043e\u0432\u0430","\u0441\u043b\u043e\u0432\u043e","\u0441\u043b\u043e\u0451\u043a","\u0441\u043b\u044e\u0434\u0430","\u0441\u043b\u044e\u043d\u0430","\u0441\u043b\u044e\u043d\u0438","\u0441\u043c\u0430\u0437\u044c","\u0441\u043c\u0430\u0439\u043b","\u0441\u043c\u0430\u0440\u0442","\u0441\u043c\u0435\u043d\u0430","\u0441\u043c\u0435\u0440\u0447","\u0441\u043c\u0435\u0441\u044c","\u0441\u043c\u0435\u0442\u0430","\u0441\u043c\u043e\u043a\u0438","\u0441\u043c\u043e\u043b\u0430","\u0441\u043c\u043e\u043b\u044c","\u0441\u043c\u043e\u0442\u0440","\u0441\u043c\u0440\u0430\u0434","\u0441\u043c\u0441\u043a\u0430","\u0441\u043c\u0443\u0437\u0438","\u0441\u043c\u0443\u0440\u044c","\u0441\u043c\u0443\u0442\u0430","\u0441\u043c\u0443\u0442\u044c","\u0441\u043c\u044b\u0441\u043b","\u0441\u043d\u0430\u0444\u0444","\u0441\u043d\u0435\u0434\u044c","\u0441\u043d\u0435\u0439\u043a","\u0441\u043d\u0438\u0442\u0447","\u0441\u043d\u0438\u0442\u044c","\u0441\u043d\u043e\u0441\u0438","\u0441\u043d\u044b\u0442\u044c","\u0441\u043e\u0431\u0435\u0441","\u0441\u043e\u0431\u043e\u0440","\u0441\u043e\u0432\u0435\u0442","\u0441\u043e\u0432\u0438\u043a","\u0441\u043e\u0432\u043e\u043a","\u0441\u043e\u0433\u0440\u0430","\u0441\u043e\u0434\u0436\u0443","\u0441\u043e\u0434\u043e\u043c","\u0441\u043e\u0437\u044b\u0432","\u0441\u043e\u0439\u043c\u0430","\u0441\u043e\u043a\u0435\u0440","\u0441\u043e\u043a\u0435\u0442","\u0441\u043e\u043a\u043e\u043b","\u0441\u043e\u043b\u0435\u043d","\u0441\u043e\u043b\u0435\u044f","\u0441\u043e\u043b\u0438\u0434","\u0441\u043e\u043b\u043a\u0430","\u0441\u043e\u043b\u043e\u0434","\u0441\u043e\u043b\u043e\u043f","\u0441\u043e\u043b\u044f\u043a","\u0441\u043e\u043b\u044f\u0440","\u0441\u043e\u043c\u0438\u0442","\u0441\u043e\u043c\u043c\u0430","\u0441\u043e\u043c\u043e\u043d","\u0441\u043e\u043d\u0430\u0440","\u0441\u043e\u043d\u0435\u0442","\u0441\u043e\u043f\u043a\u0430","\u0441\u043e\u043f\u043b\u043e","\u0441\u043e\u043f\u043b\u044f","\u0441\u043e\u043f\u043e\u0440","\u0441\u043e\u0440\u0431\u0435","\u0441\u043e\u0440\u0433\u043e","\u0441\u043e\u0440\u0438\u0442","\u0441\u043e\u0440\u043e\u043a","\u0441\u043e\u0440\u043e\u043c","\u0441\u043e\u0440\u0443\u0441","\u0441\u043e\u0441\u0435\u0446","\u0441\u043e\u0441\u043a\u0430","\u0441\u043e\u0441\u043d\u0430","\u0441\u043e\u0441\u043e\u043a","\u0441\u043e\u0441\u0443\u0434","\u0441\u043e\u0442\u0430\u044f","\u0441\u043e\u0442\u0438\u043a","\u0441\u043e\u0442\u043a\u0430","\u0441\u043e\u0442\u043d\u044f","\u0441\u043e\u0444\u0438\u0442","\u0441\u043e\u0445\u0440\u0430","\u0441\u043e\u0448\u043a\u0430","\u0441\u043f\u0430\u0433\u0438","\u0441\u043f\u0430\u0437\u043c","\u0441\u043f\u0430\u0439\u043a","\u0441\u043f\u0430\u0439\u0441","\u0441\u043f\u0435\u043a\u0442","\u0441\u043f\u0435\u0441\u044c","\u0441\u043f\u0438\u0434\u044b","\u0441\u043f\u0438\u043d\u0430","\u0441\u043f\u0438\u0440\u0442","\u0441\u043f\u0438\u0446\u0430","\u0441\u043f\u043b\u0430\u0432","\u0441\u043f\u043b\u0438\u043d","\u0441\u043f\u043b\u0438\u0442","\u0441\u043f\u043b\u0438\u0444","\u0441\u043f\u043b\u043e\u0433","\u0441\u043f\u043e\u043d\u0436","\u0441\u043f\u043e\u0440\u0430","\u0441\u043f\u043e\u0440\u0442","\u0441\u043f\u0440\u0435\u0434","\u0441\u043f\u0440\u0435\u0439","\u0441\u043f\u0440\u043e\u0441","\u0441\u043f\u0440\u0443\u0442","\u0441\u043f\u0443\u043c\u0430","\u0441\u043f\u0443\u0440\u0442","\u0441\u043f\u0443\u0441\u043a","\u0441\u043f\u044e\u0440\u043a","\u0441\u0440\u0430\u043a\u0430","\u0441\u0440\u0430\u043d\u044c","\u0441\u0440\u0435\u0434\u0430","\u0441\u0441\u0430\u043a\u0430","\u0441\u0441\u043e\u0440\u0430","\u0441\u0441\u0443\u0434\u0430","\u0441\u0441\u044b\u043b\u044c","\u0441\u0442\u0430\u0434\u043e","\u0441\u0442\u0430\u043b\u044c","\u0441\u0442\u0430\u043d\u0441","\u0441\u0442\u0430\u043d\u0442","\u0441\u0442\u0430\u0440\u0442","\u0441\u0442\u0430\u0440\u044c","\u0441\u0442\u0430\u0442\u0430","\u0441\u0442\u0430\u0442\u044c","\u0441\u0442\u0430\u0443\u0442","\u0441\u0442\u0432\u043e\u043b","\u0441\u0442\u0432\u043e\u0440","\u0441\u0442\u0435\u0433\u0430","\u0441\u0442\u0435\u0436\u044c","\u0441\u0442\u0435\u0437\u044f","\u0441\u0442\u0435\u0439\u043a","\u0441\u0442\u0435\u043a\u0430","\u0441\u0442\u0435\u043b\u0430","\u0441\u0442\u0435\u043b\u0441","\u0441\u0442\u0435\u043b\u044c","\u0441\u0442\u0435\u043d\u0430","\u0441\u0442\u0435\u043d\u0434","\u0441\u0442\u0435\u043d\u0442","\u0441\u0442\u0435\u043f\u0441","\u0441\u0442\u0435\u043f\u044c","\u0441\u0442\u0438\u043b\u043e","\u0441\u0442\u0438\u043b\u044c","\u0441\u0442\u043a\u043b\u043e","\u0441\u0442\u043e\u043a\u0441","\u0441\u0442\u043e\u043b\u0430","\u0441\u0442\u043e\u043b\u0431","\u0441\u0442\u043e\u043b\u043f","\u0441\u0442\u043e\u043c\u0430","\u0441\u0442\u043e\u043f\u0430","\u0441\u0442\u043e\u0440\u0430","\u0441\u0442\u043e\u0443\u043d","\u0441\u0442\u043e\u044f\u043a","\u0441\u0442\u0440\u0430\u0437","\u0441\u0442\u0440\u0430\u0445","\u0441\u0442\u0440\u0435\u043b","\u0441\u0442\u0440\u0451\u043c","\u0441\u0442\u0440\u0438\u043a","\u0441\u0442\u0440\u0438\u043c","\u0441\u0442\u0440\u0438\u043f","\u0441\u0442\u0440\u0438\u0442","\u0441\u0442\u0440\u043e\u0431","\u0441\u0442\u0440\u043e\u0439","\u0441\u0442\u0440\u043e\u043f","\u0441\u0442\u0440\u0443\u0433","\u0441\u0442\u0440\u0443\u043f","\u0441\u0442\u0440\u0443\u044f","\u0441\u0442\u0443\u0436\u0430","\u0441\u0442\u0443\u043f\u0430","\u0441\u0442\u044b\u043d\u044c","\u0441\u0443\u0430\u0440\u0435","\u0441\u0443\u0432\u043e\u0439","\u0441\u0443\u0434\u043d\u043e","\u0441\u0443\u0434\u043e\u043a","\u0441\u0443\u0435\u0442\u0430","\u0441\u0443\u0438\u043d\u0433","\u0441\u0443\u043a\u043a\u0430","\u0441\u0443\u043a\u043d\u043e","\u0441\u0443\u043a\u0440\u0435","\u0441\u0443\u043b\u0435\u044f","\u0441\u0443\u043b\u043e\u0439","\u0441\u0443\u043b\u0443\u043a","\u0441\u0443\u043b\u044b\u043a","\u0441\u0443\u043c\u0430\u0445","\u0441\u0443\u043c\u0435\u0442","\u0441\u0443\u043c\u043a\u0430","\u0441\u0443\u043c\u043c\u0430","\u0441\u0443\u043d\u043d\u0430","\u0441\u0443\u043f\u0430\u0442","\u0441\u0443\u043f\u0435\u0440","\u0441\u0443\u043f\u0435\u0446","\u0441\u0443\u043f\u0438\u043d","\u0441\u0443\u0440\u0436\u0430","\u0441\u0443\u0440\u0438\u043a","\u0441\u0443\u0440\u0438\u0442","\u0441\u0443\u0440\u043c\u0430","\u0441\u0443\u0440\u043d\u0430","\u0441\u0443\u0441\u0430\u043a","\u0441\u0443\u0441\u0435\u043a","\u0441\u0443\u0441\u043b\u043e","\u0441\u0443\u0442\u0430\u0436","\u0441\u0443\u0442\u043a\u0438","\u0441\u0443\u0442\u0440\u0430","\u0441\u0443\u0444\u043b\u0435","\u0441\u0443\u0445\u0430\u044f","\u0441\u0443\u0445\u0438\u0439","\u0441\u0443\u0445\u044b\u0439","\u0441\u0443\u0447\u043d\u044f","\u0441\u0443\u0447\u043e\u043a","\u0441\u0443\u0447\u044c\u0451","\u0441\u0443\u0448\u043a\u0430","\u0441\u0444\u0435\u0440\u0430","\u0441\u0445\u0435\u043c\u0430","\u0441\u0445\u0438\u043c\u0430","\u0441\u0445\u0440\u043e\u043d","\u0441\u0446\u0430\u043a\u0438","\u0441\u0446\u0435\u043d\u0430","\u0441\u0447\u0451\u0442\u044b","\u0441\u044a\u0435\u0437\u0434","\u0441\u044b\u043f\u0435\u0446","\u0441\u044b\u0440\u0435\u0446","\u0441\u044b\u0440\u043e\u043a","\u0441\u044b\u0440\u044c\u0451","\u0441\u044b\u0447\u0443\u0433","\u0441\u044d\u043d\u0442\u043e","\u0441\u044e\u0436\u0435\u0442","\u0441\u044e\u0438\u0442\u0430","\u0441\u044e\u0440\u043a\u043e","\u0441\u044f\u0432\u043a\u0430","\u0441\u044f\u0436\u043e\u043a","\u0441\u044f\u043d\u0446\u0438","\u0442\u0430\u0431\u0430\u043a","\u0442\u0430\u0431\u0435\u0441","\u0442\u0430\u0431\u0438\u044f","\u0442\u0430\u0431\u043b\u0430","\u0442\u0430\u0431\u043b\u043e","\u0442\u0430\u0431\u043e\u0440","\u0442\u0430\u0431\u0443\u043d","\u0442\u0430\u0432\u0430\u0444","\u0442\u0430\u0432\u043e\u0442","\u0442\u0430\u0432\u0440\u043e","\u0442\u0430\u0433\u0430\u043d","\u0442\u0430\u0437\u0435\u0440","\u0442\u0430\u0437\u0438\u043a","\u0442\u0430\u0438\u0442\u0438","\u0442\u0430\u0439\u0433\u0430","\u0442\u0430\u0439\u0438\u0442","\u0442\u0430\u0439\u043a\u0430","\u0442\u0430\u0439\u043d\u0430","\u0442\u0430\u0439\u0442\u043b","\u0442\u0430\u0439\u0444\u0430","\u0442\u0430\u043a\u0441\u0430","\u0442\u0430\u043a\u0441\u0438","\u0442\u0430\u043a\u044b\u0440","\u0442\u0430\u043b\u0435\u0440","\u0442\u0430\u043b\u0435\u0441","\u0442\u0430\u043b\u0435\u0446","\u0442\u0430\u043b\u0438\u043a","\u0442\u0430\u043b\u0438\u0442","\u0442\u0430\u043b\u0438\u044f","\u0442\u0430\u043b\u043e\u043d","\u0442\u0430\u043b\u0443\u0441","\u0442\u0430\u043b\u044c\u043a","\u0442\u0430\u043b\u044c\u044f","\u0442\u0430\u043c\u0433\u0430","\u0442\u0430\u043d\u0433\u043b","\u0442\u0430\u043d\u0433\u043e","\u0442\u0430\u043d\u0435\u0446","\u0442\u0430\u043d\u0438\u043d","\u0442\u0430\u043d\u043a\u0430","\u0442\u0430\u043e\u043b\u0443","\u0442\u0430\u043f\u0430\u0441","\u0442\u0430\u043f\u043a\u0430","\u0442\u0430\u043f\u043e\u043a","\u0442\u0430\u0440\u0430\u043d","\u0442\u0430\u0440\u0430\u0444","\u0442\u0430\u0440\u0438\u0444","\u0442\u0430\u0441\u043a\u0430","\u0442\u0430\u0442\u0440\u0430","\u0442\u0430\u0444\u0438\u044f","\u0442\u0430\u0444\u0442\u0430","\u0442\u0430\u0444\u044c\u044f","\u0442\u0430\u0445\u0442\u0430","\u0442\u0430\u0447\u043a\u0430","\u0442\u0430\u0448\u043a\u0430","\u0442\u0430\u044d\u043b\u044c","\u0442\u0432\u0435\u0440\u043a","\u0442\u0432\u0438\u043a\u0441","\u0442\u0432\u0438\u0441\u0442","\u0442\u0432\u0438\u0442\u0442","\u0442\u0435\u0430\u0442\u0440","\u0442\u0435\u0437\u0438\u0441","\u0442\u0435\u0438\u0437\u043c","\u0442\u0435\u0439\u0431\u043b","\u0442\u0435\u043a\u0441\u0442","\u0442\u0435\u043b\u0435\u0448","\u0442\u0435\u043b\u0438\u043a","\u0442\u0435\u043b\u0438\u044f","\u0442\u0435\u043b\u043b\u0441","\u0442\u0435\u043c\u0431\u0440","\u0442\u0435\u043c\u043a\u0430","\u0442\u0435\u043d\u0433\u0435","\u0442\u0435\u043d\u0451\u043a","\u0442\u0435\u043d\u0438\u044f","\u0442\u0435\u043d\u043e\u0440","\u0442\u0435\u043f\u043b\u043e","\u0442\u0435\u0440\u0430\u0442","\u0442\u0435\u0440\u0435\u043c","\u0442\u0451\u0440\u043a\u0430","\u0442\u0435\u0440\u043c\u0430","\u0442\u0435\u0440\u043c\u044b","\u0442\u0435\u0440\u043d\u043e","\u0442\u0435\u0440\u0440\u0430","\u0442\u0435\u0441\u0430\u043a","\u0442\u0435\u0441\u0438\u0441","\u0442\u0451\u0441\u043a\u0430","\u0442\u0435\u0441\u043b\u0430","\u0442\u0435\u0441\u043b\u043e","\u0442\u0435\u0441\u0442\u0430","\u0442\u0435\u0441\u0442\u043e","\u0442\u0435\u0442\u0440\u0438","\u0442\u0435\u0442\u0443\u043c","\u0442\u0435\u0444\u0440\u0430","\u0442\u0435\u0445\u0430\u043d","\u0442\u0435\u0445\u043d\u043e","\u0442\u0435\u0447\u043a\u0430","\u0442\u0451\u0448\u043a\u0430","\u0442\u0438\u0430\u0440\u0430","\u0442\u0438\u0437\u0435\u0440","\u0442\u0438\u0439\u0438\u043d","\u0442\u0438\u043a\u0435\u0440","\u0442\u0438\u043b\u043b\u0430","\u0442\u0438\u043b\u043b\u044c","\u0442\u0438\u043b\u044c\u0442","\u0442\u0438\u043c\u0438\u043d","\u0442\u0438\u043c\u043e\u043b","\u0442\u0438\u043c\u0443\u0441","\u0442\u0438\u043f\u0430\u0436","\u0442\u0438\u043f\u0435\u0446","\u0442\u0438\u043f\u0438\u043a","\u0442\u0438\u043f\u043b\u0435","\u0442\u0438\u043f\u0443\u043d","\u0442\u0438\u0440\u0430\u0436","\u0442\u0438\u0440\u0438\u0442","\u0442\u0438\u0441\u043a\u0438","\u0442\u0438\u0442\u0430\u043d","\u0442\u0438\u0442\u0438\u043d","\u0442\u0438\u0442\u043b\u0430","\u0442\u0438\u0442\u043b\u043e","\u0442\u0438\u0442\u0443\u043b","\u0442\u0438\u0444\u043e\u043d","\u0442\u0438\u0445\u0438\u0442","\u0442\u043a\u0430\u043d\u044c","\u0442\u043c\u0435\u0437\u0430","\u0442\u043e\u0430\u043b\u0430","\u0442\u043e\u0432\u0430\u0440","\u0442\u043e\u0434\u0435\u0441","\u0442\u043e\u0437\u0438\u043a","\u0442\u043e\u043a\u0430\u0439","\u0442\u043e\u043a\u0435\u043d","\u0442\u043e\u043b\u0430\u0440","\u0442\u043e\u043b\u043e\u0441","\u0442\u043e\u043b\u043f\u0430","\u0442\u043e\u043b\u0449\u0430","\u0442\u043e\u043b\u0449\u044c","\u0442\u043e\u043c\u0430\u043d","\u0442\u043e\u043c\u0430\u0442","\u0442\u043e\u043c\u0438\u043a","\u0442\u043e\u043c\u043e\u0441","\u0442\u043e\u043d\u0430\u0440","\u0442\u043e\u043d\u0434\u043e","\u0442\u043e\u043d\u0435\u0440","\u0442\u043e\u043d\u0438\u0439","\u0442\u043e\u043d\u0438\u043a","\u0442\u043e\u043d\u0438\u044f","\u0442\u043e\u043d\u043d\u0430","\u0442\u043e\u043d\u0443\u0441","\u0442\u043e\u043d\u0444\u0430","\u0442\u043e\u043f\u0430\u0437","\u0442\u043e\u043f\u0438\u043a","\u0442\u043e\u043f\u0438\u044f","\u0442\u043e\u043f\u043a\u0430","\u0442\u043e\u043f\u043e\u0440","\u0442\u043e\u043f\u043e\u0441","\u0442\u043e\u043f\u043e\u0442","\u0442\u043e\u0440\u0431\u0430","\u0442\u043e\u0440\u0433\u0438","\u0442\u043e\u0440\u0435\u0446","\u0442\u043e\u0440\u0438\u0438","\u0442\u043e\u0440\u0438\u0439","\u0442\u043e\u0440\u0438\u0442","\u0442\u043e\u0440\u043e\u043d","\u0442\u043e\u0440\u043e\u0441","\u0442\u043e\u0440\u0443\u0441","\u0442\u043e\u0441\u043a\u0430","\u0442\u043e\u0442\u0430\u043b","\u0442\u043e\u0442\u0435\u043c","\u0442\u043e\u0444\u0443\u0441","\u0442\u043e\u0447\u043a\u0430","\u0442\u043e\u0449\u0430\u043a","\u0442\u0440\u0430\u0431\u043b","\u0442\u0440\u0430\u0432\u0430","\u0442\u0440\u0430\u0439\u0431","\u0442\u0440\u0430\u0439\u043a","\u0442\u0440\u0430\u0439\u0442","\u0442\u0440\u0430\u043a\u0442","\u0442\u0440\u0430\u043c\u0430","\u0442\u0440\u0430\u043c\u043f","\u0442\u0440\u0430\u043d\u0441","\u0442\u0440\u0430\u043d\u0448","\u0442\u0440\u0430\u043f\u043f","\u0442\u0440\u0430\u0441\u0441","\u0442\u0440\u0430\u0441\u0442","\u0442\u0440\u0430\u0442\u0430","\u0442\u0440\u0430\u0443\u0440","\u0442\u0440\u0435\u0431\u0430","\u0442\u0440\u0435\u0439\u0434","\u0442\u0440\u0435\u0439\u043b","\u0442\u0440\u0435\u043b\u044c","\u0442\u0440\u0435\u043c\u0430","\u0442\u0440\u0435\u043d\u0434","\u0442\u0440\u0435\u043d\u0447","\u0442\u0440\u0435\u0441\u043a","\u0442\u0440\u0435\u0441\u0442","\u0442\u0440\u0435\u0442\u044c","\u0442\u0440\u0435\u0443\u0445","\u0442\u0440\u0435\u0444\u0430","\u0442\u0440\u0435\u0444\u044b","\u0442\u0440\u0451\u0445\u0430","\u0442\u0440\u0438\u0430\u043b","\u0442\u0440\u0438\u0430\u0441","\u0442\u0440\u0438\u0431\u0430","\u0442\u0440\u0438\u0435\u0440","\u0442\u0440\u0438\u0437\u043c","\u0442\u0440\u0438\u043a\u043e","\u0442\u0440\u0438\u043e\u0434","\u0442\u0440\u0438\u043e\u043d","\u0442\u0440\u0438\u043f\u0441","\u0442\u0440\u043e\u043c\u0431","\u0442\u0440\u043e\u043c\u043f","\u0442\u0440\u043e\u043d\u0430","\u0442\u0440\u043e\u043f\u0430","\u0442\u0440\u043e\u044f\u043a","\u0442\u0440\u043e\u044f\u043d","\u0442\u0440\u0443\u0431\u0430","\u0442\u0440\u0443\u0441\u044b","\u0442\u0440\u0443\u0445\u0430","\u0442\u0440\u044e\u043c\u043e","\u0442\u0441\u0443\u0433\u0430","\u0442\u0443\u0431\u0438\u043a","\u0442\u0443\u0431\u0443\u0441","\u0442\u0443\u0433\u0440\u0430","\u0442\u0443\u0436\u0431\u0430","\u0442\u0443\u0437\u0438\u043a","\u0442\u0443\u0439\u043e\u043d","\u0442\u0443\u043b\u0438\u0439","\u0442\u0443\u043b\u0438\u0442","\u0442\u0443\u043b\u0443\u043a","\u0442\u0443\u043b\u0443\u043f","\u0442\u0443\u043b\u044c\u0447","\u0442\u0443\u043b\u044c\u044f","\u0442\u0443\u043c\u0430\u043a","\u0442\u0443\u043c\u0430\u043d","\u0442\u0443\u043c\u0431\u0430","\u0442\u0443\u043c\u043e\u0440","\u0442\u0443\u043d\u0438\u0442","\u0442\u0443\u043f\u0435\u0439","\u0442\u0443\u043f\u0435\u0446","\u0442\u0443\u043f\u0438\u043a","\u0442\u0443\u043f\u044c\u0451","\u0442\u0443\u0440\u0430\u043a","\u0442\u0443\u0440\u043a\u0430","\u0442\u0443\u0440\u043c\u0430","\u0442\u0443\u0440\u043d\u0435","\u0442\u0443\u0441\u0451\u0436","\u0442\u0443\u0442\u043e\u0440","\u0442\u0443\u0442\u0442\u0438","\u0442\u0443\u0444\u043b\u044f","\u0442\u0443\u0444\u0442\u0430","\u0442\u0443\u0447\u043a\u0430","\u0442\u0443\u0448\u043a\u0430","\u0442\u0445\u0438\u043d\u0430","\u0442\u0448\u0443\u0432\u0430","\u0442\u0449\u0435\u0442\u0430","\u0442\u044b\u0439\u044b\u043d","\u0442\u044b\u043a\u0432\u0430","\u0442\u044b\u043d\u043e\u043a","\u0442\u044b\u0440\u043a\u0430","\u0442\u044b\u0440\u043b\u043e","\u0442\u044b\u0440\u0441\u0430","\u0442\u044b\u0447\u043e\u043a","\u0442\u044d\u043d\u0438\u0442","\u0442\u044d\u043d\u0441\u044e","\u0442\u044d\u0448\u043a\u0430","\u0442\u044e\u0431\u0430\u0436","\u0442\u044e\u0431\u0438\u043a","\u0442\u044e\u043d\u0435\u0440","\u0442\u044e\u043d\u0438\u043a","\u0442\u044e\u0440\u0438\u043a","\u0442\u044e\u0440\u044e\u043a","\u0442\u044e\u0442\u044e\u043d","\u0442\u044e\u0444\u044f\u043a","\u0442\u044e\u0447\u043e\u043a","\u0442\u044f\u0431\u043b\u043e","\u0442\u044f\u0433\u043b\u043e","\u0442\u044f\u0433\u0443\u043d","\u0442\u044f\u0436\u0431\u0430","\u0442\u044f\u0436\u043a\u0430","\u0442\u044f\u043f\u043a\u0430","\u0443\u0430\u0437\u0438\u043a","\u0443\u0431\u0440\u043e\u0434","\u0443\u0431\u0440\u0443\u0441","\u0443\u0431\u044b\u043b\u044c","\u0443\u0432\u0435\u0438\u0442","\u0443\u0432\u0440\u0430\u0436","\u0443\u0433\u043e\u0434\u0430","\u0443\u0433\u043e\u043b\u044c","\u0443\u0433\u043e\u0440\u044c","\u0443\u0433\u0440\u0435\u0432","\u0443\u0434\u0430\u043b\u044c","\u0443\u0434\u0430\u0447\u0430","\u0443\u0434\u0435\u0440\u0436","\u0443\u0434\u0438\u043b\u0430","\u0443\u0436\u0430\u043a\u0430","\u0443\u0436\u0438\u0449\u0435","\u0443\u0437\u0432\u0430\u0440","\u0443\u0437\u0434\u0446\u044b","\u0443\u0437\u0435\u0440\u043a","\u0443\u0437\u0438\u043d\u0430","\u0443\u0437\u043a\u0430\u044f","\u0443\u0437\u0443\u0440\u0430","\u0443\u043a\u043b\u0430\u0434","\u0443\u043a\u043b\u043e\u043d","\u0443\u043a\u0440\u043e\u043f","\u0443\u043a\u0440\u044e\u043a","\u0443\u043a\u0441\u0443\u0441","\u0443\u043b\u0438\u043a\u0430","\u0443\u043b\u0438\u0446\u0430","\u0443\u043b\u043e\u0432\u043e","\u0443\u043c\u0430\u043c\u0438","\u0443\u043c\u0431\u0438\u0442","\u0443\u043c\u0431\u043e\u043d","\u0443\u043c\u0431\u0440\u0430","\u0443\u043c\u0438\u0449\u0435","\u0443\u043c\u043e\u043b\u043a","\u0443\u043c\u043e\u0440\u0430","\u0443\u043d\u0430\u043c\u0438","\u0443\u043d\u0431\u0430\u043d","\u0443\u043d\u0438\u0430\u0442","\u0443\u043d\u0446\u0438\u044f","\u0443\u043d\u044b\u043b\u044c","\u0443\u043f\u0430\u0433\u0438","\u0443\u043f\u0440\u0451\u043a","\u0443\u0440\u0430\u0437\u0430","\u0443\u0440\u0435\u0438\u0434","\u0443\u0440\u0435\u043c\u0430","\u0443\u0440\u0451\u043c\u0430","\u0443\u0440\u0438\u043d\u0430","\u0443\u0440\u043c\u0430\u043d","\u0443\u0440\u0441\u043e\u043d","\u0443\u0440\u0442\u0438\u0442","\u0443\u0440\u0443\u0441\u0438","\u0443\u0441\u0438\u043a\u0438","\u0443\u0441\u0438\u0449\u0435","\u0443\u0441\u043f\u0435\u0445","\u0443\u0441\u0442\u0430\u0432","\u0443\u0441\u0442\u043e\u0439","\u0443\u0441\u0442\u0443\u043f","\u0443\u0441\u0442\u044c\u0435","\u0443\u0442\u0435\u0440\u044f","\u0443\u0442\u0435\u0445\u0430","\u0443\u0442\u0438\u043b\u044c","\u0443\u0445\u0432\u0430\u0442","\u0443\u0445\u043e\u0436\u044c","\u0443\u0447\u0430\u0433\u0430","\u0443\u0447\u0451\u0431\u0430","\u0443\u0447\u0445\u043e\u0437","\u0443\u0448\u0438\u0446\u0430","\u0443\u0448\u043a\u0443\u0439","\u0443\u0449\u0435\u0440\u0431","\u0444\u0430\u0431\u0437\u0430","\u0444\u0430\u0431\u0440\u0430","\u0444\u0430\u0432\u043e\u0440","\u0444\u0430\u0432\u0443\u0441","\u0444\u0430\u0433\u043e\u0442","\u0444\u0430\u0434\u0436\u0440","\u0444\u0430\u0434\u043e\u043c","\u0444\u0430\u0437\u0438\u0441","\u0444\u0430\u0439\u0435\u0440","\u0444\u0430\u043a\u0430\u043f","\u0444\u0430\u043a\u0435\u043b","\u0444\u0430\u043b\u0430\u043a","\u0444\u0430\u043b\u0434\u0430","\u0444\u0430\u043b\u044c\u0446","\u0444\u0430\u043c\u0431\u043b","\u0444\u0430\u043d\u0437\u0430","\u0444\u0430\u043d\u043a\u0438","\u0444\u0430\u043d\u0442\u0430","\u0444\u0430\u0440\u0430\u0434","\u0444\u0430\u0440\u043c\u0430","\u0444\u0430\u0440\u0441\u0438","\u0444\u0430\u0440\u0446\u0430","\u0444\u0430\u0441\u0430\u0434","\u0444\u0430\u0441\u0435\u0442","\u0444\u0430\u0441\u043a\u0430","\u0444\u0430\u0441\u043e\u043d","\u0444\u0430\u0441\u0442\u044b","\u0444\u0430\u0442\u043e\u043c","\u0444\u0430\u0442\u0442\u0430","\u0444\u0430\u0442\u0443\u043c","\u0444\u0430\u0443\u043d\u0430","\u0444\u0430\u0446\u0435\u0442","\u0444\u0430\u0446\u0438\u044f","\u0444\u0430\u0448\u043d\u044f","\u0444\u0430\u044f\u043d\u0441","\u0444\u0435\u0430\u0442\u0440","\u0444\u0435\u043a\u0430\u043b","\u0444\u0435\u043d\u0438\u043b","\u0444\u0435\u043d\u043e\u043b","\u0444\u0435\u0440\u0437\u044c","\u0444\u0435\u0440\u043c\u0430","\u0444\u0435\u0440\u043c\u0438","\u0444\u0435\u0441\u043a\u0430","\u0444\u0435\u0442\u0432\u0430","\u0444\u0435\u0442\u0438\u0448","\u0444\u0438\u0430\u043a\u0440","\u0444\u0438\u0430\u043b\u0430","\u0444\u0438\u0431\u0440\u0430","\u0444\u0438\u0433\u043d\u044f","\u0444\u0438\u0434\u0435\u0440","\u0444\u0438\u0434\u0436\u0438","\u0444\u0438\u0436\u043c\u0430","\u0444\u0438\u0437\u0438\u044f","\u0444\u0438\u0437\u0440\u0430","\u0444\u0438\u043a\u0441\u0430","\u0444\u0438\u043a\u0443\u0441","\u0444\u0438\u043a\u0448\u043d","\u0444\u0438\u043b\u0435\u0439","\u0444\u0438\u043b\u043b\u043e","\u0444\u0438\u043b\u0443\u043c","\u0444\u0438\u043b\u044c\u043c","\u0444\u0438\u043b\u044c\u0446","\u0444\u0438\u043c\u043e\u0437","\u0444\u0438\u043d\u0430\u043b","\u0444\u0438\u043d\u0431\u043e","\u0444\u0438\u043d\u0438\u043a","\u0444\u0438\u043d\u0438\u0448","\u0444\u0438\u043d\u043a\u0430","\u0444\u0438\u043d\u043d\u0430","\u0444\u0438\u043e\u0440\u0434","\u0444\u0438\u0440\u043c\u0430","\u0444\u0438\u0442\u0438\u043d","\u0444\u0438\u0442\u043e\u043b","\u0444\u0438\u0447\u0435\u0440","\u0444\u0438\u0448\u0430\u0439","\u0444\u0438\u0448\u043a\u0430","\u0444\u043b\u0430\u0435\u0440","\u0444\u043b\u0430\u043d\u0433","\u0444\u043b\u0430\u043d\u043a","\u0444\u043b\u0435\u0439\u043c","\u0444\u043b\u0435\u0439\u0442","\u0444\u043b\u0435\u0439\u0446","\u0444\u043b\u0435\u0448\u044c","\u0444\u043b\u0438\u0430\u043a","\u0444\u043b\u0438\u043d\u0442","\u0444\u043b\u0438\u0440\u0442","\u0444\u043b\u043e\u0430\u0442","\u0444\u043b\u043e\u043a\u0441","\u0444\u043b\u043e\u043f\u0441","\u0444\u043b\u043e\u0440\u0430","\u0444\u043b\u043e\u0441\u0441","\u0444\u043b\u0443\u0435\u0440","\u0444\u043b\u044e\u0438\u0434","\u0444\u043b\u044f\u0433\u0430","\u0444\u043e\u0431\u0438\u044f","\u0444\u043e\u043a\u0443\u0441","\u0444\u043e\u043b\u0438\u043e","\u0444\u043e\u043b\u0438\u044f","\u0444\u043e\u043b\u043e\u0441","\u0444\u043e\u043c\u0438\u0442","\u0444\u043e\u043c\u0438\u0447","\u0444\u043e\u043c\u043a\u0430","\u0444\u043e\u043d\u0434\u044e","\u0444\u043e\u043d\u043e\u043d","\u0444\u043e\u0440\u0435\u0437","\u0444\u043e\u0440\u043c\u0430","\u0444\u043e\u0440\u0442\u0435","\u0444\u043e\u0440\u0443\u043c","\u0444\u043e\u0441\u043a\u0430","\u0444\u043e\u0442\u0438\u043a","\u0444\u043e\u0442\u043a\u0430","\u0444\u043e\u0442\u043e\u043d","\u0444\u043e\u0444\u0430\u043d","\u0444\u0440\u0430\u0437\u0430","\u0444\u0440\u0430\u043d\u043a","\u0444\u0440\u0430\u0445\u0442","\u0444\u0440\u0435\u0437\u0430","\u0444\u0440\u0435\u0439\u043c","\u0444\u0440\u0435\u043d\u0434","\u0444\u0440\u0435\u043d\u0447","\u0444\u0440\u0435\u043e\u043d","\u0444\u0440\u0435\u0441\u043a","\u0444\u0440\u043e\u043d\u0442","\u0444\u0440\u0443\u043a\u0442","\u0444\u0440\u0443\u043d\u0442","\u0444\u0440\u044f\u0437\u044c","\u0444\u0443\u0433\u0430\u0441","\u0444\u0443\u0436\u0435\u0440","\u0444\u0443\u0437\u0435\u044f","\u0444\u0443\u0437\u0438\u044f","\u0444\u0443\u0437\u043e\u0440","\u0444\u0443\u043a\u0443\u0441","\u0444\u0443\u043b\u044f\u0440","\u0444\u0443\u0440\u0430\u0436","\u0444\u0443\u0440\u0430\u043d","\u0444\u0443\u0440\u0438\u043b","\u0444\u0443\u0440\u043c\u0430","\u0444\u0443\u0440\u043e\u043b","\u0444\u0443\u0440\u043e\u0440","\u0444\u0443\u0440\u0440\u0438","\u0444\u0443\u0441\u0445\u0430","\u0444\u0443\u0442\u0430\u0436","\u0444\u0443\u0442\u0435\u0440","\u0444\u0443\u0442\u043e\u043d","\u0444\u0443\u0442\u043e\u0440","\u0444\u0443\u0444\u043b\u043e","\u0444\u0443\u044d\u0442\u0435","\u0444\u0443\u044f\u0440\u0430","\u0444\u044b\u0440\u043e\u043a","\u0444\u044c\u043e\u0440\u0434","\u0444\u044c\u044e\u0436\u043d","\u0444\u044e\u0437\u0435\u043d","\u0444\u044e\u043c\u0430\u0436","\u0444\u044e\u0441\u0438\u0441","\u0445\u0430\u0431\u0430\u0440","\u0445\u0430\u0432\u043a\u0430","\u0445\u0430\u0434\u0430\u043a","\u0445\u0430\u0434\u0438\u0441","\u0445\u0430\u0439\u043a\u0443","\u0445\u0430\u0439\u043b\u043e","\u0445\u0430\u043a\u0438\u0442","\u0445\u0430\u043a\u043a\u0430","\u0445\u0430\u043b\u0430\u043b","\u0445\u0430\u043b\u0430\u0442","\u0445\u0430\u043b\u0432\u0430","\u0445\u0430\u043c\u0451\u0436","\u0445\u0430\u043c\u043e\u043d","\u0445\u0430\u043c\u044c\u0451","\u0445\u0430\u043d\u0430\u0442","\u0445\u0430\u043d\u0436\u0430","\u0445\u0430\u043d\u043a\u0430","\u0445\u0430\u043e\u043c\u0430","\u0445\u0430\u043f\u043a\u0430","\u0445\u0430\u043f\u0443\u043d","\u0445\u0430\u0440\u0430\u043c","\u0445\u0430\u0440\u0438\u043e","\u0445\u0430\u0440\u0447\u0438","\u0445\u0430\u0440\u0447\u043e","\u0445\u0430\u0441\u0442\u0430","\u0445\u0430\u0442\u043a\u0430","\u0445\u0430\u0443\u0441\u0430","\u0445\u0430\u0448\u0430\u0440","\u0445\u0432\u0430\u043b\u0430","\u0445\u0432\u043e\u0440\u044c","\u0445\u0432\u043e\u0441\u0442","\u0445\u0435\u0432\u0435\u044f","\u0445\u0435\u0434\u0435\u0440","\u0445\u0435\u0439\u0438\u0442","\u0445\u0435\u043b\u0430\u0442","\u0445\u0435\u043c\u043e\u0437","\u0445\u0435\u043d\u0434\u0436","\u0445\u0435\u0440\u0435\u043c","\u0445\u0435\u0440\u0435\u0441","\u0445\u0435\u0440\u0438\u043a","\u0445\u0435\u0440\u043d\u044f","\u0445\u0438\u0430\u0437\u043c","\u0445\u0438\u043b\u0435\u0437","\u0445\u0438\u043b\u0443\u0441","\u0445\u0438\u043c\u0438\u044f","\u0445\u0438\u043c\u043a\u0430","\u0445\u0438\u043c\u0443\u0441","\u0445\u0438\u043d\u0434\u0438","\u0445\u0438\u043d\u0438\u043d","\u0445\u0438\u043d\u043e\u043d","\u0445\u0438\u043f\u0435\u0436","\u0445\u0438\u043f\u043f\u0438","\u0445\u0438\u0442\u0438\u043d","\u0445\u0438\u0442\u043e\u043d","\u0445\u043b\u0443\u043f\u044c","\u0445\u043b\u044b\u0441\u0442","\u0445\u043b\u044e\u0441\u0442","\u0445\u043b\u044f\u0431\u044c","\u0445\u043c\u0430\u0440\u044c","\u0445\u043c\u0435\u043b\u044c","\u0445\u043c\u0443\u0440\u044c","\u0445\u043e\u0430\u043d\u0430","\u0445\u043e\u0431\u0431\u0438","\u0445\u043e\u0431\u043e\u0442","\u0445\u043e\u0434\u043a\u0430","\u0445\u043e\u0434\u043e\u043a","\u0445\u043e\u0434\u0443\u043d","\u0445\u043e\u0437\u0430\u0440","\u0445\u043e\u043a\u043a\u0443","\u0445\u043e\u043b\u0438\u043c","\u0445\u043e\u043b\u0438\u043d","\u0445\u043e\u043b\u043a\u0430","\u0445\u043e\u043b\u043e\u0434","\u0445\u043e\u043b\u043e\u043d","\u0445\u043e\u043b\u0441\u0442","\u0445\u043e\u043b\u044c\u0434","\u0445\u043e\u043c\u0443\u0442","\u0445\u043e\u043c\u044f\u043a","\u0445\u043e\u043d\u0434\u0430","\u0445\u043e\u0440\u0430\u043b","\u0445\u043e\u0440\u0430\u0440","\u0445\u043e\u0440\u0434\u0430","\u0445\u043e\u0440\u0435\u0439","\u0445\u043e\u0440\u0435\u044f","\u0445\u043e\u0440\u043e\u0441","\u0445\u043e\u0440\u0443\u0441","\u0445\u043e\u0441\u0442\u0430","\u0445\u043e\u0445\u043c\u0430","\u0445\u043e\u0445\u043e\u043b","\u0445\u043e\u0445\u043e\u0442","\u0445\u043e\u0448\u0435\u043d","\u0445\u043e\u0448\u0443\u043d","\u0445\u0440\u0435\u043d\u044c","\u0445\u0440\u0443\u0441\u0442","\u0445\u0440\u044f\u043f\u0430","\u0445\u0440\u044f\u0441\u043a","\u0445\u0440\u044f\u0441\u0442","\u0445\u0442\u043e\u043d\u044c","\u0445\u0443\u0435\u0442\u0430","\u0445\u0443\u0438\u043b\u043e","\u0445\u0443\u0438\u043d\u0430","\u0445\u0443\u0438\u0449\u0435","\u0445\u0443\u0439\u043b\u043e","\u0445\u0443\u0439\u043d\u044f","\u0445\u0443\u043a\u043e\u0443","\u0445\u0443\u043c\u0443\u0441","\u0445\u0443\u043c\u0447\u0438","\u0445\u0443\u043d\u0442\u0430","\u0445\u0443\u0440\u0430\u043b","\u0445\u0443\u0440\u043c\u0430","\u0445\u0443\u0440\u0443\u043b","\u0445\u0443\u0442\u0431\u0430","\u0445\u0443\u0442\u043e\u0440","\u0445\u0443\u0446\u043f\u0430","\u0445\u0443\u044f\u043a\u0430","\u0445\u0443\u044f\u0440\u0430","\u0445\u044b\u0447\u0438\u043d","\u0445\u044d\u0442\u043e\u043a","\u0445\u044e\u0433\u0433\u0435","\u0445\u044f\u043d\u0433\u0430","\u0446\u0430\u0431\u0430\u043d","\u0446\u0430\u043d\u0433\u0430","\u0446\u0430\u043f\u043a\u0430","\u0446\u0430\u043f\u0444\u0430","\u0446\u0430\u0440\u0433\u0430","\u0446\u0430\u0446\u043a\u0430","\u0446\u0432\u0435\u043b\u044c","\u0446\u0432\u0435\u0442\u044c","\u0446\u0435\u0432\u043a\u0430","\u0446\u0435\u0432\u044c\u0451","\u0446\u0435\u0434\u0440\u0430","\u0446\u0435\u0437\u0438\u0439","\u0446\u0435\u043b\u0438\u043a","\u0446\u0435\u043b\u043a\u0430","\u0446\u0435\u043b\u043b\u0430","\u0446\u0435\u043b\u043e\u0435","\u0446\u0435\u043b\u043e\u043c","\u0446\u0435\u043d\u043e\u0437","\u0446\u0435\u043d\u0442\u0440","\u0446\u0435\u043d\u0443\u0440","\u0446\u0435\u043f\u043a\u0430","\u0446\u0435\u043f\u043d\u044f","\u0446\u0435\u0440\u0438\u0439","\u0446\u0435\u0440\u0438\u0442","\u0446\u0435\u0440\u043a\u0430","\u0446\u0435\u0442\u0430\u043d","\u0446\u0435\u0445\u0438\u043d","\u0446\u0435\u0448\u043a\u0430","\u0446\u0437\u0438\u043d\u044c","\u0446\u0438\u0431\u0435\u0442","\u0446\u0438\u0431\u0438\u043a","\u0446\u0438\u0433\u0443\u043d","\u0446\u0438\u0437\u0438\u0442","\u0446\u0438\u043a\u043b\u044f","\u0446\u0438\u043c\u0435\u043d","\u0446\u0438\u043c\u0435\u0441","\u0446\u0438\u043c\u043e\u043b","\u0446\u0438\u043c\u0443\u0441","\u0446\u0438\u043d\u0433\u0430","\u0446\u0438\u043d\u043a\u0430","\u0446\u0438\u043f\u0430\u043e","\u0446\u0438\u0440\u0435\u043d","\u0446\u0438\u0440\u043b\u044b","\u0446\u0438\u0441\u0442\u0430","\u0446\u0438\u0442\u043e\u0437","\u0446\u0438\u0442\u0440\u0430","\u0446\u0438\u0444\u0440\u0430","\u0446\u0438\u0446\u0435\u0441","\u0446\u0438\u0446\u0438\u0442","\u0446\u043e\u043a\u043e\u0442","\u0446\u0443\u043a\u0430\u0442","\u0446\u044b\u0431\u0438\u043a","\u0446\u044b\u043d\u0433\u0430","\u0446\u044b\u043f\u043a\u0438","\u0447\u0430\u0431\u0435\u0440","\u0447\u0430\u0431\u0451\u0440","\u0447\u0430\u0434\u0440\u0430","\u0447\u0430\u0438\u0449\u0435","\u0447\u0430\u0439\u043a\u0430","\u0447\u0430\u043a\u0430\u043d","\u0447\u0430\u043a\u0440\u0430","\u0447\u0430\u043b\u043a\u0430","\u0447\u0430\u043b\u043c\u0430","\u0447\u0430\u043c\u0430\u043d","\u0447\u0430\u043e\u0438\u0442","\u0447\u0430\u043f\u0430\u043d","\u0447\u0430\u0440\u0430\u0441","\u0447\u0430\u0440\u043a\u0430","\u0447\u0430\u0440\u043e\u043a","\u0447\u0430\u0440\u044b\u043c","\u0447\u0430\u0441\u0438\u043a","\u0447\u0430\u0441\u043e\u043a","\u0447\u0430\u0441\u0442\u044c","\u0447\u0430\u0442\u0438\u043a","\u0447\u0430\u0442\u043d\u0438","\u0447\u0430\u0444\u0430\u043d","\u0447\u0430\u0448\u043a\u0430","\u0447\u0451\u0431\u043e\u0442","\u0447\u0435\u0434\u0436\u0443","\u0447\u0435\u043a\u0430\u043d","\u0447\u0435\u043a\u0430\u043f","\u0447\u0451\u043b\u043a\u0430","\u0447\u0435\u043b\u0441\u0438","\u0447\u0435\u043d\u0434\u0436","\u0447\u0435\u043d\u0438\u0442","\u0447\u0435\u043f\u0430\u043d","\u0447\u0435\u043f\u0435\u0446","\u0447\u0435\u0440\u0430\u0437","\u0447\u0435\u0440\u0432\u0430","\u0447\u0435\u0440\u0432\u0438","\u0447\u0435\u0440\u0432\u044b","\u0447\u0435\u0440\u0432\u044c","\u0447\u0435\u0440\u0451\u0434","\u0447\u0435\u0440\u0435\u043d","\u0447\u0435\u0440\u0435\u043f","\u0447\u0435\u0440\u043d\u044c","\u0447\u0435\u0440\u0442\u0430","\u0447\u0451\u0441\u043a\u0430","\u0447\u0435\u0441\u0442\u044c","\u0447\u0451\u0442\u043a\u0438","\u0447\u0435\u0445\u0438\u0442","\u0447\u0435\u0445\u043e\u043b","\u0447\u0435\u0447\u043a\u0430","\u0447\u0435\u0448\u043a\u0430","\u0447\u0435\u0448\u0443\u044f","\u0447\u0438\u0436\u0438\u043a","\u0447\u0438\u043b\u0438\u043c","\u0447\u0438\u043c\u0447\u0438","\u0447\u0438\u043d\u0430\u0440","\u0447\u0438\u043d\u043a\u0430","\u0447\u0438\u043d\u0443\u043a","\u0447\u0438\u043f\u043e\u043a","\u0447\u0438\u0440\u0435\u0439","\u0447\u0438\u0440\u0438\u043a","\u0447\u0438\u0440\u044f\u043a","\u0447\u0438\u0441\u043b\u043e","\u0447\u0438\u0442\u043a\u0430","\u0447\u0438\u0442\u0442\u0430","\u0447\u0438\u0444\u0438\u0440","\u0447\u0438\u0447\u0435\u0440","\u0447\u043e\u0431\u043e\u0442","\u0447\u043e\u0432\u0430\u043d","\u0447\u043e\u043a\u0435\u0440","\u0447\u043e\u043a\u043e\u0439","\u0447\u043e\u043f\u0438\u043a","\u0447\u043e\u043f\u0438\u043d","\u0447\u0440\u0435\u0432\u043e","\u0447\u0440\u0435\u0434\u0430","\u0447\u0442\u0438\u0432\u043e","\u0447\u0443\u0430\u043d\u0430","\u0447\u0443\u0431\u0438\u043a","\u0447\u0443\u0431\u0443\u043a","\u0447\u0443\u0432\u0430\u043b","\u0447\u0443\u0432\u044f\u043a","\u0447\u0443\u0433\u0443\u043d","\u0447\u0443\u0436\u043e\u0435","\u0447\u0443\u0439\u043a\u0430","\u0447\u0443\u043b\u0430\u043d","\u0447\u0443\u043b\u043e\u043a","\u0447\u0443\u043c\u043a\u0430","\u0447\u0443\u0440\u0430\u043a","\u0447\u0443\u0440\u0435\u043a","\u0447\u0443\u0440\u043a\u0430","\u0447\u0443\u0442\u044c\u0451","\u0447\u0443\u0447\u0445\u0435","\u0447\u0443\u0448\u043a\u0430","\u0447\u0443\u0448\u043d\u044f","\u0447\u0443\u0448\u043e\u043a","\u0448\u0430\u0431\u0430\u0448","\u0448\u0430\u0431\u0435\u0440","\u0448\u0430\u0431\u043b\u0438","\u0448\u0430\u0431\u043e\u0442","\u0448\u0430\u0431\u0443\u0440","\u0448\u0430\u0432\u0440\u0443","\u0448\u0430\u0432\u044b\u0440","\u0448\u0430\u0434\u0443\u0444","\u0448\u0430\u0436\u043e\u043a","\u0448\u0430\u0439\u0431\u0430","\u0448\u0430\u0439\u043a\u0430","\u0448\u0430\u043a\u0442\u0438","\u0448\u0430\u043b\u0430\u0448","\u0448\u0430\u043b\u043e\u0442","\u0448\u0430\u043c\u0431\u043e","\u0448\u0430\u043c\u0438\u0440","\u0448\u0430\u043c\u043c\u0430","\u0448\u0430\u043c\u043e\u0442","\u0448\u0430\u043c\u0443\u0430","\u0448\u0430\u043d\u0435\u0446","\u0448\u0430\u043d\u043a\u0440","\u0448\u0430\u043d\u0442\u0438","\u0448\u0430\u043f\u043a\u0430","\u0448\u0430\u0440\u0438\u043a","\u0448\u0430\u0441\u043b\u0430","\u0448\u0430\u0441\u0441\u0435","\u0448\u0430\u0441\u0441\u0438","\u0448\u0430\u0442\u0451\u0440","\u0448\u0430\u0442\u0438\u044f","\u0448\u0430\u0442\u0442\u043b","\u0448\u0430\u0442\u0443\u043d","\u0448\u0430\u0442\u0443\u0448","\u0448\u0430\u0443\u0439\u044f","\u0448\u0430\u0445\u0435\u0446","\u0448\u0430\u0445\u043d\u0430","\u0448\u0430\u0445\u0442\u0430","\u0448\u0430\u0448\u043a\u0430","\u0448\u0430\u0448\u043a\u0438","\u0448\u0430\u0448\u043d\u0438","\u0448\u0432\u0430\u043b\u044c","\u0448\u0432\u0430\u043d\u043a","\u0448\u0432\u0430\u0440\u0430","\u0448\u0432\u0430\u0440\u0442","\u0448\u0432\u0435\u043f\u0441","\u0448\u0432\u0435\u0440\u0442","\u0448\u0435\u0432\u0435\u0440","\u0448\u0435\u0432\u0440\u043e","\u0448\u0435\u0432\u044f\u043a","\u0448\u0435\u0439\u043a\u0430","\u0448\u0435\u0439\u043c\u0430","\u0448\u0435\u043b\u0435\u043f","\u0448\u0435\u043b\u043e\u043c","\u0448\u0435\u043b\u043e\u043d","\u0448\u0435\u043b\u044c\u0444","\u0448\u0451\u043f\u043e\u0442","\u0448\u0435\u0440\u0440\u0438","\u0448\u0438\u0430\u0446\u0443","\u0448\u0438\u0431\u0435\u0440","\u0448\u0438\u0438\u0437\u043c","\u0448\u0438\u043b\u044c\u0435","\u0448\u0438\u043c\u043c\u0438","\u0448\u0438\u043d\u0438\u043b","\u0448\u0438\u043d\u043a\u0430","\u0448\u0438\u043d\u043e\u043a","\u0448\u0438\u043f\u0438\u043a","\u0448\u0438\u043f\u043e\u043a","\u0448\u0438\u0440\u043c\u0430","\u0448\u0438\u0442\u0432\u043e","\u0448\u0438\u0442\u0438\u043a","\u0448\u0438\u0442\u044c\u0451","\u0448\u0438\u0444\u0435\u0440","\u0448\u0438\u0444\u043e\u043d","\u0448\u0438\u0445\u0430\u043d","\u0448\u0438\u0445\u0442\u0430","\u0448\u0438\u0448\u0430\u043a","\u0448\u0438\u0448\u043a\u0430","\u0448\u043a\u0430\u043b\u0430","\u0448\u043a\u0430\u043d\u0442","\u0448\u043a\u0432\u0430\u043b","\u0448\u043a\u0435\u0440\u0442","\u0448\u043a\u0435\u0440\u044b","\u0448\u043a\u043e\u0434\u0430","\u0448\u043a\u043e\u043b\u0430","\u0448\u043a\u0443\u0440\u0430","\u0448\u043b\u0430\u043d\u0433","\u0448\u043b\u0435\u0439\u0444","\u0448\u043b\u0438\u0445\u0442","\u0448\u043b\u0438\u0446\u0430","\u0448\u043b\u044f\u043f\u0430","\u0448\u043c\u0430\u043b\u044c","\u0448\u043c\u0430\u0440\u0430","\u0448\u043c\u0443\u0440\u0430","\u0448\u043c\u044f\u043a\u0430","\u0448\u043d\u0430\u043f\u0441","\u0448\u043d\u0435\u043a\u0430","\u0448\u043d\u0438\u0444\u0442","\u0448\u043d\u044f\u0432\u0430","\u0448\u043d\u044f\u0433\u0430","\u0448\u043e\u0431\u043b\u0430","\u0448\u043e\u0434\u0434\u0438","\u0448\u043e\u043a\u0435\u0440","\u0448\u043e\u043f\u0435\u0440","\u0448\u043e\u043f\u043e\u0442","\u0448\u043e\u0440\u043d\u044f","\u0448\u043e\u0440\u043e\u0445","\u0448\u043e\u0440\u0442\u044b","\u0448\u043e\u0441\u0441\u0435","\u0448\u043e\u0441\u0441\u044b","\u0448\u043e\u0444\u0430\u0440","\u0448\u043f\u0430\u0433\u0430","\u0448\u043f\u0430\u043b\u0430","\u0448\u043f\u0438\u043b\u044c","\u0448\u043f\u043e\u043d\u0430","\u0448\u043f\u043e\u0440\u0430","\u0448\u043f\u0440\u0438\u0446","\u0448\u043f\u0443\u043b\u044f","\u0448\u043f\u0443\u043d\u0442","\u0448\u0440\u0435\u043d\u0446","\u0448\u0440\u0438\u043c\u0441","\u0448\u0440\u0438\u0444\u0442","\u0448\u0442\u0430\u043c\u0431","\u0448\u0442\u0430\u043c\u043c","\u0448\u0442\u0430\u043c\u043f","\u0448\u0442\u0430\u043d\u0446","\u0448\u0442\u0430\u043d\u044b","\u0448\u0442\u0435\u0439\u043d","\u0448\u0442\u0435\u0440\u0442","\u0448\u0442\u0438\u043b\u044c","\u0448\u0442\u0438\u0444\u0442","\u0448\u0442\u043e\u0440\u0430","\u0448\u0442\u043e\u0440\u043c","\u0448\u0442\u0440\u0430\u0444","\u0448\u0442\u0440\u0435\u043a","\u0448\u0442\u0440\u0438\u0445","\u0448\u0442\u0443\u043a\u0430","\u0448\u0442\u0443\u0440\u043c","\u0448\u0442\u044b\u0440\u044c","\u0448\u0443\u0431\u0430\u0442","\u0448\u0443\u0431\u0435\u0440","\u0448\u0443\u0431\u043a\u0430","\u0448\u0443\u0433\u0430\u0439","\u0448\u0443\u0439\u0446\u0430","\u0448\u0443\u043b\u044e\u043c","\u0448\u0443\u043c\u043e\u043a","\u0448\u0443\u0440\u043f\u0430","\u0448\u0443\u0440\u0443\u043f","\u0448\u0443\u0442\u0435\u0440","\u0448\u0443\u0442\u043a\u0430","\u0448\u0443\u0445\u0435\u0440","\u0448\u0443\u0448\u0443\u043d","\u0448\u0445\u0435\u0440\u0430","\u0448\u0445\u0443\u043d\u0430","\u0448\u044e\u0442\u0442\u0435","\u0449\u0435\u0431\u0435\u0442","\u0449\u0451\u043a\u043e\u0442","\u0449\u0451\u043b\u043a\u0430","\u0449\u0451\u043b\u043e\u043a","\u0449\u0435\u043d\u043a\u0430","\u0449\u0435\u043f\u043a\u0430","\u0449\u0451\u0442\u043a\u0430","\u0449\u0451\u0442\u043a\u0438","\u0449\u0451\u0447\u043a\u0430","\u0449\u0438\u043f\u0435\u0446","\u0449\u0438\u043f\u043a\u0430","\u0449\u0438\u043f\u043e\u043a","\u0449\u0438\u043f\u0446\u044b","\u0449\u0438\u0442\u043e\u043a","\u0449\u0438\u0448\u043a\u0438","\u0449\u0443\u043f\u0438\u043a","\u044b\u0433\u0440\u0435\u043a","\u044d\u0432\u0435\u0438\u0442","\u044d\u0432\u0435\u043d\u0442","\u044d\u0433\u0438\u0434\u0430","\u044d\u0433\u043b\u0435\u0442","\u044d\u0433\u0440\u0435\u0442","\u044d\u0434\u0435\u043c\u0430","\u044d\u0434\u0438\u043a\u0442","\u044d\u0439\u0434\u043e\u0441","\u044d\u043a\u0437\u0438\u0442","\u044d\u043a\u0437\u043e\u043d","\u044d\u043a\u0437\u043e\u0442","\u044d\u043a\u043a\u0435\u0440","\u044d\u043a\u043b\u0435\u0440","\u044d\u043a\u0440\u0430\u043d","\u044d\u043a\u0441\u043e\u0434","\u044d\u043a\u0441\u043f\u0430","\u044d\u043a\u0448\u0435\u043d","\u044d\u043a\u0448\u043e\u043d","\u044d\u043b\u0438\u0438\u0442","\u044d\u043b\u0438\u0442\u0430","\u044d\u043b\u044e\u0430\u0442","\u044d\u043c\u0430\u043b\u044c","\u044d\u043c\u0431\u043e\u043b","\u044d\u043c\u043c\u0435\u0440","\u044d\u043d\u0434\u0441\u044b","\u044d\u043d\u0437\u0438\u043c","\u044d\u043d\u0438\u043e\u043d","\u044d\u043d\u0442\u0435\u0440","\u044d\u043e\u0437\u0438\u043d","\u044d\u043e\u043b\u0438\u0442","\u044d\u043e\u0446\u0435\u043d","\u044d\u043f\u0438\u043a\u0430","\u044d\u043f\u043e\u043d\u0436","\u044d\u043f\u043e\u0445\u0430","\u044d\u043f\u044e\u0440\u0430","\u044d\u0440\u0431\u0438\u0439","\u044d\u0440\u0434\u0438\u0442","\u044d\u0440\u0437\u0430\u0446","\u044d\u0440\u0438\u043a\u0430","\u044d\u0440\u043a\u0435\u0440","\u044d\u0441\u043a\u0435\u0440","\u044d\u0441\u043a\u0438\u0437","\u044d\u0441\u043f\u0440\u0438","\u044d\u0441\u0441\u0438\u0432","\u044d\u0441\u0442\u0435\u0440","\u044d\u0442\u0432\u0435\u0448","\u044d\u0442\u0438\u043a\u0430","\u044d\u0442\u043d\u043e\u0441","\u044d\u0442\u0440\u0430\u043f","\u044d\u0442\u0440\u043e\u043b","\u044d\u0448\u0430\u0440\u043f","\u044d\u0448\u0435\u043b\u044c","\u044e\u0434\u043e\u043b\u044c","\u044e\u0437\u0438\u043d\u044c","\u044e\u043a\u0430\u0442\u0430","\u044e\u043a\u043e\u043b\u0430","\u044e\u043c\u0438\u0437\u043c","\u044e\u043d\u0433\u0438\u0442","\u044e\u043d\u0438\u0444\u0430","\u044e\u0440\u0430\u0433\u0430","\u044e\u0440\u0438\u0437\u043c","\u044e\u0440\u0438\u0438\u0442","\u044e\u0440\u0444\u0430\u043a","\u044e\u0441\u0441\u0438\u0432","\u044e\u0444\u0435\u0440\u0441","\u044e\u044d\u0447\u0436\u0438","\u044f\u0431\u0435\u0434\u0430","\u044f\u0433\u0435\u043b\u044c","\u044f\u0433\u0438\u0438\u0442","\u044f\u0433\u043e\u0434\u0430","\u044f\u0433\u043e\u0438\u0442","\u044f\u0433\u0443\u0440\u0442","\u044f\u0435\u043d\u0438\u043d","\u044f\u0437\u0438\u043d\u0430","\u044f\u0438\u0447\u043a\u043e","\u044f\u0438\u0448\u043d\u044f","\u044f\u043a\u043e\u0440\u044c","\u044f\u043a\u0443\u0431\u0430","\u044f\u043b\u0431\u043e\u0442","\u044f\u043c\u0435\u0433\u0430","\u044f\u043c\u0438\u043d\u0430","\u044f\u043c\u0438\u0449\u0430","\u044f\u043c\u043f\u0435\u0446","\u044f\u043d\u0442\u0440\u0430","\u044f\u0440\u0438\u0446\u0430","\u044f\u0440\u043b\u0438\u0442","\u044f\u0440\u043b\u044b\u043a","\u044f\u0440\u0443\u0433\u0430","\u044f\u0441\u0435\u043d\u044c","\u044f\u0441\u043c\u0438\u043d","\u044f\u0441\u043f\u0438\u0441","\u044f\u0441\u0442\u0430\u043a","\u044f\u0441\u0442\u0432\u043e","\u044f\u0441\u0442\u044b\u043a","\u044f\u0442\u043e\u0432\u044c","\u044f\u0445\u043e\u043d\u0442","\u044f\u0447\u043d\u0438\u043a","\u0430\u0443\u0434\u0438\u043e","\u043d\u0430\u0440\u0434\u044b","\u043d\u0430\u0440\u044b\u0432","\u0431\u0438\u0433\u043e\u0441","\u0445\u0430\u043c\u043e\u043d","\u0443\u043c\u043e\u0440\u0430","\u0434\u043e\u043d\u0430\u0442","\u043e\u043c\u043c\u0430\u0436","\u0430\u0431\u0431\u0430\u0442","\u0430\u0440\u0433\u043e\u043d","\u0431\u043e\u0440\u0430\u043d","\u043a\u043b\u043e\u043d\u044b","\u043a\u0435\u0433\u043b\u044f","\u043a\u0430\u043c\u0435\u043e","\u0430\u043d\u0438\u043c\u0435","\u0434\u0430\u043d\u0438\u044f","\u043a\u0435\u0433\u043b\u0438","\u0440\u043e\u043a\u0435\u0440","\u0445\u0443\u0439\u043d\u044f","\u043a\u0440\u0443\u043f\u0430","\u043a\u0430\u0440\u0433\u043e","\u0430\u0431\u0440\u0435\u043a","\u0430\u0431\u0445\u0430\u0437","\u0430\u0432\u0433\u0443\u0440","\u0430\u0432\u0442\u043e\u0440","\u0430\u0433\u0430\u043c\u0430","\u0430\u0433\u043d\u0430\u0442","\u0430\u0433\u043d\u0435\u0446","\u0430\u0433\u043d\u0446\u044b","\u0430\u0433\u0443\u0442\u0438","\u0430\u0434\u0435\u043f\u0442","\u0430\u0434\u0438\u0442\u0438","\u0430\u0437\u0438\u0430\u0442","\u0430\u0439\u0441\u043e\u0440","\u0430\u043a\u0442\u0435\u0440","\u0430\u043a\u0443\u043b\u0430","\u0430\u043b\u0435\u0443\u0442","\u0430\u043b\u043a\u0430\u0448","\u0430\u043b\u043b\u0430\u0445","\u0430\u043b\u043e\u043d\u0436","\u0430\u043b\u044c\u0432\u0430","\u0430\u043c\u0431\u0430\u043b","\u0430\u043c\u0435\u0431\u0430","\u0430\u043d\u0433\u0435\u043b","\u0430\u043d\u0448\u0435\u0444","\u0430\u0440\u0433\u0443\u0441","\u0430\u0440\u0434\u0435\u043d","\u0430\u0440\u0438\u0435\u0446","\u0430\u0440\u0445\u0430\u0440","\u0430\u0441\u043a\u0435\u0440","\u0430\u0441\u043a\u0435\u0442","\u0430\u0442\u043b\u0435\u0442","\u0430\u0444\u0438\u043d\u0430","\u0430\u0445\u0435\u0435\u0446","\u0430\u0446\u0442\u0435\u043a","\u0431\u0430\u0431\u0430\u0439","\u0431\u0430\u0431\u044c\u0435","\u0431\u0430\u0433\u0440\u044b","\u0431\u0430\u043a\u0448\u0438","\u0431\u0430\u043b\u044c\u0438","\u0431\u0430\u043d\u0442\u0443","\u0431\u0430\u0440\u0438\u0447","\u0431\u0430\u0440\u043e\u043d","\u0431\u0430\u0442\u0435\u043a","\u0431\u0430\u0442\u044b\u0440","\u0431\u0430\u0445\u0443\u0441","\u0431\u0430\u0445\u0448\u0438","\u0431\u0430\u0447\u043a\u0438","\u0431\u0435\u043a\u0430\u0441","\u0431\u0435\u043b\u0435\u043a","\u0431\u0435\u043b\u0435\u0446","\u0431\u0435\u043b\u043e\u0435","\u0431\u0435\u043b\u044b\u0439","\u0431\u0435\u043b\u0430\u044f","\u0431\u0435\u043b\u044c\u0435","\u0431\u0435\u043b\u044f\u043a","\u0431\u0435\u0440\u0434\u043e","\u0431\u0435\u0440\u0446\u0435","\u0431\u0438\u0432\u043d\u0438","\u0431\u0438\u0433\u0443\u0441","\u0431\u0438\u0437\u043e\u043d","\u0431\u0438\u043e\u043d\u0442","\u0431\u0438\u0440\u044e\u043a","\u0431\u0438\u0440\u044e\u0447","\u0431\u0438\u0442\u044b\u0439","\u0431\u0438\u0442\u044c\u0435","\u0431\u0438\u0442\u044e\u0433","\u0431\u043b\u043e\u0445\u0430","\u0431\u043b\u044e\u043c\u0441","\u0431\u043b\u044f\u0434\u044c","\u0431\u043e\u0431\u0435\u0440","\u0431\u043e\u0431\u0438\u043a","\u0431\u043e\u0433\u0430\u0447","\u0431\u043e\u0434\u0443\u043d","\u0431\u043e\u0439\u0446\u044b","\u0431\u043e\u0436\u043e\u043a","\u0431\u043e\u043d\u0437\u0430","\u0431\u043e\u043d\u043d\u0430","\u0431\u043e\u0440\u0446\u044b","\u0431\u043e\u0440\u043e\u0432","\u0431\u043e\u0441\u044f\u043a","\u0431\u043e\u0447\u0430\u0440","\u0431\u043e\u044f\u0440\u0435","\u0431\u0440\u0430\u0432\u043e","\u0431\u0440\u0438\u0442\u0442","\u0431\u0443\u0433\u0440\u044b","\u0431\u0443\u0434\u043d\u0438","\u0431\u0443\u0437\u0443\u043d","\u0431\u0443\u043a\u0430\u043d","\u0431\u0443\u0440\u044f\u0442","\u0431\u0443\u0442\u0443\u0437","\u0431\u044b\u043b\u044c\u0435","\u0431\u044b\u0442\u044c\u0435","\u0431\u044d\u043d\u0434\u0438","\u0432\u0430\u043b\u0430\u0445","\u0432\u0430\u043b\u0435\u043a","\u0432\u0430\u043b\u0435\u0440","\u0432\u0430\u043b\u0435\u0442","\u0432\u0430\u043b\u0443\u0445","\u0432\u0430\u0440\u0430\u043d","\u0432\u0430\u0440\u044f\u0433","\u0432\u0430\u0445\u043d\u044f","\u0432\u0434\u043e\u0432\u0430","\u0432\u0435\u0434\u0443\u043d","\u0432\u0435\u0437\u0438\u0440","\u0432\u0435\u0439\u043a\u0430","\u0432\u0435\u043b\u0438\u0442","\u0432\u0435\u043b\u044c\u0441","\u0432\u0435\u043d\u0433\u0440","\u0432\u0435\u043d\u0435\u0434","\u0432\u0435\u043d\u0435\u0442","\u0432\u0435\u043d\u0446\u044b","\u0432\u0435\u043f\u0440\u044c","\u0432\u0435\u0442\u0440\u044b","\u0432\u0435\u0449\u0435\u0435","\u0432\u0435\u0449\u0443\u043d","\u0432\u0437\u043b\u0435\u0442","\u0432\u0437\u043c\u0435\u0442","\u0432\u0438\u0432\u0435\u0440","\u0432\u0438\u0442\u0438\u044f","\u0432\u0438\u0442\u044c\u0435","\u0432\u0438\u0445\u0440\u044b","\u0432\u0438\u0448\u043d\u0443","\u0432\u043e\u0431\u043b\u0430","\u0432\u043e\u0433\u0443\u043b","\u0432\u043e\u0436\u0430\u043a","\u0432\u043e\u0436\u0434\u044c","\u0432\u043e\u043b\u043e\u0445","\u0432\u043e\u043b\u0445\u0432","\u0432\u043e\u0440\u043e\u0433","\u0432\u043e\u0440\u043e\u043d","\u0432\u043e\u0440\u044c\u0435","\u0432\u043e\u0442\u044f\u043a","\u0432\u043e\u0448\u043a\u0430","\u0432\u043e\u044f\u043a\u0430","\u0432\u0440\u0430\u043b\u044c","\u0432\u0440\u0443\u0448\u0430","\u0432\u044b\u0432\u0435\u0441","\u0432\u044b\u0434\u0440\u0430","\u0432\u044b\u0437\u043e\u043b","\u0432\u044b\u043c\u0430\u0445","\u0432\u044b\u0442\u044c\u0435","\u0432\u044b\u0447\u0435\u0442","\u0432\u044b\u0448\u0430\u0433","\u0432\u044f\u0442\u0438\u0447","\u0432\u044f\u0442\u043a\u0430","\u0433\u0430\u0434\u0435\u0441","\u0433\u0430\u043d\u0435\u0446","\u0433\u0430\u043d\u0446\u044b","\u0433\u0430\u043d\u0437\u0430","\u0433\u0430\u043d\u043a\u0430","\u0433\u0430\u0443\u0447\u043e","\u0433\u0435\u0440\u043b\u0441","\u0433\u0435\u0440\u043e\u0439","\u0433\u0435\u0442\u0438\u0442","\u0433\u0438\u0430\u0434\u044b","\u0433\u0438\u0434\u0440\u0430","\u0433\u0438\u0435\u043d\u0430","\u0433\u0438\u043b\u044f\u043a","\u0433\u0438\u0442\u0430\u043d","\u0433\u043b\u0435\u043d\u044c","\u0433\u043b\u0438\u043f\u0442","\u0433\u043b\u0438\u0441\u0442","\u0433\u043e\u043b\u0435\u043c","\u0433\u043e\u043b\u044b\u0439","\u0433\u043e\u043b\u0430\u044f","\u0433\u043e\u043b\u044b\u0435","\u0433\u043e\u043b\u044c\u0434","\u0433\u043e\u043b\u044c\u0435","\u0433\u043e\u043b\u044f\u043a","\u0433\u043e\u043d\u0435\u0446","\u0433\u043e\u043d\u0446\u044b","\u0433\u043e\u0440\u0430\u043b","\u0433\u043e\u0440\u0446\u044b","\u0433\u043e\u0440\u044e\u043d","\u0433\u043e\u0441\u0442\u044c","\u0433\u0440\u0430\u043d\u0434","\u0433\u0440\u0435\u0437\u0430","\u0433\u0443\u0431\u0430\u043d","\u0433\u0443\u043b\u0430\u0433","\u0433\u0443\u0440\u0430\u043d","\u0433\u0443\u0440\u0438\u044f","\u0433\u0443\u0441\u0430\u0440","\u0433\u0443\u0441\u0435\u043a","\u0433\u0443\u0441\u0438\u0442","\u0433\u0443\u0446\u0443\u043b","\u0433\u044d\u043f\u044d\u0443","\u0433\u044e\u0440\u0437\u0430","\u0434\u0430\u0439\u043d\u0430","\u0434\u0430\u0439\u0440\u0430","\u0434\u0435\u0431\u0438\u043b","\u0434\u0435\u0431\u0440\u044c","\u0434\u0435\u0432\u043a\u0430","\u0434\u0435\u0433\u0442\u0438","\u0434\u0435\u0434\u043a\u0430","\u0434\u0435\u0434\u043e\u043a","\u0434\u0435\u0436\u043a\u0430","\u0434\u0435\u0438\u0441\u0442","\u0434\u0435\u043b\u0435\u0436","\u0434\u0435\u043b\u0435\u0446","\u0434\u0435\u043d\u0434\u0438","\u0434\u0435\u043d\u0435\u043a","\u0434\u0435\u0440\u0435\u043d","\u0434\u0435\u0442\u043a\u0430","\u0434\u0436\u0438\u043d\u043d","\u0434\u0438\u0430\u043d\u0430","\u0434\u0438\u043a\u0438\u0439","\u0434\u0438\u043a\u0430\u044f","\u0434\u0438\u043a\u0438\u0435","\u0434\u0438\u043b\u0435\u0440","\u0434\u043e\u043d\u044c\u044f","\u0434\u043e\u043a\u0435\u0440","\u0434\u043e\u043c\u0435\u043a","\u0434\u043e\u043c\u0435\u0440","\u0434\u043e\u043d\u0435\u0446","\u0434\u043e\u043d\u0446\u044b","\u0434\u043e\u043d\u043d\u0430","\u0434\u043e\u043d\u043e\u0440","\u0434\u043e\u0441\u0443\u0433","\u0434\u043e\u0444\u0438\u043d","\u0434\u0440\u0435\u043c\u0430","\u0434\u0440\u043e\u0437\u0434","\u0434\u0440\u043e\u0444\u0430","\u0434\u0440\u0443\u0438\u0434","\u0434\u0443\u0431\u0446\u044b","\u0434\u0443\u0431\u044c\u0435","\u0434\u0443\u0434\u0430\u043a","\u0434\u0443\u043b\u0435\u0431","\u0434\u0443\u043c\u0435\u0446","\u0434\u0443\u043c\u0446\u044b","\u0434\u0443\u043d\u0435\u0446","\u0434\u0443\u043d\u0446\u044b","\u0434\u0443\u0440\u043d\u0438","\u0434\u0443\u0442\u044b\u0448","\u0434\u0443\u0442\u044c\u0435","\u0434\u0449\u0435\u0440\u044c","\u0434\u044b\u043b\u0434\u0430","\u0434\u044f\u0434\u0435\u043a","\u0434\u044f\u0434\u044c\u044f","\u0434\u044f\u0442\u0435\u043b","\u0434\u044f\u0442\u043b\u044b","\u0435\u0432\u043d\u0443\u0445","\u0435\u0432\u0440\u0435\u0439","\u0435\u0433\u0435\u0440\u044c","\u0435\u0433\u043e\u0437\u0430","\u0435\u0436\u0438\u0445\u0430","\u0435\u0436\u0430\u0442\u0430","\u0435\u0437\u0434\u043e\u043a","\u0435\u043c\u0435\u043b\u044f","\u0435\u0441\u0430\u0443\u043b","\u0435\u0441\u0441\u0435\u0439","\u0435\u0445\u0438\u0434\u0430","\u0436\u0435\u043b\u043d\u0430","\u0436\u0435\u043b\u043e\u0431","\u0436\u0435\u043d\u0438\u0445","\u0436\u0435\u043d\u043a\u0430","\u0436\u0435\u0440\u0435\u0445","\u0436\u0438\u0432\u0435\u0446","\u0436\u0438\u0432\u0446\u044b","\u0436\u0438\u0432\u043e\u0435","\u0436\u0438\u0432\u044b\u0435","\u0436\u0438\u0432\u044c\u0435","\u0436\u0438\u0434\u043e\u043a","\u0436\u0438\u043b\u0435\u0446","\u0436\u0438\u043b\u043e\u0435","\u0436\u0438\u043b\u044c\u0435","\u0436\u0438\u043d\u043a\u0430","\u0436\u0438\u0440\u0430\u0444","\u0436\u0438\u0440\u044f\u043a","\u0436\u0438\u0442\u044c\u0435","\u0436\u043d\u0438\u0446\u0430","\u0436\u043e\u043a\u0435\u0439","\u0436\u0440\u0438\u0446\u0430","\u0436\u0443\u043b\u0430\u043d","\u0436\u0443\u043b\u0438\u043a","\u0436\u0443\u043b\u044c\u0435","\u0436\u0443\u0440\u043a\u0430","\u0436\u0443\u0447\u043a\u0430","\u0437\u0430\u0432\u0443\u0447","\u0437\u0430\u0439\u043c\u044b","\u0437\u0430\u0438\u043a\u0430","\u0437\u0430\u0439\u043a\u0430","\u0437\u0430\u043a\u0443\u043f","\u0437\u0430\u043b\u0435\u0442","\u0437\u0430\u043c\u0435\u0442","\u0437\u0430\u0442\u0435\u043a","\u0437\u0430\u0442\u0435\u0441","\u0437\u0430\u0442\u044f\u0433","\u0437\u0430\u0447\u0435\u0441","\u0437\u0430\u0447\u0435\u0442","\u0437\u0430\u0439\u0446\u044b","\u0437\u0432\u0435\u0440\u044c","\u0437\u0435\u043c\u0435\u0446","\u0437\u0435\u043c\u0446\u044b","\u0437\u043b\u044e\u043a\u0430","\u0437\u043e\u043c\u0431\u0438","\u0437\u0443\u0431\u044c\u044f","\u0437\u0443\u0431\u0446\u044b","\u0437\u0443\u0434\u043d\u0438","\u0437\u044f\u0442\u0435\u043a","\u0437\u044f\u0442\u0438\u043a","\u0438\u0431\u043b\u0438\u0441","\u0438\u0432\u0430\u0441\u0438","\u0438\u0433\u0440\u0435\u0446","\u0438\u0433\u0440\u043e\u043a","\u0438\u0433\u0440\u0443\u043d","\u0438\u0434\u0438\u043e\u0442","\u0438\u0435\u0440\u0435\u0439","\u0438\u0437\u0431\u0430\u0447","\u0438\u0437\u0433\u043e\u0439","\u0438\u0437\u0438\u0434\u0430","\u0438\u0437\u043b\u0435\u0442","\u0438\u0437\u044e\u0431\u0440","\u0438\u043a\u0442\u0443\u0441","\u0438\u043b\u044c\u043a\u0430","\u0438\u043c\u0435\u043d\u0430","\u0438\u043d\u0433\u0443\u0448","\u0438\u043d\u0434\u0440\u0430","\u0438\u043d\u0434\u0443\u0441","\u0438\u043d\u0434\u044e\u043a","\u0438\u043e\u043c\u0435\u043d","\u0438\u043e\u043c\u0443\u0434","\u0438\u043e\u043c\u0443\u0442","\u0438\u043e\u043d\u0438\u044f","\u0438\u0440\u0431\u0438\u0441","\u0438\u0440\u0438\u0434\u0430","\u0438\u0441\u0438\u0434\u0430","\u0438\u0441\u0442\u0435\u0446","\u0438\u0441\u0442\u0446\u044b","\u0438\u0442\u044d\u044d\u0440","\u0438\u0443\u0434\u0435\u0439","\u0438\u0444\u0440\u0438\u0442","\u0439\u043e\u043c\u0435\u043d","\u043a\u0430\u0430\u0431\u0430","\u043a\u0430\u0431\u0430\u043d","\u043a\u0430\u0431\u0438\u043b","\u043a\u0430\u0433\u0430\u043d","\u043a\u0430\u0434\u0435\u0442","\u043a\u0430\u0434\u0438\u0448","\u043a\u0430\u0437\u0430\u043a","\u043a\u0430\u0437\u0430\u0445","\u043a\u0430\u0439\u0440\u0430","\u043a\u0430\u043b\u0430\u043d","\u043a\u0430\u043b\u0431\u0430","\u043a\u0430\u043b\u0438\u0444","\u043a\u0430\u043c\u043d\u0438","\u043a\u0430\u043c\u0441\u0430","\u043a\u0430\u043d\u044e\u043a","\u043a\u0430\u0440\u0438\u0431","\u043a\u0430\u0441\u0438\u043a","\u043a\u0430\u0446\u0430\u043f","\u043a\u0430\u0446\u0438\u043a","\u043a\u0430\u0448\u043b\u0438","\u043a\u0430\u0448\u0443\u0431","\u043a\u0430\u0449\u0435\u0439","\u043a\u0435\u043d\u0430\u0440","\u043a\u0438\u0434\u0430\u0441","\u043a\u0438\u043c\u0432\u0440","\u043a\u0438\u043d\u0438\u043a","\u043a\u0438\u043f\u043a\u0430","\u043a\u043b\u0435\u043a\u0442","\u043a\u043b\u0435\u0440\u043a","\u043a\u043b\u0435\u0441\u0442","\u043a\u043b\u0435\u0444\u0442","\u043a\u043b\u043e\u0443\u043d","\u043a\u043b\u0443\u0448\u0430","\u043a\u043b\u044e\u0444\u0442","\u043a\u043b\u044f\u0447\u0430","\u043a\u043d\u0438\u043a\u0441","\u043a\u043d\u044f\u0437\u044c","\u043a\u043e\u0430\u043b\u0430","\u043a\u043e\u0430\u0442\u0430","\u043a\u043e\u0431\u043b\u044b","\u043a\u043e\u0431\u0440\u0430","\u043a\u043e\u0432\u0430\u0447","\u043a\u043e\u0432\u0435\u0440","\u043a\u043e\u0432\u0440\u044b","\u043a\u043e\u0433\u0442\u0438","\u043a\u043e\u0436\u044c\u0435","\u043a\u043e\u0437\u0435\u043b","\u043a\u043e\u0439\u043e\u0442","\u043a\u043e\u043a\u0435\u0442","\u043a\u043e\u043b\u044c\u044f","\u043a\u043e\u043b\u043b\u0438","\u043a\u043e\u043c\u044c\u044f","\u043a\u043e\u043c\u0430\u0440","\u043a\u043e\u043c\u043b\u0438","\u043a\u043e\u043c\u0438\u043a","\u043a\u043e\u043d\u0435\u043a","\u043a\u043e\u043d\u0446\u044b","\u043a\u043e\u043d\u044e\u0445","\u043a\u043e\u043f\u0430\u0447","\u043a\u043e\u043f\u0435\u0440","\u043a\u043e\u043f\u0440\u044b","\u043a\u043e\u043f\u0443\u043d","\u043a\u043e\u043f\u044c\u0435","\u043a\u043e\u0440\u0430\u043d","\u043a\u043e\u0440\u043d\u0438","\u043a\u043e\u0440\u0446\u044b","\u043a\u043e\u0440\u0435\u0448","\u043a\u043e\u0440\u0447\u044c","\u043a\u043e\u0440\u044c\u0435","\u043a\u043e\u0440\u044f\u043a","\u043a\u043e\u0441\u0430\u0447","\u043a\u043e\u0441\u0430\u044f","\u043a\u043e\u0441\u044b\u0435","\u043a\u043e\u0441\u0435\u0446","\u043a\u043e\u0441\u0446\u044b","\u043a\u043e\u0441\u043c\u0430","\u043a\u043e\u0441\u043e\u0439","\u043a\u043e\u0441\u044c\u0435","\u043a\u043e\u0442\u0435\u043b","\u043a\u043e\u0442\u0438\u043a","\u043a\u043e\u0442\u043e\u043a","\u043a\u043e\u0446\u0438\u0442","\u043a\u043e\u0447\u043d\u0438","\u043a\u043e\u0447\u0435\u0442","\u043a\u043e\u0449\u0435\u0439","\u043a\u0440\u0430\u043b\u044f","\u043a\u0440\u0435\u043e\u043b","\u043a\u0440\u043e\u0430\u0442","\u043a\u0443\u0434\u043b\u044b","\u043a\u0443\u0437\u0435\u043d","\u043a\u0443\u043a\u0448\u0430","\u043a\u0443\u043b\u0430\u0436","\u043a\u0443\u043b\u0430\u043d","\u043a\u0443\u043b\u0435\u043a","\u043a\u0443\u043b\u0438\u043a","\u043a\u0443\u043c\u0436\u0430","\u043a\u0443\u043c\u044b\u043a","\u043a\u0443\u043d\u0430\u043a","\u043a\u0443\u043d\u044c\u0438","\u043a\u0443\u043f\u0435\u0446","\u043a\u0443\u043f\u0446\u044b","\u043a\u0443\u0440\u0432\u0430","\u043a\u0443\u0440\u0435\u0446","\u043a\u0443\u0440\u0446\u044b","\u043a\u0443\u0442\u0435\u0436","\u043a\u0443\u0442\u0443\u043c","\u043a\u0443\u0447\u0435\u0440","\u043a\u0445\u043c\u0435\u0440","\u043b\u0430\u0431\u0435\u0442","\u043b\u0430\u0437\u043a\u0430","\u043b\u0430\u043a\u0435\u0439","\u043b\u0430\u043a\u0435\u0446","\u043b\u0430\u043a\u0446\u044b","\u043b\u0430\u043c\u0443\u0442","\u043b\u0430\u043d\u043a\u0430","\u043b\u0430\u043f\u0442\u0438","\u043b\u0430\u0440\u0433\u043e","\u043b\u0430\u0440\u0435\u043a","\u043b\u0430\u0440\u0446\u044b","\u043b\u0430\u0442\u0438\u043d","\u043b\u0430\u0442\u044b\u0448","\u043b\u0430\u0447\u043a\u0430","\u043b\u0435\u0432\u0438\u0442","\u043b\u0435\u0432\u0448\u0430","\u043b\u0435\u0432\u044b\u0439","\u043b\u0435\u0433\u0430\u0442","\u043b\u0435\u0433\u0430\u0448","\u043b\u0435\u0436\u043d\u0438","\u043b\u0435\u0436\u043a\u0430","\u043b\u0435\u043a\u0430\u0436","\u043b\u0435\u043c\u0435\u0448","\u043b\u0435\u043c\u0443\u0440","\u043b\u0435\u0442\u0443\u043d","\u043b\u0435\u0448\u0430\u043a","\u043b\u0435\u0448\u0438\u0439","\u043b\u0435\u0449\u0438\u043a","\u043b\u0438\u0432\u043d\u0438","\u043b\u0438\u0432\u043a\u0430","\u043b\u0438\u0434\u0435\u0440","\u043b\u0438\u043a\u0435\u0440","\u043b\u0438\u043d\u0435\u043a","\u043b\u0438\u043f\u0446\u044b","\u043b\u0438\u0440\u0438\u043a","\u043b\u0438\u0441\u043a\u0430","\u043b\u0438\u0442\u044c\u0435","\u043b\u0438\u0445\u0430\u0447","\u043b\u043e\u0431\u0430\u043d","\u043b\u043e\u0432\u0435\u0446","\u043b\u043e\u0432\u0446\u044b","\u043b\u043e\u0433\u0438\u043a","\u043b\u043e\u043a\u0442\u0438","\u043b\u043e\u043c\u0442\u0438","\u043b\u043e\u0448\u0430\u043a","\u043b\u0443\u0431\u044c\u044f","\u043b\u044b\u0441\u0443\u043d","\u043b\u044b\u0441\u044b\u0439","\u043b\u044e\u0431\u0432\u0438","\u043b\u044e\u0431\u043e\u0435","\u043b\u044e\u0431\u043e\u0439","\u043b\u044e\u0431\u0430\u044f","\u043b\u044e\u0431\u044b\u0435","\u043b\u044e\u0440\u0438\u043a","\u043b\u044e\u0442\u0438\u0447","\u043b\u044f\u0433\u0432\u0430","\u043b\u044f\u0440\u0432\u0430","\u043b\u044f\u0448\u043a\u0430","\u043c\u0430\u0433\u0438\u043a","\u043c\u0430\u0433\u043e\u0442","\u043c\u0430\u0434\u0430\u043c","\u043c\u0430\u0437\u043b\u043e","\u043c\u0430\u0437\u0443\u0440","\u043c\u0430\u0439\u0435\u0440","\u043c\u0430\u0439\u043e\u0440","\u043c\u0430\u043b\u0435\u043a","\u043c\u0430\u043b\u0435\u0446","\u043c\u0430\u043b\u043e\u0435","\u043c\u0430\u043b\u044b\u0439","\u043c\u0430\u043b\u044c\u0435","\u043c\u0430\u043b\u044f\u0440","\u043c\u0430\u043c\u0430\u0439","\u043c\u0430\u043c\u0430\u043d","\u043c\u0430\u043c\u043a\u0430","\u043c\u0430\u043d\u0441\u0438","\u043c\u0430\u043d\u0443\u043b","\u043c\u0430\u043e\u0440\u0438","\u043c\u0430\u0440\u0430\u043b","\u043c\u0430\u0440\u0430\u043d","\u043c\u0430\u0441\u0430\u0438","\u043c\u0430\u0441\u043e\u043d","\u043c\u0430\u0442\u043e\u043a","\u043c\u0430\u0445\u0430\u043d","\u043c\u0430\u0447\u043e\u043a","\u043c\u0435\u0434\u0435\u044f","\u043c\u0435\u0434\u0438\u043a","\u043c\u0435\u043a\u043a\u0430","\u043c\u0435\u043b\u0438\u0441","\u043c\u0435\u0440\u0435\u044f","\u043c\u0435\u0441\u044c\u0435","\u043c\u0435\u0442\u0435\u043a","\u043c\u0435\u0442\u0438\u0441","\u043c\u0438\u0433\u0430\u0447","\u043c\u0438\u043b\u043e\u0435","\u043c\u0438\u043b\u044b\u0439","\u043c\u0438\u043b\u0430\u044f","\u043c\u0438\u043b\u044b\u0435","\u043c\u0438\u043b\u044c\u0442","\u043c\u0438\u043c\u0438\u043a","\u043c\u0438\u043d\u0435\u0440","\u043c\u0438\u043d\u043e\u0441","\u043c\u0438\u0440\u0430\u0431","\u043c\u0438\u0440\u0437\u0430","\u043c\u0438\u0440\u043e\u043d","\u043c\u0438\u0448\u043a\u0430","\u043c\u0438\u0448\u0443\u043a","\u043c\u043e\u0433\u043e\u043b","\u043c\u043e\u0439\u0432\u0430","\u043c\u043e\u0439\u0440\u0430","\u043c\u043e\u043a\u043e\u0439","\u043c\u043e\u043b\u043e\u0445","\u043c\u043e\u043d\u0430\u0445","\u043c\u043e\u0441\u043b\u044b","\u043c\u043e\u0441\u044c\u0435","\u043c\u043e\u0445\u043d\u0430","\u043c\u0443\u0434\u0430\u043a","\u043c\u0443\u0436\u044c\u044f","\u043c\u0443\u0436\u0438\u043a","\u043c\u0443\u043b\u0430\u0442","\u043c\u0443\u043b\u0435\u043a","\u043c\u0443\u043b\u043b\u0430","\u043c\u0443\u043c\u0438\u0435","\u043c\u0443\u0440\u0430\u0448","\u043c\u0443\u0440\u0434\u0430","\u043c\u0443\u0440\u0437\u0430","\u043c\u0443\u0440\u0438\u043d","\u043c\u0443\u0440\u043a\u0430","\u043c\u0443\u0448\u0430\u0440","\u043c\u044b\u043c\u0440\u0430","\u043c\u044b\u0442\u044c\u0435","\u043c\u044e\u0440\u0438\u0434","\u043c\u044f\u043c\u043b\u044f","\u043c\u044f\u0442\u044c\u0435","\u043d\u0430\u0431\u043e\u0431","\u043d\u0430\u0439\u043c\u044b","\u043d\u0430\u0436\u0438\u0433","\u043d\u0430\u0437\u0435\u043c","\u043d\u0430\u043a\u043e\u043b","\u043d\u0430\u043a\u0440\u0430","\u043d\u0430\u043b\u0435\u0442","\u043d\u0430\u043b\u0438\u043c","\u043d\u0430\u043c\u0435\u043a","\u043d\u0430\u043c\u0435\u0442","\u043d\u0430\u043d\u0430\u0439","\u043d\u0430\u043d\u0434\u0443","\u043d\u0430\u0440\u0443\u0447","\u043d\u0430\u0442\u0435\u043a","\u043d\u0430\u0442\u0435\u0441","\u043d\u0430\u0445\u0430\u043b","\u043d\u0430\u0447\u0435\u0441","\u043d\u0430\u0447\u0435\u0442","\u043d\u0430\u044f\u0434\u0430","\u043d\u0435\u0432\u0435\u0440","\u043d\u0435\u0433\u0443\u0441","\u043d\u0435\u043c\u0435\u0446","\u043d\u0435\u043c\u0446\u044b","\u043d\u0435\u043c\u043a\u0430","\u043d\u0435\u043c\u043e\u0439","\u043d\u0435\u043c\u0430\u044f","\u043d\u0435\u043c\u044b\u0435","\u043d\u0435\u043d\u0435\u0446","\u043d\u0435\u043d\u0446\u044b","\u043d\u0435\u043d\u043a\u0430","\u043d\u0435\u0440\u043a\u0430","\u043d\u0435\u0440\u043f\u0430","\u043d\u0435\u0441\u0443\u043d","\u043d\u0435\u0442\u044f\u0433","\u043d\u0438\u0432\u0443\u0445","\u043d\u0438\u043a\u0441\u0430","\u043d\u0438\u043a\u0442\u043e","\u043d\u0438\u043b\u043e\u0442","\u043d\u0438\u043c\u0444\u0430","\u043d\u0438\u0447\u0442\u043e","\u043d\u0438\u0449\u0438\u0439","\u043d\u0438\u0449\u0430\u044f","\u043d\u0438\u0449\u0438\u0435","\u043d\u043e\u0432\u0438\u043a","\u043d\u043e\u0433\u0430\u0439","\u043d\u043e\u0433\u0442\u0438","\u043d\u043e\u0439\u043e\u043d","\u043d\u043e\u043c\u0430\u0434","\u043d\u043e\u0441\u0430\u0447","\u043d\u043e\u0447\u0432\u0430","\u043d\u0443\u0434\u0433\u0430","\u043d\u0443\u043a\u0435\u0440","\u043d\u044b\u0440\u0435\u0446","\u043d\u044b\u0440\u0446\u044b","\u043d\u044b\u0442\u0438\u043a","\u043d\u044b\u0442\u044c\u0435","\u043e\u0431\u0434\u0435\u043b","\u043e\u0431\u043b\u0435\u0442","\u043e\u0431\u043c\u0435\u0442","\u043e\u0431\u043e\u0434\u044c","\u043e\u0431\u0447\u0435\u0442","\u043e\u0431\u044a\u0435\u043c","\u043e\u0432\u0438\u0441\u0442","\u043e\u0432\u043e\u0449\u044c","\u043e\u0432\u0447\u0430\u0440","\u043e\u0434\u0435\u0436\u0430","\u043e\u0439\u0440\u043e\u0442","\u043e\u043a\u0430\u043f\u0438","\u043e\u043a\u043e\u0435\u043c","\u043e\u043a\u0443\u043d\u044c","\u043e\u043b\u0435\u043d\u044c","\u043e\u043c\u0443\u043b\u044c","\u043e\u043f\u044f\u0442\u0430","\u043e\u043f\u0442\u0438\u043a","\u043e\u0440\u0438\u043e\u043d","\u043e\u0440\u043b\u0430\u043d","\u043e\u0440\u043b\u0438\u043a","\u043e\u0440\u0444\u0435\u0439","\u043e\u0441\u0435\u0442\u0440","\u043e\u0441\u043b\u0438\u043a","\u043e\u0441\u043b\u044f\u043a","\u043e\u0441\u043c\u0430\u043d","\u043e\u0441\u043e\u0431\u0430","\u043e\u0441\u043e\u0431\u044c","\u043e\u0441\u043e\u0435\u0434","\u043e\u0441\u0442\u0430\u0448","\u043e\u0441\u0442\u044f\u043a","\u043e\u0442\u043b\u0435\u0442","\u043e\u0442\u043c\u043e\u043b","\u043e\u0442\u0440\u043e\u043a","\u043e\u0442\u0442\u0435\u043a","\u043e\u0442\u0447\u0435\u0442","\u043e\u0442\u0447\u0438\u043c","\u043e\u0442\u044a\u0435\u043c","\u043e\u0444\u0435\u043d\u044f","\u043e\u0444\u0444\u0438\u0441","\u043e\u0447\u043d\u0438\u043a","\u043e\u0448\u043a\u0443\u0439","\u043f\u0430\u0434\u043b\u0430","\u043f\u0430\u043b\u0430\u0447","\u043f\u0430\u043b\u0438\u044f","\u043f\u0430\u043d\u0435\u0440","\u043f\u0430\u043d\u044b\u0447","\u043f\u0430\u043f\u0430\u0445","\u043f\u0430\u043f\u0435\u0436","\u043f\u0430\u0440\u043d\u0438","\u043f\u0430\u0440\u0438\u044f","\u043f\u0430\u0440\u043d\u044f","\u043f\u0430\u0441\u044e\u043a","\u043f\u0430\u0442\u0430\u043d","\u043f\u0430\u0442\u0435\u0440","\u043f\u0430\u0442\u0443\u0430","\u043f\u0430\u0446\u0430\u043d","\u043f\u0430\u044f\u043b\u043e","\u043f\u0435\u0432\u043d\u0438","\u043f\u0435\u0432\u0435\u0446","\u043f\u0435\u0432\u0446\u044b","\u043f\u0435\u0432\u0443\u043d","\u043f\u0435\u0433\u0430\u0441","\u043f\u0435\u043d\u0433\u0435","\u043f\u0435\u043d\u0435\u043a","\u043f\u0435\u043f\u043b\u044b","\u043f\u0435\u0440\u0446\u044b","\u043f\u0435\u0440\u043a\u0430","\u043f\u0435\u0440\u043b\u043e","\u043f\u0435\u0440\u044c\u044f","\u043f\u0435\u0441\u0446\u044b","\u043f\u0435\u0441\u0438\u043a","\u043f\u0435\u0442\u0435\u043b","\u043f\u0435\u0442\u0443\u0445","\u043f\u0435\u0448\u0435\u0446","\u043f\u0435\u0448\u0446\u044b","\u043f\u0435\u0448\u0438\u0439","\u043f\u0435\u0448\u0430\u044f","\u043f\u0435\u0448\u0438\u0435","\u043f\u0438\u0436\u043e\u043d","\u043f\u0438\u043a\u0448\u0430","\u043f\u0438\u043b\u0430\u0442","\u043f\u0438\u043c\u043e\u043a","\u043f\u0438\u0440\u0430\u0442","\u043f\u0438\u0441\u0435\u0446","\u043f\u0438\u0441\u0446\u044b","\u043f\u0438\u0441\u0443\u043d","\u043f\u0438\u0442\u043e\u043a","\u043f\u0438\u0442\u0443\u0445","\u043f\u0438\u0442\u044c\u0435","\u043f\u0438\u0444\u0438\u044f","\u043f\u043b\u0430\u043a\u0435","\u043f\u043b\u0435\u0441\u043e","\u043f\u043b\u043e\u0445\u043e","\u043f\u043e\u0432\u0430\u0440","\u043f\u043e\u0437\u0435\u043c","\u043f\u043e\u0437\u0435\u0440","\u043f\u043e\u0437\u044b\u043a","\u043f\u043e\u043b\u0435\u0442","\u043f\u043e\u043b\u0438\u043f","\u043f\u043e\u043b\u0438\u0440","\u043f\u043e\u043b\u044f\u043a","\u043f\u043e\u043b\u044f\u0448","\u043f\u043e\u043c\u0435\u0442","\u043f\u043e\u043c\u043e\u0440","\u043f\u043e\u043f\u0438\u043a","\u043f\u043e\u0440\u043e\u0437","\u043f\u043e\u0440\u043e\u0441","\u043f\u043e\u0441\u043b\u044b","\u043f\u043e\u0442\u0435\u043a","\u043f\u043e\u0445\u0443\u043b","\u043f\u043e\u0447\u0435\u0442","\u043f\u0440\u0438\u0430\u043f","\u043f\u0440\u0438\u0435\u043c","\u043f\u0440\u0438\u043d\u0446","\u043f\u0440\u0438\u043e\u0440","\u043f\u0440\u043e\u0435\u043c","\u043f\u0440\u043e\u0444\u0438","\u043f\u0440\u0443\u0441\u0441","\u043f\u0440\u044f\u0445\u0430","\u043f\u0441\u0430\u0440\u044c","\u043f\u0441\u0438\u0446\u0430","\u043f\u0442\u0430\u0445\u0430","\u043f\u0442\u0438\u0446\u0430","\u043f\u0443\u0437\u0430\u043d","\u043f\u0443\u0437\u0430\u0447","\u043f\u0443\u043d\u0446\u044b","\u043f\u0443\u0441\u0442\u043e","\u043f\u0443\u0442\u0446\u044b","\u043f\u0443\u0447\u043a\u0430","\u043f\u0443\u0448\u0442\u0430","\u043f\u0447\u0435\u043b\u0430","\u043f\u044c\u0435\u0440\u043e","\u043f\u044d\u0442\u044d\u0443","\u043f\u044f\u0442\u044b\u0439","\u043f\u044f\u0442\u0430\u044f","\u043f\u044f\u0442\u044b\u0435","\u0440\u0430\u0434\u0436\u0430","\u0440\u0430\u043a\u0443\u043d","\u0440\u0430\u043a\u0448\u0430","\u0440\u0430\u043d\u0446\u044b","\u0440\u0430\u0442\u0430\u0439","\u0440\u0430\u0447\u043e\u043a","\u0440\u0435\u0433\u0438\u044f","\u0440\u0435\u0437\u0446\u044b","\u0440\u0435\u043b\u0438\u0442","\u0440\u0435\u043c\u0435\u0437","\u0440\u0435\u043c\u043d\u0438","\u0440\u0435\u043f\u044c\u0438","\u0440\u0435\u043f\u044c\u0435","\u0440\u0436\u0438\u0446\u0430","\u0440\u0438\u043a\u0448\u0430","\u0440\u0438\u043f\u0443\u0441","\u0440\u0438\u0442\u043e\u0440","\u0440\u0438\u0448\u0442\u0430","\u0440\u043e\u0431\u043e\u0442","\u0440\u043e\u0432\u043d\u044f","\u0440\u043e\u0433\u0430\u0447","\u0440\u043e\u0434\u0438\u0447","\u0440\u043e\u0436\u0430\u043a","\u0440\u043e\u0436\u043d\u044b","\u0440\u043e\u043d\u0436\u0430","\u0440\u0443\u0431\u0446\u044b","\u0440\u0443\u0436\u044c\u0435","\u0440\u0443\u043c\u044b\u043d","\u0440\u0443\u043d\u0435\u0446","\u0440\u0443\u043d\u0446\u044b","\u0440\u0443\u0441\u0430\u043a","\u0440\u0443\u0447\u044c\u0438","\u0440\u044b\u0431\u0430\u043a","\u0440\u044b\u0431\u0435\u0446","\u0440\u044b\u0431\u0446\u044b","\u0440\u044b\u0431\u043a\u0430","\u0440\u044b\u0436\u0430\u043a","\u0440\u044b\u0436\u0435\u0439","\u0440\u044b\u0441\u0430\u043a","\u0440\u044b\u0442\u044c\u0435","\u0440\u044f\u0431\u0435\u0446","\u0440\u044f\u0431\u0446\u044b","\u0440\u044f\u0431\u043e\u0439","\u0440\u044f\u0431\u0430\u044f","\u0440\u044f\u0431\u044b\u0435","\u0440\u044f\u0431\u043e\u043a","\u0441\u0430\u0431\u0435\u0439","\u0441\u0430\u0431\u0438\u043d","\u0441\u0430\u0432\u043a\u0430","\u0441\u0430\u0433\u0430\u043d","\u0441\u0430\u0433\u0438\u0431","\u0441\u0430\u0434\u0436\u0430","\u0441\u0430\u0437\u0430\u043d","\u0441\u0430\u0439\u0433\u0430","\u0441\u0430\u0439\u0434\u0430","\u0441\u0430\u0439\u0440\u0430","\u0441\u0430\u043b\u0430\u0440","\u0441\u0430\u043b\u0435\u043f","\u0441\u0430\u043b\u043a\u0430","\u0441\u0430\u043c\u0435\u0446","\u0441\u0430\u043c\u0446\u044b","\u0441\u0430\u043c\u043a\u0430","\u0441\u0430\u043f\u0435\u0440","\u0441\u0430\u0440\u044b\u0447","\u0441\u0430\u0442\u0438\u0440","\u0441\u0430\u0445\u0438\u0431","\u0441\u0432\u0430\u0445\u0430","\u0441\u0432\u043e\u044f\u043a","\u0441\u0435\u0432\u0430\u043a","\u0441\u0435\u0432\u0435\u0446","\u0441\u0435\u0432\u0446\u044b","\u0441\u0435\u0434\u043e\u043a","\u0441\u0435\u0437\u043d\u0438","\u0441\u0435\u0439\u0438\u0434","\u0441\u0435\u043c\u0433\u0430","\u0441\u0435\u043c\u0438\u0442","\u0441\u0435\u0440\u043d\u0430","\u0441\u0435\u0440\u043e\u0435","\u0441\u0435\u0440\u044b\u0439","\u0441\u0435\u0440\u0430\u044f","\u0441\u0435\u0440\u044b\u0435","\u0441\u0435\u0443\u043d\u0447","\u0441\u0438\u0432\u043a\u0430","\u0441\u0438\u0432\u043a\u043e","\u0441\u0438\u0432\u0443\u0447","\u0441\u0438\u0432\u044b\u0439","\u0441\u0438\u0432\u0430\u044f","\u0441\u0438\u0432\u044b\u0435","\u0441\u0438\u0434\u043d\u0438","\u0441\u0438\u0436\u043e\u043a","\u0441\u0438\u0437\u0438\u0444","\u0441\u0438\u0437\u044f\u043a","\u0441\u0438\u043b\u0430\u0447","\u0441\u0438\u043b\u0435\u043d","\u0441\u0438\u043b\u044c\u0444","\u0441\u0438\u043d\u0435\u0446","\u0441\u0438\u043d\u0446\u044b","\u0441\u0438\u043d\u044f\u044f","\u0441\u0438\u043d\u0438\u0435","\u0441\u0438\u043f\u0430\u0439","\u0441\u0438\u0440\u0435\u043d","\u0441\u0438\u0440\u0438\u043d","\u0441\u0438\u0440\u044b\u0439","\u0441\u0438\u0440\u0430\u044f","\u0441\u0438\u0440\u044b\u0435","\u0441\u0438\u0442\u0446\u044b","\u0441\u043a\u0435\u043f\u0442","\u0441\u043a\u0443\u043d\u0441","\u0441\u043b\u0443\u0433\u0430","\u0441\u043b\u0443\u0434\u0430","\u0441\u043b\u0443\u043a\u0430","\u0441\u043c\u0435\u0440\u0434","\u0441\u043d\u043e\u0445\u0430","\u0441\u043e\u0432\u043a\u0430","\u0441\u043e\u0439\u043a\u0430","\u0441\u043e\u0439\u043e\u0442","\u0441\u043e\u043c\u0438\u043a","\u0441\u043e\u043c\u044c\u0435","\u0441\u043e\u043f\u0443\u043d","\u0441\u043e\u0441\u0435\u0434","\u0441\u043e\u0441\u0446\u044b","\u0441\u043e\u0441\u0443\u043d","\u0441\u043e\u0442\u044b\u0439","\u0441\u043e\u0445\u0430\u0447","\u0441\u043e\u0447\u043d\u0438","\u0441\u043e\u0447\u043e\u043a","\u0441\u0440\u043e\u0441\u0442","\u0441\u0442\u0435\u043d\u044c","\u0441\u0442\u0435\u0440\u0445","\u0441\u0442\u0438\u043a\u0441","\u0441\u0442\u043e\u0438\u043a","\u0441\u0442\u0440\u0430\u0436","\u0441\u0442\u0440\u0438\u0436","\u0441\u0442\u0443\u043f\u044c","\u0441\u0442\u044b\u0434\u044c","\u0441\u0443\u0434\u0430\u043a","\u0441\u0443\u0434\u0438\u044f","\u0441\u0443\u0434\u044c\u044f","\u0441\u0443\u0447\u044c\u044f","\u0441\u0443\u043f\u0446\u044b","\u0441\u0443\u0440\u043e\u043a","\u0441\u0443\u0447\u043a\u0430","\u0441\u0443\u0447\u044c\u0435","\u0441\u0443\u0449\u0435\u0435","\u0441\u0443\u0449\u0438\u043a","\u0441\u0446\u0438\u043d\u043a","\u0441\u044b\u043d\u043e\u043a","\u0441\u044b\u043f\u0446\u044b","\u0441\u044b\u0440\u0446\u044b","\u0441\u044b\u0440\u0446\u0430","\u0441\u044b\u0440\u044c\u0435","\u0441\u044b\u0442\u044b\u0439","\u0441\u044b\u0442\u0430\u044f","\u0441\u044b\u0442\u044b\u0435","\u0441\u044b\u0449\u0438\u043a","\u0441\u044d\u0448\u044d\u0430","\u0441\u044f\u0431\u0435\u0440","\u0442\u0430\u0439\u0446\u044b","\u0442\u0430\u0439\u043c\u0441","\u0442\u0430\u043a\u043e\u0435","\u0442\u0430\u043b\u0430\u043d","\u0442\u0430\u043b\u0438\u0431","\u0442\u0430\u043b\u044b\u0448","\u0442\u0430\u043c\u0438\u043b","\u0442\u0430\u043d\u0446\u044b","\u0442\u0430\u043d\u043e\u043a","\u0442\u0430\u043f\u0435\u0440","\u0442\u0430\u043f\u0438\u0440","\u0442\u0430\u0440\u044b\u043d","\u0442\u0430\u0442\u043a\u0430","\u0442\u0430\u0443\u044d\u0440","\u0442\u0430\u0446\u0435\u0442","\u0442\u0432\u0430\u0440\u044c","\u0442\u0435\u0437\u0435\u0439","\u0442\u0435\u0437\u043a\u0430","\u0442\u0435\u0438\u0441\u0442","\u0442\u0435\u043b\u0435\u043a","\u0442\u0435\u043b\u0435\u0446","\u0442\u0435\u043b\u043a\u0430","\u0442\u0435\u043b\u043e\u043a","\u0442\u0435\u043d\u0435\u043a","\u0442\u0435\u0440\u0435\u0446","\u0442\u0435\u0440\u0446\u044b","\u0442\u0435\u0440\u043a\u0430","\u0442\u0435\u0441\u0435\u0439","\u0442\u0435\u0441\u043a\u0430","\u0442\u0435\u0441\u0442\u044c","\u0442\u0435\u0442\u043a\u0430","\u0442\u0435\u0443\u0440\u0433","\u0442\u0435\u0448\u043a\u0430","\u0442\u0438\u0433\u043b\u0438","\u0442\u0438\u043d\u043a\u0430","\u0442\u0438\u043f\u0446\u044b","\u0442\u0438\u0440\u0430\u043d","\u0442\u0438\u0444\u0443\u0441","\u0442\u043b\u0435\u043d\u044c","\u0442\u043e\u0439\u043e\u043d","\u0442\u043e\u043b\u0430\u0439","\u0442\u043e\u043c\u043c\u0438","\u0442\u043e\u0440\u0446\u044b","\u0442\u043e\u0443\u044d\u0440","\u0442\u043e\u0445\u0430\u0440","\u0442\u0443\u0430\u043b\u044c","\u0442\u0443\u0433\u0430\u0439","\u0442\u0443\u0433\u0443\u043d","\u0442\u0443\u043a\u0430\u043d","\u0442\u0443\u043b\u0435\u0441","\u0442\u0443\u043b\u0443\u0437","\u0442\u0443\u043b\u0443\u043d","\u0442\u0443\u043b\u044f\u043a","\u0442\u0443\u043d\u0435\u0446","\u0442\u0443\u043d\u0446\u044b","\u0442\u0443\u043f\u0446\u044b","\u0442\u0443\u043f\u044b\u0448","\u0442\u0443\u0440\u0430\u0447","\u0442\u0443\u0440\u043e\u043a","\u0442\u044e\u0440\u0431\u0435","\u0442\u044e\u0440\u043e\u043a","\u0442\u044f\u0433\u0430\u0447","\u0443\u0433\u043b\u0438\u0447","\u0443\u0434\u0438\u043b\u043e","\u0443\u0434\u044d\u0433\u0435","\u0443\u0434\u044d\u0445\u0435","\u0443\u0436\u0430\u0442\u0430","\u0443\u0437\u0430\u043d\u0441","\u0443\u0437\u0431\u0435\u043a","\u0443\u0437\u0434\u0446\u0430","\u0443\u0437\u043d\u0438\u043a","\u0443\u0439\u0433\u0443\u0440","\u0443\u043a\u043b\u0435\u044f","\u0443\u043a\u0440\u0443\u0442","\u0443\u043b\u044b\u0431\u0430","\u0443\u043b\u044c\u0442\u0430","\u0443\u043c\u043d\u0438\u043a","\u0443\u043c\u043d\u044b\u0439","\u0443\u043c\u043d\u0430\u044f","\u0443\u043c\u043d\u044b\u0435","\u0443\u043d\u0438\u043e\u043d","\u0443\u043d\u0442\u0435\u0440","\u0443\u043f\u0440\u0435\u043a","\u0443\u043f\u044b\u0440\u044c","\u0443\u0440\u0430\u0440\u0442","\u0443\u0441\u0438\u043d\u0430","\u0443\u0441\u043a\u043e\u043a","\u0443\u0442\u0438\u0446\u0430","\u0443\u0445\u0430\u0440\u044c","\u0443\u0445\u0440\u044f\u0431","\u0443\u0447\u0435\u0431\u0430","\u0443\u0448\u043d\u0438\u043a","\u0444\u0430\u0437\u0430\u043d","\u0444\u0430\u043a\u0438\u0440","\u0444\u0430\u043d\u0430\u0442","\u0444\u0430\u0442\u044e\u0439","\u0444\u0435\u043b\u043b\u043e","\u0444\u0435\u043b\u043e\u043d","\u0444\u0435\u043d\u0438\u0439","\u0444\u0435\u0442\u044e\u043a","\u0444\u0435\u0444\u0435\u0440","\u0444\u0438\u0433\u043b\u0438","\u0444\u0438\u0437\u0438\u043a","\u0444\u0438\u043b\u0435\u0440","\u0444\u0438\u043b\u0438\u043d","\u0444\u0438\u043b\u043e\u043d","\u0444\u043b\u044f\u0435\u0440","\u0444\u0440\u0430\u0435\u0440","\u0444\u0440\u0430\u043d\u0442","\u0444\u0443\u0440\u0438\u044f","\u0444\u044e\u0440\u0435\u0440","\u0445\u0430\u0437\u0430\u0440","\u0445\u0430\u043a\u0430\u0441","\u0445\u0430\u043a\u0435\u0440","\u0445\u0430\u043b\u0434\u0430","\u0445\u0430\u043b\u0438\u0444","\u0445\u0430\u043c\u0438\u0442","\u0445\u0430\u043c\u043a\u0430","\u0445\u0430\u043c\u043b\u043e","\u0445\u0430\u043c\u0441\u0430","\u0445\u0430\u043c\u044c\u0435","\u0445\u0430\u043d\u0442\u044b","\u0445\u0430\u043d\u0448\u0430","\u0445\u0430\u0440\u043e\u043d","\u0445\u0430\u0441\u0438\u0434","\u0445\u0430\u0448\u0438\u0448","\u0445\u0435\u0434\u0438\u0432","\u0445\u0438\u043b\u044b\u0439","\u0445\u0438\u043b\u0430\u044f","\u0445\u0438\u043b\u044b\u0435","\u0445\u0438\u043b\u044f\u043a","\u0445\u0438\u043c\u0438\u043a","\u0445\u0438\u0440\u043e\u0442","\u0445\u043b\u0435\u0441\u0442","\u0445\u043c\u0430\u0440\u0430","\u0445\u043c\u044b\u0440\u044c","\u0445\u043e\u0434\u043d\u0438","\u0445\u043e\u0434\u0436\u0430","\u0445\u043e\u0434\u0438\u043a","\u0445\u043e\u043b\u043e\u043f","\u0445\u043e\u043b\u0443\u0439","\u0445\u043e\u0445\u043b\u044b","\u0445\u0443\u0434\u043e\u0435","\u0445\u0443\u043d\u043d\u0443","\u0445\u0443\u0440\u0434\u0430","\u0446\u0430\u0434\u0438\u043a","\u0446\u0430\u043f\u043b\u044f","\u0446\u0430\u043f\u0443\u043d","\u0446\u0430\u0440\u0435\u043a","\u0446\u0432\u0435\u0442\u044b","\u0446\u0435\u0432\u044c\u0435","\u0446\u0435\u043f\u043d\u0438","\u0446\u0435\u0444\u0435\u0439","\u0446\u0438\u043d\u0430\u0448","\u0446\u0438\u043d\u0438\u043a","\u0446\u0438\u0440\u0438\u043a","\u0446\u0443\u0446\u0438\u043a","\u0446\u044b\u0433\u0430\u043d","\u0446\u044b\u043f\u043a\u0430","\u0446\u044d\u0440\u044d\u0443","\u0447\u0430\u0431\u0430\u043d","\u0447\u0430\u0431\u0440\u044b","\u0447\u0430\u043b\u044b\u0439","\u0447\u0430\u043b\u0430\u044f","\u0447\u0430\u043b\u044b\u0435","\u0447\u0435\u0431\u0430\u043a","\u0447\u0435\u0431\u043e\u0442","\u0447\u0435\u043b\u043a\u0430","\u0447\u0435\u043c\u0435\u0440","\u0447\u0435\u043f\u0446\u044b","\u0447\u0435\u0440\u0435\u0434","\u0447\u0435\u0440\u0435\u0437","\u0447\u0435\u0441\u043a\u0430","\u0447\u0435\u0442\u043a\u0430","\u0447\u0435\u0445\u043b\u044b","\u0447\u0435\u0447\u0435\u043d","\u0447\u0435\u0447\u0435\u0442","\u0447\u0438\u0431\u0438\u0441","\u0447\u0438\u043f\u0441\u044b","\u0447\u0438\u0440\u044c\u0438","\u0447\u0438\u0440\u043e\u043a","\u0447\u0438\u0440\u0443\u0441","\u0447\u043e\u043c\u0433\u0430","\u0447\u043e\u043c\u043f\u0438","\u0447\u0442\u0438\u0446\u0430","\u0447\u0443\u0432\u0430\u043a","\u0447\u0443\u0432\u0430\u0448","\u0447\u0443\u0434\u0430\u043a","\u0447\u0443\u0434\u0438\u043a","\u0447\u0443\u0436\u0430\u043a","\u0447\u0443\u0436\u0430\u044f","\u0447\u0443\u0436\u0438\u0435","\u0447\u0443\u043a\u0447\u0430","\u0447\u0443\u043c\u0430\u043a","\u0447\u0443\u0442\u044c\u0435","\u0447\u0443\u0445\u043d\u0430","\u0448\u0430\u0431\u0430\u0442","\u0448\u0430\u0431\u0440\u044b","\u0448\u0430\u0431\u043b\u044f","\u0448\u0430\u0432\u043a\u0430","\u0448\u0430\u043a\u0430\u043b","\u0448\u0430\u043b\u043e\u043d","\u0448\u0430\u043b\u0443\u043d","\u0448\u0430\u043c\u0430\u043d","\u0448\u0430\u043d\u0446\u044b","\u0448\u0430\u0440\u0430\u043f","\u0448\u0430\u0442\u0435\u043d","\u0448\u0430\u0442\u0435\u0440","\u0448\u0430\u0442\u0440\u044b","\u0448\u0430\u0444\u0435\u0440","\u0448\u0435\u0438\u0449\u0430","\u0448\u0435\u043c\u0430\u044f","\u0448\u0435\u043f\u043e\u0442","\u0448\u0435\u0440\u0438\u0444","\u0448\u0438\u0437\u0438\u043a","\u0448\u0438\u043b\u044c\u044f","\u0448\u0438\u043d\u0442\u043e","\u0448\u0438\u043f\u0443\u043d","\u0448\u0438\u0442\u044c\u0435","\u0448\u043b\u044e\u0445\u0430","\u0448\u043c\u0435\u043b\u044c","\u0448\u043c\u043e\u043b\u044c","\u0448\u043d\u044f\u043a\u0430","\u0448\u043e\u0440\u0435\u0446","\u0448\u043e\u0440\u0446\u044b","\u0448\u043e\u0440\u043a\u0430","\u0448\u043e\u0444\u0435\u0440","\u0448\u043f\u0430\u043d\u0430","\u0448\u043f\u0435\u043d\u044c","\u0448\u043f\u0438\u043d\u044c","\u0448\u043f\u0438\u043e\u043d","\u0448\u043f\u0440\u043e\u0442","\u0448\u043f\u044b\u043d\u044c","\u0448\u0442\u0440\u0430\u0431","\u0448\u0442\u044b\u043a\u0430","\u0448\u0443\u0434\u0440\u0430","\u0448\u0443\u043b\u0435\u0440","\u0448\u0443\u043c\u0435\u0440","\u0448\u0443\u0440\u0438\u043d","\u0448\u0443\u0440\u044f\u043a","\u0449\u0435\u0431\u043d\u0438","\u0449\u0435\u0433\u043e\u043b","\u0449\u0435\u0433\u043b\u044b","\u0449\u0435\u043a\u043e\u0442","\u0449\u0435\u043b\u043e\u043a","\u0449\u0435\u043d\u043e\u043a","\u0449\u0435\u043f\u044c\u0435","\u0449\u0435\u0440\u0431\u0430","\u0449\u0435\u0442\u043a\u0430","\u0449\u0435\u0447\u043a\u0430","\u0449\u0438\u0442\u043d\u0438","\u0449\u0438\u0442\u0438\u043a","\u0449\u0443\u043f\u043a\u0430","\u0449\u0443\u0440\u043a\u0430","\u0449\u0443\u0440\u043e\u043a","\u0449\u0443\u0447\u043a\u0430","\u044d\u043b\u0435\u0430\u0442","\u044d\u043b\u043b\u0438\u043d","\u044d\u043f\u0430\u0440\u0445","\u044d\u0440\u0430\u0442\u0430","\u044d\u0441\u0434\u0435\u043a","\u044d\u0441\u0441\u0435\u043d","\u044d\u0441\u0442\u0435\u0442","\u044d\u0444\u0438\u043e\u043f","\u044e\u0433\u0440\u0438\u0447","\u044e\u0433\u0443\u0440\u0442","\u044e\u0436\u0430\u043d\u0435","\u044e\u0437\u0438\u0441\u0442","\u044e\u043d\u0438\u043e\u0440","\u044e\u043d\u0438\u0446\u0430","\u044e\u043d\u043a\u0435\u0440","\u044e\u043d\u043a\u043e\u0440","\u044e\u043d\u043d\u0430\u0442","\u044e\u043d\u043e\u043d\u0430","\u044e\u043d\u043e\u0448\u0430","\u044e\u0440\u0438\u0441\u0442","\u044f\u0433\u0443\u0430\u0440","\u044f\u043c\u0449\u0438\u043a","\u044f\u0440\u044b\u0433\u0430","\u044f\u0441\u044b\u0440\u044c","\u044f\u0442\u0432\u044f\u0433","\u043b\u0430\u0441\u0442\u044b","\u0431\u043b\u044f\u0442\u044c","\u0440\u0435\u0448\u0430\u043b\u0430","\u043f\u0440\u0430\u0432\u0430","\u0442\u043e\u0433\u0434\u0430","\u0437\u0434\u0435\u0441\u044c","\u0431\u043e\u043b\u0435\u0435","\u043c\u0435\u0436\u0434\u0443","\u043f\u043e\u0439\u0442\u0438","\u0442\u0430\u043a\u0436\u0435","\u0432\u0437\u044f\u0442\u044c","\u0432\u0434\u0440\u0443\u0433","\u043f\u043e\u0447\u0442\u0438","\u043e\u0431\u0449\u0438\u0439","\u0441\u0440\u0430\u0437\u0443","\u0432\u044b\u0439\u0442\u0438","\u043c\u043d\u043e\u0433\u043e","\u043d\u0430\u0439\u0442\u0438","\u0436\u0434\u0430\u0442\u044c","\u043d\u0443\u0436\u043d\u043e","\u043a\u0440\u043e\u043c\u0435","\u043e\u043f\u044f\u0442\u044c","\u0441\u043d\u043e\u0432\u0430","\u0441\u0440\u0435\u0434\u0438","\u0440\u044f\u0434\u043e\u043c","\u043d\u0430\u0437\u0430\u0434","\u0436\u0438\u0432\u043e\u0439","\u0432\u0435\u0441\u0442\u0438","\u0434\u0430\u0432\u043d\u043e","\u0447\u0430\u0441\u0442\u043e","\u0441\u043c\u043e\u0447\u044c","\u0434\u043e\u043b\u0433\u043e","\u0437\u0430\u0447\u0435\u043c","\u0437\u0430\u0442\u0435\u043c","\u0434\u0430\u043b\u0435\u0435","\u043e\u043a\u043e\u043b\u043e","\u0431\u0440\u0430\u0442\u044c","\u0446\u0435\u043b\u044b\u0439","\u0441\u043f\u0430\u0442\u044c","\u043c\u043e\u0436\u0435\u0442","\u043f\u0443\u0441\u0442\u044c","\u0432\u043e\u0439\u0442\u0438","\u0435\u0445\u0430\u0442\u044c","\u043c\u0435\u043d\u0435\u0435","\u0441\u0435\u0441\u0442\u044c","\u0431\u0443\u0434\u0442\u043e","\u0443\u043c\u0435\u0442\u044c","\u0432\u0441\u0435\u0433\u043e","\u0434\u043e\u043c\u043e\u0439","\u0441\u043a\u043e\u0440\u043e","\u0440\u0430\u0437\u0432\u0435","\u0442\u043e\u0447\u043d\u043e","\u0441\u043d\u044f\u0442\u044c","\u0447\u0443\u0436\u043e\u0439","\u043b\u0435\u0433\u043a\u043e","\u0441\u0442\u0438\u0445\u0438","\u0432\u0438\u0434\u043d\u043e","\u0432\u043e\u0432\u0441\u0435","\u0443\u0431\u0438\u0442\u044c","\u0441\u0442\u043e\u043b\u044c","\u043d\u0438\u043a\u0430\u043a","\u043a\u043e\u0433\u0434\u0430","\u0437\u0432\u0430\u0442\u044c","\u0432\u043d\u043e\u0432\u044c","\u043f\u0440\u044f\u043c\u043e","\u0438\u043d\u0430\u0447\u0435","\u0440\u0430\u0441\u0442\u0438","\u043d\u0435\u043a\u0438\u0439","\u0448\u0435\u0441\u0442\u044c","\u043d\u0435\u0441\u0442\u0438","\u0442\u0430\u043a\u043e\u0439","\u0447\u0442\u043e\u0431\u044b","\u043e\u0447\u0435\u043d\u044c","\u043d\u043e\u0432\u044b\u0439","\u043c\u043e\u0436\u043d\u043e","\u043a\u0430\u043a\u043e\u0439","\u043f\u043e\u0441\u043b\u0435","\u0441\u0430\u043c\u044b\u0439","\u043f\u043e\u0442\u043e\u043c","\u0438\u043c\u0435\u0442\u044c","\u0432\u0440\u043e\u0434\u0435","\u0430\u043a\u0442\u0451\u0440","\u0432\u043e\u0437\u043b\u0435","\u043b\u0443\u0447\u0448\u0435","\u043b\u0430\u0434\u043d\u043e","\u0442\u0438\u0445\u0438\u0439","\u0434\u0430\u0432\u0430\u0439","\u0431\u043e\u0436\u0438\u0439","\u044f\u0440\u043a\u0438\u0439","\u0440\u0435\u0437\u043a\u043e","\u044f\u0441\u043d\u044b\u0439","\u0434\u043e\u0439\u0442\u0438","\u0443\u0437\u043a\u0438\u0439","\u043b\u0438\u0447\u043d\u043e","\u0437\u0430\u0439\u0442\u0438","\u043d\u0435\u0447\u0442\u043e","\u0441\u0443\u0445\u043e\u0439","\u0432\u0432\u0435\u0440\u0445","\u043c\u043e\u043b\u0447\u0430","\u0443\u0447\u0438\u0442\u044c","\u0432\u0434\u043e\u043b\u044c","\u0440\u0430\u043d\u0435\u0435","\u0432\u0430\u0436\u043d\u043e","\u0440\u0435\u0434\u043a\u043e","\u0442\u0430\u043a\u043e\u0432","\u044e\u0436\u043d\u044b\u0439","\u0432\u0441\u043b\u0435\u0434","\u043e\u0441\u043e\u0431\u043e","\u0447\u0438\u0441\u0442\u043e","\u0441\u043e\u0439\u0442\u0438","\u0441\u0434\u0430\u0442\u044c","\u043f\u0443\u0442\u0451\u043c","\u0432\u0435\u0440\u043d\u043e","\u043b\u0435\u0437\u0442\u044c","\u043a\u0430\u043a\u043e\u0432","\u0447\u0451\u0442\u043a\u043e","\u0432\u0435\u0437\u0434\u0435","\u0432\u0435\u0437\u0442\u0438","\u0441\u0437\u0430\u0434\u0438","\u0432\u043d\u0438\u0437\u0443","\u0442\u0451\u0442\u043a\u0430","\u044f\u043a\u043e\u0431\u044b","\u0448\u043e\u0444\u0451\u0440","\u0432\u0440\u0430\u0442\u044c","\u043f\u043b\u044b\u0442\u044c","\u0440\u043e\u0432\u043d\u043e","\u043e\u0440\u0430\u0442\u044c","\u043d\u0438\u0433\u0434\u0435","\u0437\u0430\u043c\u0443\u0436","\u0436\u0438\u043b\u043e\u0439","\u0432\u0435\u0447\u043d\u043e","\u044f\u0432\u043d\u044b\u0439","\u0441\u0438\u044f\u0442\u044c","\u0432\u0441\u043b\u0443\u0445","\u043c\u044f\u0433\u043a\u043e","\u0445\u0443\u0434\u043e\u0439","\u0441\u0435\u0434\u043e\u0439","\u043c\u0430\u043b\u044b\u0448","\u043e\u0434\u0435\u0442\u044c","\u0441\u043b\u0435\u0432\u0430","\u0433\u043d\u0430\u0442\u044c","\u043d\u044b\u043d\u0447\u0435","\u0441\u043d\u0438\u0437\u0443","\u0441\u0431\u0438\u0442\u044c","\u0441\u043b\u0430\u0431\u043e","\u0441\u044b\u0440\u043e\u0439","\u0442\u0435\u0441\u043d\u043e","\u0441\u043c\u0435\u0442\u044c","\u0432\u0441\u044e\u0434\u0443","\u043b\u043e\u0432\u043a\u043e","\u0441\u0432\u044b\u0448\u0435","\u043f\u0440\u043e\u0447\u044c","\u0432\u043b\u0435\u0447\u044c","\u0442\u0443\u043f\u043e\u0439","\u043f\u043e\u043b\u043d\u043e","\u0440\u0432\u0430\u0442\u044c","\u043c\u0430\u043c\u0438\u043d","\u0441\u0436\u0435\u0447\u044c","\u0433\u0440\u0443\u0431\u043e","\u0441\u043f\u0435\u0442\u044c","\u043e\u0441\u0442\u0440\u043e","\u0441\u0436\u0430\u0442\u044c","\u0441\u043c\u0435\u043b\u043e","\u043e\u0432\u043e\u0449\u0438","\u0436\u0430\u0440\u043a\u043e","\u0440\u0430\u0437\u043e\u043c","\u043a\u043e\u0437\u0451\u043b","\u0441\u0431\u043e\u043a\u0443","\u0442\u0435\u043c\u043d\u043e","\u043d\u0435\u0436\u043d\u043e","\u0432\u0434\u0432\u043e\u0435","\u0432\u0434\u0430\u043b\u0438","\u0431\u044b\u043b\u043e\u0439","\u0433\u043e\u0440\u0434\u043e","\u043b\u044e\u0441\u0438\u043d","\u0441\u043a\u043e\u043b\u044c","\u0436\u0430\u0434\u043d\u043e","\u0431\u043e\u0441\u043e\u0439","\u0432\u0432\u0438\u0434\u0443","\u0433\u043b\u0443\u043f\u043e","\u0435\u0436\u0435\u043b\u0438","\u043c\u0438\u0440\u043d\u043e","\u043d\u0438\u0437\u043a\u043e","\u0436\u0440\u0430\u0442\u044c","\u0433\u0443\u0441\u0442\u043e","\u0448\u0443\u043c\u043d\u043e","\u0434\u0430\u0440\u043e\u043c","\u043a\u0440\u0443\u0442\u043e","\u0441\u0448\u0438\u0442\u044c","\u043e\u0436\u0438\u0442\u044c","\u0440\u0430\u0432\u043d\u043e","\u0443\u043c\u0435\u043b\u043e","\u0440\u043e\u0431\u043a\u043e","\u0447\u0443\u044f\u0442\u044c","\u0431\u043e\u0434\u0440\u043e","\u0433\u043b\u0443\u0445\u043e","\u043d\u0435\u0433\u0434\u0435","\u0432\u043b\u0435\u0432\u043e","\u0442\u043e\u0449\u0438\u0439","\u0432\u043e\u0432\u0441\u044e","\u0442\u0430\u0439\u043d\u043e","\u0442\u0443\u0433\u043e\u0439","\u0442\u0430\u044f\u0442\u044c","\u0449\u0435\u0434\u0440\u043e","\u0436\u0443\u0442\u043a\u043e","\u043c\u0435\u043b\u043a\u043e","\u0431\u0435\u0433\u043e\u043c","\u043b\u0433\u0430\u0442\u044c","\u043f\u0430\u043f\u0438\u043d","\u0442\u043e\u043d\u043a\u043e","\u0442\u0430\u0438\u0442\u044c","\u0433\u0440\u0435\u0442\u044c","\u043d\u0435\u043a\u0442\u043e","\u0441\u0443\u0449\u0438\u0439","\u0445\u043c\u0443\u0440\u043e","\u0431\u0430\u0431\u0438\u0439","\u0443\u044e\u0442\u043d\u043e","\u043b\u0438\u0445\u043e\u0439","\u0441\u0435\u044f\u0442\u044c","\u0441\u0432\u0435\u0440\u0445","\u043d\u0430\u0437\u043b\u043e","\u0438\u0445\u043d\u0438\u0439","\u0431\u0443\u0440\u044b\u0439","\u0431\u0443\u0440\u043d\u043e","\u0447\u0443\u0434\u043e\u043c","\u0438\u0437\u0432\u043d\u0435","\u043c\u044f\u0442\u044b\u0439","\u043b\u044e\u0442\u044b\u0439","\u0432\u0434\u0430\u043b\u044c","\u0442\u044f\u0436\u043a\u043e","\u0445\u0438\u0442\u0440\u043e","\u0431\u0443\u0434\u0434\u0430","\u043d\u0430\u0431\u043e\u043a","\u043d\u0430\u0433\u043b\u043e","\u0431\u0430\u0440\u0438\u043d","\u043c\u044e\u0441\u043b\u0438","\u0442\u0430\u043f\u043a\u0438","\u0441\u043b\u0438\u0432\u044b","\u043f\u0438\u0434\u043e\u0440"]
target_words1 = ["\u043a\u043e\u043d\u0435\u0446", "\u043d\u0438\u043a\u0442\u043e", "\u0437\u0435\u043c\u043b\u044f", "\u043f\u0440\u0430\u0432\u043e", "\u0434\u0432\u0435\u0440\u044c", "\u043e\u0431\u0440\u0430\u0437", "\u0437\u0430\u043a\u043e\u043d", "\u0432\u043e\u0439\u043d\u0430", "\u0441\u0442\u0438\u043b\u044c", "\u043a\u043d\u0438\u0433\u0430", "\u0447\u0438\u0441\u043b\u043e", "\u043d\u0430\u0440\u043e\u0434", "\u0444\u043e\u0440\u043c\u0430", "\u0441\u0432\u044f\u0437\u044c", "\u0443\u043b\u0438\u0446\u0430", "\u043c\u044b\u0441\u043b\u044c", "\u043c\u0435\u0441\u044f\u0446", "\u0448\u043a\u043e\u043b\u0430", "\u0442\u0435\u0430\u0442\u0440", "\u0440\u0443\u0431\u043b\u044c", "\u0441\u043c\u044b\u0441\u043b", "\u043e\u0440\u0433\u0430\u043d", "\u0440\u044b\u043d\u043e\u043a", "\u0441\u0435\u043c\u044c\u044f", "\u0446\u0435\u043d\u0442\u0440", "\u043e\u0442\u0432\u0435\u0442", "\u0430\u0432\u0442\u043e\u0440", "\u0441\u0442\u0435\u043d\u0430", "\u0441\u043e\u0432\u0435\u0442", "\u0433\u043b\u0430\u0432\u0430", "\u043d\u0430\u0443\u043a\u0430", "\u043f\u043b\u0435\u0447\u043e", "\u0442\u043e\u0447\u043a\u0430", "\u043f\u0430\u043b\u0435\u0446", "\u043d\u043e\u043c\u0435\u0440", "\u043c\u0435\u0442\u043e\u0434", "\u0444\u0438\u043b\u044c\u043c", "\u0433\u043e\u0441\u0442\u044c", "\u043a\u0440\u043e\u0432\u044c", "\u0440\u0430\u0439\u043e\u043d", "\u043a\u043b\u0430\u0441\u0441", "\u0433\u0435\u0440\u043e\u0439", "\u0441\u043f\u0438\u043d\u0430", "\u0441\u0446\u0435\u043d\u0430", "\u043e\u0431\u044a\u0435\u043c", "\u0431\u0435\u0440\u0435\u0433", "\u0444\u0438\u0440\u043c\u0430", "\u0437\u0430\u0432\u043e\u0434", "\u043f\u0435\u0441\u043d\u044f", "\u0440\u043e\u043c\u0430\u043d", "\u0441\u0442\u0438\u0445\u0438", "\u043f\u043e\u0432\u043e\u0434", "\u0443\u0441\u043f\u0435\u0445", "\u0432\u044b\u0445\u043e\u0434", "\u0442\u0435\u043a\u0441\u0442", "\u043f\u0443\u043d\u043a\u0442", "\u043b\u0438\u043d\u0438\u044f", "\u0441\u0440\u0435\u0434\u0430", "\u0432\u043e\u043b\u043e\u0441", "\u0432\u0435\u0442\u0435\u0440", "\u043e\u0433\u043e\u043d\u044c", "\u0433\u0440\u0443\u0434\u044c", "\u043a\u043e\u0433\u0434\u0430", "\u0441\u0442\u0440\u0430\u0445", "\u0441\u0443\u043c\u043c\u0430", "\u0441\u0444\u0435\u0440\u0430", "\u0432\u0447\u0435\u0440\u0430", "\u043c\u0443\u0436\u0438\u043a", "\u043d\u0435\u043c\u0435\u0446", "\u0432\u044b\u0431\u043e\u0440", "\u043c\u0430\u0441\u0441\u0430", "\u0448\u0435\u0441\u0442\u044c", "\u0441\u043b\u0430\u0432\u0430", "\u043a\u0443\u0445\u043d\u044f", "\u0432\u0440\u0435\u043c\u044f", "\u0436\u0438\u0437\u043d\u044c", "\u0441\u043b\u043e\u0432\u043e", "\u043c\u0435\u0441\u0442\u043e", "\u043e\u0442\u0434\u0435\u043b", "\u0442\u043e\u0432\u0430\u0440", "\u0430\u043a\u0442\u0435\u0440", "\u0441\u043b\u0435\u0437\u0430", "\u0432\u044b\u0432\u043e\u0434", "\u043d\u043e\u0440\u043c\u0430", "\u0440\u0430\u043c\u043a\u0430", "\u043f\u0440\u0438\u0435\u043c", "\u0440\u0435\u0436\u0438\u043c", "\u0446\u0435\u043b\u043e\u0435", "\u043f\u043e\u0438\u0441\u043a", "\u043d\u0430\u043b\u043e\u0433", "\u0434\u043e\u0445\u043e\u0434", "\u043a\u0430\u0440\u0442\u0430", "\u0430\u043a\u0446\u0438\u044f", "\u0441\u043e\u0441\u0435\u0434", "\u0444\u0440\u0430\u0437\u0430", "\u0442\u043e\u043b\u043f\u0430", "\u0441\u0435\u0440\u044b\u0439", "\u0432\u043e\u043b\u043d\u0430", "\u043f\u0442\u0438\u0446\u0430", "\u0437\u0430\u043f\u0430\u0445", "\u0432\u043e\u0434\u043a\u0430", "\u043f\u043e\u0435\u0437\u0434", "\u0430\u0434\u0440\u0435\u0441", "\u043b\u0438\u0434\u0435\u0440", "\u0432\u0435\u0441\u043d\u0430", "\u044d\u043f\u043e\u0445\u0430", "\u0437\u0430\u043f\u0430\u0434", "\u0442\u0430\u0439\u043d\u0430", "\u0442\u0440\u0430\u0432\u0430", "\u043c\u0443\u0437\u0435\u0439", "\u043a\u043d\u044f\u0437\u044c", "\u0441\u0443\u0442\u043a\u0438", "\u0441\u0443\u0434\u044c\u044f", "\u043a\u0440\u044b\u0448\u0430", "\u043f\u043e\u0442\u043e\u043a", "\u0447\u0435\u0441\u0442\u044c", "\u0435\u0432\u0440\u0435\u0439", "\u0441\u043e\u0442\u043d\u044f", "\u0434\u043e\u0436\u0434\u044c", "\u0442\u0440\u0443\u0431\u0430", "\u043e\u0441\u0435\u043d\u044c", "\u043f\u044c\u0435\u0441\u0430", "\u0447\u0435\u0440\u0442\u0430", "\u043a\u0443\u0441\u043e\u043a", "\u0431\u0438\u043b\u0435\u0442", "\u043c\u0430\u0441\u043b\u043e", "\u044d\u043a\u0440\u0430\u043d", "\u043a\u0430\u043d\u0430\u043b", "\u0432\u0430\u0433\u043e\u043d", "\u0434\u0443\u0440\u0430\u043a", "\u0441\u0435\u0437\u043e\u043d", "\u0437\u0430\u043f\u0430\u0441", "\u0434\u043b\u0438\u043d\u0430", "\u043a\u0440\u044b\u043b\u043e", "\u043e\u043a\u0440\u0443\u0433", "\u0434\u043e\u0441\u043a\u0430", "\u043f\u043e\u043b\u0435\u0442", "\u043f\u0430\u043a\u0435\u0442", "\u0436\u0438\u0432\u043e\u0442", "\u0441\u043c\u0435\u043d\u0430", "\u043f\u043e\u0440\u043e\u0433", "\u0431\u0443\u043a\u0432\u0430", "\u043b\u043e\u0434\u043a\u0430", "\u0441\u0435\u0440\u0438\u044f", "\u0448\u0443\u0442\u043a\u0430", "\u043a\u0443\u043b\u0430\u043a", "\u043d\u0435\u0444\u0442\u044c", "\u0446\u0438\u0444\u0440\u0430", "\u0441\u0430\u043f\u043e\u0433", "\u0436\u0438\u043b\u044c\u0435", "\u043c\u0435\u0448\u043e\u043a", "\u043e\u0442\u043a\u0430\u0437", "\u0437\u0430\u043c\u043e\u043a", "\u0434\u0438\u0432\u0430\u043d", "\u0434\u043e\u0431\u0440\u043e", "\u043f\u043e\u043a\u043e\u0439", "\u0432\u0441\u043b\u0435\u0434", "\u043a\u043e\u0441\u0442\u044c", "\u0441\u043f\u043e\u0440\u0442", "\u043c\u0430\u0439\u043e\u0440", "\u043e\u0442\u0434\u044b\u0445", "\u0440\u0443\u0447\u043a\u0430", "\u043c\u0435\u0447\u0442\u0430", "\u0441\u044e\u0436\u0435\u0442", "\u0440\u0443\u0431\u0435\u0436", "\u043a\u0440\u0435\u0441\u0442", "\u0432\u0437\u0440\u044b\u0432", "\u043f\u043e\u0447\u0432\u0430", "\u0437\u0430\u043a\u0430\u0437", "\u0448\u0442\u0443\u043a\u0430", "\u0441\u0443\u043c\u043a\u0430", "\u0445\u0432\u043e\u0441\u0442", "\u043f\u0435\u0441\u043e\u043a", "\u043e\u0437\u0435\u0440\u043e", "\u0441\u0442\u0440\u043e\u0439", "\u0434\u043e\u0447\u043a\u0430", "\u0442\u0430\u043d\u0435\u0446", "\u043d\u0430\u0431\u043e\u0440", "\u0433\u043e\u0440\u043b\u043e", "\u043f\u043b\u0430\u0442\u0430", "\u0441\u044a\u0435\u0437\u0434", "\u043a\u043e\u0448\u043a\u0430", "\u043f\u044f\u0442\u043d\u043e", "\u0442\u043a\u0430\u043d\u044c", "\u0432\u0438\u0437\u0438\u0442", "\u043e\u043a\u0435\u0430\u043d", "\u043f\u0430\u0443\u0437\u0430", "\u0441\u0442\u0432\u043e\u043b", "\u0442\u0435\u043f\u043b\u043e", "\u0440\u0430\u0434\u0438\u043e", "\u0437\u0432\u0435\u0440\u044c", "\u043d\u0430\u0446\u0438\u044f", "\u0431\u0430\u043d\u043a\u0430", "\u043c\u0435\u0442\u0440\u043e", "\u043e\u0442\u0447\u0435\u0442", "\u043e\u0431\u043c\u0435\u043d", "\u0442\u043e\u0441\u043a\u0430", "\u0437\u0430\u0431\u043e\u0440", "\u043c\u043e\u0440\u043e\u0437", "\u043c\u0430\u0440\u043a\u0430", "\u0433\u0440\u044f\u0437\u044c", "\u0441\u043f\u0440\u043e\u0441", "\u043c\u043e\u0442\u043e\u0440", "\u0441\u0435\u0432\u0435\u0440", "\u0441\u043a\u043b\u0430\u0434", "\u043c\u043e\u0442\u0438\u0432", "\u0430\u0440\u0435\u0441\u0442", "\u043e\u0442\u0440\u044f\u0434", "\u043d\u0443\u0436\u0434\u0430", "\u0441\u043e\u0431\u043e\u0440", "\u0441\u0430\u043b\u043e\u043d", "\u043a\u0430\u043f\u043b\u044f", "\u043f\u043e\u0436\u0430\u0440", "\u0445\u043e\u043b\u043e\u0434", "\u0442\u0435\u0442\u043a\u0430", "\u043e\u0431\u0438\u0434\u0430", "\u0432\u043a\u043b\u0430\u0434", "\u0433\u043e\u043d\u043a\u0430", "\u0440\u0443\u043a\u0430\u0432", "\u0442\u0443\u043c\u0430\u043d", "\u0448\u043e\u0444\u0435\u0440", "\u0430\u0442\u0430\u043a\u0430", "\u0438\u0433\u0440\u043e\u043a", "\u0442\u043e\u043d\u043d\u0430", "\u0432\u043e\u0436\u0434\u044c", "\u043e\u0440\u0434\u0435\u043d", "\u044e\u043d\u043e\u0448\u0430", "\u0431\u0430\u0431\u043a\u0430", "\u0432\u044b\u0437\u043e\u0432", "\u043f\u043e\u043b\u043a\u0430", "\u0431\u0440\u043e\u0432\u044c", "\u0443\u0434\u0430\u0447\u0430", "\u0431\u044b\u0442\u0438\u0435", "\u0448\u0430\u043f\u043a\u0430", "\u043b\u043e\u0436\u043a\u0430", "\u0431\u0435\u043b\u043e\u043a", "\u0433\u043e\u043b\u043e\u0434", "\u043e\u0445\u043e\u0442\u0430", "\u0434\u043e\u043c\u0438\u043a", "\u0437\u0430\u043c\u0443\u0436", "\u0432\u0435\u0442\u043a\u0430", "\u0431\u0430\u0448\u043d\u044f", "\u0442\u0430\u0440\u0438\u0444", "\u0430\u0433\u0435\u043d\u0442", "\u0433\u0435\u043d\u0438\u0439", "\u0441\u0430\u0445\u0430\u0440", "\u0431\u043b\u0430\u0433\u043e", "\u0441\u043c\u0435\u0441\u044c", "\u0440\u0430\u0437\u0443\u043c", "\u043f\u043b\u0438\u0442\u0430", "\u0431\u0440\u044e\u043a\u0438", "\u043c\u044b\u0448\u0446\u0430", "\u043f\u043e\u0441\u043e\u043b", "\u043f\u0430\u0447\u043a\u0430", "\u0448\u043e\u0441\u0441\u0435", "\u0438\u043a\u043e\u043d\u0430", "\u0445\u0430\u043b\u0430\u0442", "\u0438\u0434\u0435\u0430\u043b", "\u043b\u0435\u043d\u0442\u0430", "\u0431\u043e\u043c\u0431\u0430", "\u0448\u0442\u0430\u043c\u043c", "\u0443\u0449\u0435\u0440\u0431", "\u0430\u0440\u0445\u0438\u0432", "\u0432\u0441\u043b\u0443\u0445", "\u0441\u044b\u0440\u044c\u0435", "\u0448\u0442\u0430\u043d\u044b", "\u043f\u043e\u0445\u043e\u0434", "\u043c\u043e\u0440\u0434\u0430", "\u043f\u0440\u043e\u0437\u0430", "\u043c\u0430\u0441\u043a\u0430", "\u043e\u043f\u0435\u0440\u0430", "\u0448\u043b\u044f\u043f\u0430", "\u0431\u0435\u043b\u044c\u0435", "\u0432\u0435\u0434\u0440\u043e", "\u043b\u0430\u043c\u043f\u0430", "\u043c\u0430\u043b\u044b\u0448", "\u043e\u0431\u043b\u0438\u043a", "\u043a\u043e\u043f\u0438\u044f", "\u0443\u0433\u043e\u043b\u044c", "\u0443\u0441\u0442\u0430\u0432", "\u0441\u0443\u0434\u043d\u043e", "\u0447\u0430\u0448\u043a\u0430", "\u044d\u043b\u0438\u0442\u0430", "\u0432\u0435\u0442\u0432\u044c", "\u043f\u043e\u0447\u0442\u0430", "\u043f\u0430\u043b\u043a\u0430", "\u0430\u043a\u0442\u0438\u0432", "\u0441\u0442\u043e\u043b\u0431", "\u0430\u043d\u0433\u0435\u043b", "\u0448\u0442\u0440\u0430\u0444", "\u043e\u0442\u0435\u043b\u044c", "\u043c\u044b\u0448\u043a\u0430", "\u043d\u043e\u0436\u043a\u0430", "\u0441\u0432\u0435\u0447\u0430", "\u0431\u043b\u044e\u0434\u043e", "\u0433\u0440\u0430\u043d\u044c", "\u0441\u043a\u0430\u043b\u0430", "\u0442\u0430\u043a\u0441\u0438", "\u0437\u0435\u0440\u043d\u043e", "\u0441\u043e\u0441\u043d\u0430", "\u0444\u043e\u0440\u0443\u043c", "\u0431\u043b\u0435\u0441\u043a", "\u043f\u0430\u043f\u043a\u0430", "\u0441\u0442\u0435\u043f\u044c", "\u043a\u043e\u0432\u0435\u0440", "\u043f\u043b\u0430\u043c\u044f", "\u0442\u0440\u0435\u0442\u044c", "\u043f\u0435\u0432\u0435\u0446", "\u0440\u0443\u0436\u044c\u0435", "\u0440\u0435\u0447\u043a\u0430", "\u0441\u044b\u0440\u043e\u0439", "\u0444\u0438\u043d\u0430\u043b", "\u043d\u0435\u0434\u0440\u0430", "\u0434\u0435\u0432\u043a\u0430", "\u0437\u043d\u0430\u043c\u044f", "\u0440\u044e\u043c\u043a\u0430", "\u0438\u0441\u0445\u043e\u0434", "\u0437\u0432\u0435\u043d\u043e", "\u0433\u0440\u0438\u043f\u043f", "\u043f\u0435\u0447\u043a\u0430", "\u0448\u0435\u043f\u043e\u0442", "\u043f\u0438\u043b\u043e\u0442", "\u043f\u043b\u043e\u0442\u044c", "\u0441\u0430\u043c\u0435\u0446", "\u0447\u0435\u0440\u0435\u043f", "\u0431\u043e\u0447\u043a\u0430", "\u0433\u0430\u0440\u0430\u0436", "\u0441\u043e\u0441\u0443\u0434", "\u043f\u043e\u044d\u043c\u0430", "\u0441\u043a\u043b\u043e\u043d", "\u043b\u0430\u0432\u043a\u0430", "\u0431\u043e\u043a\u0430\u043b", "\u043a\u0438\u0441\u0442\u044c", "\u0438\u0434\u0438\u043e\u0442", "\u043a\u043e\u0439\u043a\u0430", "\u0442\u0435\u0441\u043d\u043e", "\u044f\u043a\u043e\u0440\u044c", "\u0434\u0440\u043e\u0432\u0430", "\u0432\u043e\u0440\u043e\u0442", "\u043e\u043f\u043e\u0440\u0430", "\u0443\u0447\u0435\u0431\u0430", "\u0442\u043e\u043f\u043e\u0440", "\u0444\u0438\u0437\u0438\u043a", "\u0431\u0430\u0440\u043e\u043d", "\u0431\u0430\u0440\u0430\u043a", "\u043a\u0443\u043a\u043b\u0430", "\u0436\u0435\u043d\u0438\u0445", "\u043d\u043e\u0441\u043e\u043a", "\u043e\u0431\u0443\u0432\u044c", "\u043d\u0430\u043c\u0435\u043a", "\u043a\u043e\u0442\u0435\u043b", "\u043f\u0440\u0438\u043d\u0446", "\u043f\u0443\u0448\u043a\u0430", "\u0431\u0435\u0434\u0440\u043e", "\u043f\u043e\u0440\u044b\u0432", "\u0441\u0435\u0442\u043a\u0430", "\u0441\u0430\u0440\u0430\u0439", "\u043e\u043f\u0440\u043e\u0441", "\u0441\u0442\u0430\u0440\u0442", "\u0448\u043a\u0443\u0440\u0430", "\u0434\u0440\u0430\u043a\u0430", "\u043f\u0440\u043e\u0431\u0430", "\u043a\u0430\u0437\u0430\u043a", "\u043c\u0443\u0441\u043e\u0440", "\u043c\u043e\u0440\u044f\u043a", "\u0441\u0430\u043c\u043a\u0430", "\u0442\u0443\u0444\u043b\u044f", "\u0445\u0438\u043c\u0438\u044f", "\u043c\u0430\u0434\u0430\u043c", "\u043f\u043b\u0435\u043c\u044f", "\u043a\u0443\u0431\u043e\u043a", "\u044f\u0433\u043e\u0434\u0430", "\u0431\u0438\u0442\u0432\u0430", "\u043c\u043e\u043d\u0430\u0445", "\u0433\u0440\u0430\u043c\u043c", "\u0434\u044b\u0440\u043a\u0430", "\u043f\u043e\u0431\u0435\u0433", "\u0431\u0430\u0437\u0430\u0440", "\u0434\u0440\u0430\u043c\u0430", "\u043e\u0442\u0431\u043e\u0440", "\u0441\u043f\u0438\u0440\u0442", "\u043a\u0443\u0437\u043e\u0432", "\u0432\u0430\u043d\u043d\u0430", "\u043a\u0430\u043d\u043e\u043d", "\u0440\u044b\u0431\u0430\u043a", "\u0440\u0435\u0431\u0440\u043e", "\u0444\u0440\u0443\u043a\u0442", "\u0432\u0434\u043e\u0432\u0430", "\u0433\u0440\u0443\u0431\u043e", "\u0431\u0430\u043b\u0435\u0442", "\u0432\u0435\u0441\u0442\u044c", "\u043a\u0430\u0437\u043d\u044c", "\u043d\u0430\u0432\u044b\u043a", "\u043f\u0435\u043d\u0438\u0435", "\u0440\u0443\u0447\u0435\u0439", "\u0432\u0437\u0432\u043e\u0434", "\u043a\u0430\u0441\u0441\u0430", "\u0449\u0435\u043d\u043e\u043a", "\u0432\u0437\u0434\u043e\u0445", "\u044e\u0440\u0438\u0441\u0442", "\u0448\u0430\u0445\u0442\u0430", "\u0431\u0443\u043a\u0435\u0442", "\u0442\u0438\u0440\u0430\u0436", "\u0448\u043f\u0438\u043e\u043d", "\u0436\u0438\u043b\u0435\u0446", "\u0442\u0435\u0437\u0438\u0441", "\u0437\u0430\u043b\u043e\u0433", "\u0441\u0442\u0440\u0443\u044f", "\u043e\u0447\u0435\u0440\u043a", "\u0431\u0443\u0444\u0435\u0442", "\u0436\u0430\u0436\u0434\u0430", "\u0441\u0434\u0430\u0447\u0430", "\u043e\u0432\u043e\u0449\u0438", "\u043f\u0430\u0441\u0442\u044c", "\u043d\u0438\u0442\u043a\u0430", "\u0440\u043e\u044f\u043b\u044c", "\u043f\u0430\u0446\u0430\u043d", "\u0441\u0443\u0435\u0442\u0430", "\u043a\u043e\u0437\u0435\u043b", "\u043d\u0430\u0440\u044f\u0434", "\u0442\u0430\u0439\u0433\u0430", "\u043f\u0435\u0442\u043b\u044f", "\u0442\u0435\u043c\u043d\u043e", "\u043f\u043b\u0430\u0441\u0442", "\u043e\u0431\u043c\u0430\u043d", "\u0448\u0430\u0440\u0438\u043a", "\u0440\u044b\u0447\u0430\u0433", "\u0437\u043b\u043e\u0431\u0430", "\u043f\u0435\u0441\u043d\u044c", "\u0441\u0442\u0430\u0434\u043e", "\u0433\u043b\u0438\u043d\u0430", "\u0432\u044b\u0435\u0437\u0434", "\u0437\u0430\u043a\u0430\u0442", "\u0432\u0438\u0441\u043e\u043a", "\u043e\u0431\u0437\u043e\u0440", "\u0437\u0430\u0432\u0435\u0442", "\u0438\u0441\u043a\u0440\u0430", "\u043f\u043e\u0433\u043e\u043d", "\u0432\u0437\u043d\u043e\u0441", "\u0441\u043b\u0443\u0433\u0430", "\u043a\u0440\u044b\u0441\u0430", "\u043a\u0443\u043b\u044c\u0442", "\u0441\u044b\u043d\u043e\u043a", "\u0434\u043e\u0432\u043e\u0434", "\u0441\u043f\u0443\u0441\u043a", "\u0447\u0430\u0439\u043a\u0430", "\u0433\u0440\u0443\u043d\u0442", "\u043e\u0431\u044b\u0441\u043a", "\u0437\u0430\u0440\u044f\u0434", "\u043a\u0443\u043f\u043e\u043b", "\u043e\u0442\u0437\u044b\u0432", "\u043f\u043e\u0437\u043e\u0440", "\u0432\u043e\u043f\u043b\u044c", "\u043a\u0430\u0442\u0435\u0440", "\u0446\u044b\u0433\u0430\u043d", "\u0430\u043b\u043c\u0430\u0437", "\u043c\u0438\u043d\u0443\u0441", "\u043e\u0431\u0440\u044f\u0434", "\u043f\u043e\u043b\u044f\u043a", "\u0442\u0443\u043f\u0438\u043a", "\u0442\u0440\u043e\u043f\u0430", "\u0440\u0435\u043b\u044c\u0441", "\u0434\u0438\u043b\u0435\u0440", "\u0444\u0435\u0440\u043c\u0430", "\u0433\u043e\u0440\u043a\u0430", "\u043f\u0438\u0440\u043e\u0433", "\u0441\u0442\u0430\u043b\u044c", "\u0442\u0430\u0431\u0430\u043a", "\u043f\u043e\u0447\u043a\u0430", "\u0442\u0432\u0430\u0440\u044c", "\u0437\u0430\u043b\u0438\u0432", "\u0438\u0441\u043f\u0443\u0433", "\u0430\u043b\u043b\u0435\u044f", "\u0432\u0438\u043b\u043a\u0430", "\u043f\u0435\u0442\u0443\u0445", "\u0444\u043e\u043a\u0443\u0441", "\u043f\u043e\u043a\u0430\u0437", "\u043e\u0432\u0440\u0430\u0433", "\u0433\u0440\u043e\u0437\u0430", "\u043f\u043e\u043b\u044e\u0441", "\u0440\u0443\u0441\u043b\u043e", "\u0431\u0443\u0434\u043a\u0430", "\u043e\u043b\u0435\u043d\u044c", "\u0440\u0443\u0431\u043a\u0430", "\u0440\u044b\u0431\u043a\u0430", "\u0442\u0440\u0443\u0441\u044b", "\u0431\u043e\u0441\u043e\u0439", "\u0441\u0430\u043b\u0430\u0442", "\u0431\u0430\u043d\u0434\u0430", "\u0431\u043e\u0440\u0435\u0446", "\u043e\u0431\u0440\u044b\u0432", "\u0445\u043e\u0445\u043e\u0442", "\u0432\u0432\u0438\u0434\u0443", "\u043a\u043e\u043c\u0430\u0440", "\u043c\u0430\u0439\u043a\u0430", "\u043f\u0430\u0440\u0443\u0441", "\u043f\u0435\u0440\u0435\u0446", "\u043f\u0438\u0440\u0430\u0442", "\u0441\u0442\u0435\u043d\u0434", "\u0438\u0441\u043b\u0430\u043c", "\u0443\u043f\u0440\u0435\u043a", "\u0445\u0443\u0442\u043e\u0440", "\u0431\u0438\u0440\u0436\u0430", "\u0438\u0441\u0442\u043e\u043a", "\u043d\u0430\u043b\u0435\u0442", "\u0444\u0430\u0441\u0430\u0434", "\u0438\u0441\u0442\u0435\u0446", "\u043f\u044b\u0442\u043a\u0430", "\u0442\u0443\u0440\u043e\u043a", "\u044d\u0442\u0438\u043a\u0430", "\u0432\u0438\u0441\u043a\u0438", "\u043e\u0431\u043a\u043e\u043c", "\u0432\u043b\u0430\u0433\u0430", "\u0441\u0435\u0430\u043d\u0441", "\u043c\u044d\u0440\u0438\u044f", "\u043f\u0430\u043b\u0430\u0447", "\u043a\u043b\u043e\u0443\u043d", "\u0442\u0440\u0435\u0441\u043a", "\u0431\u0430\u0448\u043a\u0430", "\u0441\u043f\u043b\u0430\u0432", "\u0441\u0441\u043e\u0440\u0430", "\u043c\u0435\u0434\u0438\u043a", "\u043d\u0430\u0441\u043e\u0441", "\u0441\u043a\u0443\u043a\u0430", "\u0448\u043a\u0430\u043b\u0430", "\u0430\u0440\u0435\u043d\u0430", "\u0438\u043c\u0438\u0434\u0436", "\u043b\u0430\u0441\u043a\u0430", "\u0442\u0438\u0442\u0443\u043b", "\u0434\u0430\u0440\u043e\u043c", "\u043a\u0438\u043e\u0441\u043a", "\u043a\u0440\u0443\u0442\u043e", "\u043f\u0430\u0440\u0442\u0430", "\u043f\u043e\u0432\u0430\u0440", "\u0437\u0430\u0442\u0435\u044f", "\u043f\u0430\u0440\u0430\u0434", "\u043f\u0443\u0447\u043e\u043a", "\u0433\u0430\u0437\u043e\u043d", "\u043f\u044f\u0442\u043a\u0430", "\u043a\u0440\u043e\u043d\u0430", "\u0441\u0438\u043d\u044f\u043a", "\u0440\u044b\u0432\u043e\u043a", "\u0432\u0437\u043b\u0435\u0442", "\u043f\u0435\u043f\u0435\u043b", "\u043f\u043e\u0440\u043e\u043a", "\u0441\u0435\u0434\u043b\u043e", "\u0448\u0442\u0443\u0440\u043c", "\u043f\u0440\u043e\u0435\u043c", "\u0431\u043e\u0434\u0440\u043e", "\u0434\u0440\u043e\u0436\u044c", "\u043a\u0430\u044e\u0442\u0430", "\u043a\u0438\u0448\u043a\u0430", "\u043a\u0443\u043f\u0435\u0446", "\u0441\u0434\u0432\u0438\u0433", "\u0448\u0442\u043e\u0440\u0430", "\u0431\u0440\u0435\u043c\u044f", "\u043f\u0430\u0444\u043e\u0441", "\u043a\u0435\u043f\u043a\u0430", "\u043c\u0438\u0441\u043a\u0430", "\u0431\u0430\u0433\u0430\u0436", "\u043a\u043e\u043d\u0435\u043a", "\u043c\u0430\u043d\u0435\u0436", "\u043c\u0435\u0441\u0442\u044c", "\u0447\u0443\u043b\u043e\u043a", "\u0434\u0440\u044f\u043d\u044c", "\u043a\u0443\u043c\u0438\u0440", "\u0441\u0430\u0434\u0438\u043a", "\u0441\u0432\u0438\u0441\u0442", "\u0430\u0437\u0430\u0440\u0442", "\u0433\u0435\u043d\u043e\u043c", "\u0433\u0443\u0434\u043e\u043a", "\u043a\u0432\u043e\u0442\u0430", "\u043b\u0438\u043c\u043e\u043d", "\u0441\u0442\u043e\u043f\u0430", "\u0431\u0435\u043b\u043a\u0430", "\u043e\u0431\u0445\u043e\u0434", "\u0440\u043e\u043b\u0438\u043a", "\u0441\u043b\u044e\u043d\u0430", "\u043c\u0435\u043b\u043a\u043e", "\u043e\u043a\u043b\u0430\u0434", "\u0431\u0435\u0433\u043e\u043c", "\u0431\u0435\u0442\u043e\u043d", "\u043a\u0430\u043c\u0438\u043d", "\u043f\u0443\u043b\u044c\u0442", "\u0433\u0430\u043c\u043c\u0430", "\u043c\u0435\u0440\u043a\u0430", "\u0430\u0444\u0438\u0448\u0430", "\u0432\u0430\u0445\u0442\u0430", "\u0433\u0440\u0443\u0434\u0430", "\u0440\u043e\u0434\u043d\u044f", "\u0434\u043e\u043d\u043e\u0441", "\u043e\u0441\u043e\u0431\u0430", "\u0448\u043e\u0440\u043e\u0445", "\u0448\u0442\u0430\u043c\u043f", "\u0431\u043b\u0430\u043d\u043a", "\u0431\u0440\u044e\u0445\u043e", "\u0433\u0440\u0443\u0448\u0430", "\u0434\u0435\u043c\u043e\u043d", "\u043c\u0430\u0433\u0438\u044f", "\u0441\u043a\u0440\u0438\u043f", "\u0442\u0430\u043b\u0438\u044f", "\u0442\u0435\u0441\u0442\u043e", "\u0432\u044b\u0448\u043a\u0430", "\u043b\u0430\u0440\u0435\u043a", "\u0441\u043a\u0432\u0435\u0440", "\u0432\u0438\u043b\u043b\u0430", "\u043b\u0430\u043f\u043a\u0430", "\u043a\u0443\u0447\u043a\u0430", "\u043f\u0447\u0435\u043b\u0430", "\u0441\u043a\u0443\u043b\u0430", "\u0430\u0431\u0437\u0430\u0446", "\u043f\u0430\u0441\u0445\u0430", "\u0444\u0440\u0430\u043d\u043a", "\u0431\u0443\u0433\u043e\u0440", "\u043d\u0430\u043f\u043e\u0440", "\u043a\u0430\u0437\u043d\u0430", "\u043a\u043e\u043f\u044c\u0435", "\u0431\u0430\u043b\u043a\u0430", "\u0432\u0435\u0441\u043b\u043e", "\u0437\u0430\u0447\u0435\u0442", "\u043a\u0443\u0431\u0438\u043a", "\u0447\u0435\u0440\u0432\u044c", "\u0449\u0435\u0442\u043a\u0430", "\u0434\u043e\u0441\u0443\u0433", "\u0432\u0438\u0445\u0440\u044c", "\u0447\u0443\u0442\u044c\u0435", "\u0432\u0435\u043d\u0435\u0446", "\u043a\u043e\u0441\u044f\u043a", "\u043d\u044e\u0430\u043d\u0441", "\u0448\u0430\u0448\u043a\u0430", "\u0447\u0443\u0434\u0430\u043a", "\u0432\u044a\u0435\u0437\u0434", "\u0432\u044b\u043b\u0435\u0442", "\u0433\u043b\u0443\u0431\u044c", "\u043a\u043e\u0440\u043c\u0430", "\u043f\u0443\u043b\u044c\u0441", "\u0448\u0438\u0448\u043a\u0430", "\u0430\u043a\u0443\u043b\u0430", "\u043e\u0442\u0441\u0435\u043a", "\u043f\u0440\u0438\u044e\u0442", "\u0441\u0432\u0438\u0442\u0430", "\u043a\u0440\u0430\u0436\u0430", "\u043b\u0438\u043b\u0438\u044f", "\u0444\u0430\u043a\u0435\u043b", "\u0448\u043b\u0430\u043d\u0433", "\u0430\u0440\u0431\u0443\u0437", "\u0432\u0435\u043d\u043e\u043a", "\u0434\u0438\u0435\u0442\u0430", "\u043a\u043e\u0440\u043a\u0430", "\u0434\u0443\u044d\u043b\u044c", "\u043a\u0430\u043d\u0430\u0442", "\u0440\u0443\u0431\u0438\u043d", "\u0431\u0430\u0440\u0430\u043d", "\u0445\u043e\u043b\u0441\u0442", "\u043a\u043e\u043b\u0435\u044f", "\u043c\u0430\u0444\u0438\u044f", "\u0440\u0430\u0446\u0438\u044f", "\u0431\u0443\u0434\u043d\u0438", "\u0438\u0437\u043d\u043e\u0441", "\u043a\u043e\u043c\u043e\u043a", "\u0440\u043e\u0431\u043e\u0442", "\u0433\u0440\u0430\u043d\u0442", "\u0430\u043b\u043b\u0430\u0445", "\u0431\u0443\u0440\u043d\u043e", "\u0433\u043b\u044b\u0431\u0430", "\u0438\u0437\u0433\u0438\u0431", "\u0440\u0438\u0444\u043c\u0430", "\u0444\u043b\u0430\u043d\u0433", "\u044d\u0441\u043a\u0438\u0437", "\u0431\u0443\u0445\u0442\u0430", "\u043c\u0430\u043d\u0435\u0440", "\u043d\u0443\u0442\u0440\u043e", "\u0442\u0430\u0447\u043a\u0430", "\u0444\u0438\u043d\u0438\u0448", "\u0445\u0438\u043c\u0438\u043a", "\u0447\u0443\u0434\u043e\u043c", "\u0430\u0431\u043e\u0440\u0442", "\u0432\u0430\u043b\u044c\u0441", "\u0441\u043e\u043a\u043e\u043b", "\u0447\u0443\u043a\u0447\u0430", "\u0431\u0440\u043e\u043d\u044f", "\u0432\u043e\u0437\u043d\u044f", "\u043a\u0430\u0441\u043a\u0430", "\u0441\u0430\u043b\u044e\u0442", "\u0431\u0430\u0440\u0436\u0430", "\u0434\u0435\u0431\u044e\u0442", "\u0434\u0435\u0432\u0438\u0437", "\u043c\u0430\u043a\u0435\u0442", "\u043e\u043f\u0435\u043a\u0430", "\u043f\u043e\u0434\u043e\u043b", "\u043f\u043e\u0447\u0435\u0442", "\u0442\u043e\u043b\u0449\u0430", "\u0432\u044b\u0432\u043e\u0437", "\u0440\u0435\u043d\u0442\u0430", "\u0446\u0435\u0440\u043a\u0430", "\u043a\u0430\u0442\u043e\u043a", "\u043e\u0442\u0440\u044b\u0432", "\u0443\u043a\u043b\u043e\u043d", "\u0448\u043f\u0440\u0438\u0446", "\u0432\u0435\u043d\u0438\u043a", "\u0436\u0443\u043b\u0438\u043a", "\u043a\u043e\u0444\u0442\u0430", "\u043d\u0435\u0434\u0443\u0433", "\u043e\u0441\u043e\u0431\u044c", "\u043e\u0442\u0431\u043e\u0439", "\u0442\u0435\u0441\u0442\u044c", "\u0431\u043e\u0433\u0430\u0447", "\u0432\u0434\u0430\u043b\u044c", "\u043c\u044f\u0447\u0438\u043a", "\u043d\u0430\u0432\u043e\u0437", "\u043e\u0449\u0443\u043f\u044c", "\u0441\u0438\u0440\u043e\u043f", "\u0442\u0440\u0435\u0441\u0442", "\u0431\u0443\u0434\u0434\u0430", "\u0432\u0438\u0434\u0435\u043e", "\u0432\u0438\u0448\u043d\u044f", "\u0434\u043e\u043d\u043e\u0440", "\u0434\u044b\u043c\u043e\u043a", "\u043a\u0430\u043f\u043e\u0442", "\u0441\u043c\u043e\u043b\u0430", "\u0441\u044b\u0449\u0438\u043a", "\u0431\u0430\u0440\u0438\u043d", "\u0432\u044b\u043a\u0443\u043f", "\u0437\u0430\u0445\u043e\u0434", "\u043f\u0438\u0442\u043e\u043d"]

In [80]:
# source 2: https://github.com/ronanru/wordle-ru/blob/main/src/words.ts
allowed_words2 = ['абаец', 'абака', 'абаси', 'аббат', 'абвер', 'абзац', 'абзим', 'абиоз', 'абома', 'аборт', 'абрек', 'абрин', 'абрис', 'аброн', 'абсит', 'абунг', 'абура', 'абхаз', 'абцуг', 'абшид', 'аваги', 'аваль', 'аванс', 'авгит', 'авгур', 'авеню', 'аверс', 'авизо', 'авлет', 'авлос', 'авост', 'авось', 'аврал', 'авран', 'автол', 'автор', 'авуар', 'агава', 'агама', 'агами', 'аганц', 'агапе', 'аггел', 'агема', 'агенс', 'агент', 'агнат', 'агнел', 'агнец', 'агоги', 'агого', 'агона', 'агора', 'аграф', 'агуйя', 'агути', 'адель', 'адепт', 'адник', 'адоба', 'адрес', 'адрон', 'адряс', 'адыге', 'ажгон', 'азара', 'азарт', 'азери', 'азиат', 'азиец', 'азиль', 'азина', 'азога', 'айван', 'аймак', 'айова', 'айоли', 'айран', 'айрен', 'айрон', 'айсор', 'айтыс', 'акажу', 'акант', 'акара', 'акаша', 'акбаш', 'акиба', 'акита', 'аккит', 'акмит', 'акнит', 'акори', 'акреп', 'акрил', 'акрит', 'аксин', 'аксис', 'аксон', 'актер', 'актив', 'актин', 'актор', 'акула', 'акциз', 'акция', 'алаец', 'алаит', 'алапа', 'аларм', 'алаша', 'алгид', 'алгия', 'алгол', 'алдар', 'алеец', 'алеут', 'алиби', 'аличи', 'алкан', 'алкен', 'алкид', 'алкил', 'алкин', 'аллен', 'аллея', 'аллил', 'аллит', 'аллод', 'аллюр', 'алмаз', 'алмуд', 'алоза', 'алтей', 'алтын', 'алунд', 'алфер', 'алчба', 'алыча', 'альба', 'альфа', 'амаас', 'амбал', 'амбар', 'амбра', 'амбре', 'амвон', 'амеба', 'амеро', 'аминь', 'амиур', 'аммин', 'ампер', 'ампир', 'анаша', 'ангар', 'ангас', 'ангел', 'англы', 'ангоб', 'ангон', 'андид', 'андоб', 'андон', 'аниец', 'анима', 'аниме', 'анион', 'анкер', 'анкош', 'анкус', 'аннит', 'аноит', 'анойя', 'анона', 'анонс', 'ансар', 'антем', 'антик', 'антия', 'антре', 'антур', 'антье', 'анфас', 'анчар', 'аншеф', 'аньюм', 'аорта', 'апайо', 'апана', 'апарт', 'апачи', 'апекс', 'апиин', 'апиол', 'апион', 'аплит', 'аплом', 'апноэ', 'апорт', 'апрок', 'апрон', 'апрош', 'апсар', 'аптих', 'апфль', 'араза', 'арама', 'араме', 'арара', 'арбуз', 'аргал', 'аргас', 'аргон', 'аргус', 'арден', 'ареал', 'арека', 'арена', 'арест', 'арзис', 'ариец', 'арион', 'аркад', 'аркал', 'аркан', 'аркат', 'аркоз', 'армет', 'армин', 'армия', 'армюр', 'армяк', 'ароил', 'арпан', 'аррау', 'аррип', 'арсин', 'арсол', 'артиг', 'артос', 'артха', 'архар', 'архат', 'архей', 'архея', 'архив', 'арчак', 'аршин', 'асадо', 'асана', 'асафо', 'аскер', 'аскет', 'аскон', 'аскос', 'аскот', 'аспер', 'аспид', 'аспит', 'ассиз', 'астат', 'астер', 'астма', 'астра', 'асура', 'асцит', 'атака', 'атлас', 'атлет', 'атман', 'атолл', 'атони', 'атрий', 'аттан', 'аттик', 'аудио', 'аудит', 'аурат', 'аурин', 'аурон', 'афган', 'афера', 'афиша', 'афтоз', 'ахеец', 'ахилл', 'ахоит', 'ахунд', 'ацена', 'ацера', 'ацтек', 'ачали', 'ачоли', 'ашико', 'ашрам', 'аштец', 'аэроб', 'аэрон', 'аянец', 'аятит', 'аяцин', 'бабид', 'бабка', 'бабоа', 'багаж', 'багги', 'багер', 'багет', 'багор', 'багуа', 'бадан', 'бадик', 'бадис', 'бадук', 'бадья', 'базан', 'базар', 'базис', 'базит', 'байга', 'байда', 'байер', 'байза', 'байка', 'бакан', 'бакве', 'бакен', 'бакля', 'бакор', 'бакун', 'балда', 'балдж', 'балет', 'балия', 'балка', 'балок', 'балти', 'балун', 'балык', 'бальи', 'баляш', 'бамия', 'бамум', 'банан', 'банар', 'банги', 'банда', 'банди', 'бандо', 'банка', 'банко', 'банту', 'банши', 'барак', 'баран', 'барат', 'барба', 'барда', 'бареж', 'баржа', 'барий', 'барин', 'барит', 'барич', 'бария', 'барка', 'бармы', 'барон', 'баррэ', 'барыш', 'баска', 'басма', 'басня', 'басок', 'басон', 'бастр', 'батан', 'батат', 'батва', 'батик', 'батог', 'батон', 'батос', 'батун', 'батут', 'батыр', 'бауит', 'бауле', 'баунс', 'бафия', 'бахта', 'бахча', 'бахши', 'бацен', 'бачок', 'башня', 'бебут', 'бегун', 'беджа', 'бедро', 'безик', 'бейгл', 'бейза', 'бейка', 'бейра', 'бекар', 'бекас', 'бекон', 'белги', 'белек', 'белец', 'белит', 'белка', 'белок', 'белье', 'бельт', 'беляк', 'беляш', 'бемба', 'бемит', 'бенди', 'бенто', 'бердо', 'берег', 'берет', 'берма', 'берси', 'берсо', 'берта', 'берце', 'берцо', 'бесса', 'бетон', 'бетта', 'бефол', 'биакс', 'бибоп', 'бивак', 'бигль', 'бигос', 'бидон', 'бизон', 'бикит', 'бикол', 'бикса', 'билби', 'билет', 'билин', 'билль', 'бинго', 'бинди', 'бинду', 'бинет', 'бином', 'бинор', 'биоза', 'бионт', 'биота', 'бипак', 'бипед', 'бипер', 'биржа', 'бирка', 'бирюк', 'бирюч', 'бисер', 'бисса', 'бисти', 'бистр', 'битва', 'битка', 'биток', 'битум', 'битье', 'битюг', 'бифас', 'бишон', 'бишоп', 'благо', 'блажь', 'бланк', 'бланш', 'бласт', 'блеск', 'близь', 'блинд', 'блинт', 'блоха', 'блуза', 'блюдо', 'бляха', 'бобер', 'бобик', 'бобок', 'богач', 'бодхи', 'бодяк', 'божба', 'божок', 'бозах', 'бозец', 'бозон', 'бойга', 'бойит', 'бойки', 'бойня', 'бокаж', 'бокал', 'бокит', 'бокка', 'бокой', 'болет', 'болид', 'болюс', 'бомба', 'бомбо', 'бонга', 'бонго', 'бонгу', 'бонет', 'бонза', 'бонка', 'бонмо', 'бонна', 'бонус', 'боопс', 'боран', 'борат', 'бордо', 'борей', 'борец', 'борид', 'борий', 'борил', 'борин', 'боров', 'борол', 'борть', 'босяк', 'ботва', 'ботик', 'ботус', 'боция', 'бочаг', 'бочар', 'бочка', 'бочче', 'браво', 'брага', 'бракк', 'брана', 'брань', 'браса', 'брасс', 'браха', 'бреве', 'брейк', 'бремя', 'бренд', 'брешь', 'бриар', 'бридж', 'бриза', 'бритт', 'бровь', 'бронх', 'бронь', 'броня', 'броше', 'брошь', 'брыжи', 'брыла', 'брыль', 'брюки', 'брюхо', 'бубал', 'бубен', 'бубна', 'бубон', 'бувье', 'бугай', 'бугор', 'будда', 'будзю', 'будин', 'будка', 'будни', 'будра', 'бузун', 'буква', 'букер', 'букет', 'букле', 'букля', 'букол', 'букса', 'букша', 'булат', 'булка', 'булла', 'бульб', 'бунак', 'бурав', 'бурак', 'буран', 'бурат', 'бурда', 'бурка', 'бурма', 'бурре', 'бурса', 'бурун', 'бурят', 'бутан', 'бутен', 'бутик', 'бутил', 'бутин', 'бутит', 'бутия', 'бутон', 'бутса', 'бутуз', 'бутут', 'буфер', 'буфет', 'буфол', 'бухит', 'бухта', 'быдло', 'былка', 'былое', 'былье', 'бытие', 'бытье', 'бычок', 'бювар', 'бювет', 'бюкса', 'вабик', 'ваган', 'вагон', 'вазон', 'вайда', 'вайма', 'вакат', 'вакка', 'вакса', 'вакус', 'вакуф', 'валах', 'валеж', 'валек', 'валер', 'валет', 'валец', 'валик', 'валил', 'валин', 'валка', 'валок', 'валуй', 'валун', 'валух', 'вальс', 'ванга', 'ванда', 'ванец', 'ванна', 'ванта', 'вапор', 'варан', 'варва', 'варек', 'варка', 'варна', 'варяг', 'варяк', 'ватер', 'ватин', 'ватка', 'ватты', 'вафля', 'вахня', 'вахта', 'вброс', 'вдова', 'вебер', 'веган', 'ведда', 'ведро', 'ведун', 'вежда', 'везир', 'вейка', 'векша', 'велец', 'велик', 'велит', 'велум', 'вельд', 'вельс', 'велюр', 'венге', 'венгр', 'венда', 'венед', 'венет', 'венец', 'веник', 'венит', 'венка', 'венок', 'вента', 'венус', 'вепрь', 'верба', 'вервь', 'веред', 'верея', 'верже', 'верит', 'верпа', 'версо', 'верфь', 'верша', 'весло', 'весна', 'весок', 'веста', 'весть', 'ветал', 'ветвь', 'ветер', 'ветка', 'ветла', 'вечер', 'вечин', 'вешка', 'вещун', 'взбег', 'взвар', 'взвод', 'взвоз', 'вздор', 'вздох', 'взлет', 'взлив', 'взлом', 'взмах', 'взмет', 'взмыв', 'взнос', 'взрез', 'взрыв', 'взрыд', 'взъем', 'вивер', 'вигна', 'видам', 'видео', 'видиа', 'видик', 'видок', 'визаж', 'визир', 'визит', 'визор', 'вийот', 'викит', 'вилик', 'вилка', 'вилла', 'вилок', 'винея', 'винил', 'винин', 'винир', 'винол', 'винту', 'виола', 'випий', 'вираж', 'вирус', 'вирши', 'виска', 'виски', 'висок', 'вития', 'виток', 'виттл', 'витье', 'вихор', 'вихрь', 'вицин', 'вишап', 'вишня', 'вклад', 'влага', 'вобла', 'вогул', 'водка', 'вожак', 'вождь', 'вожжа', 'возка', 'возня', 'возок', 'война', 'вокал', 'вокар', 'волан', 'волна', 'волок', 'волос', 'волоф', 'волхв', 'вольт', 'волюм', 'вомер', 'вопль', 'ворог', 'ворон', 'ворот', 'ворох', 'ворса', 'ворст', 'ворье', 'вотум', 'вотяк', 'вояка', 'впуск', 'враки', 'враль', 'врата', 'время', 'вруша', 'вскип', 'вскок', 'вспых', 'всход', 'втора', 'вуаль', 'вчера', 'въезд', 'выбег', 'выбой', 'выбор', 'вывал', 'вывес', 'вывих', 'вывод', 'вывоз', 'выгар', 'выгиб', 'выгон', 'выгул', 'выдел', 'выдох', 'выдра', 'выдув', 'выезд', 'выжиг', 'выжим', 'выжла', 'вызов', 'выкат', 'выкид', 'выкол', 'выкос', 'выкуп', 'выкус', 'вылаз', 'вылет', 'вылив', 'вылов', 'вылом', 'вымах', 'вымет', 'вымич', 'вынос', 'выпад', 'выпал', 'выпар', 'выпас', 'выпор', 'выпот', 'вырез', 'выруб', 'вырыв', 'высев', 'высед', 'высол', 'вытие', 'вытье', 'выход', 'вычет', 'вышаг', 'вышка', 'вьюга', 'вьяна', 'вязка', 'вятец', 'вятич', 'вятка', 'габар', 'гавит', 'гавот', 'гагат', 'газер', 'газон', 'гайда', 'гайка', 'гайно', 'гайот', 'галец', 'галит', 'галка', 'галла', 'галле', 'галло', 'галон', 'галоп', 'галун', 'галут', 'гамак', 'гаман', 'гамба', 'гамма', 'гамон', 'ганаш', 'ганда', 'ганец', 'ганий', 'ганит', 'гания', 'ганка', 'гараж', 'гарда', 'гарде', 'гарди', 'гареа', 'гарем', 'гарна', 'гаруа', 'гарус', 'гаста', 'гаусс', 'гаучо', 'гахам', 'гачек', 'гашиш', 'гаюин', 'гбайя', 'гвалт', 'гваяк', 'гевея', 'гезит', 'гейша', 'гекат', 'гекта', 'гелий', 'гелин', 'гелит', 'гемам', 'гемин', 'гемма', 'генез', 'гений', 'генин', 'геном', 'генри', 'генро', 'геоид', 'герит', 'герлс', 'герма', 'герой', 'гетит', 'гетра', 'гетто', 'гжель', 'гибка', 'гидра', 'гиена', 'гижда', 'гилея', 'гилия', 'гиляк', 'гимен', 'гинец', 'гинея', 'гипак', 'гипюр', 'гирих', 'гирло', 'гирча', 'гитан', 'гитов', 'гичка', 'глава', 'главк', 'гладь', 'глайд', 'глеба', 'глефа', 'глиеж', 'глина', 'глинт', 'глиоз', 'глипт', 'глист', 'глубь', 'глушь', 'глыба', 'глюон', 'глясе', 'гмина', 'гнейс', 'гнида', 'гниль', 'гнома', 'гобан', 'гобой', 'говор', 'гогот', 'гоист', 'гокко', 'голар', 'голем', 'голет', 'голец', 'голик', 'голод', 'голос', 'голыш', 'гольд', 'голье', 'гольф', 'голяк', 'гомбо', 'гомон', 'гонан', 'гонга', 'гонец', 'гонит', 'гонка', 'гонок', 'гонор', 'гопак', 'гопан', 'гопен', 'горал', 'горец', 'гореш', 'горжа', 'горит', 'горка', 'горло', 'город', 'горох', 'горст', 'гость', 'готра', 'гофер', 'гофре', 'гохуа', 'гошма', 'граве', 'грайк', 'грайм', 'грамм', 'грана', 'гранд', 'гранж', 'грано', 'грант', 'грань', 'граус', 'графа', 'графт', 'грахт', 'гребо', 'грежа', 'греза', 'грейд', 'грена', 'греча', 'грива', 'гридь', 'гриль', 'гриот', 'грипп', 'гроза', 'грозд', 'гроот', 'гросс', 'гроул', 'гроут', 'груба', 'груда', 'грудь', 'грунт', 'груси', 'груша', 'грыжа', 'гряда', 'грязь', 'гуава', 'гуано', 'гуань', 'гуари', 'гуаро', 'гуато', 'гуашь', 'губан', 'губач', 'губка', 'гугол', 'гудок', 'гужон', 'гузка', 'гуиро', 'гукар', 'гукер', 'гукор', 'гулям', 'гуляш', 'гумай', 'гумат', 'гумин', 'гумит', 'гумма', 'гумми', 'гумно', 'гумус', 'гунди', 'гунду', 'гуппи', 'гупти', 'гуран', 'гурда', 'гурия', 'гуркх', 'гурма', 'гурон', 'гусак', 'гусан', 'гусар', 'гусек', 'гусит', 'гусла', 'гусле', 'гусли', 'гутта', 'гуцул', 'гушти', 'гэгэн', 'гэнро', 'гювеч', 'гюлеш', 'гюреш', 'гюрза', 'дабка', 'давин', 'давка', 'дадан', 'дайка', 'дайме', 'дайна', 'дайра', 'далер', 'далит', 'далия', 'даман', 'дамба', 'дамка', 'дамно', 'дампо', 'данио', 'дараф', 'дарик', 'дарси', 'дартс', 'датер', 'датив', 'даури', 'дафла', 'дафна', 'дацан', 'дацит', 'дверь', 'двояк', 'дебай', 'дебен', 'дебет', 'дебил', 'дебит', 'дебош', 'дебри', 'дебют', 'девиз', 'девка', 'девон', 'дегра', 'деизм', 'деист', 'декан', 'декор', 'дележ', 'делец', 'делли', 'демон', 'демос', 'денар', 'денди', 'деним', 'денье', 'дерби', 'дерен', 'дерма', 'дерть', 'десна', 'дессу', 'десть', 'детва', 'детка', 'дефис', 'децен', 'децим', 'децин', 'джайв', 'джаса', 'джати', 'джига', 'джинн', 'джойн', 'джола', 'джолт', 'джума', 'дзета', 'дзюдо', 'диада', 'диана', 'диван', 'дидим', 'диета', 'дизен', 'дийод', 'дилдо', 'дилей', 'дилен', 'дилер', 'дильс', 'димер', 'динар', 'динас', 'динат', 'динги', 'динго', 'динер', 'динит', 'динка', 'динод', 'диоат', 'диола', 'диоон', 'дипир', 'дирит', 'диско', 'дихас', 'дихта', 'дичок', 'диэдр', 'длань', 'длина', 'днище', 'добла', 'добор', 'добра', 'добро', 'довга', 'довод', 'довоз', 'догма', 'догон', 'догра', 'догри', 'додзе', 'додин', 'дождь', 'дожим', 'дожин', 'дозор', 'дойка', 'дойна', 'дойра', 'докер', 'долив', 'долма', 'долок', 'долом', 'домен', 'домер', 'домик', 'домит', 'домна', 'домол', 'домра', 'донго', 'донец', 'донка', 'донна', 'донор', 'донос', 'донум', 'донья', 'допан', 'досев', 'доска', 'досол', 'досуг', 'досыл', 'досье', 'доула', 'дофин', 'доход', 'дочка', 'драга', 'драек', 'драже', 'драйв', 'драка', 'драма', 'дрань', 'драпа', 'драфт', 'древо', 'дрейф', 'дрель', 'дрема', 'дрена', 'дриин', 'дрифт', 'дробь', 'дрова', 'дрога', 'дроги', 'дрожь', 'дрозд', 'дронт', 'дросс', 'дрофа', 'друза', 'друид', 'дрюит', 'дрязг', 'дрянь', 'дуала', 'дуаль', 'дуант', 'дубка', 'дубль', 'дубье', 'дувал', 'дуван', 'дудка', 'дудук', 'дужка', 'дузен', 'дукат', 'дукер', 'дулеб', 'дулун', 'думец', 'думка', 'дунит', 'дунст', 'дуоди', 'дуоль', 'дупло', 'дурак', 'дурил', 'дурол', 'дурра', 'дурро', 'дусен', 'дутар', 'дутик', 'дутыш', 'дутье', 'духан', 'дучка', 'душка', 'душок', 'дуэль', 'дхоти', 'дщерь', 'дыбка', 'дымка', 'дырка', 'дышло', 'дюбек', 'дюкер', 'дюкит', 'дюрен', 'дюрок', 'дюшес', 'дякло', 'дятел', 'евнух', 'еврей', 'егерь', 'егоза', 'ежиха', 'ездка', 'ездок', 'елань', 'емчак', 'емшан', 'енаит', 'еналь', 'еноат', 'еноил', 'ересь', 'ерибо', 'ерика', 'ерник', 'ероол', 'ершик', 'есаул', 'ессей', 'ехида', 'жабка', 'жабра', 'жажда', 'жакан', 'жакет', 'жакоб', 'жалон', 'жарка', 'жарма', 'жатва', 'жатка', 'жвало', 'жедин', 'жейра', 'желна', 'желоб', 'желть', 'желчь', 'жених', 'жеода', 'жердь', 'жерех', 'жерло', 'жеста', 'жесть', 'жетон', 'живец', 'живое', 'живой', 'живот', 'живье', 'жижка', 'жизнь', 'жилет', 'жилец', 'жилка', 'жилье', 'жират', 'жираф', 'жирши', 'жиряк', 'житие', 'житье', 'жмудь', 'жница', 'жокей', 'жрица', 'жуаль', 'жудец', 'жулан', 'жулик', 'жулье', 'жумар', 'жупан', 'жупел', 'журка', 'жучка', 'жучок', 'забег', 'забой', 'забор', 'завал', 'завет', 'завод', 'завоз', 'завуч', 'загар', 'загиб', 'загон', 'загул', 'задел', 'задир', 'задок', 'задор', 'заеда', 'заезд', 'зажим', 'зажор', 'зазор', 'зазыв', 'заика', 'заказ', 'закал', 'закат', 'закол', 'закон', 'закоп', 'закуп', 'закус', 'закят', 'залет', 'залив', 'зализ', 'залог', 'залом', 'замах', 'замер', 'замес', 'замет', 'замин', 'замия', 'замок', 'замор', 'замша', 'занде', 'зандр', 'занкл', 'занос', 'запад', 'запал', 'запас', 'запах', 'запев', 'запил', 'запой', 'запор', 'зарез', 'зарин', 'зарок', 'заруб', 'заряд', 'засев', 'засов', 'засол', 'засор', 'засос', 'засыл', 'засып', 'затек', 'затес', 'затея', 'заток', 'затон', 'затоп', 'затор', 'заумь', 'захаб', 'заход', 'зацеп', 'зачес', 'зачет', 'зачин', 'зашаг', 'защип', 'звено', 'зверь', 'зебра', 'зевок', 'зелот', 'зелье', 'зельц', 'земец', 'земля', 'зенит', 'зерет', 'зерец', 'зерно', 'зернь', 'зефир', 'зилот', 'зимоз', 'зиндж', 'зипун', 'злато', 'злоба', 'злюка', 'знамя', 'знать', 'зобик', 'золка', 'золон', 'зоман', 'зомби', 'зооид', 'зорит', 'зраза', 'зубан', 'зубец', 'зубок', 'зулец', 'зулус', 'зумпф', 'зуньи', 'зурна', 'зыбун', 'зэчка', 'зябра', 'иайдо', 'иваси', 'ивина', 'ивняк', 'иврит', 'игиль', 'игрек', 'игрец', 'игрок', 'игрун', 'идаит', 'идеал', 'идель', 'иджма', 'идиот', 'идоза', 'идома', 'идрил', 'иенит', 'иерей', 'ижица', 'ижора', 'избач', 'извет', 'извив', 'извод', 'извоз', 'изгиб', 'изгой', 'излет', 'излив', 'излом', 'измол', 'измор', 'износ', 'изоль', 'изъян', 'изыск', 'изюбр', 'икаит', 'икако', 'икона', 'икота', 'иксия', 'иктус', 'икура', 'илама', 'илеит', 'илеус', 'илиец', 'илиша', 'иллит', 'илоед', 'илька', 'имаго', 'имади', 'имама', 'имейл', 'имидж', 'инвар', 'ингуш', 'индан', 'индат', 'индау', 'инден', 'индид', 'индий', 'индио', 'индит', 'индол', 'индон', 'индри', 'индус', 'индюк', 'инжир', 'инион', 'инкор', 'инкуб', 'иноат', 'инпут', 'интим', 'инуит', 'инула', 'инфак', 'иозис', 'иолит', 'иомуд', 'ионий', 'ионик', 'ионит', 'ионол', 'ионон', 'иофоб', 'иоцит', 'иппон', 'иприт', 'ирбис', 'ирена', 'ирмос', 'искра', 'искус', 'ислам', 'исмин', 'испуг', 'иссоп', 'истец', 'истод', 'исток', 'исход', 'итоит', 'иудей', 'ифрит', 'ихрам', 'ициан', 'ишиас', 'ишпан', 'ишрак', 'ишхан', 'йенец', 'йогин', 'йодат', 'йодид', 'йодил', 'йодин', 'йодит', 'йодль', 'йодол', 'йодюр', 'йомен', 'йошта', 'кабак', 'кабан', 'кабат', 'кабил', 'кабир', 'кабре', 'кабул', 'кавал', 'кавия', 'кагал', 'каган', 'кагат', 'кагор', 'кадет', 'кадиш', 'кадка', 'кадло', 'кадры', 'кадык', 'казак', 'казан', 'казах', 'казна', 'казнь', 'казус', 'кайен', 'кайин', 'кайка', 'кайла', 'кайло', 'кайма', 'кайра', 'кайса', 'кайчи', 'какао', 'калам', 'калан', 'калао', 'калах', 'калач', 'калаш', 'калий', 'калка', 'калла', 'калот', 'калта', 'калым', 'калья', 'камба', 'камео', 'камец', 'камея', 'камин', 'камка', 'кампа', 'камса', 'камус', 'камча', 'камыш', 'канак', 'канал', 'канат', 'канва', 'канец', 'канза', 'канзу', 'канит', 'канна', 'канон', 'каноэ', 'канун', 'канфа', 'канюк', 'капеж', 'капер', 'капля', 'капок', 'капор', 'капот', 'каппа', 'капри', 'капса', 'капча', 'каран', 'карас', 'карат', 'карга', 'карго', 'карда', 'карел', 'карен', 'кариб', 'кария', 'карла', 'карло', 'карма', 'карра', 'карри', 'карст', 'карта', 'карча', 'касба', 'касик', 'каска', 'каско', 'касог', 'касса', 'каста', 'катар', 'катер', 'катет', 'катин', 'катла', 'катля', 'катод', 'каток', 'катта', 'катун', 'катык', 'катыш', 'кауза', 'каури', 'кафир', 'кахец', 'кахон', 'кацик', 'качим', 'качин', 'качка', 'качок', 'качча', 'кашка', 'кашне', 'кашпо', 'кашуб', 'каюта', 'квадр', 'квант', 'кварк', 'кварц', 'квача', 'квени', 'квест', 'квилт', 'кволл', 'квота', 'кебаб', 'кегль', 'кегля', 'кедди', 'кедет', 'кедец', 'кезец', 'кейвы', 'кейит', 'кекит', 'кекчи', 'келпи', 'кельт', 'келья', 'кенар', 'кенаф', 'кепка', 'керес', 'керит', 'керма', 'кетен', 'кетил', 'кетин', 'кетоз', 'кетол', 'кетон', 'кефир', 'кечак', 'кечуа', 'кешью', 'кианг', 'кибла', 'кибуц', 'кивер', 'кивок', 'кидар', 'кидас', 'кидка', 'кидус', 'кижуч', 'кизил', 'кизяк', 'кикер', 'килец', 'килик', 'килим', 'кимак', 'кимвр', 'кимчи', 'кинга', 'кинед', 'кинза', 'киник', 'кинин', 'киноа', 'кинто', 'киоск', 'кипер', 'кирея', 'кирза', 'кирка', 'кирха', 'кисас', 'кисет', 'кисея', 'кисси', 'киста', 'кисть', 'китит', 'кифоз', 'кичка', 'кишка', 'клаве', 'кладь', 'клака', 'кларк', 'класс', 'клатч', 'клеол', 'клерк', 'клерс', 'клест', 'клеть', 'клефт', 'клехо', 'клещи', 'клика', 'клико', 'кликс', 'клинт', 'клинч', 'клипа', 'клипп', 'клипс', 'клица', 'клише', 'клопс', 'клоун', 'клупп', 'клуша', 'клюка', 'клюфт', 'кляча', 'кнель', 'кнехт', 'книга', 'кница', 'князь', 'коала', 'коата', 'коати', 'кобан', 'кобея', 'кобза', 'кобия', 'кобол', 'кобра', 'кобуз', 'кобыз', 'ковар', 'ковен', 'ковер', 'ковит', 'ковка', 'когай', 'когия', 'кодак', 'кодек', 'кодер', 'кодон', 'кожан', 'кожух', 'кожье', 'козел', 'козлы', 'кознь', 'коипу', 'койка', 'койне', 'койот', 'кокер', 'кокет', 'кокни', 'кокон', 'кокор', 'кокос', 'кокур', 'колба', 'колер', 'колет', 'колея', 'колин', 'колит', 'колка', 'колли', 'колло', 'колоб', 'колок', 'колон', 'колос', 'колун', 'колхи', 'колье', 'кольт', 'колюр', 'комар', 'комби', 'комбо', 'комик', 'комит', 'комма', 'комми', 'комод', 'комок', 'комуз', 'конга', 'конго', 'кондо', 'конек', 'конец', 'конзо', 'коник', 'конит', 'конка', 'консо', 'конто', 'конус', 'конфи', 'конха', 'конюх', 'копал', 'копан', 'копач', 'копер', 'копие', 'копир', 'копис', 'копия', 'копка', 'копна', 'коппа', 'копра', 'копун', 'копыл', 'копье', 'корат', 'корба', 'корда', 'корка', 'корма', 'короа', 'короб', 'корсо', 'корча', 'корье', 'коряк', 'косач', 'косец', 'косма', 'косой', 'косок', 'кость', 'косяк', 'котел', 'котик', 'кофан', 'кофта', 'кохит', 'кохия', 'кочан', 'кочка', 'кошер', 'кошка', 'кошма', 'кошок', 'кошти', 'кощей', 'коэль', 'крабб', 'крага', 'кража', 'краза', 'крайт', 'кракс', 'краля', 'крамп', 'кранч', 'крапп', 'краса', 'крауч', 'крафт', 'кредл', 'кредо', 'крезь', 'крень', 'креод', 'креол', 'крепь', 'кресс', 'крест', 'криль', 'крион', 'крица', 'кроат', 'кробу', 'кровь', 'кроки', 'кроль', 'крона', 'кросс', 'крота', 'кроха', 'кроше', 'круиз', 'крупа', 'круча', 'крыло', 'крыса', 'крыша', 'ксоан', 'ктырь', 'кубан', 'кубба', 'кубик', 'кубит', 'кубок', 'кубыз', 'кувас', 'кувез', 'кудри', 'кузен', 'кузов', 'кукан', 'кукер', 'кукиш', 'кукла', 'кукри', 'куксу', 'кукша', 'кулаж', 'кулак', 'кулан', 'кулек', 'кулер', 'кулет', 'кулеш', 'кулик', 'кулич', 'кулия', 'кулон', 'кульм', 'культ', 'кумай', 'куман', 'кумач', 'кумжа', 'кумил', 'кумин', 'кумир', 'кумол', 'кумык', 'кумыс', 'кунак', 'купаж', 'купец', 'купля', 'купол', 'купон', 'купор', 'кураж', 'курай', 'курак', 'кураш', 'курба', 'курес', 'курет', 'куреш', 'курин', 'курия', 'курок', 'курол', 'курос', 'курсы', 'курта', 'курум', 'курут', 'куруц', 'куруш', 'курши', 'куряк', 'куряш', 'кусок', 'кутас', 'кутеж', 'кутин', 'кутис', 'кутия', 'куток', 'кутра', 'кутум', 'кутья', 'кутюм', 'куфея', 'куфия', 'кухня', 'кучер', 'кучия', 'кушак', 'кхаси', 'кхмер', 'кхонд', 'кылыы', 'кэмпо', 'кэндо', 'кэроб', 'кэтти', 'кювет', 'кюйши', 'кюлот', 'кюрий', 'кюрин', 'кюрит', 'кюсле', 'кюфта', 'кяриз', 'лабаз', 'лабео', 'лабет', 'лаваж', 'лаваш', 'лавис', 'лавка', 'лавра', 'лагер', 'лагид', 'лагун', 'ладан', 'ладин', 'ладья', 'лазер', 'лазка', 'лайба', 'лайка', 'лакей', 'лакец', 'лакса', 'лалич', 'ламин', 'лампа', 'ламут', 'ланго', 'ланда', 'ландо', 'ланды', 'ланка', 'ланце', 'лапка', 'лапта', 'лапша', 'ларва', 'ларга', 'ларго', 'ларек', 'ларец', 'ларич', 'ларра', 'ласка', 'лассо', 'латес', 'латин', 'латит', 'латка', 'латте', 'латук', 'латыш', 'латьи', 'лауда', 'лафет', 'лафит', 'лахар', 'лахта', 'лаццо', 'лачка', 'лебда', 'левак', 'леван', 'левин', 'левит', 'левша', 'легат', 'легаш', 'легуа', 'ледий', 'ледин', 'лежак', 'лежка', 'лейас', 'лейбл', 'лейка', 'лейма', 'лекаж', 'лекиф', 'лекса', 'лемех', 'лемки', 'лемма', 'лемур', 'ленец', 'ленка', 'ленок', 'лента', 'ленто', 'леоне', 'лепет', 'лепка', 'лепра', 'лепта', 'лепча', 'лерез', 'лерка', 'леска', 'лесса', 'лесть', 'лесха', 'летка', 'леток', 'летун', 'леший', 'лжица', 'лиаза', 'лиана', 'лиард', 'либор', 'либра', 'ливан', 'ливер', 'ливка', 'лигур', 'лидар', 'лидер', 'лидин', 'лидит', 'лидор', 'лизат', 'лизил', 'лизин', 'лизис', 'лизол', 'лизун', 'ликей', 'ликер', 'ликит', 'ликод', 'лилия', 'лиман', 'лимба', 'лимбо', 'лимес', 'лимец', 'лимит', 'лимма', 'лимон', 'лимфа', 'линек', 'линза', 'линин', 'линит', 'линия', 'липец', 'липид', 'липоз', 'липси', 'лирик', 'литас', 'литер', 'литий', 'литик', 'литин', 'лития', 'литка', 'литоз', 'литол', 'литра', 'литье', 'лихач', 'лихва', 'лихен', 'лихия', 'лицей', 'лицин', 'личер', 'лишай', 'лишек', 'лоаоз', 'лобан', 'лобби', 'лобик', 'лобин', 'лобио', 'лобит', 'лобия', 'лобок', 'лобот', 'ловец', 'ловля', 'ловоа', 'логик', 'логин', 'лоден', 'лодка', 'ложка', 'ложок', 'локва', 'локер', 'локон', 'локус', 'ломка', 'лонжа', 'лопух', 'лорел', 'лорум', 'лоток', 'лотос', 'лофин', 'лохии', 'лохмы', 'лоция', 'лошак', 'лубок', 'лузга', 'лузец', 'лукно', 'лукум', 'лунда', 'лунит', 'лунка', 'лупан', 'лупка', 'луток', 'лухья', 'лучок', 'лушеи', 'лыжня', 'лысун', 'лычка', 'льяло', 'любер', 'любка', 'люгер', 'людус', 'люмен', 'люнет', 'люпин', 'люпус', 'люрик', 'люстр', 'лютик', 'лютит', 'лютич', 'лютня', 'люцит', 'лябан', 'ляжка', 'лямка', 'ляпис', 'ляссе', 'ляшка', 'маака', 'мабуб', 'мабуи', 'мавка', 'мавля', 'магар', 'магик', 'магия', 'магма', 'магог', 'магот', 'мадам', 'мадия', 'мажор', 'мазар', 'мазер', 'мазка', 'мазня', 'мазок', 'мазур', 'мазут', 'майду', 'майка', 'майко', 'майна', 'майор', 'макам', 'макао', 'макет', 'маком', 'макси', 'макуа', 'малат', 'малек', 'малео', 'малец', 'малик', 'малка', 'малли', 'малыш', 'мальм', 'мальт', 'маляр', 'мамак', 'маман', 'мамба', 'мамбо', 'мамец', 'мамка', 'манап', 'манас', 'манат', 'манга', 'манго', 'манде', 'манеб', 'манеж', 'манер', 'манец', 'мание', 'мания', 'манка', 'манки', 'манко', 'манкс', 'манна', 'манок', 'манор', 'манси', 'манта', 'манто', 'манту', 'манул', 'мануш', 'маори', 'марае', 'марал', 'маран', 'мараэ', 'марго', 'маржа', 'марка', 'марко', 'марку', 'марля', 'марши', 'масаи', 'маска', 'масло', 'масон', 'масса', 'массе', 'масть', 'матка', 'матты', 'мауля', 'маунд', 'мафит', 'мафия', 'махан', 'махди', 'махра', 'махры', 'мацис', 'мачок', 'мачта', 'мбоши', 'меала', 'мегом', 'медиа', 'медик', 'медио', 'медия', 'медли', 'медок', 'медье', 'медяк', 'мезга', 'мезил', 'мезон', 'мейоз', 'мелба', 'мелем', 'мелик', 'мелис', 'мелит', 'мелия', 'мелок', 'мелон', 'мелос', 'мелье', 'менаж', 'менде', 'менек', 'мензи', 'менид', 'мерва', 'мерет', 'мерея', 'мерин', 'мерка', 'мерло', 'мероу', 'месна', 'месса', 'место', 'месть', 'месье', 'месяц', 'метан', 'метек', 'метен', 'метиз', 'метик', 'метил', 'метин', 'метис', 'метка', 'метла', 'метод', 'метол', 'метоп', 'метро', 'меццо', 'мечта', 'мешка', 'мешок', 'мивок', 'мигас', 'мигма', 'мидия', 'мизер', 'миква', 'микоз', 'микст', 'милая', 'милон', 'милый', 'милье', 'мимик', 'минаи', 'минал', 'минер', 'минет', 'минец', 'минея', 'миний', 'миним', 'минор', 'минус', 'миома', 'мираб', 'мираж', 'мирза', 'мирин', 'мирок', 'мирон', 'мирра', 'миска', 'мисла', 'митоз', 'митра', 'михна', 'мишка', 'мнонг', 'мобер', 'могар', 'могол', 'могул', 'могур', 'модал', 'модем', 'модий', 'модус', 'модфа', 'моизм', 'моист', 'мойва', 'мойка', 'мойра', 'мокет', 'мокик', 'мокко', 'мокой', 'мокра', 'мокса', 'мокша', 'молва', 'молка', 'молот', 'молох', 'моляр', 'момме', 'момот', 'монах', 'монго', 'мондо', 'мония', 'моном', 'мопед', 'моран', 'морда', 'морея', 'морин', 'морит', 'мория', 'мороз', 'морок', 'морон', 'морфа', 'морцо', 'моряк', 'мосит', 'мосол', 'мосье', 'мотал', 'мотет', 'мотив', 'мотка', 'мотня', 'моток', 'мотор', 'мотто', 'мофет', 'мохер', 'мохит', 'мохна', 'мочка', 'мошка', 'мошна', 'мубах', 'мугам', 'мудра', 'мужик', 'музей', 'мукор', 'мулао', 'мулат', 'мулла', 'мульк', 'муляж', 'мумие', 'мумия', 'мунго', 'мунгу', 'мунда', 'мунит', 'мураш', 'мурза', 'мурин', 'мурит', 'мурка', 'мурук', 'мурья', 'мусат', 'мусор', 'мутар', 'мутон', 'мутул', 'муфта', 'мухур', 'муцин', 'мушар', 'мушир', 'мушка', 'мцыри', 'мысль', 'мысок', 'мытье', 'мышей', 'мышка', 'мышца', 'мэлан', 'мэрия', 'мэрон', 'мюзет', 'мюзле', 'мюрид', 'мюсли', 'мякиш', 'мялка', 'мямля', 'мясцо', 'мятеж', 'мятие', 'мятье', 'мячик', 'набам', 'набат', 'набег', 'набла', 'набоб', 'набор', 'наваб', 'навал', 'навар', 'нават', 'навах', 'навес', 'навет', 'навис', 'навит', 'навоз', 'навой', 'навык', 'наган', 'нагар', 'нагиб', 'нагон', 'нагул', 'надел', 'надир', 'надой', 'надув', 'наезд', 'нажиг', 'нажим', 'нажин', 'найра', 'наказ', 'накал', 'накат', 'накид', 'накол', 'накос', 'накра', 'налед', 'налеп', 'налет', 'налив', 'налим', 'налог', 'намаз', 'намек', 'намет', 'намец', 'намин', 'намка', 'намол', 'намыв', 'нанди', 'нанду', 'нанка', 'нанос', 'напев', 'напой', 'напор', 'наппа', 'нарды', 'нарез', 'народ', 'нарта', 'нарыв', 'наряд', 'насад', 'насос', 'насса', 'насып', 'натек', 'натес', 'натир', 'натяг', 'наука', 'нафта', 'нахал', 'наход', 'нахур', 'нация', 'начес', 'начет', 'начин', 'начос', 'наяда', 'нгала', 'нгони', 'невар', 'невер', 'невма', 'невод', 'невус', 'негус', 'недра', 'недуг', 'нейит', 'немая', 'немец', 'немич', 'немка', 'немой', 'ненец', 'нения', 'ненка', 'непер', 'непот', 'нерил', 'нерка', 'нерол', 'нерпа', 'несун', 'несяк', 'нетто', 'неуют', 'нефть', 'нечет', 'нзима', 'нивоз', 'нидус', 'низам', 'низин', 'низка', 'низок', 'никаб', 'никах', 'никса', 'нилас', 'нилит', 'нилот', 'нилум', 'нимат', 'нимец', 'нимит', 'нимфа', 'нирал', 'нисан', 'нисса', 'нитка', 'нитон', 'ничья', 'нишец', 'нищая', 'нищий', 'нобль', 'новик', 'новус', 'ногат', 'ногул', 'нодья', 'ножик', 'ножка', 'ножны', 'нойон', 'нокке', 'номад', 'номен', 'номер', 'нонан', 'нонен', 'нонет', 'нонил', 'нонин', 'нопол', 'норит', 'нория', 'норка', 'норма', 'норов', 'носач', 'носик', 'носка', 'носок', 'нотис', 'ноэль', 'ноэма', 'нубук', 'нудеж', 'нужда', 'нукер', 'нумик', 'нураг', 'нутка', 'нутро', 'нырок', 'нытик', 'нытье', 'ньюфи', 'ньяик', 'ньяла', 'нэцке', 'нюанс', 'нюхач', 'оазис', 'обвал', 'обвес', 'обвив', 'обвод', 'обвоз', 'обгон', 'обдел', 'обдир', 'обдув', 'обжиг', 'обжим', 'обжин', 'обзол', 'обзор', 'обида', 'обкат', 'обком', 'обкос', 'облет', 'облик', 'облов', 'облог', 'облой', 'облом', 'обман', 'обмен', 'обмер', 'обмет', 'обмин', 'обмол', 'обмыв', 'обнос', 'обора', 'образ', 'обрат', 'обрез', 'оброк', 'оброн', 'обруб', 'обруч', 'обрыв', 'обряд', 'обсев', 'обувь', 'обуза', 'обход', 'обчет', 'объем', 'обыск', 'овизм', 'овина', 'овист', 'овоид', 'овраг', 'овсец', 'овсюг', 'овула', 'овчар', 'огарь', 'огива', 'огони', 'огонь', 'огрех', 'одаль', 'оддит', 'одеон', 'одурь', 'оелит', 'ожива', 'ожика', 'озена', 'озеро', 'озимь', 'озноб', 'оидий', 'ойлер', 'ойран', 'ойрат', 'ойрот', 'окаем', 'окапи', 'окара', 'окарь', 'океан', 'окись', 'оклад', 'оклик', 'оковы', 'окоем', 'окорм', 'окрас', 'окрик', 'окрол', 'округ', 'оксан', 'оксер', 'оксет', 'оксид', 'оксил', 'оксим', 'оксин', 'оксол', 'оксон', 'октан', 'октен', 'октет', 'октил', 'октин', 'октод', 'октол', 'окунь', 'олеат', 'олеил', 'олеин', 'олень', 'олеум', 'олива', 'олимп', 'олифа', 'олово', 'олтен', 'олуша', 'ольха', 'омаха', 'омега', 'омела', 'омето', 'омлет', 'оммаж', 'оммин', 'омуль', 'онагр', 'оникс', 'онинг', 'онкоз', 'онмун', 'онуча', 'ончиа', 'оолит', 'оолог', 'оофаг', 'ооцит', 'опала', 'опара', 'опека', 'опера', 'опиат', 'опись', 'опиум', 'оплот', 'оплыв', 'опоек', 'опока', 'опора', 'опрос', 'опсин', 'оптик', 'опхоз', 'опция', 'орава', 'орало', 'ораон', 'орарь', 'орбис', 'орган', 'оргия', 'оргон', 'орден', 'ордер', 'ореид', 'ореол', 'ориби', 'орикс', 'оркан', 'орлан', 'орлец', 'орлик', 'орлит', 'орлоп', 'орляк', 'орнат', 'орсин', 'ортез', 'ортит', 'ортоз', 'орфик', 'орхит', 'орцин', 'оршад', 'осада', 'оседж', 'осень', 'осетр', 'осина', 'оскал', 'ослик', 'ослоп', 'осляк', 'осман', 'осмат', 'осмий', 'осмия', 'осмол', 'осмос', 'особа', 'особь', 'осоед', 'осока', 'остан', 'осташ', 'остит', 'остов', 'остоз', 'остол', 'остяк', 'осыпь', 'отава', 'отара', 'отбел', 'отбив', 'отбой', 'отбор', 'отвал', 'отвар', 'отвес', 'ответ', 'отвод', 'отвоз', 'отгиб', 'отгон', 'отгул', 'отдел', 'отдух', 'отдых', 'отель', 'отжиг', 'отжим', 'отзол', 'отзыв', 'отказ', 'откат', 'откол', 'откос', 'откуп', 'откус', 'отлет', 'отлив', 'отлов', 'отлуп', 'отмах', 'отмер', 'относ', 'отоми', 'отпад', 'отпал', 'отпор', 'отрез', 'отрог', 'отрок', 'отруб', 'отрыв', 'отряд', 'отсев', 'отсек', 'отсос', 'оттек', 'отток', 'отход', 'отцеп', 'отчал', 'отчет', 'отчим', 'отшиб', 'отщеп', 'отъем', 'оулец', 'офеня', 'офиаз', 'офихт', 'офорт', 'офрис', 'офсет', 'офуро', 'офшор', 'охват', 'охота', 'очава', 'очерк', 'очкур', 'очник', 'ошеек', 'ощупь', 'павин', 'пагон', 'пагур', 'падеж', 'падре', 'падуб', 'падья', 'пайба', 'пайза', 'пайка', 'пайол', 'пайса', 'пайют', 'пакер', 'пакет', 'пакля', 'палас', 'палач', 'палаш', 'палех', 'палец', 'палий', 'палия', 'палка', 'палла', 'палму', 'пампа', 'панаш', 'панда', 'панер', 'панир', 'панке', 'панна', 'панно', 'панты', 'панус', 'паныч', 'панье', 'панья', 'паоло', 'папец', 'папка', 'парад', 'параф', 'парез', 'парео', 'парик', 'пария', 'парка', 'парма', 'парня', 'парод', 'паром', 'парта', 'парти', 'парус', 'парфе', 'парча', 'парша', 'пасма', 'пасмо', 'пассе', 'паста', 'пасть', 'пасха', 'пасюк', 'патан', 'патас', 'паташ', 'патер', 'патет', 'патио', 'патра', 'патуа', 'пауза', 'пафос', 'пахви', 'пахит', 'пахса', 'пахта', 'пацан', 'пачка', 'пачук', 'пашня', 'певец', 'певун', 'пеган', 'пегас', 'пегаш', 'пейот', 'пекан', 'пекло', 'пелеа', 'пелит', 'пелог', 'пелот', 'пелта', 'пелям', 'пемза', 'пемон', 'пенал', 'пенам', 'пенге', 'пенго', 'пенек', 'пение', 'пенис', 'пенка', 'пенни', 'пеняз', 'пепел', 'пепси', 'перга', 'перед', 'перец', 'перит', 'перка', 'перко', 'пермь', 'перно', 'перси', 'перст', 'песец', 'песнь', 'песня', 'песок', 'песто', 'петаз', 'петит', 'петля', 'петух', 'печец', 'печка', 'пешва', 'пешец', 'пеший', 'пешка', 'пешня', 'пиала', 'пиано', 'пигон', 'пижма', 'пижон', 'пикан', 'пикап', 'пикер', 'пикет', 'пикон', 'пикси', 'пикша', 'пилав', 'пилея', 'пилит', 'пилка', 'пилон', 'пилот', 'пилум', 'пиляр', 'пинан', 'пинас', 'пинго', 'пинен', 'пинит', 'пиния', 'пинка', 'пинна', 'пинок', 'пинол', 'пинта', 'пипка', 'пиран', 'пират', 'пирен', 'пирин', 'пирит', 'пирке', 'пирог', 'пирон', 'пироп', 'писец', 'писун', 'питар', 'питон', 'питта', 'питха', 'питье', 'пифия', 'пифос', 'пихта', 'пицен', 'пицит', 'пицца', 'пищик', 'плавт', 'плавь', 'плаке', 'пламя', 'планш', 'пласт', 'плата', 'плато', 'плаун', 'плаха', 'плебс', 'плева', 'плеер', 'плеея', 'племя', 'плена', 'плеск', 'плеть', 'плехт', 'плечо', 'плешь', 'плинт', 'плита', 'плица', 'плока', 'плоть', 'плохо', 'побег', 'побои', 'побор', 'повал', 'повар', 'повет', 'повод', 'повоз', 'повой', 'погиб', 'погон', 'подий', 'подир', 'подол', 'поезд', 'пожар', 'пожня', 'пожог', 'позем', 'позер', 'позор', 'позыв', 'поиск', 'пойка', 'пойло', 'пойма', 'пойнт', 'показ', 'покат', 'покер', 'покой', 'покос', 'покус', 'полаз', 'полба', 'полет', 'полив', 'полиз', 'полик', 'полип', 'полир', 'полис', 'полия', 'полка', 'полог', 'полоз', 'полок', 'полом', 'полон', 'полье', 'полюс', 'поляк', 'помак', 'помет', 'помин', 'помои', 'помол', 'помор', 'помпа', 'понго', 'понит', 'понор', 'понос', 'понсо', 'понту', 'понур', 'пончо', 'понюх', 'попка', 'попса', 'порез', 'порей', 'порин', 'порка', 'порно', 'порог', 'порок', 'порох', 'порса', 'порск', 'порто', 'поруб', 'порча', 'порыв', 'посад', 'посев', 'посол', 'посох', 'посул', 'посыл', 'потаж', 'потай', 'поташ', 'потек', 'потир', 'поток', 'потоп', 'поттл', 'потто', 'потяг', 'поход', 'почва', 'почет', 'почин', 'почка', 'почта', 'пошиб', 'пошив', 'поэма', 'право', 'прайд', 'прайс', 'прана', 'пранг', 'пранк', 'праща', 'прево', 'прель', 'преон', 'пресс', 'прета', 'прием', 'прима', 'принт', 'принц', 'прион', 'приор', 'причт', 'приют', 'проба', 'проем', 'проза', 'пропс', 'просо', 'профи', 'прусс', 'пруто', 'прыск', 'прыть', 'прядь', 'пряжа', 'пряха', 'псарь', 'псина', 'псица', 'псише', 'псоит', 'птаха', 'птица', 'пуант', 'пугач', 'пуджа', 'пудра', 'пузан', 'пулка', 'пульс', 'пульт', 'пунан', 'пункт', 'пунто', 'пупок', 'пурга', 'пурин', 'пурка', 'путец', 'путик', 'путля', 'путти', 'путук', 'пучок', 'пушка', 'пушок', 'пушта', 'пушту', 'пчела', 'пшено', 'пыжик', 'пырей', 'пытка', 'пышка', 'пьеза', 'пьеса', 'пялка', 'пясть', 'пятак', 'пятка', 'пятно', 'пяток', 'рабад', 'рабат', 'рабби', 'равви', 'рагия', 'радар', 'раджа', 'раджм', 'радий', 'радио', 'радон', 'разор', 'разум', 'раина', 'райка', 'район', 'райтс', 'райят', 'ракат', 'ракия', 'ракля', 'ракун', 'ракша', 'ралли', 'рамен', 'рамка', 'рампа', 'ранет', 'ранец', 'ранчо', 'рапид', 'рапис', 'расса', 'растр', 'ратай', 'ратин', 'ратит', 'ратха', 'раунд', 'рафид', 'рафия', 'рахис', 'рахит', 'рацея', 'рация', 'рачок', 'рашад', 'рвань', 'рвота', 'рдест', 'ребаб', 'ребек', 'ребис', 'ребро', 'ребус', 'ревун', 'регби', 'регги', 'редан', 'редис', 'редиф', 'редия', 'редут', 'режим', 'резак', 'резец', 'резит', 'резка', 'резня', 'резол', 'резон', 'резус', 'реизм', 'рейка', 'рекан', 'рекон', 'ректо', 'релиз', 'релин', 'релит', 'рельс', 'ремез', 'ремен', 'ремиз', 'ренат', 'рендж', 'ренет', 'рений', 'ренин', 'ренит', 'рента', 'репей', 'репер', 'репка', 'репье', 'ретен', 'ретро', 'ретур', 'рефюж', 'речка', 'решка', 'ржище', 'рибат', 'рибит', 'ридер', 'риель', 'рижец', 'ризод', 'рикша', 'рилли', 'ринги', 'ринит', 'рипка', 'рипус', 'риска', 'рисса', 'ритон', 'ритор', 'рифей', 'рифли', 'рифма', 'рицин', 'ришта', 'робот', 'ровня', 'рогач', 'рогоз', 'родан', 'родат', 'родео', 'родий', 'родит', 'родич', 'родня', 'рожки', 'рожок', 'рожон', 'розан', 'розга', 'розит', 'рознь', 'ройба', 'ройка', 'рокер', 'рокки', 'рокот', 'рокош', 'ролик', 'роман', 'рондо', 'ронжа', 'ронин', 'ронит', 'ропак', 'ропот', 'ростр', 'ротан', 'ротин', 'ротон', 'ротор', 'роуит', 'рохля', 'рояль', 'ртуть', 'руаит', 'рубаб', 'рубаи', 'рубеж', 'рубец', 'рубин', 'рубка', 'рубль', 'рубус', 'рудит', 'рудяк', 'ружье', 'рузец', 'руина', 'рукав', 'рулет', 'рулик', 'рулон', 'румба', 'румпа', 'румын', 'рунди', 'рунец', 'рунка', 'рупия', 'рупор', 'русак', 'русин', 'русич', 'русло', 'руспо', 'рутил', 'рутин', 'руфер', 'руфия', 'руфол', 'ручей', 'ручка', 'рыбак', 'рыбец', 'рыбка', 'рывок', 'рыжей', 'рыжий', 'рыжик', 'рылец', 'рында', 'рынок', 'рысак', 'рысца', 'рытье', 'рычаг', 'рэйит', 'рэкет', 'рэлей', 'рэнга', 'рэпер', 'рюмка', 'рюшка', 'рябец', 'рябок', 'рядно', 'рядок', 'ряжец', 'ряска', 'рясна', 'саами', 'сабан', 'сабей', 'сабза', 'сабля', 'сабор', 'сабра', 'сабур', 'саван', 'савар', 'сават', 'савка', 'саган', 'сагиб', 'саджа', 'садик', 'садка', 'садок', 'сазан', 'саиди', 'сайба', 'сайга', 'сайда', 'сайка', 'сайра', 'саква', 'сакля', 'сакма', 'сакта', 'салад', 'салат', 'сален', 'салеп', 'салит', 'салка', 'салол', 'салон', 'салоп', 'салун', 'салют', 'саман', 'самба', 'самбо', 'самец', 'самка', 'самса', 'самум', 'санго', 'санид', 'санки', 'сапаи', 'сапер', 'сапка', 'сапог', 'сапун', 'сарай', 'сарга', 'саржа', 'сарма', 'сарод', 'сарос', 'сарыч', 'сасак', 'сатин', 'сатир', 'сауна', 'саунд', 'сафра', 'сахар', 'сахем', 'сахиб', 'сачок', 'сбора', 'сброд', 'сброс', 'сбруя', 'свази', 'свара', 'сваха', 'сведа', 'свеит', 'свеча', 'свиль', 'свинг', 'свист', 'свита', 'свити', 'свитч', 'свора', 'свояк', 'связь', 'сглаз', 'сдача', 'сдвиг', 'сдоба', 'сеанс', 'север', 'севец', 'севин', 'севит', 'севок', 'сегун', 'седан', 'седзи', 'седло', 'седок', 'седум', 'сеево', 'сезам', 'сезон', 'сейба', 'сейсм', 'сейша', 'секач', 'секко', 'секта', 'селар', 'селен', 'селин', 'селфи', 'селям', 'семга', 'семис', 'семит', 'семпл', 'семья', 'сенаж', 'сенат', 'сенжи', 'сенна', 'сенон', 'сенэн', 'сепаж', 'сепия', 'септа', 'серак', 'серау', 'серер', 'серил', 'серин', 'серир', 'сериф', 'серия', 'серка', 'серко', 'серна', 'серсо', 'серум', 'серый', 'сетка', 'сетор', 'сетье', 'сеунч', 'сечат', 'сечка', 'сиаль', 'сибас', 'сиван', 'сивка', 'сивко', 'сивуч', 'сигла', 'сигма', 'сидер', 'сиджо', 'сидор', 'сиена', 'сизиф', 'сизяк', 'сикль', 'сикоз', 'сикон', 'силал', 'силан', 'силат', 'силач', 'силен', 'силил', 'силин', 'силка', 'силок', 'силол', 'силон', 'силос', 'силур', 'сильф', 'симец', 'синай', 'синап', 'сингл', 'синдх', 'синец', 'синид', 'синин', 'синод', 'синти', 'синто', 'синус', 'синяк', 'сипай', 'сипах', 'сирен', 'сирин', 'сироп', 'сисла', 'систр', 'ситар', 'ситец', 'ситио', 'ситро', 'сифон', 'скала', 'скало', 'скань', 'скарб', 'скарн', 'скаут', 'сквер', 'сквиз', 'сквоб', 'сквот', 'сквош', 'скейт', 'скена', 'скетч', 'скиба', 'скирд', 'скирр', 'склад', 'склеп', 'склон', 'скоба', 'скоец', 'скопа', 'скорм', 'скотч', 'скраб', 'скрап', 'скреп', 'скрип', 'скрэб', 'скудо', 'скука', 'скула', 'скунс', 'слава', 'слайд', 'слайс', 'слань', 'слега', 'слеза', 'сленг', 'слива', 'слизь', 'слинг', 'слово', 'слуга', 'слюда', 'слюна', 'смазь', 'смайл', 'смена', 'смерд', 'смерч', 'смесь', 'смета', 'смола', 'смоль', 'смотр', 'смрад', 'смузи', 'смута', 'смысл', 'снафф', 'снедь', 'сноха', 'сныть', 'снэки', 'собес', 'собор', 'совет', 'совик', 'совка', 'совок', 'соджу', 'содом', 'созыв', 'сойка', 'сойма', 'сойон', 'сойот', 'сокер', 'сокет', 'сокол', 'солар', 'солея', 'солид', 'солка', 'солод', 'соляр', 'сомар', 'сомик', 'сомит', 'сомма', 'сомон', 'сонар', 'сонет', 'сопка', 'сопло', 'сопор', 'соппа', 'сорго', 'сорец', 'сорит', 'сорок', 'сорус', 'сосед', 'сосец', 'соска', 'сосна', 'сосок', 'сосуд', 'сосун', 'сотка', 'сотня', 'софит', 'сошка', 'сояги', 'спаги', 'спазм', 'спайк', 'спата', 'спекл', 'спель', 'спесо', 'спесь', 'спина', 'спирт', 'спица', 'сплав', 'сплин', 'сплит', 'спонж', 'спора', 'спорт', 'спред', 'спрей', 'спрос', 'спрус', 'спрут', 'спурт', 'спуск', 'спюрк', 'среда', 'сринг', 'срост', 'ссора', 'ссуда', 'стадо', 'стаза', 'сталь', 'станс', 'стант', 'старт', 'старь', 'стать', 'ствол', 'створ', 'стезя', 'стейк', 'стека', 'стела', 'стело', 'стель', 'стена', 'стенд', 'стент', 'степс', 'степь', 'стерх', 'стило', 'стиль', 'стоик', 'стокс', 'стола', 'столб', 'столп', 'стома', 'стопа', 'стоун', 'стояк', 'страж', 'страз', 'страх', 'стриж', 'стрик', 'стрип', 'стрит', 'стрия', 'строб', 'строй', 'строп', 'струг', 'струп', 'струя', 'стужа', 'ступа', 'суаре', 'судак', 'судан', 'судза', 'судия', 'судно', 'судок', 'судья', 'суета', 'сукно', 'сукре', 'сулея', 'сулла', 'сулой', 'сумах', 'сумец', 'сумка', 'сумма', 'суоми', 'супат', 'супер', 'супиз', 'супин', 'сурец', 'суржа', 'сурик', 'сурит', 'сурма', 'сурна', 'сурок', 'сурра', 'сусак', 'сусек', 'сусло', 'сусук', 'сутаж', 'сутки', 'сутра', 'суфет', 'суфий', 'суфле', 'сухая', 'сучок', 'сучье', 'сушка', 'сущее', 'сфекс', 'сфера', 'схват', 'схема', 'схима', 'схрон', 'сцена', 'сцинк', 'счеты', 'съезд', 'сыпец', 'сырба', 'сырец', 'сырок', 'сырть', 'сырье', 'сычик', 'сычуг', 'сыщик', 'сэбин', 'сэнто', 'сэсэн', 'сюжет', 'сюита', 'сюрко', 'сяжок', 'сянци', 'табак', 'табес', 'табия', 'табла', 'табло', 'табор', 'табун', 'тавит', 'тавот', 'тавро', 'тагал', 'таган', 'тазик', 'таино', 'таита', 'тайга', 'тайка', 'тайна', 'тайра', 'тайфи', 'тайша', 'такин', 'такка', 'такри', 'такса', 'такси', 'такыр', 'такья', 'талак', 'талан', 'талер', 'талес', 'талиб', 'талик', 'талит', 'талия', 'талон', 'талус', 'талыш', 'тальк', 'талья', 'тамас', 'тамга', 'тамил', 'тамуз', 'тамур', 'танга', 'танго', 'танец', 'танид', 'танин', 'танка', 'танто', 'танхо', 'таолу', 'тапас', 'тапер', 'тапес', 'тапир', 'тапка', 'таран', 'тараф', 'тарга', 'тареа', 'тарец', 'тариф', 'таска', 'татка', 'тафос', 'тафта', 'тафья', 'тахит', 'тахта', 'тацет', 'тачка', 'ташка', 'таэль', 'тварь', 'твист', 'твиши', 'театр', 'тебби', 'тевет', 'тевяк', 'тезин', 'тезис', 'тезка', 'теизм', 'теист', 'текке', 'текст', 'телец', 'телий', 'телик', 'телит', 'телка', 'телок', 'телом', 'телос', 'тембр', 'темне', 'тенар', 'тенга', 'тенге', 'тенда', 'тенек', 'тенил', 'тенит', 'тения', 'тенор', 'тепло', 'тераи', 'терем', 'терец', 'терка', 'терма', 'термы', 'терно', 'тесак', 'теска', 'тесла', 'тесло', 'тесто', 'тесть', 'тетин', 'тетка', 'тетра', 'теург', 'тефра', 'техно', 'теций', 'течка', 'тешка', 'тиара', 'тибет', 'тигон', 'тигре', 'тидея', 'тизер', 'тикар', 'тикер', 'тилла', 'тилль', 'тимар', 'тимин', 'тимит', 'тимол', 'тимус', 'тиоат', 'типаж', 'типец', 'типун', 'тираж', 'тирам', 'тиран', 'тирит', 'тиски', 'титан', 'титин', 'титло', 'титон', 'титул', 'тиуль', 'тифия', 'тифон', 'тихит', 'ткань', 'тлако', 'тобет', 'товар', 'тодес', 'тодея', 'тозил', 'тоизм', 'тойон', 'токай', 'токен', 'токин', 'толай', 'толан', 'толар', 'толил', 'толоа', 'толос', 'толпа', 'толща', 'томан', 'томар', 'томат', 'томец', 'томин', 'томич', 'тонар', 'тонга', 'тондо', 'тонел', 'тонер', 'тонец', 'тоний', 'тоник', 'тонит', 'тонна', 'тонус', 'тонфа', 'топаз', 'топик', 'топка', 'топор', 'топос', 'топот', 'торба', 'торет', 'торец', 'торий', 'торит', 'торон', 'торос', 'торус', 'тоска', 'тосол', 'тотем', 'тофус', 'тохар', 'точка', 'точок', 'трава', 'травы', 'трайк', 'тракт', 'трама', 'трамп', 'транк', 'транс', 'транш', 'трапп', 'трасс', 'траст', 'трата', 'траур', 'треба', 'трейд', 'трель', 'трема', 'тренд', 'трент', 'тренч', 'трень', 'треск', 'трест', 'треть', 'треух', 'трефа', 'треха', 'триаж', 'триал', 'триас', 'триба', 'трига', 'триди', 'триен', 'триер', 'тризм', 'тризн', 'триил', 'триин', 'трико', 'триод', 'триол', 'трион', 'трипс', 'трокс', 'тролл', 'тромб', 'тромп', 'трона', 'тронк', 'тропа', 'трояк', 'труба', 'трусы', 'труха', 'трюмо', 'тсуга', 'туаль', 'тубаж', 'тубус', 'тувит', 'тугай', 'тугра', 'тугун', 'тузик', 'туизл', 'туйан', 'туйен', 'туйон', 'тукан', 'тулес', 'тулий', 'тулит', 'тулуп', 'тулья', 'туляк', 'тумак', 'туман', 'тумба', 'тумен', 'тумор', 'тумыр', 'тунга', 'тунец', 'туонг', 'тупей', 'тупец', 'тупик', 'турач', 'турец', 'турка', 'турма', 'турне', 'турок', 'турус', 'турча', 'тутак', 'тутор', 'тутси', 'тутти', 'туфля', 'туцзя', 'тушин', 'тушка', 'тхару', 'тхебе', 'тщета', 'тыква', 'тынок', 'тырса', 'тычок', 'тэнно', 'тюбаж', 'тюбик', 'тювик', 'тюмыр', 'тюнер', 'тюник', 'тюпец', 'тюрбе', 'тюрбо', 'тюрик', 'тюркю', 'тюрок', 'тюфяк', 'тюшен', 'тябло', 'тягач', 'тягло', 'тягун', 'тяжба', 'тяпка', 'убрус', 'убыль', 'увеит', 'увраж', 'увула', 'угода', 'уголь', 'угорь', 'удаль', 'удама', 'удана', 'удача', 'удерж', 'удила', 'удэге', 'узанс', 'узбек', 'узбой', 'узвар', 'уздцы', 'узерк', 'узина', 'узник', 'узура', 'уигит', 'уилец', 'уинал', 'уинец', 'уипет', 'уйгур', 'уклад', 'уклея', 'уклон', 'укроп', 'уксус', 'улика', 'улица', 'улыба', 'ульва', 'улька', 'умами', 'умбит', 'умбон', 'умбра', 'умбры', 'умиак', 'умник', 'умора', 'унаби', 'унжак', 'униат', 'унион', 'унтер', 'унция', 'упрек', 'упырь', 'ураец', 'ураза', 'урали', 'ураса', 'ургит', 'урдит', 'уреид', 'урема', 'урина', 'урсан', 'уртит', 'уруту', 'уруть', 'ускок', 'уснея', 'успех', 'устав', 'усташ', 'устой', 'уступ', 'устье', 'утеря', 'утеха', 'утиль', 'утица', 'ухарь', 'ухват', 'уцмий', 'учеба', 'учинг', 'учком', 'учлет', 'учхоз', 'ушкуй', 'ушник', 'ущерб', 'уярец', 'фабра', 'фавид', 'фавор', 'фавус', 'фагот', 'фаден', 'фазан', 'фазер', 'фазис', 'фазия', 'фазон', 'файда', 'файер', 'факел', 'факир', 'факих', 'фалда', 'фален', 'фальц', 'фанат', 'фанер', 'фанза', 'фанта', 'фанти', 'фанус', 'фарад', 'фарси', 'фасад', 'фасет', 'фаска', 'фасон', 'фасты', 'фатеи', 'фатин', 'фатом', 'фатум', 'фауна', 'фацет', 'фация', 'фаянс', 'фегея', 'фекал', 'фелло', 'фенак', 'фенат', 'фенек', 'фений', 'фенил', 'фенин', 'фенит', 'фенол', 'феном', 'фенон', 'ферзь', 'ферма', 'ферми', 'фесец', 'феска', 'фетва', 'фетиш', 'фетоз', 'фетюк', 'фиада', 'фиакр', 'фибра', 'фидер', 'фижмы', 'физик', 'фикус', 'филей', 'филер', 'филид', 'филин', 'филия', 'филум', 'фильм', 'фильц', 'фимоз', 'финал', 'финик', 'финиш', 'финка', 'финки', 'финна', 'финта', 'фиорд', 'фирер', 'фирма', 'фитан', 'фитил', 'фитин', 'фитол', 'фитон', 'фицин', 'фишка', 'фланг', 'фланк', 'флеин', 'флейм', 'флейт', 'флейц', 'флешь', 'флиак', 'флинг', 'флинт', 'флирт', 'флокк', 'флокс', 'флопс', 'флора', 'флосс', 'флоут', 'флуер', 'флюат', 'флюид', 'флюта', 'флюте', 'фляга', 'фляер', 'фляки', 'фобия', 'фокон', 'фокус', 'фолат', 'фолио', 'фолия', 'фомка', 'фомоз', 'фомор', 'фондю', 'фонон', 'фория', 'форма', 'форон', 'форос', 'форте', 'форум', 'фоска', 'фосса', 'фотка', 'фотон', 'фофан', 'фраза', 'франк', 'франт', 'фрахт', 'фреза', 'фрейм', 'френч', 'фреон', 'фрина', 'фриул', 'фронт', 'фрукт', 'фугас', 'фужер', 'фузея', 'фузит', 'фузия', 'фукия', 'фукус', 'фуляр', 'фураж', 'фуран', 'фурил', 'фурин', 'фурия', 'фурка', 'фурма', 'фурор', 'фуста', 'футер', 'футон', 'футор', 'фуэте', 'фьорд', 'фьюжн', 'фюзен', 'фюзит', 'фюрер', 'хавал', 'хадак', 'хаджв', 'хаджи', 'хадис', 'хазан', 'хазар', 'хайку', 'хайло', 'хакас', 'хакер', 'хакея', 'хаким', 'хакит', 'халал', 'халат', 'халва', 'халео', 'халиф', 'халон', 'халха', 'хамит', 'хамка', 'хамон', 'хамса', 'хамсе', 'хамье', 'ханжа', 'ханка', 'ханос', 'ханум', 'ханша', 'харам', 'харза', 'харра', 'харчо', 'хасид', 'хаски', 'хасса', 'хаста', 'хастл', 'хатиб', 'хатка', 'хатты', 'хатун', 'хатыб', 'хауса', 'хафиз', 'хашар', 'хвала', 'хвост', 'хедер', 'хедже', 'хедив', 'хейит', 'хейти', 'хекат', 'хелат', 'хемоз', 'хеник', 'хенна', 'херем', 'херес', 'херик', 'хиазм', 'хивец', 'хилер', 'хилус', 'химик', 'химия', 'химус', 'хинди', 'хинду', 'хинен', 'хинид', 'хинин', 'хинит', 'хинол', 'хинон', 'хиппи', 'хирот', 'хитин', 'хитон', 'хлест', 'хлупь', 'хлыст', 'хлюст', 'хлябь', 'хмель', 'хмурь', 'хоана', 'хобби', 'хобец', 'хобот', 'ховея', 'ходжа', 'ходок', 'ходун', 'хоким', 'хокку', 'холан', 'холат', 'холин', 'холка', 'холод', 'холоп', 'холст', 'холуй', 'хольд', 'хольк', 'хомус', 'хомут', 'хомяк', 'хондо', 'хопеш', 'хорал', 'хорда', 'хорег', 'хорей', 'хорек', 'хорея', 'хорон', 'хорос', 'хоста', 'хохма', 'хохол', 'хохот', 'хошун', 'хруст', 'худия', 'худое', 'хулок', 'хулос', 'хумса', 'хумус', 'хунну', 'хунта', 'хуплу', 'хурал', 'хурма', 'хутба', 'хутия', 'хутор', 'хутун', 'хычин', 'хьенг', 'хянга', 'цадик', 'цанга', 'цанит', 'цапка', 'цапля', 'цапфа', 'царга', 'царек', 'цахур', 'цверг', 'цевин', 'цевка', 'цевье', 'цедра', 'цезий', 'целая', 'целер', 'целик', 'целла', 'целое', 'целом', 'ценоз', 'центр', 'ценур', 'цепея', 'церан', 'церат', 'церва', 'церий', 'церин', 'церит', 'цетан', 'цетен', 'цетил', 'цехин', 'цзинь', 'цибик', 'цигун', 'циема', 'цизит', 'цикас', 'цикля', 'цимат', 'цимен', 'цимес', 'цимол', 'цинаш', 'цинга', 'цинеб', 'циник', 'цинна', 'ципао', 'цирик', 'циста', 'цитоз', 'цитра', 'цифра', 'цокор', 'цокот', 'цуйка', 'цукат', 'цыган', 'цыпки', 'цяншу', 'чабан', 'чабер', 'чаган', 'чагра', 'чадар', 'чадец', 'чадра', 'чайка', 'чайот', 'чакан', 'чакма', 'чакра', 'чалка', 'чалма', 'чалоп', 'чанах', 'чанги', 'чанго', 'чанди', 'чанка', 'чаоит', 'чапсы', 'чардж', 'чарец', 'чарка', 'чарок', 'часть', 'чатни', 'чашка', 'чебак', 'чебер', 'чебот', 'чекан', 'челак', 'челка', 'чемер', 'ченит', 'чепан', 'чепец', 'черва', 'червь', 'черед', 'через', 'череп', 'чернь', 'черта', 'черут', 'ческа', 'честь', 'четки', 'чехит', 'чехол', 'чечен', 'чечет', 'чечил', 'чешка', 'чешуя', 'чибис', 'чибча', 'чижик', 'чилим', 'чинар', 'чинга', 'чинка', 'чинук', 'чипсы', 'чирей', 'чирок', 'чирус', 'число', 'читер', 'читка', 'читта', 'чокве', 'чокер', 'чокой', 'чомга', 'чомпи', 'чорба', 'чрево', 'чреда', 'чтиво', 'чтица', 'чубук', 'чувал', 'чувач', 'чуваш', 'чувяк', 'чугун', 'чудак', 'чудик', 'чужак', 'чужое', 'чуйка', 'чукур', 'чукча', 'чулан', 'чулок', 'чумак', 'чумка', 'чунга', 'чунец', 'чурак', 'чурек', 'чурка', 'чутье', 'чухна', 'чучхе', 'чушка', 'шабаш', 'шабер', 'шабли', 'шабот', 'шабур', 'шавка', 'шадуф', 'шаири', 'шайба', 'шайка', 'шакал', 'шакти', 'шакья', 'шалаш', 'шалон', 'шалот', 'шалун', 'шаман', 'шамес', 'шамот', 'шанга', 'шанец', 'шанза', 'шанкр', 'шапка', 'шарав', 'шарик', 'шарка', 'шарпи', 'шаруа', 'шасла', 'шассе', 'шасси', 'шатар', 'шатен', 'шатер', 'шатра', 'шаттл', 'шатун', 'шафер', 'шахид', 'шахин', 'шахта', 'шашка', 'шашки', 'шашни', 'шванк', 'швара', 'шварт', 'шверт', 'швунг', 'шеваж', 'шевер', 'шевро', 'шейка', 'шейха', 'шелеп', 'шелом', 'шелон', 'шелти', 'шельф', 'шеляг', 'шемая', 'шемиз', 'шенит', 'шепот', 'шериф', 'шерпа', 'шерри', 'шефша', 'шиацу', 'шибер', 'шиизм', 'шимми', 'шинка', 'шинок', 'шипик', 'шипок', 'шипун', 'ширма', 'ширяш', 'шитик', 'шитье', 'шифер', 'шифон', 'шихан', 'шихта', 'шишак', 'шишка', 'шишок', 'шкала', 'шкант', 'шквал', 'шкерт', 'шкода', 'школа', 'шкура', 'шланг', 'шлейф', 'шлица', 'шлюсс', 'шляпа', 'шмель', 'шнапс', 'шнека', 'шнитт', 'шнява', 'шокер', 'шомаж', 'шопер', 'шопин', 'шорец', 'шорея', 'шорит', 'шорка', 'шорня', 'шорох', 'шорты', 'шоссе', 'шофар', 'шофер', 'шошон', 'шпага', 'шпала', 'шпень', 'шпиль', 'шпион', 'шпирт', 'шпона', 'шпора', 'шприц', 'шпрот', 'шпрух', 'шпуля', 'шпунт', 'шранг', 'шратт', 'шримс', 'шрифт', 'штамб', 'штамм', 'штамп', 'штаны', 'штейн', 'штерт', 'штиль', 'штифт', 'штора', 'шторм', 'штраф', 'штрек', 'штрих', 'штроп', 'штука', 'штурм', 'штырь', 'шубат', 'шубер', 'шувыр', 'шугай', 'шудра', 'шужук', 'шуйца', 'шулер', 'шулик', 'шумер', 'шумка', 'шунья', 'шурин', 'шурпа', 'шуруп', 'шутер', 'шутка', 'шушун', 'шхеры', 'шхоут', 'шхуна', 'шютте', 'щебет', 'щегол', 'щекот', 'щелок', 'щенок', 'щепка', 'щетка', 'щечка', 'щипец', 'щипка', 'щипок', 'щипцы', 'щитик', 'щиток', 'щокур', 'щупик', 'щупка', 'щурка', 'щурок', 'щучка', 'эвеит', 'эвенк', 'эвлит', 'эгида', 'эгрет', 'эдеин', 'эджер', 'эдикт', 'эдукт', 'эйдос', 'эйфоб', 'эквол', 'экзон', 'экзот', 'эклер', 'экран', 'эксод', 'экшен', 'элеат', 'элект', 'элиит', 'элита', 'эллин', 'эмаль', 'эмбия', 'эмбол', 'эммер', 'эмпат', 'эмсец', 'эмшер', 'энзим', 'энрик', 'энтез', 'эозин', 'эозит', 'эозон', 'эолит', 'эоцен', 'эпарх', 'эпика', 'эпонж', 'эпоха', 'эпохе', 'эрбий', 'эргин', 'эрдит', 'эрзац', 'эрика', 'эркер', 'эррит', 'эртли', 'эртуг', 'эсдек', 'эскер', 'эскиз', 'эснаф', 'эспри', 'эссен', 'эстет', 'эсток', 'эсцин', 'эталь', 'этвеш', 'этика', 'этмия', 'этнос', 'этрол', 'эулит', 'эуфон', 'эфапс', 'эфель', 'эфиоп', 'эфира', 'эфрин', 'эхмея', 'эшара', 'эшарп', 'эшель', 'юанит', 'юглон', 'югрич', 'юддит', 'юдоль', 'юзист', 'юитка', 'юката', 'юкола', 'юлина', 'юмизм', 'юнайт', 'юнгит', 'юниор', 'юница', 'юнкер', 'юнкор', 'юннат', 'юнона', 'юноша', 'юриит', 'юрист', 'юрфак', 'ютаит', 'юферс', 'юшман', 'ябеда', 'ябиру', 'ягеин', 'ягель', 'ягиит', 'ягода', 'ягоит', 'ягуар', 'яичко', 'якана', 'якорь', 'ялапа', 'ялбот', 'ямаец', 'ямщик', 'ямынь', 'янтра', 'яодао', 'япикс', 'ярица', 'ярлит', 'ярлык', 'яруро', 'ярыга', 'ясень', 'яспис', 'яссец', 'яство', 'ястык', 'ясырь', 'ятвяг', 'ятовь', 'яхонт', 'яшмак']

In [19]:
# source 3: https://rudle.vercel.app/
target_words3 = ['аббат', 'абзац', 'аборт', 'абрек', 'абрис', 'абхаз', 'абцуг', 'абшид', 'аваль', 'аванс', 'авгит', 'авгур', 'авеню', 'аверс', 'авизо', 'авось', 'аврал', 'авран', 'автол', 'автор', 'агава', 'агама', 'агами', 'агент', 'агнат', 'агнец', 'аграф', 'агути', 'адепт', 'адрес', 'адряс', 'адски', 'ажгон', 'азарт', 'азиат', 'аймак', 'айран', 'айрол', 'айсор', 'акажу', 'акант', 'аканф', 'акать', 'аксон', 'актер', 'актив', 'актин', 'акула', 'акциз', 'акция', 'алгол', 'алеть', 'алеут', 'алиби', 'алкил', 'аллах', 'аллея', 'аллод', 'аллюр', 'алмаз', 'алтей', 'алтея', 'алтын', 'алчба', 'алыча', 'альфа', 'амбар', 'амбра', 'амбре', 'амвон', 'амеба', 'аминь', 'ампер', 'ампир', 'ангар', 'ангел', 'ангоб', 'анион', 'анкер', 'анона', 'анонс', 'антик', 'антре', 'анфас', 'анчар', 'аорта', 'апекс', 'апноэ', 'апорт', 'апрош', 'арбуз', 'аргал', 'аргон', 'аргус', 'ареал', 'арена', 'арест', 'ариец', 'аркан', 'армия', 'армюр', 'армяк', 'арсин', 'артос', 'архар', 'архив', 'аршин', 'аскер', 'аскет', 'аспид', 'астат', 'астма', 'астра', 'асцит', 'атака', 'атлас', 'атлет', 'атолл', 'атрий', 'аттик', 'афера', 'афиша', 'ахать', 'ахеец', 'ацтек', 'аэроб', 'аэрон', 'бабах', 'бабий', 'бабка', 'бабье', 'багаж', 'багер', 'багет', 'багор', 'бадан', 'бадья', 'базар', 'базис', 'байка', 'бакан', 'бакен', 'балда', 'балет', 'балка', 'балок', 'балык', 'банан', 'банда', 'банка', 'банту', 'барак', 'баран', 'барда', 'бареж', 'баржа', 'барий', 'барин', 'барит', 'барич', 'барка', 'бармы', 'барон', 'барыш', 'баска', 'басма', 'басня', 'басок', 'басон', 'баста', 'бастр', 'батан', 'батат', 'батог', 'батон', 'батуд', 'батут', 'батыр', 'бахча', 'бачки', 'бачок', 'башка', 'башня', 'баять', 'бдеть', 'бебут', 'бегом', 'бегун', 'бедро', 'бейка', 'бекар', 'бекас', 'бекон', 'белек', 'белец', 'белка', 'белок', 'белые', 'белый', 'белье', 'беляк', 'беляш', 'бердо', 'берег', 'берет', 'берма', 'берце', 'берцо', 'бетон', 'бивак', 'бивни', 'бигус', 'бидон', 'бизон', 'билет', 'билль', 'бином', 'бионт', 'биржа', 'бирка', 'бирюк', 'бирюч', 'бисер', 'бистр', 'битва', 'битки', 'биток', 'битум', 'битый', 'битье', 'битюг', 'благо', 'блажь', 'бланк', 'блеск', 'блинт', 'блоха', 'блуза', 'блюдо', 'блюмс', 'бляха', 'бобби', 'бобер', 'бобок', 'богач', 'бодун', 'бодяк', 'божба', 'божий', 'божок', 'бозон', 'бойня', 'бокаж', 'бокал', 'боком', 'более', 'болид', 'болюс', 'бомба', 'бонза', 'бонмо', 'бонна', 'борат', 'бордо', 'борей', 'борец', 'борид', 'боров', 'борок', 'борть', 'босой', 'босяк', 'ботва', 'ботик', 'бочаг', 'бочар', 'бочка', 'бочок', 'браво', 'брага', 'брада', 'брань', 'брасс', 'брать', 'бремя', 'брешь', 'бридж', 'бритт', 'брить', 'брови', 'бровь', 'бронх', 'броня', 'брошь', 'брыжи', 'брыла', 'брылы', 'брысь', 'брюки', 'брюхо', 'бубал', 'бубен', 'бубна', 'бубны', 'бубон', 'бугай', 'бугор', 'будка', 'будни', 'будра', 'будто', 'бузун', 'букан', 'буква', 'букет', 'букле', 'букли', 'букля', 'букса', 'булат', 'булга', 'булка', 'булла', 'бульк', 'бурав', 'бурак', 'буран', 'бурда', 'бурка', 'бурки', 'бурре', 'бурса', 'бурун', 'бурый', 'бурят', 'бутан', 'бутил', 'бутон', 'бутса', 'бутсы', 'бутуз', 'буфер', 'буфет', 'бухта', 'быдло', 'былой', 'былье', 'бытие', 'бытье', 'бычий', 'бычок', 'бьюик', 'бювар', 'бювет', 'бюкса', 'бяшка', 'вабик', 'вагон', 'важно', 'вазон', 'вакса', 'валах', 'валек', 'валет', 'валец', 'валик', 'валка', 'валок', 'валом', 'валуй', 'валун', 'валух', 'вальс', 'ванна', 'ванта', 'ванты', 'вапор', 'варан', 'варка', 'варяг', 'ватер', 'ватин', 'ватка', 'вафля', 'вахня', 'вахта', 'ваять', 'вбить', 'вброд', 'вверх', 'ввиду', 'ввить', 'вволю', 'ввысь', 'вдали', 'вдаль', 'вдвое', 'вдеть', 'вдова', 'вдоль', 'вдруг', 'вдуть', 'ведро', 'ведун', 'вежда', 'вежды', 'везде', 'везти', 'вейка', 'векша', 'велюр', 'венгр', 'венды', 'венед', 'венет', 'венец', 'веник', 'венка', 'венок', 'вента', 'вепрь', 'верба', 'вервь', 'веред', 'верес', 'верея', 'верже', 'верки', 'верно', 'верть', 'верфь', 'верхи', 'верша', 'вески', 'весло', 'весна', 'весок', 'вести', 'весть', 'ветвь', 'ветер', 'ветка', 'ветла', 'вечер', 'вечор', 'вешка', 'вещий', 'вещун', 'веять', 'вжать', 'вживе', 'взаем', 'взвар', 'взвод', 'взвоз', 'вздор', 'вздох', 'взлет', 'взлом', 'взмах', 'взмет', 'взмыв', 'взнос', 'взрез', 'взрыв', 'взыск', 'взять', 'виват', 'вивер', 'видам', 'видик', 'видно', 'визир', 'визит', 'вилка', 'вилла', 'вилок', 'винил', 'винол', 'винцо', 'виола', 'вираж', 'вирус', 'вирши', 'виски', 'висок', 'вития', 'витой', 'виток', 'витье', 'вихор', 'вихрь', 'вишня', 'вклад', 'вкось', 'вкруг', 'вкупе', 'влага', 'власы', 'влево', 'влечь', 'влить', 'влюбе', 'вмять', 'внаем', 'внизу', 'внове', 'вновь', 'внука', 'внять', 'вобла', 'вовек', 'вовне', 'вовсе', 'вовсю', 'вогул', 'водка', 'вожак', 'вождь', 'вожжа', 'вожжи', 'возка', 'возле', 'возня', 'возок', 'война', 'войти', 'волан', 'волга', 'волей', 'волна', 'волок', 'волос', 'волох', 'волхв', 'вольт', 'волюм', 'вопль', 'ворог', 'ворон', 'ворот', 'ворох', 'вотум', 'вотще', 'вотяк', 'вошка', 'вояка', 'впечь', 'впить', 'впору', 'впрок', 'впуск', 'враки', 'враль', 'врата', 'врать', 'время', 'вроде', 'врозь', 'вруша', 'врыть', 'всего', 'вслед', 'вслух', 'всход', 'всюду', 'всяко', 'втечь', 'втора', 'втрое', 'втуне', 'вуаль', 'вчера', 'вчуже', 'вширь', 'вшить', 'въезд', 'въяве', 'въявь', 'выбег', 'выбор', 'вывал', 'вывес', 'вывих', 'вывод', 'вывоз', 'выгар', 'выгиб', 'выгон', 'выгул', 'выдел', 'выдох', 'выдра', 'выезд', 'выжиг', 'выжим', 'вызов', 'вызол', 'выйти', 'выкат', 'выкос', 'выкуп', 'выкус', 'вылаз', 'вылет', 'вылов', 'вылом', 'вымол', 'вынос', 'выпад', 'выпал', 'выпас', 'выпек', 'выпор', 'выпот', 'вырез', 'выруб', 'высев', 'вытье', 'выход', 'вычет', 'вышаг', 'вышка', 'вьюга', 'вязка', 'вялый', 'вятич', 'вятка', 'вящий', 'гавот', 'гагат', 'газик', 'газон', 'гайка', 'галка', 'галлы', 'галоп', 'галун', 'гамак', 'гамма', 'ганаш', 'ганец', 'ганка', 'гараж', 'гарда', 'гарем', 'гарус', 'гаусс', 'гачек', 'гашиш', 'гвалт', 'гевея', 'гейша', 'гелий', 'гемма', 'гений', 'генри', 'геоид', 'герма', 'герой', 'гетра', 'гетры', 'гетто', 'гибка', 'гидра', 'гиена', 'гилея', 'гиляк', 'гинея', 'гипюр', 'гирло', 'гитов', 'гичка', 'глава', 'главк', 'гладь', 'глаза', 'глень', 'глина', 'глипт', 'глист', 'глубь', 'глупо', 'глушь', 'глыба', 'глядь', 'глядя', 'глясе', 'гмина', 'гнать', 'гнейс', 'гнида', 'гниль', 'гнить', 'гнома', 'гнусь', 'гнуть', 'гобой', 'говор', 'гогот', 'годик', 'годок', 'гокко', 'голец', 'голик', 'голод', 'голос', 'голый', 'голыш', 'гольд', 'голье', 'гольф', 'голяк', 'гомон', 'гонец', 'гонка', 'гонки', 'гонок', 'гонор', 'гопак', 'гопля', 'горал', 'горец', 'горка', 'горло', 'город', 'горох', 'горст', 'горюн', 'гость', 'гофры', 'граве', 'грамм', 'гранд', 'грань', 'графа', 'греза', 'грена', 'греть', 'греча', 'грива', 'гридь', 'грипп', 'гроза', 'грозд', 'гросс', 'гроши', 'груда', 'грудь', 'грунт', 'груша', 'грыжа', 'гряда', 'грязи', 'грязь', 'гуава', 'гуано', 'гуашь', 'губан', 'губка', 'гудок', 'гужом', 'гузка', 'гузно', 'гуляш', 'гумма', 'гумми', 'гумно', 'гумус', 'гуппи', 'гуран', 'гурия', 'гусак', 'гусар', 'гусек', 'гусем', 'гусит', 'гусли', 'густи', 'гуцул', 'гюрза', 'давка', 'давно', 'давок', 'дайна', 'дайра', 'далее', 'далия', 'дамба', 'дамка', 'дамно', 'дарма', 'даром', 'дацан', 'дачка', 'двери', 'дверь', 'дебет', 'дебил', 'дебит', 'дебош', 'дебри', 'дебют', 'девиз', 'девка', 'девон', 'дедка', 'дедов', 'дежка', 'деизм', 'деист', 'декан', 'декор', 'дележ', 'делец', 'демон', 'демос', 'денди', 'денек', 'денно', 'дерба', 'дерби', 'дерен', 'дерма', 'дерть', 'десна', 'десть', 'детва', 'детка', 'детки', 'дефис', 'джинн', 'дзета', 'дзинь', 'дзюдо', 'диван', 'диета', 'дикий', 'динар', 'динас', 'динго', 'дихта', 'дичок', 'длань', 'длина', 'длить', 'днесь', 'днище', 'добор', 'добро', 'довод', 'догма', 'дождь', 'дозор', 'доить', 'дойка', 'дойна', 'дойра', 'дойти', 'докер', 'долго', 'долее', 'долой', 'домен', 'домик', 'домна', 'домой', 'домок', 'домра', 'донец', 'донка', 'донна', 'донор', 'донос', 'донце', 'донья', 'доска', 'досол', 'досуг', 'досье', 'дотла', 'дофин', 'доход', 'дочка', 'драга', 'драже', 'драйв', 'драка', 'драма', 'дрань', 'драть', 'древо', 'дрейф', 'дрель', 'дрема', 'дрена', 'дробь', 'дрова', 'дрога', 'дроги', 'дрожь', 'дрозд', 'дросс', 'дрофа', 'друза', 'друид', 'дрязг', 'дрянь', 'дубка', 'дубль', 'дубок', 'дубье', 'дуван', 'дудак', 'дудка', 'дудки', 'дужка', 'дукат', 'дулеб', 'думец', 'думка', 'дупло', 'дурак', 'дурий', 'дурно', 'дурра', 'дурро', 'дутар', 'дутик', 'дутый', 'дутыш', 'дутье', 'духан', 'духов', 'духом', 'душка', 'душно', 'душок', 'дуэль', 'дщерь', 'дыбом', 'дылда', 'дымка', 'дымок', 'дырка', 'дышло', 'дюбек', 'дюжий', 'дюкер', 'дюшес', 'дядин', 'дятел', 'евнух', 'еврей', 'егерь', 'егоза', 'едать', 'едкий', 'ежели', 'ежить', 'ежиха', 'ездка', 'ездок', 'езжай', 'ей-ей', 'екать', 'елико', 'емкий', 'емшан', 'ересь', 'ерник', 'ершик', 'ершом', 'есаул', 'ехать', 'ехида', 'жабий', 'жабка', 'жабра', 'жабры', 'жажда', 'жакан', 'жакет', 'жамка', 'жарка', 'жарко', 'жарок', 'жатва', 'жатка', 'жатый', 'жвала', 'жвало', 'жвалы', 'ждать', 'жевок', 'желна', 'желоб', 'желть', 'желчь', 'женин', 'жених', 'женка', 'жеода', 'жердь', 'жерех', 'жерло', 'жесть', 'жетон', 'живец', 'живой', 'живот', 'жиган', 'жизнь', 'жилет', 'жилец', 'жилка', 'жилой', 'жилье', 'жираф', 'жирок', 'жиряк', 'житие', 'житье', 'жменя', 'жмудь', 'жмыхи', 'жнива', 'жниво', 'жница', 'жокей', 'жрать', 'жрица', 'жулан', 'жулик', 'жулье', 'жупан', 'жупел', 'жутко', 'жучка', 'жучок', 'забег', 'забой', 'забор', 'завал', 'завет', 'завод', 'завоз', 'завуч', 'загар', 'загиб', 'загон', 'загул', 'задел', 'задик', 'задок', 'задом', 'задор', 'заезд', 'зажим', 'зажин', 'зажор', 'зазор', 'зазыв', 'заика', 'зайка', 'зайти', 'заказ', 'закал', 'закат', 'закол', 'закон', 'закуп', 'закут', 'залет', 'залив', 'зализ', 'залог', 'залом', 'замах', 'замер', 'замес', 'замет', 'замок', 'замор', 'замуж', 'замша', 'занос', 'запад', 'запал', 'запас', 'запах', 'запев', 'запой', 'запор', 'зараз', 'зарез', 'зарод', 'зарок', 'заруб', 'заряд', 'засев', 'засим', 'засов', 'засол', 'засор', 'засос', 'затек', 'затем', 'затес', 'затея', 'заток', 'затон', 'затор', 'затяг', 'заумь', 'заход', 'зацеп', 'зачем', 'зачес', 'зачет', 'зачин', 'звать', 'звено', 'зверь', 'здесь', 'зебра', 'зевок', 'зелот', 'зелье', 'зельц', 'земец', 'земля', 'земно', 'зенит', 'зенки', 'зерно', 'зернь', 'зефир', 'зимой', 'зипун', 'зиять', 'злато', 'злеть', 'злить', 'злоба', 'злюка', 'знамо', 'знамя', 'знать', 'золка', 'зоман', 'зраза', 'зразы', 'зреть', 'зубец', 'зубик', 'зубок', 'зулус', 'зумпф', 'зурна', 'зыбка', 'зыбун', 'зятев', 'зятек', 'иберы', 'иваси', 'ивина', 'ивняк', 'игрек', 'игрец', 'игрок', 'игрун', 'идеал', 'идиом', 'идиот', 'иерей', 'ижица', 'из-за', 'избач', 'извет', 'извив', 'извне', 'извод', 'извоз', 'изгиб', 'изгой', 'излет', 'излом', 'измор', 'износ', 'изъян', 'изыск', 'изюбр', 'икать', 'икона', 'икота', 'иксия', 'иктус', 'ильин', 'илька', 'имаго', 'иметь', 'иначе', 'инвар', 'ингуш', 'индау', 'индий', 'индол', 'индус', 'индюк', 'инжир', 'инкор', 'инока', 'иомен', 'ионий', 'ионит', 'иприт', 'ирбис', 'ирмос', 'ирный', 'искра', 'искус', 'ислам', 'испод', 'испуг', 'иссоп', 'истец', 'истод', 'исток', 'истый', 'исход', 'итого', 'иудей', 'ихний', 'ичиги', 'ишиас', 'йодид', 'йомен', 'кааба', 'кабак', 'кабан', 'кабил', 'кабул', 'кавун', 'кагал', 'каган', 'кагат', 'кагор', 'кадет', 'кадий', 'кадка', 'кадры', 'кадык', 'казак', 'казан', 'казах', 'казна', 'казнь', 'казус', 'кайла', 'кайло', 'кайма', 'кайра', 'какао', 'каков', 'какой', 'калам', 'калач', 'калий', 'калин', 'калиф', 'калым', 'камея', 'камин', 'камка', 'камса', 'камча', 'камыш', 'канал', 'канат', 'канва', 'канна', 'канон', 'каноэ', 'канун', 'канюк', 'капер', 'капли', 'капля', 'капок', 'капор', 'капот', 'капут', 'карат', 'карга', 'карго', 'карда', 'карел', 'карий', 'карла', 'карма', 'карст', 'карта', 'карча', 'каска', 'каско', 'касса', 'каста', 'катар', 'катер', 'катет', 'катод', 'каток', 'катыш', 'кацап', 'качка', 'кашка', 'кашне', 'кашпо', 'кашуб', 'каюта', 'квант', 'кварк', 'кварц', 'квиты', 'квота', 'кегли', 'кегль', 'кегля', 'кекур', 'кельт', 'келья', 'кенар', 'кенаф', 'кепка', 'кетон', 'кефир', 'кечуа', 'кзади', 'кианг', 'кивер', 'кивок', 'кивот', 'кидас', 'кизил', 'кизяк', 'килим', 'киник', 'киоск', 'кирза', 'кирка', 'кирха', 'кисет', 'кисея', 'киска', 'киста', 'кисть', 'кифоз', 'кичка', 'кишка', 'кишмя', 'кладь', 'клака', 'класс', 'клерк', 'клест', 'клеть', 'клещи', 'клика', 'клико', 'клинч', 'клише', 'клоун', 'клуня', 'клупп', 'клуша', 'клюка', 'клюфт', 'кляча', 'кнели', 'кнель', 'кнехт', 'книга', 'книзу', 'кница', 'князь', 'коала', 'кобел', 'кобза', 'кобра', 'ковач', 'ковер', 'ковка', 'когда', 'кодак', 'кожан', 'кожух', 'козел', 'козий', 'козлы', 'козни', 'козон', 'койка', 'койне', 'койот', 'кокет', 'кокон', 'кокор', 'кокос', 'колба', 'колер', 'колет', 'колея', 'колит', 'колка', 'колли', 'колоб', 'колок', 'колон', 'колос', 'колун', 'колье', 'кольт', 'колюр', 'комар', 'комик', 'комма', 'комми', 'комод', 'комок', 'комуз', 'конек', 'конец', 'коник', 'конка', 'конто', 'конус', 'конюх', 'копал', 'копач', 'копер', 'копия', 'копка', 'копна', 'копра', 'копты', 'копун', 'копыл', 'копье', 'коран', 'корда', 'корец', 'корка', 'корма', 'короб', 'корча', 'корчи', 'корье', 'коряк', 'косач', 'косая', 'косец', 'космы', 'косой', 'косок', 'кость', 'косяк', 'котел', 'котик', 'кофей', 'кофта', 'кохия', 'кочан', 'кочет', 'кочка', 'кошер', 'кошка', 'кошма', 'кощей', 'крага', 'краги', 'кража', 'краля', 'крапп', 'краса', 'краше', 'кредо', 'креол', 'крепь', 'кресс', 'крест', 'крица', 'кровь', 'кроки', 'кроль', 'кроме', 'крона', 'кросс', 'кроха', 'круиз', 'крупа', 'круча', 'крыло', 'крыса', 'крыть', 'крыша', 'кряду', 'кубик', 'кубок', 'кувез', 'кудри', 'кузен', 'кузня', 'кузов', 'кукан', 'кукиш', 'кукла', 'кулаж', 'кулак', 'кулан', 'кулек', 'кулер', 'кулеш', 'кулик', 'кулич', 'кулон', 'культ', 'кумач', 'кумжа', 'кумир', 'кумов', 'кумык', 'кумыс', 'кунак', 'куний', 'купаж', 'купец', 'купля', 'купно', 'купол', 'купон', 'кураж', 'курва', 'курий', 'курия', 'курка', 'курок', 'курсы', 'курья', 'кусок', 'кутас', 'кутеж', 'куток', 'кутум', 'кутья', 'кухня', 'куцый', 'кучер', 'кучка', 'кушак', 'кхмер', 'кювет', 'кюрий', 'кюрин', 'кяриз', 'лабаз', 'лаваш', 'лавка', 'лавра', 'лагун', 'ладан', 'ладно', 'ладья', 'лазер', 'лайба', 'лайда', 'лайка', 'лакей', 'лампа', 'ламут', 'ландо', 'ланды', 'ланка', 'лапка', 'лапта', 'лапша', 'ларго', 'ларек', 'ларец', 'ласка', 'лассо', 'ласты', 'латка', 'латук', 'латыш', 'лафет', 'лафит', 'лачка', 'лачок', 'лаять', 'лбина', 'лбище', 'лгать', 'левак', 'левее', 'левит', 'левша', 'левый', 'легат', 'легаш', 'легко', 'ледок', 'лежак', 'лежка', 'лежмя', 'лезть', 'лейка', 'лекаж', 'лемех', 'лемма', 'лемур', 'ленок', 'лента', 'ленто', 'ленца', 'лепет', 'лепка', 'лепра', 'лепта', 'леска', 'лесок', 'лесть', 'летка', 'леток', 'летом', 'летун', 'лешак', 'леший', 'лещик', 'лиана', 'ливер', 'ливмя', 'лидер', 'лизин', 'лизис', 'лизол', 'лизун', 'ликер', 'лилия', 'лиман', 'лимит', 'лимон', 'лимфа', 'линек', 'линза', 'линия', 'липец', 'липка', 'липси', 'лирик', 'лисий', 'литер', 'литий', 'лития', 'литка', 'литой', 'литье', 'лихач', 'лихва', 'лихой', 'лицей', 'лишай', 'лишек', 'лобан', 'лобби', 'лобик', 'лобио', 'лобия', 'лобок', 'ловец', 'ловля', 'логик', 'логос', 'лодка', 'ложка', 'ложок', 'локва', 'локон', 'ломик', 'ломка', 'лонжа', 'лопух', 'лоток', 'лотос', 'лохмы', 'лоция', 'лошак', 'лубок', 'лужок', 'лузга', 'лукно', 'лунка', 'лупка', 'лучик', 'лучок', 'лучше', 'лыжня', 'лысун', 'лысый', 'лычки', 'лычко', 'льяло', 'любая', 'любой', 'люмен', 'люнет', 'люпин', 'люпус', 'лютик', 'лютич', 'лютня', 'лютый', 'ляжка', 'лямка', 'ляпис', 'лярва', 'ляшка', 'магия', 'магма', 'магот', 'мадам', 'мадия', 'маета', 'мажор', 'мазер', 'мазик', 'мазка', 'мазло', 'мазня', 'мазок', 'мазур', 'мазут', 'майка', 'майна', 'майор', 'макао', 'макет', 'маков', 'макса', 'малек', 'малец', 'малик', 'малка', 'малое', 'малый', 'малыш', 'малье', 'маляр', 'мамин', 'мамка', 'мамон', 'манго', 'манеж', 'манер', 'мание', 'мания', 'манка', 'манко', 'манна', 'манок', 'манси', 'манто', 'манул', 'марал', 'маран', 'марго', 'маржа', 'марка', 'марля', 'марши', 'маска', 'масло', 'масон', 'масса', 'масть', 'матка', 'мафия', 'махра', 'махры', 'мачок', 'мачта', 'маять', 'мегом', 'медик', 'медок', 'медяк', 'между', 'мезга', 'мезон', 'мелис', 'мелок', 'мелос', 'менее', 'мерея', 'мерин', 'мерка', 'мерси', 'месса', 'мести', 'место', 'месть', 'месье', 'месяц', 'метан', 'метек', 'метил', 'метис', 'метка', 'метла', 'метод', 'метол', 'метоп', 'метро', 'меццо', 'мечта', 'мешок', 'мигач', 'мигли', 'мигом', 'мидия', 'мизер', 'микоз', 'микст', 'милая', 'милка', 'милок', 'милый', 'минер', 'минея', 'минор', 'минус', 'миома', 'мираж', 'мирза', 'мирок', 'мирон', 'мирра', 'мирта', 'миска', 'митоз', 'митра', 'мишка', 'млеко', 'млеть', 'мнить', 'много', 'могар', 'модус', 'можно', 'мозги', 'мойва', 'мойка', 'мойра', 'мокко', 'мокша', 'молва', 'молвь', 'молот', 'молох', 'молча', 'моляр', 'монах', 'моном', 'мопед', 'морда', 'морзе', 'мороз', 'морок', 'морфа', 'морцо', 'моряк', 'мосол', 'мосье', 'мотет', 'мотив', 'мотка', 'мотня', 'моток', 'мотор', 'мотто', 'мохер', 'мохны', 'мочка', 'мошка', 'мошна', 'мразь', 'мрежа', 'мужик', 'музей', 'мулат', 'мулек', 'мулла', 'муляж', 'мумие', 'мумия', 'мунда', 'мураш', 'мурда', 'мурза', 'мурло', 'мурья', 'мусор', 'мутон', 'муфта', 'муцин', 'мучка', 'мушка', 'мцыри', 'мчать', 'мшить', 'мымра', 'мысик', 'мысль', 'мысок', 'мытый', 'мытье', 'мышей', 'мыший', 'мышка', 'мышца', 'мэлан', 'мэрия', 'мэрон', 'мюзет', 'мюрид', 'мякиш', 'мялка', 'мямля', 'мяско', 'мясцо', 'мятеж', 'мятие', 'мятый', 'мятье', 'мячик', 'на-ка', 'набат', 'набег', 'набоб', 'набок', 'набор', 'навал', 'навар', 'навек', 'навес', 'навет', 'навий', 'навис', 'навоз', 'навой', 'навык', 'наган', 'нагар', 'нагиб', 'нагой', 'нагон', 'нагул', 'надел', 'надир', 'надой', 'надув', 'наезд', 'нажим', 'нажин', 'назад', 'назем', 'назло', 'найти', 'наказ', 'накал', 'накат', 'накос', 'налет', 'налив', 'налим', 'налог', 'намаз', 'намек', 'намет', 'намин', 'намол', 'намыв', 'нанду', 'нанка', 'нанос', 'напев', 'напой', 'напор', 'нарды', 'нарез', 'народ', 'нарта', 'нарты', 'нарыв', 'наряд', 'насад', 'насос', 'насып', 'натек', 'натяг', 'наука', 'нахал', 'нация', 'начес', 'начет', 'начин', 'наяву', 'наяда', 'невод', 'негде', 'негус', 'недра', 'недуг', 'нейти', 'некий', 'некто', 'немая', 'немец', 'немка', 'немой', 'ненец', 'нения', 'ненка', 'непер', 'нервы', 'нерка', 'нерол', 'нерпа', 'нести', 'несть', 'нетто', 'нефть', 'нечет', 'нечто', 'нешто', 'ни-ни', 'нивоз', 'нивух', 'нигде', 'низка', 'низок', 'низом', 'никак', 'никой', 'никто', 'нилот', 'нимфа', 'нисан', 'нитка', 'нитон', 'ничей', 'ничто', 'ничья', 'нищая', 'нищий', 'новик', 'новое', 'новый', 'ножик', 'ножка', 'ножны', 'нойон', 'нолик', 'номад', 'номер', 'нонет', 'нория', 'норка', 'норма', 'норов', 'носач', 'носик', 'носка', 'носки', 'носок', 'нотис', 'нотка', 'ночва', 'ночка', 'ночью', 'нощно', 'ну-ка', 'ну-ну', 'нудно', 'нужда', 'нужно', 'нукер', 'нулик', 'нумер', 'нутро', 'нынче', 'нырок', 'нытик', 'нытье', 'нюанс', 'нянин', 'оазис', 'обвал', 'обвес', 'обвод', 'обвоз', 'обгон', 'обдир', 'обдув', 'обжиг', 'обжим', 'обжин', 'обжог', 'обзол', 'обзор', 'обида', 'обить', 'обкат', 'обком', 'обкос', 'облет', 'облик', 'облов', 'облог', 'облом', 'обман', 'обмен', 'обмер', 'обмет', 'обмин', 'обмол', 'обмыв', 'обнос', 'обора', 'образ', 'обрат', 'обрез', 'оброк', 'обруб', 'обруч', 'обрыв', 'обряд', 'обсев', 'обувь', 'обуза', 'обуть', 'обход', 'общий', 'объем', 'обыск', 'овамо', 'овить', 'овощи', 'овраг', 'овсец', 'овсюг', 'овчар', 'огонь', 'огрех', 'одаль', 'одежа', 'одеон', 'одеть', 'одурь', 'ожечь', 'ожить', 'оземь', 'озеро', 'озимь', 'озноб', 'ойрот', 'окапи', 'окать', 'океан', 'окись', 'оклад', 'оклик', 'оковы', 'окоем', 'около', 'окорм', 'окрас', 'окрик', 'окрол', 'округ', 'оксид', 'октан', 'октет', 'октод', 'окунь', 'олеат', 'олеин', 'олень', 'олеум', 'олива', 'олифа', 'олово', 'ольха', 'омега', 'омела', 'омлет', 'омуль', 'омыть', 'онагр', 'оникс', 'онуча', 'онучи', 'оолит', 'опала', 'опара', 'опека', 'опера', 'опись', 'опить', 'опиум', 'оплот', 'оплыв', 'опоек', 'опока', 'опора', 'опрос', 'оптик', 'оптом', 'опять', 'орава', 'орала', 'орало', 'орарь', 'орать', 'орган', 'оргия', 'орден', 'ордер', 'ореол', 'оркан', 'орлан', 'орлец', 'орлий', 'орлик', 'орляк', 'ороки', 'орочи', 'ортит', 'оршад', 'осада', 'осень', 'осетр', 'осечь', 'осина', 'оскал', 'ослик', 'ослоп', 'осляк', 'осман', 'осмий', 'осмол', 'осмос', 'особа', 'особо', 'особь', 'осоед', 'осока', 'остит', 'остов', 'остяк', 'осыпь', 'отава', 'отара', 'отбив', 'отбой', 'отбор', 'отвал', 'отвар', 'отвес', 'ответ', 'отвод', 'отвоз', 'отгиб', 'отгон', 'отгул', 'отдел', 'отдух', 'отдых', 'отель', 'отечь', 'отжиг', 'отжим', 'отжог', 'отзол', 'отзыв', 'отказ', 'откат', 'откол', 'откос', 'откуп', 'откус', 'отлет', 'отлив', 'отлов', 'отлуп', 'отмах', 'относ', 'отпад', 'отпор', 'отрез', 'отрог', 'отрок', 'отруб', 'отрыв', 'отряд', 'отсев', 'отсек', 'отсос', 'оттек', 'отток', 'отход', 'отцеп', 'отцов', 'отчал', 'отчет', 'отчий', 'отчим', 'отшиб', 'отщеп', 'отъем', 'офеня', 'офорт', 'офсет', 'охала', 'охать', 'охват', 'охота', 'очень', 'очерк', 'очкур', 'очный', 'ошеек', 'ошуюю', 'ощупь', 'павий', 'падеж', 'падло', 'падре', 'падуб', 'падун', 'пазок', 'пайза', 'пайка', 'пакет', 'пакля', 'палас', 'палац', 'палач', 'палаш', 'палец', 'палея', 'палия', 'палка', 'палый', 'пампа', 'панаш', 'панда', 'панна', 'панно', 'панты', 'паныч', 'папин', 'папка', 'парад', 'параф', 'парез', 'парик', 'пария', 'парка', 'парод', 'парок', 'паром', 'парта', 'парус', 'парча', 'парша', 'пасмо', 'пассы', 'паста', 'пасти', 'пасть', 'пасха', 'пасюк', 'патан', 'патер', 'патла', 'патлы', 'патуа', 'пауза', 'пафос', 'пахви', 'пахит', 'пахта', 'пацан', 'пачка', 'пашня', 'паять', 'певец', 'певун', 'пегас', 'пегий', 'пейсы', 'пекан', 'пекло', 'пелит', 'пемза', 'пенал', 'пенек', 'пение', 'пенис', 'пенка', 'пенни', 'пенье', 'пепел', 'перга', 'перед', 'перец', 'перка', 'перси', 'перст', 'песец', 'песий', 'песик', 'пески', 'песнь', 'песня', 'песок', 'петел', 'петит', 'петля', 'петух', 'печка', 'пеший', 'пешка', 'пешня', 'пещур', 'пиала', 'пиано', 'пивко', 'пивцо', 'пигус', 'пижма', 'пижон', 'пикап', 'пикет', 'пикон', 'пикша', 'пилав', 'пилка', 'пилон', 'пилот', 'пиния', 'пинок', 'пинта', 'пипка', 'пират', 'пирит', 'пирке', 'пирог', 'пирон', 'пироп', 'писец', 'писун', 'питие', 'питон', 'питух', 'питье', 'пифия', 'пифос', 'пихта', 'пищик', 'плавь', 'плаке', 'пламя', 'пласт', 'плата', 'плато', 'плаун', 'плаха', 'плебс', 'плева', 'племя', 'плена', 'плеск', 'плеть', 'плечи', 'плечо', 'плешь', 'плита', 'плица', 'плоть', 'плохо', 'плыть', 'плюха', 'пнище', 'пнуть', 'по-за', 'побег', 'побои', 'побор', 'повал', 'повар', 'повет', 'повод', 'повой', 'погиб', 'погон', 'подий', 'подле', 'подог', 'подол', 'поезд', 'пожар', 'пожня', 'пожог', 'позем', 'позер', 'позже', 'позор', 'позыв', 'поиск', 'поить', 'пойка', 'пойло', 'пойма', 'пойти', 'показ', 'покат', 'покер', 'покои', 'покой', 'покос', 'полба', 'полет', 'полив', 'полип', 'полир', 'полис', 'полка', 'полно', 'полог', 'полоз', 'полок', 'полом', 'полон', 'полть', 'полый', 'полюс', 'поляк', 'помет', 'помин', 'помои', 'помол', 'помор', 'помпа', 'понос', 'понур', 'пончо', 'понюх', 'попик', 'попка', 'попов', 'порез', 'порей', 'порка', 'порог', 'пороз', 'порой', 'порок', 'порос', 'порох', 'порою', 'порск', 'порто', 'порты', 'порча', 'порыв', 'посад', 'посев', 'после', 'посол', 'посох', 'посул', 'посыл', 'поташ', 'потек', 'потир', 'поток', 'потом', 'потоп', 'потяг', 'поход', 'похул', 'почва', 'почем', 'почет', 'почин', 'почка', 'почта', 'почти', 'почто', 'пошиб', 'пошив', 'поэма', 'право', 'праща', 'предо', 'прель', 'пресс', 'преть', 'прием', 'прима', 'принц', 'приор', 'приуз', 'причт', 'приют', 'проба', 'проем', 'проза', 'пропс', 'просо', 'прочь', 'прусс', 'прыть', 'прядь', 'пряжа', 'прямо', 'пряха', 'псарь', 'псина', 'псица', 'птаха', 'птица', 'пуант', 'пугач', 'пудик', 'пудра', 'пузан', 'пузач', 'пукля', 'пульс', 'пульт', 'пункт', 'пупок', 'пурга', 'пурин', 'пурка', 'пусть', 'путем', 'путец', 'путло', 'путля', 'пуфик', 'пучок', 'пушка', 'пушок', 'пушту', 'пущий', 'пхать', 'пчела', 'пшено', 'пыжик', 'пырей', 'пытка', 'пышка', 'пьеза', 'пьеро', 'пьеса', 'пялка', 'пясть', 'пятак', 'пятая', 'пятка', 'пятно', 'пяток', 'пятый', 'пятью', 'рабат', 'рабий', 'равно', 'радар', 'раджа', 'радий', 'радио', 'радон', 'ражий', 'разве', 'разик', 'разок', 'разом', 'разор', 'разум', 'раина', 'район', 'ракия', 'ракля', 'ракун', 'ракша', 'ралли', 'рамка', 'рампа', 'ранее', 'ранет', 'ранец', 'ранка', 'ранчо', 'рапид', 'расти', 'растр', 'ратай', 'ратин', 'раунд', 'рафия', 'рахис', 'рахит', 'рацея', 'рация', 'рачий', 'рачок', 'рвань', 'рвать', 'рвота', 'рдест', 'рдеть', 'ребро', 'ребус', 'ревмя', 'ревун', 'регби', 'регия', 'регот', 'редан', 'редис', 'редут', 'режим', 'резак', 'резец', 'резит', 'резка', 'резня', 'резол', 'резон', 'резус', 'рейка', 'релит', 'рельс', 'ремез', 'ремиз', 'ренет', 'рений', 'ренин', 'рента', 'репей', 'репер', 'репка', 'речка', 'решка', 'реять', 'ржавь', 'ржать', 'ржище', 'ризка', 'рикша', 'ринит', 'рипус', 'риска', 'ритор', 'рифма', 'рицин', 'ришта', 'робот', 'ровик', 'ровно', 'ровня', 'рогач', 'рогоз', 'родан', 'родий', 'родич', 'родня', 'рожки', 'рожок', 'рожон', 'розан', 'розга', 'розно', 'рознь', 'роить', 'ройба', 'рокот', 'ролик', 'роман', 'рондо', 'ронжа', 'ропак', 'ропот', 'ростр', 'ротик', 'роток', 'ротон', 'ротор', 'рохля', 'рояль', 'ртище', 'ртуть', 'рубеж', 'рубец', 'рубин', 'рубка', 'рубль', 'ругмя', 'ругня', 'рудой', 'рудый', 'рудяк', 'ружье', 'руина', 'руины', 'рукав', 'рулет', 'рулон', 'румба', 'румын', 'рунец', 'рупия', 'рупор', 'русак', 'русин', 'русло', 'русый', 'рутил', 'ручей', 'ручка', 'рыбак', 'рыбец', 'рыбий', 'рыбка', 'рывок', 'рыжей', 'рыжий', 'рыжик', 'рында', 'рынок', 'рысак', 'рысий', 'рысца', 'рысью', 'рытый', 'рытье', 'рычаг', 'рэкет', 'рюмка', 'рюшка', 'рябой', 'рябок', 'рядно', 'рядок', 'рядом', 'ряска', 'ряшка', 'саами', 'сабан', 'сабза', 'сабля', 'сабур', 'саван', 'савка', 'саган', 'сагиб', 'саджа', 'садик', 'садка', 'садок', 'сазан', 'сайга', 'сайда', 'сайка', 'сайра', 'саква', 'сакля', 'сакма', 'салат', 'салка', 'салки', 'салол', 'салон', 'салоп', 'салют', 'саман', 'самба', 'самбо', 'самец', 'самка', 'самум', 'самый', 'санки', 'сапер', 'сапка', 'сапог', 'сапун', 'сарай', 'саран', 'саржа', 'сарыч', 'сатин', 'сатир', 'сахар', 'сахиб', 'сачок', 'сбить', 'сбоку', 'сборы', 'сброд', 'сброс', 'сбруя', 'сбыть', 'свара', 'сваха', 'свежо', 'сверх', 'свеча', 'свиль', 'свинг', 'свист', 'свита', 'свить', 'свора', 'свояк', 'свыше', 'связь', 'сглаз', 'сдать', 'сдача', 'сдвиг', 'сдоба', 'сдуру', 'сдуть', 'сеанс', 'севак', 'север', 'севец', 'севок', 'седло', 'седой', 'седок', 'сезам', 'сезон', 'сейша', 'секач', 'секта', 'селен', 'селин', 'семга', 'семик', 'семит', 'семью', 'семья', 'сенаж', 'сенат', 'сенна', 'сенцо', 'сенцы', 'сепия', 'серия', 'серка', 'серко', 'серна', 'серсо', 'серум', 'серый', 'серяк', 'сесть', 'сетка', 'сечка', 'сеять', 'сжать', 'сжечь', 'сжить', 'сзади', 'сивер', 'сивка', 'сивко', 'сивуч', 'сивый', 'сигма', 'сидка', 'сидмя', 'сиена', 'сижок', 'сизиф', 'сизый', 'сизяк', 'сикоз', 'силач', 'силен', 'силой', 'силок', 'силон', 'силос', 'силур', 'сильф', 'синап', 'синец', 'синий', 'синод', 'синус', 'синяк', 'сипай', 'сирин', 'сироп', 'сирый', 'систр', 'ситар', 'ситец', 'ситро', 'ситце', 'сифон', 'сиять', 'скала', 'скало', 'скань', 'скарб', 'скарн', 'скаут', 'сквер', 'скетч', 'скирд', 'склад', 'склеп', 'склиз', 'склон', 'скоба', 'сколь', 'скопа', 'скора', 'скорм', 'скоро', 'скотт', 'скрап', 'скреп', 'скрип', 'скудо', 'скука', 'скула', 'скунс', 'слабо', 'слава', 'слайд', 'слать', 'слева', 'слега', 'слеза', 'сленг', 'слечь', 'слива', 'слизь', 'слить', 'слово', 'слоек', 'слоны', 'слуга', 'слыть', 'слышь', 'слюда', 'слюна', 'слюни', 'смазь', 'смалу', 'смена', 'смерд', 'смерч', 'смесь', 'смета', 'сметь', 'смола', 'смоль', 'смотр', 'смочь', 'смрад', 'смута', 'смысл', 'смыть', 'смять', 'снедь', 'снизу', 'снова', 'сноси', 'сноха', 'сныть', 'снять', 'собес', 'собор', 'совет', 'совий', 'совик', 'совка', 'совок', 'согды', 'содом', 'созыв', 'сойка', 'сойма', 'сойот', 'сойти', 'сокол', 'солея', 'солка', 'солод', 'сомон', 'сонет', 'сопка', 'сопли', 'сопло', 'сопля', 'сопор', 'сопун', 'сорго', 'сорит', 'сорок', 'сором', 'сорус', 'сосед', 'сосец', 'соска', 'сосна', 'сосок', 'сосуд', 'сосун', 'сосцы', 'сотая', 'сотка', 'сотня', 'сотый', 'софит', 'сочок', 'сошка', 'спазм', 'спать', 'спесь', 'спеть', 'спечь', 'спина', 'спирт', 'спить', 'спица', 'сплав', 'сплин', 'спора', 'спорт', 'спрос', 'спрут', 'спурт', 'спуск', 'сразу', 'среда', 'среди', 'средь', 'сроду', 'срост', 'срыву', 'срыть', 'сряду', 'ссечь', 'ссора', 'ссуда', 'стадо', 'стаза', 'сталь', 'станс', 'старт', 'старь', 'стать', 'ствол', 'створ', 'стега', 'стезя', 'стека', 'стела', 'стена', 'стенд', 'степс', 'степь', 'стечь', 'стило', 'стиль', 'стихи', 'стоик', 'стокс', 'столб', 'столп', 'столь', 'стопа', 'стояк', 'страж', 'страз', 'страх', 'стриж', 'строй', 'строп', 'струг', 'струп', 'струя', 'стужа', 'ступа', 'стыть', 'суаре', 'сувой', 'судак', 'судия', 'судно', 'судок', 'судья', 'суета', 'сукин', 'сукно', 'сулея', 'сулой', 'сумах', 'сумет', 'сумка', 'сумма', 'сунна', 'суоми', 'супец', 'супин', 'суржа', 'сурик', 'сурна', 'сурок', 'сусак', 'сусек', 'сусло', 'сутаж', 'сутки', 'сутра', 'суфле', 'сухая', 'сухой', 'сучий', 'сучка', 'сучок', 'сучье', 'сушка', 'сущий', 'сущик', 'сфера', 'схват', 'схема', 'схима', 'сцена', 'сцинк', 'счеты', 'сшить', 'съезд', 'сынов', 'сынок', 'сыпец', 'сырец', 'сырой', 'сырок', 'сырть', 'сырца', 'сырье', 'сытый', 'сычуг', 'сыщик', 'сюжет', 'сюита', 'сяжки', 'сяжок', 'сякой', 'табак', 'табес', 'табло', 'табор', 'табун', 'тавот', 'тавро', 'тавры', 'таган', 'тазик', 'таить', 'тайга', 'тайна', 'также', 'такка', 'таков', 'такой', 'такса', 'такси', 'такыр', 'талан', 'талер', 'талес', 'талик', 'талия', 'талон', 'талый', 'талыш', 'тальк', 'талья', 'тамга', 'тамил', 'танго', 'танец', 'танин', 'тапер', 'тапир', 'тапка', 'тапки', 'таран', 'тариф', 'таска', 'татка', 'тафта', 'тафья', 'тахта', 'тацет', 'тачка', 'ташка', 'таять', 'тварь', 'твист', 'театр', 'тезис', 'тезка', 'теизм', 'теист', 'текст', 'телец', 'телка', 'телок', 'тембр', 'темно', 'тенек', 'тенор', 'тепло', 'терем', 'терец', 'терка', 'термы', 'тесак', 'теска', 'тесло', 'тесно', 'тесто', 'тесть', 'тетин', 'тетка', 'теург', 'течка', 'тешка', 'тещин', 'тиара', 'тимол', 'типаж', 'типец', 'типик', 'типун', 'тираж', 'тиран', 'тиски', 'титан', 'титла', 'титло', 'титул', 'тифон', 'тихий', 'ткань', 'ткать', 'тлеть', 'тлить', 'тмить', 'то-то', 'товар', 'тогда', 'тодес', 'тойон', 'токай', 'токмо', 'толай', 'толки', 'толпа', 'толща', 'толщь', 'томат', 'томик', 'тондо', 'тонна', 'тонус', 'топаз', 'топка', 'топор', 'топот', 'торба', 'торги', 'торец', 'торий', 'торит', 'торос', 'тоска', 'тотем', 'тохар', 'точка', 'точно', 'точок', 'тошно', 'тощий', 'трава', 'травы', 'тракт', 'транс', 'трапп', 'трата', 'траур', 'треба', 'трель', 'трема', 'треск', 'трест', 'треть', 'треух', 'трефа', 'трефы', 'триас', 'триба', 'триер', 'тризм', 'трико', 'триод', 'тромб', 'тромп', 'тропа', 'трояк', 'труба', 'трусы', 'труха', 'трюмо', 'тубус', 'тугой', 'тугун', 'тузик', 'тукан', 'тулес', 'тулий', 'тулуз', 'тулук', 'тулуп', 'тулья', 'туляк', 'тумак', 'туман', 'тумба', 'тумор', 'тунец', 'тупей', 'тупец', 'тупик', 'тупой', 'тупыш', 'турач', 'турий', 'турне', 'турок', 'тутор', 'тутти', 'туфли', 'туфля', 'тучка', 'тушин', 'тушка', 'тщета', 'тыква', 'тынок', 'тырло', 'тырса', 'тычок', 'тю-тю', 'тюбик', 'тюник', 'тюрки', 'тюрок', 'тютюн', 'тюфяк', 'тючок', 'тябло', 'тягач', 'тягло', 'тяжба', 'тяжко', 'тяпка', 'убить', 'убрус', 'убыль', 'увить', 'увраж', 'угода', 'уголь', 'угорь', 'удаль', 'удача', 'удерж', 'удила', 'удить', 'удэге', 'уесть', 'ужать', 'ужели', 'ужель', 'узбек', 'уздцы', 'узерк', 'узина', 'узить', 'узкий', 'узник', 'уйгур', 'укать', 'уклад', 'уклон', 'укроп', 'укрут', 'укрюк', 'уксус', 'улика', 'улита', 'улица', 'уличи', 'улыба', 'улюлю', 'умбра', 'умбры', 'уметь', 'умище', 'умник', 'умный', 'умолк', 'умора', 'умыть', 'умять', 'униат', 'унтер', 'унция', 'унять', 'упечь', 'упрек', 'упырь', 'урема', 'урина', 'урман', 'усечь', 'усики', 'усище', 'усищи', 'успех', 'устав', 'устой', 'уступ', 'устье', 'утеря', 'утеха', 'утечь', 'утиль', 'утица', 'утлый', 'утром', 'ухарь', 'ухать', 'ухват', 'ухожь', 'учеба', 'учить', 'учхоз', 'ушить', 'ушица', 'ушкан', 'ушкуй', 'ушник', 'ушной', 'ущерб', 'уютно', 'фабра', 'фавор', 'фавус', 'фагот', 'фазан', 'фазис', 'факел', 'факир', 'фалда', 'фальц', 'фанза', 'фанты', 'фасад', 'фасет', 'фаска', 'фасон', 'фатум', 'фатюй', 'фауна', 'фацет', 'фация', 'фаянс', 'фений', 'фенил', 'фенол', 'ферзь', 'ферма', 'феска', 'фетиш', 'фетюк', 'фефер', 'фиакр', 'фибра', 'фибры', 'фигли', 'фидер', 'фижмы', 'физик', 'физия', 'фикус', 'филей', 'филер', 'филин', 'фильм', 'фильц', 'фимоз', 'финал', 'финик', 'финиш', 'финка', 'финна', 'фиорд', 'фирма', 'фитин', 'фишка', 'фланг', 'фланк', 'флейц', 'флешь', 'флинт', 'флирт', 'флокс', 'флора', 'флюид', 'фляга', 'фобия', 'фокус', 'фолио', 'фомин', 'фомка', 'фонон', 'форма', 'форте', 'форум', 'фоска', 'фотон', 'фофан', 'фраер', 'фраза', 'франк', 'франт', 'фрахт', 'фреза', 'френч', 'фреон', 'фронт', 'фрукт', 'фугас', 'фужер', 'фузея', 'фукус', 'фуляр', 'фураж', 'фурия', 'фурма', 'фурор', 'футер', 'футор', 'фуэте', 'фырок', 'фьорд', 'фюйть', 'фюрер', 'ха-ха', 'хабар', 'хаджи', 'хазар', 'хайло', 'хакас', 'халат', 'халва', 'халда', 'халиф', 'хамит', 'хамка', 'хамов', 'хамса', 'хамье', 'ханжа', 'ханты', 'ханша', 'хапун', 'харчи', 'харчо', 'хасид', 'хатка', 'хауса', 'хаять', 'хвала', 'хвать', 'хворь', 'хвост', 'хе-хе', 'хедер', 'хедив', 'херес', 'херик', 'хетты', 'хи-хи', 'хиазм', 'хилус', 'хилый', 'хиляк', 'химик', 'химия', 'химус', 'хинди', 'хинин', 'хиппи', 'хитин', 'хитон', 'хлест', 'хлупь', 'хлыст', 'хлюст', 'хлябь', 'хмарь', 'хмель', 'хмурь', 'хмырь', 'хо-хо', 'хобби', 'хобот', 'ходжа', 'ходик', 'ходка', 'ходок', 'ходун', 'холка', 'холод', 'холоп', 'холст', 'холуй', 'хомут', 'хомяк', 'хорал', 'хорда', 'хорей', 'хорек', 'хорея', 'хорон', 'хохма', 'хохол', 'хохот', 'хруст', 'хряпа', 'хряск', 'хряст', 'худой', 'хунта', 'хурал', 'хурда', 'хурма', 'хутор', 'цадик', 'цанга', 'цапка', 'цапля', 'цапун', 'цапфа', 'царап', 'царев', 'царек', 'цахур', 'цацка', 'цвель', 'цветы', 'цевка', 'цевье', 'цедра', 'цезий', 'целик', 'целое', 'целый', 'центр', 'цепка', 'цепня', 'церий', 'цехин', 'цибик', 'цикля', 'цинаш', 'цинга', 'циник', 'цирик', 'циста', 'цитра', 'цифра', 'цокот', 'цугом', 'цукат', 'цуцик', 'цыган', 'цыпка', 'цыпки', 'чабан', 'чабер', 'чадра', 'чайка', 'чакан', 'чалка', 'чалма', 'чалый', 'чанах', 'чарка', 'часик', 'часок', 'часом', 'часть', 'чатал', 'чашка', 'чаять', 'чебак', 'чебот', 'чекан', 'челка', 'чемер', 'чепан', 'чепец', 'черва', 'черви', 'червы', 'червь', 'черед', 'через', 'череп', 'чернь', 'черта', 'ческа', 'честь', 'четий', 'четки', 'чехол', 'чечет', 'чешка', 'чешки', 'чешуя', 'чибис', 'чижик', 'чилим', 'чинар', 'чинка', 'чипсы', 'чирей', 'чирок', 'число', 'читка', 'чичер', 'чокер', 'чомга', 'чохом', 'чрево', 'чреда', 'чтиво', 'чтить', 'чтица', 'чтобы', 'чубук', 'чувал', 'чуваш', 'чувяк', 'чугун', 'чудак', 'чудик', 'чудно', 'чудом', 'чужак', 'чужая', 'чужое', 'чужой', 'чуйка', 'чукча', 'чулан', 'чулки', 'чулок', 'чумак', 'чурек', 'чурка', 'чуток', 'чутье', 'чухна', 'чушка', 'чуять', 'чхать', 'шабаш', 'шабер', 'шабли', 'шабот', 'шавка', 'шагом', 'шажок', 'шайба', 'шайка', 'шакал', 'шалаш', 'шалун', 'шалый', 'шаман', 'шамот', 'шанец', 'шанкр', 'шапка', 'шарап', 'шарах', 'шарик', 'шасла', 'шассе', 'шасси', 'шасть', 'шатен', 'шатер', 'шатия', 'шатун', 'шафер', 'шахта', 'шашка', 'шашни', 'шваль', 'шванк', 'швара', 'шварк', 'шварт', 'швырк', 'шевро', 'шеища', 'шейка', 'шелеп', 'шелом', 'шельф', 'шемая', 'шепот', 'шериф', 'шесть', 'шибер', 'шиизм', 'шимми', 'шинок', 'шипун', 'ширма', 'ширмы', 'шитво', 'шитик', 'шитый', 'шитье', 'шифер', 'шифон', 'шихан', 'шихта', 'шишак', 'шишка', 'шкала', 'шквал', 'шкерт', 'шкода', 'школа', 'шкура', 'шланг', 'шлейф', 'шлюха', 'шляпа', 'шмель', 'шнапс', 'шнека', 'шорец', 'шорка', 'шорня', 'шорох', 'шорты', 'шоссе', 'шофер', 'шпага', 'шпала', 'шпана', 'шпень', 'шпиль', 'шпинь', 'шпион', 'шпона', 'шпора', 'шприц', 'шпрот', 'шпуля', 'шпунт', 'шпынь', 'шримс', 'шрифт', 'штамб', 'штамм', 'штамп', 'штаны', 'штейн', 'штиль', 'штифт', 'штора', 'шторм', 'штраф', 'штрек', 'штрих', 'штука', 'штурм', 'штырь', 'шуаны', 'шубка', 'шугай', 'шудра', 'шуйца', 'шулер', 'шумер', 'шумок', 'шурин', 'шурпа', 'шуруп', 'шутка', 'шутов', 'шушун', 'шхеры', 'шхуна', 'щаной', 'щебет', 'щегол', 'щекот', 'щелка', 'щелок', 'щенок', 'щепка', 'щетка', 'щечка', 'щипец', 'щипка', 'щипок', 'щипцы', 'щитик', 'щиток', 'щупик', 'щурка', 'щурок', 'щучий', 'щучка', 'эвенк', 'эгида', 'эгрет', 'эдикт', 'экзот', 'эклер', 'экран', 'элеат', 'элита', 'эллин', 'эмаль', 'эммер', 'энзим', 'энный', 'эозин', 'эолит', 'эолов', 'эоцен', 'эпика', 'эпонж', 'эпоха', 'эпюра', 'эрбий', 'эрзац', 'эркер', 'эсдек', 'эскиз', 'эстет', 'этвеш', 'этика', 'эфиоп', 'юдоль', 'южнее', 'южный', 'юзист', 'юкола', 'юлить', 'юнеть', 'юниор', 'юница', 'юнкер', 'юнкор', 'юннат', 'юноша', 'юрист', 'юркий', 'юрьев', 'юферс', 'ябеда', 'явить', 'явный', 'ягель', 'ягода', 'ягуар', 'яичко', 'якать', 'якобы', 'якорь', 'ялбот', 'ямаец', 'ямина', 'ямища', 'ямный', 'ямщик', 'ярить', 'ярица', 'яркий', 'ярлык', 'яруга', 'ярыга', 'ясень', 'ясмин', 'ясный', 'яспис', 'яства', 'яство', 'ястык', 'ясырь', 'ятвяг', 'ятовь', 'яхонт', 'ячать', 'ячный']
allowed_words3 = ['аббат', 'абзац', 'аборт', 'абрек', 'абрис', 'абхаз', 'абцуг', 'абшид', 'аваль', 'аванс', 'авгит', 'авгур', 'авеню', 'аверс', 'авизо', 'авось', 'аврал', 'авран', 'автол', 'автор', 'агава', 'агама', 'агами', 'агент', 'агнат', 'агнец', 'аграф', 'агути', 'адепт', 'адрес', 'адряс', 'адски', 'ажгон', 'азарт', 'азиат', 'аймак', 'айран', 'айрол', 'айсор', 'акажу', 'акант', 'аканф', 'акать', 'аксон', 'актер', 'актив', 'актин', 'акула', 'акциз', 'акция', 'алгол', 'алеть', 'алеут', 'алиби', 'алкил', 'аллах', 'аллея', 'аллод', 'аллюр', 'алмаз', 'алтей', 'алтея', 'алтын', 'алчба', 'алыча', 'альфа', 'амбар', 'амбра', 'амбре', 'амвон', 'амеба', 'аминь', 'ампер', 'ампир', 'ангар', 'ангел', 'ангоб', 'анион', 'анкер', 'анона', 'анонс', 'антик', 'антре', 'анфас', 'анчар', 'аорта', 'апекс', 'апноэ', 'апорт', 'апрош', 'арбуз', 'аргал', 'аргон', 'аргус', 'ареал', 'арена', 'арест', 'ариец', 'аркан', 'армия', 'армюр', 'армяк', 'арсин', 'артос', 'архар', 'архив', 'аршин', 'аскер', 'аскет', 'аспид', 'астат', 'астма', 'астра', 'асцит', 'атака', 'атлас', 'атлет', 'атолл', 'атрий', 'аттик', 'афера', 'афиша', 'ахать', 'ахеец', 'ацтек', 'аэроб', 'аэрон', 'бабах', 'бабий', 'бабка', 'бабье', 'багаж', 'багер', 'багет', 'багор', 'бадан', 'бадья', 'базар', 'базис', 'байка', 'бакан', 'бакен', 'балда', 'балет', 'балка', 'балок', 'балык', 'банан', 'банда', 'банка', 'банту', 'барак', 'баран', 'барда', 'бареж', 'баржа', 'барий', 'барин', 'барит', 'барич', 'барка', 'бармы', 'барон', 'барыш', 'баска', 'басма', 'басня', 'басок', 'басон', 'баста', 'бастр', 'батан', 'батат', 'батог', 'батон', 'батуд', 'батут', 'батыр', 'бахча', 'бачки', 'бачок', 'башка', 'башня', 'баять', 'бдеть', 'бебут', 'бегом', 'бегун', 'бедро', 'бейка', 'бекар', 'бекас', 'бекон', 'белек', 'белец', 'белка', 'белок', 'белые', 'белый', 'белье', 'беляк', 'беляш', 'бердо', 'берег', 'берет', 'берма', 'берце', 'берцо', 'бетон', 'бивак', 'бивни', 'бигус', 'бидон', 'бизон', 'билет', 'билль', 'бином', 'бионт', 'биржа', 'бирка', 'бирюк', 'бирюч', 'бисер', 'бистр', 'битва', 'битки', 'биток', 'битум', 'битый', 'битье', 'битюг', 'благо', 'блажь', 'бланк', 'блеск', 'блинт', 'блоха', 'блуза', 'блюдо', 'блюмс', 'бляха', 'бобби', 'бобер', 'бобок', 'богач', 'бодун', 'бодяк', 'божба', 'божий', 'божок', 'бозон', 'бойня', 'бокаж', 'бокал', 'боком', 'более', 'болид', 'болюс', 'бомба', 'бонза', 'бонмо', 'бонна', 'борат', 'бордо', 'борей', 'борец', 'борид', 'боров', 'борок', 'борть', 'босой', 'босяк', 'ботва', 'ботик', 'бочаг', 'бочар', 'бочка', 'бочок', 'браво', 'брага', 'брада', 'брань', 'брасс', 'брать', 'бремя', 'брешь', 'бридж', 'бритт', 'брить', 'брови', 'бровь', 'бронх', 'броня', 'брошь', 'брыжи', 'брыла', 'брылы', 'брысь', 'брюки', 'брюхо', 'бубал', 'бубен', 'бубна', 'бубны', 'бубон', 'бугай', 'бугор', 'будка', 'будни', 'будра', 'будто', 'бузун', 'букан', 'буква', 'букет', 'букле', 'букли', 'букля', 'букса', 'булат', 'булга', 'булка', 'булла', 'бульк', 'бурав', 'бурак', 'буран', 'бурда', 'бурка', 'бурки', 'бурре', 'бурса', 'бурун', 'бурый', 'бурят', 'бутан', 'бутил', 'бутон', 'бутса', 'бутсы', 'бутуз', 'буфер', 'буфет', 'бухта', 'быдло', 'былой', 'былье', 'бытие', 'бытье', 'бычий', 'бычок', 'бьюик', 'бювар', 'бювет', 'бюкса', 'бяшка', 'вабик', 'вагон', 'важно', 'вазон', 'вакса', 'валах', 'валек', 'валет', 'валец', 'валик', 'валка', 'валок', 'валом', 'валуй', 'валун', 'валух', 'вальс', 'ванна', 'ванта', 'ванты', 'вапор', 'варан', 'варка', 'варяг', 'ватер', 'ватин', 'ватка', 'вафля', 'вахня', 'вахта', 'ваять', 'вбить', 'вброд', 'вверх', 'ввиду', 'ввить', 'вволю', 'ввысь', 'вдали', 'вдаль', 'вдвое', 'вдеть', 'вдова', 'вдоль', 'вдруг', 'вдуть', 'ведро', 'ведун', 'вежда', 'вежды', 'везде', 'везти', 'вейка', 'векша', 'велюр', 'венгр', 'венды', 'венед', 'венет', 'венец', 'веник', 'венка', 'венок', 'вента', 'вепрь', 'верба', 'вервь', 'веред', 'верес', 'верея', 'верже', 'верки', 'верно', 'верть', 'верфь', 'верхи', 'верша', 'вески', 'весло', 'весна', 'весок', 'вести', 'весть', 'ветвь', 'ветер', 'ветка', 'ветла', 'вечер', 'вечор', 'вешка', 'вещий', 'вещун', 'веять', 'вжать', 'вживе', 'взаем', 'взвар', 'взвод', 'взвоз', 'вздор', 'вздох', 'взлет', 'взлом', 'взмах', 'взмет', 'взмыв', 'взнос', 'взрез', 'взрыв', 'взыск', 'взять', 'виват', 'вивер', 'видам', 'видик', 'видно', 'визир', 'визит', 'вилка', 'вилла', 'вилок', 'винил', 'винол', 'винцо', 'виола', 'вираж', 'вирус', 'вирши', 'виски', 'висок', 'вития', 'витой', 'виток', 'витье', 'вихор', 'вихрь', 'вишня', 'вклад', 'вкось', 'вкруг', 'вкупе', 'влага', 'власы', 'влево', 'влечь', 'влить', 'влюбе', 'вмять', 'внаем', 'внизу', 'внове', 'вновь', 'внука', 'внять', 'вобла', 'вовек', 'вовне', 'вовсе', 'вовсю', 'вогул', 'водка', 'вожак', 'вождь', 'вожжа', 'вожжи', 'возка', 'возле', 'возня', 'возок', 'война', 'войти', 'волан', 'волга', 'волей', 'волна', 'волок', 'волос', 'волох', 'волхв', 'вольт', 'волюм', 'вопль', 'ворог', 'ворон', 'ворот', 'ворох', 'вотум', 'вотще', 'вотяк', 'вошка', 'вояка', 'впечь', 'впить', 'впору', 'впрок', 'впуск', 'враки', 'враль', 'врата', 'врать', 'время', 'вроде', 'врозь', 'вруша', 'врыть', 'всего', 'вслед', 'вслух', 'всход', 'всюду', 'всяко', 'втечь', 'втора', 'втрое', 'втуне', 'вуаль', 'вчера', 'вчуже', 'вширь', 'вшить', 'въезд', 'въяве', 'въявь', 'выбег', 'выбор', 'вывал', 'вывес', 'вывих', 'вывод', 'вывоз', 'выгар', 'выгиб', 'выгон', 'выгул', 'выдел', 'выдох', 'выдра', 'выезд', 'выжиг', 'выжим', 'вызов', 'вызол', 'выйти', 'выкат', 'выкос', 'выкуп', 'выкус', 'вылаз', 'вылет', 'вылов', 'вылом', 'вымол', 'вынос', 'выпад', 'выпал', 'выпас', 'выпек', 'выпор', 'выпот', 'вырез', 'выруб', 'высев', 'вытье', 'выход', 'вычет', 'вышаг', 'вышка', 'вьюга', 'вязка', 'вялый', 'вятич', 'вятка', 'вящий', 'гавот', 'гагат', 'газик', 'газон', 'гайка', 'галка', 'галлы', 'галоп', 'галун', 'гамак', 'гамма', 'ганаш', 'ганец', 'ганка', 'гараж', 'гарда', 'гарем', 'гарус', 'гаусс', 'гачек', 'гашиш', 'гвалт', 'гевея', 'гейша', 'гелий', 'гемма', 'гений', 'генри', 'геоид', 'герма', 'герой', 'гетра', 'гетры', 'гетто', 'гибка', 'гидра', 'гиена', 'гилея', 'гиляк', 'гинея', 'гипюр', 'гирло', 'гитов', 'гичка', 'глава', 'главк', 'гладь', 'глаза', 'глень', 'глина', 'глипт', 'глист', 'глубь', 'глупо', 'глушь', 'глыба', 'глядь', 'глядя', 'глясе', 'гмина', 'гнать', 'гнейс', 'гнида', 'гниль', 'гнить', 'гнома', 'гнусь', 'гнуть', 'гобой', 'говор', 'гогот', 'годик', 'годок', 'гокко', 'голец', 'голик', 'голод', 'голос', 'голый', 'голыш', 'гольд', 'голье', 'гольф', 'голяк', 'гомон', 'гонец', 'гонка', 'гонки', 'гонок', 'гонор', 'гопак', 'гопля', 'горал', 'горец', 'горка', 'горло', 'город', 'горох', 'горст', 'горюн', 'гость', 'гофры', 'граве', 'грамм', 'гранд', 'грань', 'графа', 'греза', 'грена', 'греть', 'греча', 'грива', 'гридь', 'грипп', 'гроза', 'грозд', 'гросс', 'гроши', 'груда', 'грудь', 'грунт', 'груша', 'грыжа', 'гряда', 'грязи', 'грязь', 'гуава', 'гуано', 'гуашь', 'губан', 'губка', 'гудок', 'гужом', 'гузка', 'гузно', 'гуляш', 'гумма', 'гумми', 'гумно', 'гумус', 'гуппи', 'гуран', 'гурия', 'гусак', 'гусар', 'гусек', 'гусем', 'гусит', 'гусли', 'густи', 'гуцул', 'гюрза', 'давка', 'давно', 'давок', 'дайна', 'дайра', 'далее', 'далия', 'дамба', 'дамка', 'дамно', 'дарма', 'даром', 'дацан', 'дачка', 'двери', 'дверь', 'дебет', 'дебил', 'дебит', 'дебош', 'дебри', 'дебют', 'девиз', 'девка', 'девон', 'дедка', 'дедов', 'дежка', 'деизм', 'деист', 'декан', 'декор', 'дележ', 'делец', 'демон', 'демос', 'денди', 'денек', 'денно', 'дерба', 'дерби', 'дерен', 'дерма', 'дерть', 'десна', 'десть', 'детва', 'детка', 'детки', 'дефис', 'джинн', 'дзета', 'дзинь', 'дзюдо', 'диван', 'диета', 'дикий', 'динар', 'динас', 'динго', 'дихта', 'дичок', 'длань', 'длина', 'длить', 'днесь', 'днище', 'добор', 'добро', 'довод', 'догма', 'дождь', 'дозор', 'доить', 'дойка', 'дойна', 'дойра', 'дойти', 'докер', 'долго', 'долее', 'долой', 'домен', 'домик', 'домна', 'домой', 'домок', 'домра', 'донец', 'донка', 'донна', 'донор', 'донос', 'донце', 'донья', 'доска', 'досол', 'досуг', 'досье', 'дотла', 'дофин', 'доход', 'дочка', 'драга', 'драже', 'драйв', 'драка', 'драма', 'дрань', 'драть', 'древо', 'дрейф', 'дрель', 'дрема', 'дрена', 'дробь', 'дрова', 'дрога', 'дроги', 'дрожь', 'дрозд', 'дросс', 'дрофа', 'друза', 'друид', 'дрязг', 'дрянь', 'дубка', 'дубль', 'дубок', 'дубье', 'дуван', 'дудак', 'дудка', 'дудки', 'дужка', 'дукат', 'дулеб', 'думец', 'думка', 'дупло', 'дурак', 'дурий', 'дурно', 'дурра', 'дурро', 'дутар', 'дутик', 'дутый', 'дутыш', 'дутье', 'духан', 'духов', 'духом', 'душка', 'душно', 'душок', 'дуэль', 'дщерь', 'дыбом', 'дылда', 'дымка', 'дымок', 'дырка', 'дышло', 'дюбек', 'дюжий', 'дюкер', 'дюшес', 'дядин', 'дятел', 'евнух', 'еврей', 'егерь', 'егоза', 'едать', 'едкий', 'ежели', 'ежить', 'ежиха', 'ездка', 'ездок', 'езжай', 'ей-ей', 'екать', 'елико', 'емкий', 'емшан', 'ересь', 'ерник', 'ершик', 'ершом', 'есаул', 'ехать', 'ехида', 'жабий', 'жабка', 'жабра', 'жабры', 'жажда', 'жакан', 'жакет', 'жамка', 'жарка', 'жарко', 'жарок', 'жатва', 'жатка', 'жатый', 'жвала', 'жвало', 'жвалы', 'ждать', 'жевок', 'желна', 'желоб', 'желть', 'желчь', 'женин', 'жених', 'женка', 'жеода', 'жердь', 'жерех', 'жерло', 'жесть', 'жетон', 'живец', 'живой', 'живот', 'жиган', 'жизнь', 'жилет', 'жилец', 'жилка', 'жилой', 'жилье', 'жираф', 'жирок', 'жиряк', 'житие', 'житье', 'жменя', 'жмудь', 'жмыхи', 'жнива', 'жниво', 'жница', 'жокей', 'жрать', 'жрица', 'жулан', 'жулик', 'жулье', 'жупан', 'жупел', 'жутко', 'жучка', 'жучок', 'забег', 'забой', 'забор', 'завал', 'завет', 'завод', 'завоз', 'завуч', 'загар', 'загиб', 'загон', 'загул', 'задел', 'задик', 'задок', 'задом', 'задор', 'заезд', 'зажим', 'зажин', 'зажор', 'зазор', 'зазыв', 'заика', 'зайка', 'зайти', 'заказ', 'закал', 'закат', 'закол', 'закон', 'закуп', 'закут', 'залет', 'залив', 'зализ', 'залог', 'залом', 'замах', 'замер', 'замес', 'замет', 'замок', 'замор', 'замуж', 'замша', 'занос', 'запад', 'запал', 'запас', 'запах', 'запев', 'запой', 'запор', 'зараз', 'зарез', 'зарод', 'зарок', 'заруб', 'заряд', 'засев', 'засим', 'засов', 'засол', 'засор', 'засос', 'затек', 'затем', 'затес', 'затея', 'заток', 'затон', 'затор', 'затяг', 'заумь', 'заход', 'зацеп', 'зачем', 'зачес', 'зачет', 'зачин', 'звать', 'звено', 'зверь', 'здесь', 'зебра', 'зевок', 'зелот', 'зелье', 'зельц', 'земец', 'земля', 'земно', 'зенит', 'зенки', 'зерно', 'зернь', 'зефир', 'зимой', 'зипун', 'зиять', 'злато', 'злеть', 'злить', 'злоба', 'злюка', 'знамо', 'знамя', 'знать', 'золка', 'зоман', 'зраза', 'зразы', 'зреть', 'зубец', 'зубик', 'зубок', 'зулус', 'зумпф', 'зурна', 'зыбка', 'зыбун', 'зятев', 'зятек', 'иберы', 'иваси', 'ивина', 'ивняк', 'игрек', 'игрец', 'игрок', 'игрун', 'идеал', 'идиом', 'идиот', 'иерей', 'ижица', 'из-за', 'избач', 'извет', 'извив', 'извне', 'извод', 'извоз', 'изгиб', 'изгой', 'излет', 'излом', 'измор', 'износ', 'изъян', 'изыск', 'изюбр', 'икать', 'икона', 'икота', 'иксия', 'иктус', 'ильин', 'илька', 'имаго', 'иметь', 'иначе', 'инвар', 'ингуш', 'индау', 'индий', 'индол', 'индус', 'индюк', 'инжир', 'инкор', 'инока', 'иомен', 'ионий', 'ионит', 'иприт', 'ирбис', 'ирмос', 'ирный', 'искра', 'искус', 'ислам', 'испод', 'испуг', 'иссоп', 'истец', 'истод', 'исток', 'истый', 'исход', 'итого', 'иудей', 'ихний', 'ичиги', 'ишиас', 'йодид', 'йомен', 'кааба', 'кабак', 'кабан', 'кабил', 'кабул', 'кавун', 'кагал', 'каган', 'кагат', 'кагор', 'кадет', 'кадий', 'кадка', 'кадры', 'кадык', 'казак', 'казан', 'казах', 'казна', 'казнь', 'казус', 'кайла', 'кайло', 'кайма', 'кайра', 'какао', 'каков', 'какой', 'калам', 'калач', 'калий', 'калин', 'калиф', 'калым', 'камея', 'камин', 'камка', 'камса', 'камча', 'камыш', 'канал', 'канат', 'канва', 'канна', 'канон', 'каноэ', 'канун', 'канюк', 'капер', 'капли', 'капля', 'капок', 'капор', 'капот', 'капут', 'карат', 'карга', 'карго', 'карда', 'карел', 'карий', 'карла', 'карма', 'карст', 'карта', 'карча', 'каска', 'каско', 'касса', 'каста', 'катар', 'катер', 'катет', 'катод', 'каток', 'катыш', 'кацап', 'качка', 'кашка', 'кашне', 'кашпо', 'кашуб', 'каюта', 'квант', 'кварк', 'кварц', 'квиты', 'квота', 'кегли', 'кегль', 'кегля', 'кекур', 'кельт', 'келья', 'кенар', 'кенаф', 'кепка', 'кетон', 'кефир', 'кечуа', 'кзади', 'кианг', 'кивер', 'кивок', 'кивот', 'кидас', 'кизил', 'кизяк', 'килим', 'киник', 'киоск', 'кирза', 'кирка', 'кирха', 'кисет', 'кисея', 'киска', 'киста', 'кисть', 'кифоз', 'кичка', 'кишка', 'кишмя', 'кладь', 'клака', 'класс', 'клерк', 'клест', 'клеть', 'клещи', 'клика', 'клико', 'клинч', 'клише', 'клоун', 'клуня', 'клупп', 'клуша', 'клюка', 'клюфт', 'кляча', 'кнели', 'кнель', 'кнехт', 'книга', 'книзу', 'кница', 'князь', 'коала', 'кобел', 'кобза', 'кобра', 'ковач', 'ковер', 'ковка', 'когда', 'кодак', 'кожан', 'кожух', 'козел', 'козий', 'козлы', 'козни', 'козон', 'койка', 'койне', 'койот', 'кокет', 'кокон', 'кокор', 'кокос', 'колба', 'колер', 'колет', 'колея', 'колит', 'колка', 'колли', 'колоб', 'колок', 'колон', 'колос', 'колун', 'колье', 'кольт', 'колюр', 'комар', 'комик', 'комма', 'комми', 'комод', 'комок', 'комуз', 'конек', 'конец', 'коник', 'конка', 'конто', 'конус', 'конюх', 'копал', 'копач', 'копер', 'копия', 'копка', 'копна', 'копра', 'копты', 'копун', 'копыл', 'копье', 'коран', 'корда', 'корец', 'корка', 'корма', 'короб', 'корча', 'корчи', 'корье', 'коряк', 'косач', 'косая', 'косец', 'космы', 'косой', 'косок', 'кость', 'косяк', 'котел', 'котик', 'кофей', 'кофта', 'кохия', 'кочан', 'кочет', 'кочка', 'кошер', 'кошка', 'кошма', 'кощей', 'крага', 'краги', 'кража', 'краля', 'крапп', 'краса', 'краше', 'кредо', 'креол', 'крепь', 'кресс', 'крест', 'крица', 'кровь', 'кроки', 'кроль', 'кроме', 'крона', 'кросс', 'кроха', 'круиз', 'крупа', 'круча', 'крыло', 'крыса', 'крыть', 'крыша', 'кряду', 'кубик', 'кубок', 'кувез', 'кудри', 'кузен', 'кузня', 'кузов', 'кукан', 'кукиш', 'кукла', 'кулаж', 'кулак', 'кулан', 'кулек', 'кулер', 'кулеш', 'кулик', 'кулич', 'кулон', 'культ', 'кумач', 'кумжа', 'кумир', 'кумов', 'кумык', 'кумыс', 'кунак', 'куний', 'купаж', 'купец', 'купля', 'купно', 'купол', 'купон', 'кураж', 'курва', 'курий', 'курия', 'курка', 'курок', 'курсы', 'курья', 'кусок', 'кутас', 'кутеж', 'куток', 'кутум', 'кутья', 'кухня', 'куцый', 'кучер', 'кучка', 'кушак', 'кхмер', 'кювет', 'кюрий', 'кюрин', 'кяриз', 'лабаз', 'лаваш', 'лавка', 'лавра', 'лагун', 'ладан', 'ладно', 'ладья', 'лазер', 'лайба', 'лайда', 'лайка', 'лакей', 'лампа', 'ламут', 'ландо', 'ланды', 'ланка', 'лапка', 'лапта', 'лапша', 'ларго', 'ларек', 'ларец', 'ласка', 'лассо', 'ласты', 'латка', 'латук', 'латыш', 'лафет', 'лафит', 'лачка', 'лачок', 'лаять', 'лбина', 'лбище', 'лгать', 'левак', 'левее', 'левит', 'левша', 'левый', 'легат', 'легаш', 'легко', 'ледок', 'лежак', 'лежка', 'лежмя', 'лезть', 'лейка', 'лекаж', 'лемех', 'лемма', 'лемур', 'ленок', 'лента', 'ленто', 'ленца', 'лепет', 'лепка', 'лепра', 'лепта', 'леска', 'лесок', 'лесть', 'летка', 'леток', 'летом', 'летун', 'лешак', 'леший', 'лещик', 'лиана', 'ливер', 'ливмя', 'лидер', 'лизин', 'лизис', 'лизол', 'лизун', 'ликер', 'лилия', 'лиман', 'лимит', 'лимон', 'лимфа', 'линек', 'линза', 'линия', 'липец', 'липка', 'липси', 'лирик', 'лисий', 'литер', 'литий', 'лития', 'литка', 'литой', 'литье', 'лихач', 'лихва', 'лихой', 'лицей', 'лишай', 'лишек', 'лобан', 'лобби', 'лобик', 'лобио', 'лобия', 'лобок', 'ловец', 'ловля', 'логик', 'логос', 'лодка', 'ложка', 'ложок', 'локва', 'локон', 'ломик', 'ломка', 'лонжа', 'лопух', 'лоток', 'лотос', 'лохмы', 'лоция', 'лошак', 'лубок', 'лужок', 'лузга', 'лукно', 'лунка', 'лупка', 'лучик', 'лучок', 'лучше', 'лыжня', 'лысун', 'лысый', 'лычки', 'лычко', 'льяло', 'любая', 'любой', 'люмен', 'люнет', 'люпин', 'люпус', 'лютик', 'лютич', 'лютня', 'лютый', 'ляжка', 'лямка', 'ляпис', 'лярва', 'ляшка', 'магия', 'магма', 'магот', 'мадам', 'мадия', 'маета', 'мажор', 'мазер', 'мазик', 'мазка', 'мазло', 'мазня', 'мазок', 'мазур', 'мазут', 'майка', 'майна', 'майор', 'макао', 'макет', 'маков', 'макса', 'малек', 'малец', 'малик', 'малка', 'малое', 'малый', 'малыш', 'малье', 'маляр', 'мамин', 'мамка', 'мамон', 'манго', 'манеж', 'манер', 'мание', 'мания', 'манка', 'манко', 'манна', 'манок', 'манси', 'манто', 'манул', 'марал', 'маран', 'марго', 'маржа', 'марка', 'марля', 'марши', 'маска', 'масло', 'масон', 'масса', 'масть', 'матка', 'мафия', 'махра', 'махры', 'мачок', 'мачта', 'маять', 'мегом', 'медик', 'медок', 'медяк', 'между', 'мезга', 'мезон', 'мелис', 'мелок', 'мелос', 'менее', 'мерея', 'мерин', 'мерка', 'мерси', 'месса', 'мести', 'место', 'месть', 'месье', 'месяц', 'метан', 'метек', 'метил', 'метис', 'метка', 'метла', 'метод', 'метол', 'метоп', 'метро', 'меццо', 'мечта', 'мешок', 'мигач', 'мигли', 'мигом', 'мидия', 'мизер', 'микоз', 'микст', 'милая', 'милка', 'милок', 'милый', 'минер', 'минея', 'минор', 'минус', 'миома', 'мираж', 'мирза', 'мирок', 'мирон', 'мирра', 'мирта', 'миска', 'митоз', 'митра', 'мишка', 'млеко', 'млеть', 'мнить', 'много', 'могар', 'модус', 'можно', 'мозги', 'мойва', 'мойка', 'мойра', 'мокко', 'мокша', 'молва', 'молвь', 'молот', 'молох', 'молча', 'моляр', 'монах', 'моном', 'мопед', 'морда', 'морзе', 'мороз', 'морок', 'морфа', 'морцо', 'моряк', 'мосол', 'мосье', 'мотет', 'мотив', 'мотка', 'мотня', 'моток', 'мотор', 'мотто', 'мохер', 'мохны', 'мочка', 'мошка', 'мошна', 'мразь', 'мрежа', 'мужик', 'музей', 'мулат', 'мулек', 'мулла', 'муляж', 'мумие', 'мумия', 'мунда', 'мураш', 'мурда', 'мурза', 'мурло', 'мурья', 'мусор', 'мутон', 'муфта', 'муцин', 'мучка', 'мушка', 'мцыри', 'мчать', 'мшить', 'мымра', 'мысик', 'мысль', 'мысок', 'мытый', 'мытье', 'мышей', 'мыший', 'мышка', 'мышца', 'мэлан', 'мэрия', 'мэрон', 'мюзет', 'мюрид', 'мякиш', 'мялка', 'мямля', 'мяско', 'мясцо', 'мятеж', 'мятие', 'мятый', 'мятье', 'мячик', 'на-ка', 'набат', 'набег', 'набоб', 'набок', 'набор', 'навал', 'навар', 'навек', 'навес', 'навет', 'навий', 'навис', 'навоз', 'навой', 'навык', 'наган', 'нагар', 'нагиб', 'нагой', 'нагон', 'нагул', 'надел', 'надир', 'надой', 'надув', 'наезд', 'нажим', 'нажин', 'назад', 'назем', 'назло', 'найти', 'наказ', 'накал', 'накат', 'накос', 'налет', 'налив', 'налим', 'налог', 'намаз', 'намек', 'намет', 'намин', 'намол', 'намыв', 'нанду', 'нанка', 'нанос', 'напев', 'напой', 'напор', 'нарды', 'нарез', 'народ', 'нарта', 'нарты', 'нарыв', 'наряд', 'насад', 'насос', 'насып', 'натек', 'натяг', 'наука', 'нахал', 'нация', 'начес', 'начет', 'начин', 'наяву', 'наяда', 'невод', 'негде', 'негус', 'недра', 'недуг', 'нейти', 'некий', 'некто', 'немая', 'немец', 'немка', 'немой', 'ненец', 'нения', 'ненка', 'непер', 'нервы', 'нерка', 'нерол', 'нерпа', 'нести', 'несть', 'нетто', 'нефть', 'нечет', 'нечто', 'нешто', 'ни-ни', 'нивоз', 'нивух', 'нигде', 'низка', 'низок', 'низом', 'никак', 'никой', 'никто', 'нилот', 'нимфа', 'нисан', 'нитка', 'нитон', 'ничей', 'ничто', 'ничья', 'нищая', 'нищий', 'новик', 'новое', 'новый', 'ножик', 'ножка', 'ножны', 'нойон', 'нолик', 'номад', 'номер', 'нонет', 'нория', 'норка', 'норма', 'норов', 'носач', 'носик', 'носка', 'носки', 'носок', 'нотис', 'нотка', 'ночва', 'ночка', 'ночью', 'нощно', 'ну-ка', 'ну-ну', 'нудно', 'нужда', 'нужно', 'нукер', 'нулик', 'нумер', 'нутро', 'нынче', 'нырок', 'нытик', 'нытье', 'нюанс', 'нянин', 'оазис', 'обвал', 'обвес', 'обвод', 'обвоз', 'обгон', 'обдир', 'обдув', 'обжиг', 'обжим', 'обжин', 'обжог', 'обзол', 'обзор', 'обида', 'обить', 'обкат', 'обком', 'обкос', 'облет', 'облик', 'облов', 'облог', 'облом', 'обман', 'обмен', 'обмер', 'обмет', 'обмин', 'обмол', 'обмыв', 'обнос', 'обора', 'образ', 'обрат', 'обрез', 'оброк', 'обруб', 'обруч', 'обрыв', 'обряд', 'обсев', 'обувь', 'обуза', 'обуть', 'обход', 'общий', 'объем', 'обыск', 'овамо', 'овить', 'овощи', 'овраг', 'овсец', 'овсюг', 'овчар', 'огонь', 'огрех', 'одаль', 'одежа', 'одеон', 'одеть', 'одурь', 'ожечь', 'ожить', 'оземь', 'озеро', 'озимь', 'озноб', 'ойрот', 'окапи', 'окать', 'океан', 'окись', 'оклад', 'оклик', 'оковы', 'окоем', 'около', 'окорм', 'окрас', 'окрик', 'окрол', 'округ', 'оксид', 'октан', 'октет', 'октод', 'окунь', 'олеат', 'олеин', 'олень', 'олеум', 'олива', 'олифа', 'олово', 'ольха', 'омега', 'омела', 'омлет', 'омуль', 'омыть', 'онагр', 'оникс', 'онуча', 'онучи', 'оолит', 'опала', 'опара', 'опека', 'опера', 'опись', 'опить', 'опиум', 'оплот', 'оплыв', 'опоек', 'опока', 'опора', 'опрос', 'оптик', 'оптом', 'опять', 'орава', 'орала', 'орало', 'орарь', 'орать', 'орган', 'оргия', 'орден', 'ордер', 'ореол', 'оркан', 'орлан', 'орлец', 'орлий', 'орлик', 'орляк', 'ороки', 'орочи', 'ортит', 'оршад', 'осада', 'осень', 'осетр', 'осечь', 'осина', 'оскал', 'ослик', 'ослоп', 'осляк', 'осман', 'осмий', 'осмол', 'осмос', 'особа', 'особо', 'особь', 'осоед', 'осока', 'остит', 'остов', 'остяк', 'осыпь', 'отава', 'отара', 'отбив', 'отбой', 'отбор', 'отвал', 'отвар', 'отвес', 'ответ', 'отвод', 'отвоз', 'отгиб', 'отгон', 'отгул', 'отдел', 'отдух', 'отдых', 'отель', 'отечь', 'отжиг', 'отжим', 'отжог', 'отзол', 'отзыв', 'отказ', 'откат', 'откол', 'откос', 'откуп', 'откус', 'отлет', 'отлив', 'отлов', 'отлуп', 'отмах', 'относ', 'отпад', 'отпор', 'отрез', 'отрог', 'отрок', 'отруб', 'отрыв', 'отряд', 'отсев', 'отсек', 'отсос', 'оттек', 'отток', 'отход', 'отцеп', 'отцов', 'отчал', 'отчет', 'отчий', 'отчим', 'отшиб', 'отщеп', 'отъем', 'офеня', 'офорт', 'офсет', 'охала', 'охать', 'охват', 'охота', 'очень', 'очерк', 'очкур', 'очный', 'ошеек', 'ошуюю', 'ощупь', 'павий', 'падеж', 'падло', 'падре', 'падуб', 'падун', 'пазок', 'пайза', 'пайка', 'пакет', 'пакля', 'палас', 'палац', 'палач', 'палаш', 'палец', 'палея', 'палия', 'палка', 'палый', 'пампа', 'панаш', 'панда', 'панна', 'панно', 'панты', 'паныч', 'папин', 'папка', 'парад', 'параф', 'парез', 'парик', 'пария', 'парка', 'парод', 'парок', 'паром', 'парта', 'парус', 'парча', 'парша', 'пасмо', 'пассы', 'паста', 'пасти', 'пасть', 'пасха', 'пасюк', 'патан', 'патер', 'патла', 'патлы', 'патуа', 'пауза', 'пафос', 'пахви', 'пахит', 'пахта', 'пацан', 'пачка', 'пашня', 'паять', 'певец', 'певун', 'пегас', 'пегий', 'пейсы', 'пекан', 'пекло', 'пелит', 'пемза', 'пенал', 'пенек', 'пение', 'пенис', 'пенка', 'пенни', 'пенье', 'пепел', 'перга', 'перед', 'перец', 'перка', 'перси', 'перст', 'песец', 'песий', 'песик', 'пески', 'песнь', 'песня', 'песок', 'петел', 'петит', 'петля', 'петух', 'печка', 'пеший', 'пешка', 'пешня', 'пещур', 'пиала', 'пиано', 'пивко', 'пивцо', 'пигус', 'пижма', 'пижон', 'пикап', 'пикет', 'пикон', 'пикша', 'пилав', 'пилка', 'пилон', 'пилот', 'пиния', 'пинок', 'пинта', 'пипка', 'пират', 'пирит', 'пирке', 'пирог', 'пирон', 'пироп', 'писец', 'писун', 'питие', 'питон', 'питух', 'питье', 'пифия', 'пифос', 'пихта', 'пищик', 'плавь', 'плаке', 'пламя', 'пласт', 'плата', 'плато', 'плаун', 'плаха', 'плебс', 'плева', 'племя', 'плена', 'плеск', 'плеть', 'плечи', 'плечо', 'плешь', 'плита', 'плица', 'плоть', 'плохо', 'плыть', 'плюха', 'пнище', 'пнуть', 'по-за', 'побег', 'побои', 'побор', 'повал', 'повар', 'повет', 'повод', 'повой', 'погиб', 'погон', 'подий', 'подле', 'подог', 'подол', 'поезд', 'пожар', 'пожня', 'пожог', 'позем', 'позер', 'позже', 'позор', 'позыв', 'поиск', 'поить', 'пойка', 'пойло', 'пойма', 'пойти', 'показ', 'покат', 'покер', 'покои', 'покой', 'покос', 'полба', 'полет', 'полив', 'полип', 'полир', 'полис', 'полка', 'полно', 'полог', 'полоз', 'полок', 'полом', 'полон', 'полть', 'полый', 'полюс', 'поляк', 'помет', 'помин', 'помои', 'помол', 'помор', 'помпа', 'понос', 'понур', 'пончо', 'понюх', 'попик', 'попка', 'попов', 'порез', 'порей', 'порка', 'порог', 'пороз', 'порой', 'порок', 'порос', 'порох', 'порою', 'порск', 'порто', 'порты', 'порча', 'порыв', 'посад', 'посев', 'после', 'посол', 'посох', 'посул', 'посыл', 'поташ', 'потек', 'потир', 'поток', 'потом', 'потоп', 'потяг', 'поход', 'похул', 'почва', 'почем', 'почет', 'почин', 'почка', 'почта', 'почти', 'почто', 'пошиб', 'пошив', 'поэма', 'право', 'праща', 'предо', 'прель', 'пресс', 'преть', 'прием', 'прима', 'принц', 'приор', 'приуз', 'причт', 'приют', 'проба', 'проем', 'проза', 'пропс', 'просо', 'прочь', 'прусс', 'прыть', 'прядь', 'пряжа', 'прямо', 'пряха', 'псарь', 'псина', 'псица', 'птаха', 'птица', 'пуант', 'пугач', 'пудик', 'пудра', 'пузан', 'пузач', 'пукля', 'пульс', 'пульт', 'пункт', 'пупок', 'пурга', 'пурин', 'пурка', 'пусть', 'путем', 'путец', 'путло', 'путля', 'пуфик', 'пучок', 'пушка', 'пушок', 'пушту', 'пущий', 'пхать', 'пчела', 'пшено', 'пыжик', 'пырей', 'пытка', 'пышка', 'пьеза', 'пьеро', 'пьеса', 'пялка', 'пясть', 'пятак', 'пятая', 'пятка', 'пятно', 'пяток', 'пятый', 'пятью', 'рабат', 'рабий', 'равно', 'радар', 'раджа', 'радий', 'радио', 'радон', 'ражий', 'разве', 'разик', 'разок', 'разом', 'разор', 'разум', 'раина', 'район', 'ракия', 'ракля', 'ракун', 'ракша', 'ралли', 'рамка', 'рампа', 'ранее', 'ранет', 'ранец', 'ранка', 'ранчо', 'рапид', 'расти', 'растр', 'ратай', 'ратин', 'раунд', 'рафия', 'рахис', 'рахит', 'рацея', 'рация', 'рачий', 'рачок', 'рвань', 'рвать', 'рвота', 'рдест', 'рдеть', 'ребро', 'ребус', 'ревмя', 'ревун', 'регби', 'регия', 'регот', 'редан', 'редис', 'редут', 'режим', 'резак', 'резец', 'резит', 'резка', 'резня', 'резол', 'резон', 'резус', 'рейка', 'релит', 'рельс', 'ремез', 'ремиз', 'ренет', 'рений', 'ренин', 'рента', 'репей', 'репер', 'репка', 'речка', 'решка', 'реять', 'ржавь', 'ржать', 'ржище', 'ризка', 'рикша', 'ринит', 'рипус', 'риска', 'ритор', 'рифма', 'рицин', 'ришта', 'робот', 'ровик', 'ровно', 'ровня', 'рогач', 'рогоз', 'родан', 'родий', 'родич', 'родня', 'рожки', 'рожок', 'рожон', 'розан', 'розга', 'розно', 'рознь', 'роить', 'ройба', 'рокот', 'ролик', 'роман', 'рондо', 'ронжа', 'ропак', 'ропот', 'ростр', 'ротик', 'роток', 'ротон', 'ротор', 'рохля', 'рояль', 'ртище', 'ртуть', 'рубеж', 'рубец', 'рубин', 'рубка', 'рубль', 'ругмя', 'ругня', 'рудой', 'рудый', 'рудяк', 'ружье', 'руина', 'руины', 'рукав', 'рулет', 'рулон', 'румба', 'румын', 'рунец', 'рупия', 'рупор', 'русак', 'русин', 'русло', 'русый', 'рутил', 'ручей', 'ручка', 'рыбак', 'рыбец', 'рыбий', 'рыбка', 'рывок', 'рыжей', 'рыжий', 'рыжик', 'рында', 'рынок', 'рысак', 'рысий', 'рысца', 'рысью', 'рытый', 'рытье', 'рычаг', 'рэкет', 'рюмка', 'рюшка', 'рябой', 'рябок', 'рядно', 'рядок', 'рядом', 'ряска', 'ряшка', 'саами', 'сабан', 'сабза', 'сабля', 'сабур', 'саван', 'савка', 'саган', 'сагиб', 'саджа', 'садик', 'садка', 'садок', 'сазан', 'сайга', 'сайда', 'сайка', 'сайра', 'саква', 'сакля', 'сакма', 'салат', 'салка', 'салки', 'салол', 'салон', 'салоп', 'салют', 'саман', 'самба', 'самбо', 'самец', 'самка', 'самум', 'самый', 'санки', 'сапер', 'сапка', 'сапог', 'сапун', 'сарай', 'саран', 'саржа', 'сарыч', 'сатин', 'сатир', 'сахар', 'сахиб', 'сачок', 'сбить', 'сбоку', 'сборы', 'сброд', 'сброс', 'сбруя', 'сбыть', 'свара', 'сваха', 'свежо', 'сверх', 'свеча', 'свиль', 'свинг', 'свист', 'свита', 'свить', 'свора', 'свояк', 'свыше', 'связь', 'сглаз', 'сдать', 'сдача', 'сдвиг', 'сдоба', 'сдуру', 'сдуть', 'сеанс', 'севак', 'север', 'севец', 'севок', 'седло', 'седой', 'седок', 'сезам', 'сезон', 'сейша', 'секач', 'секта', 'селен', 'селин', 'семга', 'семик', 'семит', 'семью', 'семья', 'сенаж', 'сенат', 'сенна', 'сенцо', 'сенцы', 'сепия', 'серия', 'серка', 'серко', 'серна', 'серсо', 'серум', 'серый', 'серяк', 'сесть', 'сетка', 'сечка', 'сеять', 'сжать', 'сжечь', 'сжить', 'сзади', 'сивер', 'сивка', 'сивко', 'сивуч', 'сивый', 'сигма', 'сидка', 'сидмя', 'сиена', 'сижок', 'сизиф', 'сизый', 'сизяк', 'сикоз', 'силач', 'силен', 'силой', 'силок', 'силон', 'силос', 'силур', 'сильф', 'синап', 'синец', 'синий', 'синод', 'синус', 'синяк', 'сипай', 'сирин', 'сироп', 'сирый', 'систр', 'ситар', 'ситец', 'ситро', 'ситце', 'сифон', 'сиять', 'скала', 'скало', 'скань', 'скарб', 'скарн', 'скаут', 'сквер', 'скетч', 'скирд', 'склад', 'склеп', 'склиз', 'склон', 'скоба', 'сколь', 'скопа', 'скора', 'скорм', 'скоро', 'скотт', 'скрап', 'скреп', 'скрип', 'скудо', 'скука', 'скула', 'скунс', 'слабо', 'слава', 'слайд', 'слать', 'слева', 'слега', 'слеза', 'сленг', 'слечь', 'слива', 'слизь', 'слить', 'слово', 'слоек', 'слоны', 'слуга', 'слыть', 'слышь', 'слюда', 'слюна', 'слюни', 'смазь', 'смалу', 'смена', 'смерд', 'смерч', 'смесь', 'смета', 'сметь', 'смола', 'смоль', 'смотр', 'смочь', 'смрад', 'смута', 'смысл', 'смыть', 'смять', 'снедь', 'снизу', 'снова', 'сноси', 'сноха', 'сныть', 'снять', 'собес', 'собор', 'совет', 'совий', 'совик', 'совка', 'совок', 'согды', 'содом', 'созыв', 'сойка', 'сойма', 'сойот', 'сойти', 'сокол', 'солея', 'солка', 'солод', 'сомон', 'сонет', 'сопка', 'сопли', 'сопло', 'сопля', 'сопор', 'сопун', 'сорго', 'сорит', 'сорок', 'сором', 'сорус', 'сосед', 'сосец', 'соска', 'сосна', 'сосок', 'сосуд', 'сосун', 'сосцы', 'сотая', 'сотка', 'сотня', 'сотый', 'софит', 'сочок', 'сошка', 'спазм', 'спать', 'спесь', 'спеть', 'спечь', 'спина', 'спирт', 'спить', 'спица', 'сплав', 'сплин', 'спора', 'спорт', 'спрос', 'спрут', 'спурт', 'спуск', 'сразу', 'среда', 'среди', 'средь', 'сроду', 'срост', 'срыву', 'срыть', 'сряду', 'ссечь', 'ссора', 'ссуда', 'стадо', 'стаза', 'сталь', 'станс', 'старт', 'старь', 'стать', 'ствол', 'створ', 'стега', 'стезя', 'стека', 'стела', 'стена', 'стенд', 'степс', 'степь', 'стечь', 'стило', 'стиль', 'стихи', 'стоик', 'стокс', 'столб', 'столп', 'столь', 'стопа', 'стояк', 'страж', 'страз', 'страх', 'стриж', 'строй', 'строп', 'струг', 'струп', 'струя', 'стужа', 'ступа', 'стыть', 'суаре', 'сувой', 'судак', 'судия', 'судно', 'судок', 'судья', 'суета', 'сукин', 'сукно', 'сулея', 'сулой', 'сумах', 'сумет', 'сумка', 'сумма', 'сунна', 'суоми', 'супец', 'супин', 'суржа', 'сурик', 'сурна', 'сурок', 'сусак', 'сусек', 'сусло', 'сутаж', 'сутки', 'сутра', 'суфле', 'сухая', 'сухой', 'сучий', 'сучка', 'сучок', 'сучье', 'сушка', 'сущий', 'сущик', 'сфера', 'схват', 'схема', 'схима', 'сцена', 'сцинк', 'счеты', 'сшить', 'съезд', 'сынов', 'сынок', 'сыпец', 'сырец', 'сырой', 'сырок', 'сырть', 'сырца', 'сырье', 'сытый', 'сычуг', 'сыщик', 'сюжет', 'сюита', 'сяжки', 'сяжок', 'сякой', 'табак', 'табес', 'табло', 'табор', 'табун', 'тавот', 'тавро', 'тавры', 'таган', 'тазик', 'таить', 'тайга', 'тайна', 'также', 'такка', 'таков', 'такой', 'такса', 'такси', 'такыр', 'талан', 'талер', 'талес', 'талик', 'талия', 'талон', 'талый', 'талыш', 'тальк', 'талья', 'тамга', 'тамил', 'танго', 'танец', 'танин', 'тапер', 'тапир', 'тапка', 'тапки', 'таран', 'тариф', 'таска', 'татка', 'тафта', 'тафья', 'тахта', 'тацет', 'тачка', 'ташка', 'таять', 'тварь', 'твист', 'театр', 'тезис', 'тезка', 'теизм', 'теист', 'текст', 'телец', 'телка', 'телок', 'тембр', 'темно', 'тенек', 'тенор', 'тепло', 'терем', 'терец', 'терка', 'термы', 'тесак', 'теска', 'тесло', 'тесно', 'тесто', 'тесть', 'тетин', 'тетка', 'теург', 'течка', 'тешка', 'тещин', 'тиара', 'тимол', 'типаж', 'типец', 'типик', 'типун', 'тираж', 'тиран', 'тиски', 'титан', 'титла', 'титло', 'титул', 'тифон', 'тихий', 'ткань', 'ткать', 'тлеть', 'тлить', 'тмить', 'то-то', 'товар', 'тогда', 'тодес', 'тойон', 'токай', 'токмо', 'толай', 'толки', 'толпа', 'толща', 'толщь', 'томат', 'томик', 'тондо', 'тонна', 'тонус', 'топаз', 'топка', 'топор', 'топот', 'торба', 'торги', 'торец', 'торий', 'торит', 'торос', 'тоска', 'тотем', 'тохар', 'точка', 'точно', 'точок', 'тошно', 'тощий', 'трава', 'травы', 'тракт', 'транс', 'трапп', 'трата', 'траур', 'треба', 'трель', 'трема', 'треск', 'трест', 'треть', 'треух', 'трефа', 'трефы', 'триас', 'триба', 'триер', 'тризм', 'трико', 'триод', 'тромб', 'тромп', 'тропа', 'трояк', 'труба', 'трусы', 'труха', 'трюмо', 'тубус', 'тугой', 'тугун', 'тузик', 'тукан', 'тулес', 'тулий', 'тулуз', 'тулук', 'тулуп', 'тулья', 'туляк', 'тумак', 'туман', 'тумба', 'тумор', 'тунец', 'тупей', 'тупец', 'тупик', 'тупой', 'тупыш', 'турач', 'турий', 'турне', 'турок', 'тутор', 'тутти', 'туфли', 'туфля', 'тучка', 'тушин', 'тушка', 'тщета', 'тыква', 'тынок', 'тырло', 'тырса', 'тычок', 'тю-тю', 'тюбик', 'тюник', 'тюрки', 'тюрок', 'тютюн', 'тюфяк', 'тючок', 'тябло', 'тягач', 'тягло', 'тяжба', 'тяжко', 'тяпка', 'убить', 'убрус', 'убыль', 'увить', 'увраж', 'угода', 'уголь', 'угорь', 'удаль', 'удача', 'удерж', 'удила', 'удить', 'удэге', 'уесть', 'ужать', 'ужели', 'ужель', 'узбек', 'уздцы', 'узерк', 'узина', 'узить', 'узкий', 'узник', 'уйгур', 'укать', 'уклад', 'уклон', 'укроп', 'укрут', 'укрюк', 'уксус', 'улика', 'улита', 'улица', 'уличи', 'улыба', 'улюлю', 'умбра', 'умбры', 'уметь', 'умище', 'умник', 'умный', 'умолк', 'умора', 'умыть', 'умять', 'униат', 'унтер', 'унция', 'унять', 'упечь', 'упрек', 'упырь', 'урема', 'урина', 'урман', 'усечь', 'усики', 'усище', 'усищи', 'успех', 'устав', 'устой', 'уступ', 'устье', 'утеря', 'утеха', 'утечь', 'утиль', 'утица', 'утлый', 'утром', 'ухарь', 'ухать', 'ухват', 'ухожь', 'учеба', 'учить', 'учхоз', 'ушить', 'ушица', 'ушкан', 'ушкуй', 'ушник', 'ушной', 'ущерб', 'уютно', 'фабра', 'фавор', 'фавус', 'фагот', 'фазан', 'фазис', 'факел', 'факир', 'фалда', 'фальц', 'фанза', 'фанты', 'фасад', 'фасет', 'фаска', 'фасон', 'фатум', 'фатюй', 'фауна', 'фацет', 'фация', 'фаянс', 'фений', 'фенил', 'фенол', 'ферзь', 'ферма', 'феска', 'фетиш', 'фетюк', 'фефер', 'фиакр', 'фибра', 'фибры', 'фигли', 'фидер', 'фижмы', 'физик', 'физия', 'фикус', 'филей', 'филер', 'филин', 'фильм', 'фильц', 'фимоз', 'финал', 'финик', 'финиш', 'финка', 'финна', 'фиорд', 'фирма', 'фитин', 'фишка', 'фланг', 'фланк', 'флейц', 'флешь', 'флинт', 'флирт', 'флокс', 'флора', 'флюид', 'фляга', 'фобия', 'фокус', 'фолио', 'фомин', 'фомка', 'фонон', 'форма', 'форте', 'форум', 'фоска', 'фотон', 'фофан', 'фраер', 'фраза', 'франк', 'франт', 'фрахт', 'фреза', 'френч', 'фреон', 'фронт', 'фрукт', 'фугас', 'фужер', 'фузея', 'фукус', 'фуляр', 'фураж', 'фурия', 'фурма', 'фурор', 'футер', 'футор', 'фуэте', 'фырок', 'фьорд', 'фюйть', 'фюрер', 'ха-ха', 'хабар', 'хаджи', 'хазар', 'хайло', 'хакас', 'халат', 'халва', 'халда', 'халиф', 'хамит', 'хамка', 'хамов', 'хамса', 'хамье', 'ханжа', 'ханты', 'ханша', 'хапун', 'харчи', 'харчо', 'хасид', 'хатка', 'хауса', 'хаять', 'хвала', 'хвать', 'хворь', 'хвост', 'хе-хе', 'хедер', 'хедив', 'херес', 'херик', 'хетты', 'хи-хи', 'хиазм', 'хилус', 'хилый', 'хиляк', 'химик', 'химия', 'химус', 'хинди', 'хинин', 'хиппи', 'хитин', 'хитон', 'хлест', 'хлупь', 'хлыст', 'хлюст', 'хлябь', 'хмарь', 'хмель', 'хмурь', 'хмырь', 'хо-хо', 'хобби', 'хобот', 'ходжа', 'ходик', 'ходка', 'ходок', 'ходун', 'холка', 'холод', 'холоп', 'холст', 'холуй', 'хомут', 'хомяк', 'хорал', 'хорда', 'хорей', 'хорек', 'хорея', 'хорон', 'хохма', 'хохол', 'хохот', 'хруст', 'хряпа', 'хряск', 'хряст', 'худой', 'хунта', 'хурал', 'хурда', 'хурма', 'хутор', 'цадик', 'цанга', 'цапка', 'цапля', 'цапун', 'цапфа', 'царап', 'царев', 'царек', 'цахур', 'цацка', 'цвель', 'цветы', 'цевка', 'цевье', 'цедра', 'цезий', 'целик', 'целое', 'целый', 'центр', 'цепка', 'цепня', 'церий', 'цехин', 'цибик', 'цикля', 'цинаш', 'цинга', 'циник', 'цирик', 'циста', 'цитра', 'цифра', 'цокот', 'цугом', 'цукат', 'цуцик', 'цыган', 'цыпка', 'цыпки', 'чабан', 'чабер', 'чадра', 'чайка', 'чакан', 'чалка', 'чалма', 'чалый', 'чанах', 'чарка', 'часик', 'часок', 'часом', 'часть', 'чатал', 'чашка', 'чаять', 'чебак', 'чебот', 'чекан', 'челка', 'чемер', 'чепан', 'чепец', 'черва', 'черви', 'червы', 'червь', 'черед', 'через', 'череп', 'чернь', 'черта', 'ческа', 'честь', 'четий', 'четки', 'чехол', 'чечет', 'чешка', 'чешки', 'чешуя', 'чибис', 'чижик', 'чилим', 'чинар', 'чинка', 'чипсы', 'чирей', 'чирок', 'число', 'читка', 'чичер', 'чокер', 'чомга', 'чохом', 'чрево', 'чреда', 'чтиво', 'чтить', 'чтица', 'чтобы', 'чубук', 'чувал', 'чуваш', 'чувяк', 'чугун', 'чудак', 'чудик', 'чудно', 'чудом', 'чужак', 'чужая', 'чужое', 'чужой', 'чуйка', 'чукча', 'чулан', 'чулки', 'чулок', 'чумак', 'чурек', 'чурка', 'чуток', 'чутье', 'чухна', 'чушка', 'чуять', 'чхать', 'шабаш', 'шабер', 'шабли', 'шабот', 'шавка', 'шагом', 'шажок', 'шайба', 'шайка', 'шакал', 'шалаш', 'шалун', 'шалый', 'шаман', 'шамот', 'шанец', 'шанкр', 'шапка', 'шарап', 'шарах', 'шарик', 'шасла', 'шассе', 'шасси', 'шасть', 'шатен', 'шатер', 'шатия', 'шатун', 'шафер', 'шахта', 'шашка', 'шашни', 'шваль', 'шванк', 'швара', 'шварк', 'шварт', 'швырк', 'шевро', 'шеища', 'шейка', 'шелеп', 'шелом', 'шельф', 'шемая', 'шепот', 'шериф', 'шесть', 'шибер', 'шиизм', 'шимми', 'шинок', 'шипун', 'ширма', 'ширмы', 'шитво', 'шитик', 'шитый', 'шитье', 'шифер', 'шифон', 'шихан', 'шихта', 'шишак', 'шишка', 'шкала', 'шквал', 'шкерт', 'шкода', 'школа', 'шкура', 'шланг', 'шлейф', 'шлюха', 'шляпа', 'шмель', 'шнапс', 'шнека', 'шорец', 'шорка', 'шорня', 'шорох', 'шорты', 'шоссе', 'шофер', 'шпага', 'шпала', 'шпана', 'шпень', 'шпиль', 'шпинь', 'шпион', 'шпона', 'шпора', 'шприц', 'шпрот', 'шпуля', 'шпунт', 'шпынь', 'шримс', 'шрифт', 'штамб', 'штамм', 'штамп', 'штаны', 'штейн', 'штиль', 'штифт', 'штора', 'шторм', 'штраф', 'штрек', 'штрих', 'штука', 'штурм', 'штырь', 'шуаны', 'шубка', 'шугай', 'шудра', 'шуйца', 'шулер', 'шумер', 'шумок', 'шурин', 'шурпа', 'шуруп', 'шутка', 'шутов', 'шушун', 'шхеры', 'шхуна', 'щаной', 'щебет', 'щегол', 'щекот', 'щелка', 'щелок', 'щенок', 'щепка', 'щетка', 'щечка', 'щипец', 'щипка', 'щипок', 'щипцы', 'щитик', 'щиток', 'щупик', 'щурка', 'щурок', 'щучий', 'щучка', 'эвенк', 'эгида', 'эгрет', 'эдикт', 'экзот', 'эклер', 'экран', 'элеат', 'элита', 'эллин', 'эмаль', 'эммер', 'энзим', 'энный', 'эозин', 'эолит', 'эолов', 'эоцен', 'эпика', 'эпонж', 'эпоха', 'эпюра', 'эрбий', 'эрзац', 'эркер', 'эсдек', 'эскиз', 'эстет', 'этвеш', 'этика', 'эфиоп', 'юдоль', 'южнее', 'южный', 'юзист', 'юкола', 'юлить', 'юнеть', 'юниор', 'юница', 'юнкер', 'юнкор', 'юннат', 'юноша', 'юрист', 'юркий', 'юрьев', 'юферс', 'ябеда', 'явить', 'явный', 'ягель', 'ягода', 'ягуар', 'яичко', 'якать', 'якобы', 'якорь', 'ялбот', 'ямаец', 'ямина', 'ямища', 'ямный', 'ямщик', 'ярить', 'ярица', 'яркий', 'ярлык', 'яруга', 'ярыга', 'ясень', 'ясмин', 'ясный', 'яспис', 'яства', 'яство', 'ястык', 'ясырь', 'ятвяг', 'ятовь', 'яхонт', 'ячать', 'ячный']

In [20]:
# source 4: https://timmarinin.net/slovl/
target_words4 = "абзац аборт аверс аврал автор агент агнец адрес акрил актер акула акция аллея алмаз альфа амбар амеба ампир ангар ангел аниме аорта арбуз арена арест армия архив аршин аскет астра атака атлас атлет афера багет багор бадья базар балда балет балка банан банда банка барак баран барин барон басня батон башня бегун бедро бекон белка белок белье беляш берег берет бетон бидон бизон билет биржа бирка бисер битва благо блажь бланк блеск блоха блюдо бобер богач бодун бокал болид бомба борец боров ботан ботва бочка браво брань бремя брешь бровь бронь броня брошь брюки брюхо бугай бугор будка буква букет булат булка буран бурка бутан бутон буфет бухло бытие бювет вагон вакса валет валик валун вальс ванна варан варка варяг ватка вафля вдова веган ведро велик венгр венец веник венок вепрь верба верфь весло весна весть ветвь ветер ветка вечер вешка взвод вздох взнос взрыв видео визит вилка винил вираж вирус висок виток вихрь вишня вклад влага вобла водка вожак вождь вожжи война вокал волан волна волос вопль ворон ворот ворох врата время въезд выбор вывод выгул выдох выдра выезд вызов выкуп вылет вынос вырез выход вычет вышка вьюга газон галка галоп галош гамак гамма гараж гарем гений геном герой гжель гидра гиена глава гладь глина глист глыба гниль гобой говор голем голод голос гомон гонец гонка гопак горец горка горло город горох гость грамм грант грань греза греча грива гриль гроза груда грудь грунт груша грыжа гряда грязь губка гудок гуляш гумно гусак гусар гусли давка далек дамба дамка дверь дебош дебют девиз декан декор демон деним десна детка диван дилер динар длань длина днище добро довод догма дождь дозор долма домен домик домна домра донор донос доска досуг досье доход дочка драка драма древо дрейф дрель дробь дрова дрожь дрозд дронт дрофа дрянь дубль дудка дукат дупло дурак душка дымка дымок дырка дятел евнух егерь егоза жажда жакет жалко жатва желоб желчь жених жердь жерло жесть жетон живот жизнь жилет жилец жилье жираф житие жокей жрица жулик жупел забег забор завал завет завод завуч загар загон задор заказ закат закон залет залив залог залом замер замок запад запал запас запах запой запор зарок заряд засов засор затон затор зачет звено зверь зебра зевок зелье земля зенит зерно зефир знамя знать игрок идиот излом икона икота индюк инжир ирбис искра испуг истец исток исход кабак кабан кагор кадет кадка кадык казак казан казна казнь калач камео камин камыш канал канат канва канон капля капор капот капут карат карма карта каска касса каста катер катет катод каток качка кашпо квант кварк квота кебаб кегля келья кепка кефир кешью кивок кинза киоск кипеш кирка кисет киска киста кисть кишка класс клеть клоун клуша клюка кляча книга князь коала кобра ковер ковка козел койка кокон кокос колба колос колун колье кольт комар комик комод комок конец конус конюх копия копна копье корги кореш корка корма короб кости кость косяк котел котик кофта кочка кошка кража краса кредо крема крест кровь крона кросс кроха крупа круча крыло крыса крыша ксива кубик кубок кузня кузов кукла кулак кулек кулер кулик кулич кулон культ кумин кумир кумыс купаж купец купол купон кураж курок кусок кутья кухня кучер кучка кушак кювет лавка ладан ладья лазер лайка лакей лампа лапка лапта лапша ларек ларец ласка ласты лежка лейка лемур лента лепет лепка лепта леска лесть лиана ливер лидер лилия лиман лимит лимон линза линия ловец ловля логик лодка ложка локон ломка лопух лоток лотос лунка лыжня лычка ляжка лямка магия мадам мажор мазок майка майор макет малек малец малыш маляр манга манго манеж мания манка манок маржа марка марля маска масло масон масса масть матка мачта медик медок мекка мелок мерин мерка место месть месяц метан метис метка метла метод метро мечта мешок мизер милок минор минус мираж миска мишка модем мозги мойва мойка молва молот монах мопед морда мороз морок моряк мотив моток мотор мочка мошка мошна мужик музей муляж мумия мусор муфта мушка мысль мышка мышца мэрия мюсли мячик набат набор навар навес навет навоз навык наган нагар надел надир наказ накал налет налим налог намек напев напор нарды народ нарыв наряд насос наука начес невод недуг нервы нерпа нефть нимфа нитка ничья ножка ножны номер норка норма норов носок нужда нутро нырок нычка обвал обида облик обман обмен образ обрез оброк обруч обрыв обряд обувь обуза обход объем овощи овраг огонь озеро озноб океан окись оклад оковы окрас округ оксид октет окунь олень олива олово омела омлет оммаж опера опись опиум оплот опора опрос орава орган орден орлан осень осина оскал ослик остов отара отбой отбор отвар ответ отвод отгул отдел отдых отель отзыв отказ откат отрок отряд отсев отчет охват охота падеж пакет пакля палас палач палец палка панда панно папка парад парик парка паром парта парус паста пасть пасха пачка пашня певец пекан пекло пемза пенал пение пенка пепел перец перст песец песня песок петля петух печка пешка пиала пижон пикап пикет пилка пилон пилот пинок пинта пират пирог питон питье пихта пицца пламя пласт плата плато плаха плебс плеер племя плеск плеть плечо плешь плита плоть побег повар повод погон подол поезд пожар позер позор поиск пойма показ покер покой покос полба полет полис полка полог полоз полюс помет понос пончо порез порог порок порох порча порыв посев посол посох посыл поток потоп поход почва почет почка почта поэма право прайд праща пресс прием прима принт принц приют проба проем проза промо просо прыть прядь пряжа псина птица пугач пудра пульт пункт пупок пурга пучок пушка пчела пшено пытка пышка пьеса пьянь пятак пятка пятно радий радио радон разум район рамен рамка рампа ранец ранчо раунд рация рвота ребро ребус редис редут режим резак резка резон рейка рельс рента репка ретро речка решка рикша риска рифма робот ровня родео родня рожок розга рокот ролик роман ропот ротан ротор рохля рояль ртуть рубеж рубец рубин рубка рубль ружье рукав рулет рулон румба рупор русло ручей ручка рыбак рыбка рывок рынок рычаг рюмка сабля саван садик сазан салат салон салют самец самка санки сапер сапог сарай сатин сатир сауна сахар сачок свара сваты сваха свеча свист свита свора связь сдача сдвиг сдоба сеанс север седло сезон секта семга семья сенат сепия серия серна сетка силач синод синус синяк сироп ситец ситро сифон скала скарб скетч склад склеп склон скоба скотч скраб скрип скука скула скунс слава слеза слива слизь слово слуга слюда слюна смена смерч смесь смета смола смотр смрад смута смысл сноха собор совет совок сойка сокол солод сонет сопка сопли сопло сосед сосна сотня спазм спесь спина спирт спица сплав спорт спрут спуск среда ссора стадо сталь старт ствол стезя стена стенд степь стерх стиль столб стопа страж страх строй струя стужа ступа судак судно судья суета сукно сумка сумма сурок сусек сусло сутки сучок сушка сфера схема сцена съезд сынок сырок сюжет табак табло табор табун тайга тайна такса такси талия талон тальк танго танец тапер тапир тапок таран тариф тахта тачка тварь театр тезис текст телец тенор тепло терем терка тесто тесть тетка тизер типаж типун тираж тиран тиски титан ткань товар толпа томат тоник тонна тонус топаз топка топор топот торба торец тоска тотем точка трава тракт транш трата трель тренд тренч треск треть трико тромб тропа труба трусы труха трюмо тубус тукан тулуп тумак туман тумба тунец тупик турка туфля тучка тушка тыква тюбик тяпка уголь угорь удаль удача узник уклад уклон укроп уксус улика улица умора упрек упырь успех устав устье утиль ухват учеба фагот фазан факел факир фанат фасад фасон ферзь ферма фетиш фикус филин фильм финал финиш фирма фишка фланг флора фляга фокус фомка фондю форма форум фраза фреза фронт фрукт фугас фужер фурор халат халва хвост химия хитин хлыст хмель хобот холод холоп холст хомяк хорда хорей хорек хохот хруст хумус хурма хутор цапля цедра центр цинга цифра цукат цыган чайка часть чашка челка червь череп чернь черта честь чехол число чрево чугун чудак чулан чулок шабаш шайба шайка шакал шалаш шаман шапка шасси шатер шахта шашка шепот шериф ширма шишка школа шланг шляпа шмель шорох шорты шоссе шофер шпага шпала шпиль шпион шпора шприц штамп штаны штиль штора шторм штраф штука шубка шумер шурпа шутка шхуна щегол щенок щепка щетка щипок экран элита эмаль эпоха эрзац эстет юдоль юноша юрист ябеда ягода якорь ямщик".split(" ")
allowed_words4 = "абвер абзац аборт аверс аврал автор агент агнец адрес акрил актер акула акция алкаш аллея алмаз алтын алыча альпы альфа амбар амеба ампир ангар ангел аниме анкер антей анусы аорта арбуз арена арест аркан армия архив аршин аскет астра атака атлас атлет афера багет багор бадья базар балда балет балка балык банан банда банка барак баран барин барка барон барре басня батат батон башка башня бегун бедра бедро бекон белка белок белый белье беляш берег берет бетон бидон бизон билет биржа бирка бисер битва битум битюг благо блажь бланк блеск блоха блюдо бобер богач бодун бокал болид бомба борец боров ботан ботва бочка бошка браво брань бремя брешь бровь бронь броня брошь брюки брюхо бугай бугор будка буква букет булат булка буран бурда бурка бутан бутон буфет бухло бытие бювет вагон вазон вакса валет валик валун вальс ванна варан варка варяг ватка вафля вдова вдруг веган ведро велик венгр венец веник венок вепрь верба верфь весло весна весть ветвь ветер ветка вечер вешка взвод вздох взнос взрыв видео видно видок визит вилка вилок винил вираж вирус висок виток вихрь вишня вклад влага вобла водка вожак вождь вожжа вожжи возжа война вокал волан волга волна волос вопль ворон ворот ворох враги врата время въезд выбор вывод выгул выдох выдра выезд вызов выкуп вылет вынос вырез выход вычет вышка вьюга газон галка галоп галош гамак гамма гараж гарем гений геном герой гжель гидра гиена глава гладь глина глист глыба гнида гниль гобой говно говор голем голод голос гомон гонец гонка гопак горец горка горло город горох гость грамм грант грань греза греча грива гриль гроза груда грудь грунт груша грыжа гряда грязь губка гудок гуляш гумно гусак гусар гусли давка далек дамба дамка дверь дебил дебош дебри дебют девиз декан декор демон деним десна детка диван дилдо дилер динар длань длина днище добро довод догма дождь дозор докер долма домен домик домна домра донор донос доска досуг досье доход дочка драка драма древо дрейф дрель дробь дрова дрожь дрозд дронт дрофа дрянь дубль дудка дукат дупло дурак дурка душка дымка дымок дырка дятел евнух еврей егерь егоза жажда жакет жалко жатва желоб желчь жених жердь жерло жесть жетон живец живот жизнь жилет жилец жилье жираф жирок житие жокей жопка жрица жулик жупел забег забор завал завет завод завуч загар загиб загон задор заказ закат закон залет залив залог залом замер замес замок запад запал запас запах запой запор зарок заряд засов засор затон затор зачет звено зверь звуки зебра зевок зелье земля зенит зерно зефир зипун знамя знать игрок идиот излом икона икота ингуш индия индюк инжир интим ирбис искра испуг истец исток исход кабак кабан кагор кадет кадка кадык казак казан казах казна казнь кайло калач камео камея камин камыш канал канат канва канон капля капор капот капут карат карма карта каска касса каста катер катет катод каток качка кашка кашпо квант кварк квота кебаб кегля келья кепка кефир кешью кибуц кивок кинза киоск кипеш кирка кисет киска киста кисть китай кишка кишки класс клест клеть клоун клуша клюка ключи кляча книга князь коала кобра ковер ковид ковка ковры козел койка кокон кокос колба колос колун колье кольт комар комик комод комок конец конус конюх копия копна копье коран корги кореш корея корка корма короб кости кость косяк котел котик кофта кочка кошка кража краса кредо крема крест кровь кроль крона кросс кроха крупа круча крыло крыса крыша ксива кубик кубок кузня кузов кукла кулак кулек кулер кулик кулич кулон культ кумар кумин кумир кумыс кунак купаж купец купол купон кураж курва курок кусок кусты кутья кухня кучер кучка кушак кювет лавка ладан ладья лазер лайка лакей лампа лапка лапта лапша ларек ларец ласка ласты латка лежка лейка лемур ленин лента лепет лепка лепта леска лесок лесть лиана ливер лидар лидер лилия лиман лимит лимон линза линия литва лобок ловец ловля логик лодка ложка локон ломка лопух лоток лотос лунка лыжня лычка ляжка лямка магия мадам мажор мазок майка майор макет малек малец малыш маляр манга манго манеж мания манка манок маржа марка марля маска масло масон масса масть матка мачта медик медок мекка мелок мерин мерка место месть месяц метан метис метка метла метод метро мечта мешок мизер милок милфа минет минор минус мираж мирок миска митра мишка модем мозги мойва мойка мокум молва молот монах мопед морда морзе мороз морок моряк мотив мотня моток мотор мочка мошка мошна мразь мужик музей муляж мумия мусор муфта мушка мысль мышка мышца мэрия мюсли мячик набат набор навар навес навет навоз навык наган нагар надел надир наказ накал налет налим налог намаз намек напев напор нарды народ нарыв наряд насос наука начес невод недуг некто немой непал нервы нерка нерпа нефть нигер никто нимфа нитка ничья ножка ножны номер норка норма норов носик носки носок нотка ношпа нужда нутро нырок нычка обвал обида облик обман обмен образ обрез оброк обруч обрыв обряд обувь обуза обход объем овощи овраг огонь озеро озноб океан окись оклад оковы около окрас округ оксид октет окунь олень олива олово омела омлет оммаж опера опись опиум оплот опора опрос орава орган оргия орден ордер орлан осень осина оскал ослик остов отара отбой отбор отвар ответ отвод отгул отдел отдых отель отзыв отказ откат отрок отряд отсев отчет охват охота падеж пакет пакля палас палач палех палец палка памир панда панно папка парад парео париж парик парка паром парта парус паста пасть пасха пахан пацан пачка пашня певец пегас пекан пекло пемза пенал пенек пение пенис пенка пепел перец перст перун перья песец песнь песня песок песто петля петух печка пешка пиала пижон пикап пикет пилка пилон пилот пинок пинта пират пирог питон питье пихта пицца пламя пласт плата плато плаха плебс плева плеер племя плеск плеть плечо плешь плита плоть побег повар повод погон подол поезд пожар позер позор поиск пойма показ покер покой покос полба полет полис полка полог полоз полюс помет понос пончо порез порка порно порог порок порох порча порыв посев посол посох посыл поток потоп поход почва почет почка почта поэма права право прайд пранк праща пресс прием призы прима принт принц приют проба проем проза промо просо прыть прядь пряжа прямо псина птица пугач пудра пузан пульт пункт пупок пурга пучок пушка пчела пшено пытка пышка пьеса пьянь пятак пятка пятно радий радио радон разум район рамен рамка рампа ранец ранчо раунд рация рвота ребро ребус редис редут режим резак резка резон рейка рельс рента репин репка ретро речка решка рикша риска рифма робот ровня родео родня рожок розан розга розги рокот ролик роман ропот ротан ротик ротор рохля рояль ртуть рубеж рубец рубин рубка рубль ружье рукав рулет рулон румба румын рупор русло ручей ручка рыбак рыбка рывок рынок рычаг рюмка ряска сабля саван садик сазан салат салон салют самец самка санки сапер сапог сарай сатин сатир сауна сахар сачок свара сваты сваха света свеча свинг свист свита свора связь сдача сдвиг сдоба сеанс север седло сезон секта семга семит семья сенат сепия серия серна сетка сизый силач силок синод синус синяк сироп ситец ситро сифон скала скарб скетч скилл склад склеп склон скоба скотч скраб скрип скука скула скунс слава слани слеза слива слизь слова слово слуга слюда слюна смена смерд смерч смесь смета смола смоль смотр смрад смута смысл снедь снова сноха собор совет совок сойка сокол солод сонар сонет сопка сопли сопло сопля сорра сосед соска сосна сосок сотка сотня сошка спазм спесь спина спирт спица сплав спора спорт спрут спуск срань среда ссора стадо сталь старт ствол стезя стена стенд степь стерх стило стиль столб столы стопа стояк страж страх строй струя стужа ступа судак судно судья суета сукно сумка сумма сурок сусек сусло сутки сучок сушка сфера схема сцена съезд сынок сырок сюжет табак табло табор табун тайга тайна такса такси талиб талия талон тальк танго танец танцы тапер тапир тапка тапки тапок таран тариф тахта тачка тварь театр тезис текст телец телка тенор тепло терем терка тесак тесло тесто тесть тетка течка тизер типаж типун тираж тиран тиски титан ткань товар толпа томат томик тоник тонна тонус топаз топка топор топот торба торги торец тоска тотем точка трава тракт транш траст трата трель тренд тренч треск трест треть трико триод тромб тропа труба трусы труха трюмо тубус тукан тулуп тумак туман тумба тунец тупик турка турок туфли туфля тучка тушка тыква тюбик тяпка уголь угорь удаль удача узбек узник уклад уклон укроп уксус улика улица умора упрек упырь успех устав устье утиль ухват учеба фагот фазан факел факир фанат фасад фаска фасон ферзь ферма фетиш фигня фикус филин фильм финал финиш финка фирма фишка фланг флора фляга фокус фомка фондю форма форум фраза фрахт фреза фронт фрукт фугас фужер фураж фурор фюрер хабар халат халва харам хвост химия хитин хлыст хмель хобот холод холоп холст хомяк хорда хорей хорек хохот хрень хруст хтонь хумус хунта хурма хутор цапля цветы цевье цедра центр цехин цинга цифра цукат цыган чайка чарка часть чашка челка червь череп чернь черта честь чехол чипсы число чисто чрево чугун чудак чуйка чулан чулок чурка чушка шабаш шайба шайка шакал шалаш шаман шапка шарфы шасси шатер шахта шашка шепот шериф ширма шифон шишка школа шкура шланг шляпа шмель шорох шорты шоссе шофер шпага шпала шпиль шпион шпора шприц штамп штаны штиль штора шторм штраф штука шубка шумер шурпа шутка шхуна щегол щелка щенок щепка щетка щипок экран элита эмаль эпоха эрзац эстет юдоль юноша юрист ябеда ягода якорь ямщик".split(" ")

In [21]:
def combine_words(wordlists: []) -> set():
    words = set(wordlists[0])
    for wlist in wordlists[1:]:
        words = words.union(set(wlist))
    print("Total words:", len(words))
    return words

## Target words

1. Combine all target words from the data sources

In [61]:
target_words_candidates = combine_words([target_words1, target_words3, target_words4])
target_words_candidates = [item.lower() for item in target_words_candidates]

Total words: 4225


2. Load the frequency dictionary and filter out target candidates with no known word frequency

   NOTE: make sure you have the frequency dictionary ready in wordle/resources, [download link](http://dict.ruslang.ru/Freq2011.zip)

In [62]:
freq_dict = pd.read_csv("../resources/freqrnc2011.csv", sep='\t')
freq_dict['Lemma'] = freq_dict['Lemma'].map(lambda l: l.lower().strip())
freq_dict = freq_dict.groupby('Lemma')['Freq(ipm)'].sum().reset_index()

filtered_words_freqs = freq_dict[freq_dict['Lemma'].isin(target_words_candidates)]
print("Total words after filtering by freq:", filtered_words_freqs.shape[0])

Total words after filtering by freq: 2665


3. Load the clue dictionary and filter out target candidates with no clues available (NOTE: please run utils/clue_scraper.py to make sure the clue dictionary exists)

In [65]:
with open("../resources/clues.json", 'r') as file:
    clue_dict = json.load(file)

double_filtered_words_freqs = filtered_words_freqs[filtered_words_freqs['Lemma'].isin(clue_dict.keys())]
print("Total words after filtering by freq and clue:", double_filtered_words_freqs.shape[0])

Total words after filtering by freq and clue: 2162


4. Write into the file (wordle/resources/target_words.txt)

In [69]:
target_words = double_filtered_words_freqs["Lemma"]
with open("../resources/target_words.txt", "w") as file: 
    for word in target_words:
        file.write(word)
        file.write('\n')

5. Divide words into 3 difficulty groups (easy, medium, hard) based on word frequency and write into respective files

In [76]:
def write_stats(df: pd.DataFrame):
    """
    Write some word stats after binning into a txt file for future reference.
    """
    one_third = int(df.shape[0] / 3)
    two_thirds = df.shape[0] - one_third

    with open("../resources/data_stats.txt", "w") as o:
        o.write(f"Total words: {df.shape[0]}\n")
        o.write(f"High frequency: {one_third} words\n")
        o.write(f"Medium frequency: {two_thirds-one_third} words.\n")
        o.write(f"Low frequency: {df.shape[0] - two_thirds} words\n")


def create_word_lists(df: pd.DataFrame):
    """
    Sort and divide the target words into 3 equal groups (high, medium and low frequency words).
    :param df: dataframe of target words with 2 columns: Lemma and Freq(ipm)
    """
    df["Freq(ipm)"] = df["Freq(ipm)"].map(lambda f:  round(f, 2))
    df = df.sort_values(by=["Freq(ipm)"], ascending=False)
    
    write_stats(df)

    one_third = int(df.shape[0] / 3)
    two_thirds = df.shape[0] - one_third

    with open("../resources/easy_words.txt", "w") as o:
        for w in df.iloc[:one_third]['Lemma'].values:
            o.write(f"{w}\n")

    with open("../resources/medium_words.txt", "w") as o:
        for w in df.iloc[one_third+1:two_thirds]['Lemma'].values:
            o.write(f"{w}\n")

    with open("../resources/hard_words.txt", "w") as o:
        for w in df.iloc[two_thirds+1:]['Lemma'].values:
            o.write(f"{w}\n")

In [ ]:
create_word_lists(double_filtered_words_freqs)

## Allowed words

1. Combine all allowed words from the data sources

In [25]:
allowed_words_candidates = combine_words([allowed_words1, allowed_words2, allowed_words3, allowed_words4])

Total words: 8390


In [70]:
allowed_words = combine_words([allowed_words_candidates, set(target_words)])

Total words: 8390


2. Write into the file (wordle/resources/allowed_words.txt)

In [73]:
with open("../resources/allowed_words.txt", "w") as o: 
    for word in list(allowed_words):
        o.write(f"{word}\n")